In [ ]:
import os
import pandas as pd
import os
import requests
from bs4 import BeautifulSoup
from requests.exceptions import HTTPError
import numpy as np
import regex as re

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive/')

Mounted at /content/gdrive/


In [ ]:
# directory = "gdrive/MyDrive/CSCI 461/Project"

directory = "gdrive/MyDrive/Project"

df = pd.read_csv("{0}/final_datatable.csv".format(directory))

In [ ]:
df

,NDC11,Part,Image,Layout,Name
0,2322730,1,PillProjectDisc69/images/CLLLLUPGIX7J8MP1WWQ9W...,C3PI_Reference,strattera
1,2322730,1,PillProjectDisc98/images/PRNJ-AXZIQ!HUQKJJBP_D...,C3PI_Reference,strattera
2,2322730,1,PillProjectDisc10/images/79U-YY6M1UUR6F127ZMAC...,C3PI_Test,strattera
3,2322730,1,PillProjectDisc11/images/7WVFV5H74!ELFNQ_GUH92...,C3PI_Test,strattera
4,2322730,1,PillProjectDisc20/images/B4CH0R9B7PEQ6GORRX-8X...,C3PI_Test,strattera
...,...,...,...,...,...
133769,99207046730,1,PillProjectDisc103/images/TY5OVXLLOXV6H4I1TDVT...,MC_COOKED_CALIBRATED_V1.2,solodyn
133770,99207046730,1,PillProjectDisc31/images/BEIR3XK38EMGSDOZTWMUK...,MC_COOKED_CALIBRATED_V1.2,solodyn
133771,99207046730,1,PillProjectDisc69/images/CLJ1W40OS0XG5H6IVYT!N...,MC_COOKED_CALIBRATED_V1.2,solodyn
133772,99207046730,1,PillProjectDisc77/images/CSUHWDZ!XAZSEJHDANMFR...,MC_COOKED_CALIBRATED_V1.2,solodyn


In [ ]:
unique_labels = df["Name"].unique()

In [ ]:
def get_url_from_page(drug_name):
  result = None
  if drug_name == "":
    return
  try:
    url = "https://go.drugbank.com/unearth/q?query={0}&button=&searcher=drugs".format(drug_name)
    response = requests.post(url)
    result = response.url
    if "unearth" in result:
      drug_split = drug_name.split(" ")
      url = "https://go.drugbank.com/unearth/q?query={0}&button=&searcher=drugs".format(drug_split[0])
      response = requests.post(url)
      result = response.url

      if "unearth" in result:
        first_link_result =  first_link_from_response(result)
        result = first_link_result

    if not result:
      print("No result for ", drug_name)
    return result
  except HTTPError as http_err:
      print(f'HTTP error occurred: {http_err}')
  except Exception as err:
      print(f'Other error occurred: {err}')

In [ ]:
def first_link_from_response(url):
  try:
    response = requests.get(url)
    response.raise_for_status()
    response.encoding = 'utf-8'
    soup = BeautifulSoup(response.text)
    hit_tags = soup.find("h2", {"class": "hit-link"})
    ref = hit_tags.contents[0]['href']
    return "https://go.drugbank.com" + ref
  except HTTPError as http_err:
      print(f'HTTP error occurred: {http_err}')
  except Exception as err:
      print(f'Other error occurred: {err}')



In [ ]:
get_url_from_page("tylenol")

'https://go.drugbank.com/drugs/DB00316'

In [ ]:
def get_toxicity_from_page(page_url):
  try:
    response = requests.get(page_url)
    response.raise_for_status()
    response.encoding = 'utf-8'
    soup = BeautifulSoup(response.text)
    toxicity_tag = soup.find("dt", {"id": "toxicity"})
    toxicity_next_sib = toxicity_tag.find_next_sibling("dd")
    toxicity_field = toxicity_next_sib.find_all("p")[0].text

    try:
      toxicity_mouse, toxicity_rat = toxicity_field.split(";")
      mouse_start_bracket, rat_start_bracket = toxicity_mouse.find("("), toxicity_rat.find("(")
      toxicity_mouse_val, toxicity_rat_val = toxicity_mouse[7: mouse_start_bracket], toxicity_rat[7: rat_start_bracket]
      return toxicity_mouse_val.strip(), toxicity_rat_val.strip()
    except:
      return toxicity_field, toxicity_field
  except HTTPError as http_err:
      print(f'HTTP error occurred: {http_err}')
  except Exception as err:
      print(f'Other error occurred: {err}')

In [ ]:
toxicity_mouse_val, toxicity_rat_val = get_toxicity_from_page("https://go.drugbank.com/drugs/DB01017")

In [ ]:
drug_name_to_url = {}

In [ ]:
print(len(unique_labels))

1651


In [ ]:
drugs_without_urls = []

for drug_name in unique_labels:
  try:
    url = get_url_from_page(drug_name)
    if not url:
      drugs_without_urls.append(drug_name)

    drug_name_to_url[drug_name] = url
  except:
    drug_name_to_url[drug_name] = ""

Other error occurred: 'NoneType' object has no attribute 'contents'
No result for  diovanhctz
Other error occurred: 'NoneType' object has no attribute 'contents'
No result for  apapcodeine
Other error occurred: 'NoneType' object has no attribute 'contents'
No result for  ecnaprosyn
Other error occurred: 'NoneType' object has no attribute 'contents'
No result for  fexofenadinepse hr
Other error occurred: 'NoneType' object has no attribute 'contents'
No result for  trilosprintec triphasic regimen
Other error occurred: 'NoneType' object has no attribute 'contents'
No result for  amoxicilclavulanate
Other error occurred: 'NoneType' object has no attribute 'contents'
No result for  onceaweek risedronate sodium
Other error occurred: 'NoneType' object has no attribute 'contents'
No result for  omegaacid ethyl esters
Other error occurred: 'NoneType' object has no attribute 'contents'
No result for  irbesarhctz
Other error occurred: 'NoneType' object has no attribute 'contents'
No result for  l

In [ ]:
toxicity_mapping = {}

In [ ]:
from collections import defaultdict
toxicity_mapping = defaultdict(list)

for drug_name, link in drug_name_to_url.items():
  try:
    mouse_toxic, rat_toxic = get_toxicity_from_page(link)
  except:
    mouse_toxic, rat_toxic = "", ""
  toxicity_mapping[drug_name] = [mouse_toxic, rat_toxic]

Other error occurred: Invalid URL 'None': No schema supplied. Perhaps you meant http://None?
Other error occurred: Invalid URL 'None': No schema supplied. Perhaps you meant http://None?
Other error occurred: Invalid URL 'None': No schema supplied. Perhaps you meant http://None?
Other error occurred: Invalid URL 'None': No schema supplied. Perhaps you meant http://None?
Other error occurred: Invalid URL 'None': No schema supplied. Perhaps you meant http://None?
Other error occurred: Invalid URL 'None': No schema supplied. Perhaps you meant http://None?
Other error occurred: Invalid URL 'None': No schema supplied. Perhaps you meant http://None?
Other error occurred: Invalid URL 'None': No schema supplied. Perhaps you meant http://None?
Other error occurred: Invalid URL 'None': No schema supplied. Perhaps you meant http://None?
Other error occurred: Invalid URL 'None': No schema supplied. Perhaps you meant http://None?
Other error occurred: Invalid URL 'None': No schema supplied. Perhaps 

In [ ]:
from collections import defaultdict

In [ ]:
toxicity_mapping

NameError: ignored

In [ ]:
def create_new_df():
  result = []
  for index, row in df.iterrows():
    new_row = np.append(row.values, toxicity_mapping[row['Name']])
    result.append(new_row)
  return pd.DataFrame(result, columns = list(df.columns) + ['mouse toxicity', 'rat toxicity'])

In [ ]:
df_with_toxicity = create_new_df()

In [ ]:
df_with_toxicity.to_csv("gdrive/MyDrive/Project/db_with_toxicity.csv", index=False)

In [ ]:
def try_finding_toxicicity(word):
  return re.findall("\d+\s*\wg/kg", word)

In [ ]:
try_finding_toxicicity("Oral LD50 is 2000 kg/kg in rats 500 mg/kg for cat and 600 mg/kg for rabbit")

['2000 kg/kg', '500 mg/kg', '600 mg/kg']

In [ ]:
toxicity_mapping = defaultdict(list,
            {'strattera': ['There is limited clinical trial experience with atomoxetine overdose. During postmarketing, there have been fatalities reported involving a mixed ingestion overdose of atomoxetine capsules and at least one other drug. There have been no reports of death involving overdose of atomoxetine capsules alone, including intentional overdoses at amounts up to 1400 mg. In some cases of overdose involving atomoxetine, seizures have been reported. The most commonly reported symptoms accompanying acute and chronic overdoses of atomoxetine capsules were gastrointestinal symptoms, somnolence, dizziness, tremor, and abnormal behavior. Hyperactivity and agitation have also been reported. Signs and symptoms consistent with mild to moderate sympathetic nervous system activation (e.g., tachycardia, blood pressure increased, mydriasis, dry mouth) have also been observed. Most events were mild to moderate. Less commonly, there have been reports of QT prolongation and mental changes, including disorientation and hallucinations. If symptoms of overdose are suspected, a Certified Poison Control Center should be consulted for up to date guidance and advice. Because atomoxetine is highly protein-bound, dialysis is not likely to be useful in the treatment of overdose.18',
              'There is limited clinical trial experience with atomoxetine overdose. During postmarketing, there have been fatalities reported involving a mixed ingestion overdose of atomoxetine capsules and at least one other drug. There have been no reports of death involving overdose of atomoxetine capsules alone, including intentional overdoses at amounts up to 1400 mg. In some cases of overdose involving atomoxetine, seizures have been reported. The most commonly reported symptoms accompanying acute and chronic overdoses of atomoxetine capsules were gastrointestinal symptoms, somnolence, dizziness, tremor, and abnormal behavior. Hyperactivity and agitation have also been reported. Signs and symptoms consistent with mild to moderate sympathetic nervous system activation (e.g., tachycardia, blood pressure increased, mydriasis, dry mouth) have also been observed. Most events were mild to moderate. Less commonly, there have been reports of QT prolongation and mental changes, including disorientation and hallucinations. If symptoms of overdose are suspected, a Certified Poison Control Center should be consulted for up to date guidance and advice. Because atomoxetine is highly protein-bound, dialysis is not likely to be useful in the treatment of overdose.18'],
             'cymbalta': ['Overdose', 'Overdose'],
             'evista': ['LD50 and Overdose', 'LD50 and Overdose'],
             'cialis': ['Symptoms of overdose are expected to be similar to typical adverse effects which may include headache, dyspepsia, back pain, myalgia, nasopharyngitis, and dizziness.7,8 Standard supportive care is recommended. Hemodialysis is not expected to contribute significantly to tadalafil clearance.',
              'Symptoms of overdose are expected to be similar to typical adverse effects which may include headache, dyspepsia, back pain, myalgia, nasopharyngitis, and dizziness.7,8 Standard supportive care is recommended. Hemodialysis is not expected to contribute significantly to tadalafil clearance.'],
             'livalo': ['Not Available', 'Not Available'],
             'effient': ['', ''],
             'eliquis': ['Animal studies have shown an increased risk of maternal bleeding during pregnancy but no increase in fetal malformations or fetal or maternal deathsLabel. It is unknown if this animal data also translates to humans so apixaban should only be used in pregnancy if the benefits outweigh the risksLabel. It is not know whether apixaban is safe and effective in labor and during birth, though animal studies have shown an increased rate of maternal bleedingLabel. Animal studies in rats show apixaban excreted in milk, though it is not know if this also applies to humansLabel. Nursing mothers should either stop breastfeeding or stop taking apixaban depending on the risk and benefit of each optionLabel. Studies to determine safety and effectiveness in pediatric patients have yet to be performedLabel. Studies that involved geriatric patients (at least 75 years old) saw no difference in safety or effectiveness compared to younger patients, though geriatric patients at an especially advanced age may be more susceptible to adverse effectsLabel. Dosage adjustments for patients with end stage renal disease(ESRD) are based on estimates of pharmacokinetic principles and not clinical studyLabel. Patients with ESRD may experience pharmacodynamics similar to those seen in well controlled studies but it may not lead to the same clinical effectsLabel. Dosage adjustments are not necessary in mild hepatic impairmentLabel. In moderate hepatic impairment patients may already experience abnormalities in coagulation and so no dose recommendations are possibleLabel. Apixaban is not recommended for patients with severe hepatic impairmentLabel.',
              'Animal studies have shown an increased risk of maternal bleeding during pregnancy but no increase in fetal malformations or fetal or maternal deathsLabel. It is unknown if this animal data also translates to humans so apixaban should only be used in pregnancy if the benefits outweigh the risksLabel. It is not know whether apixaban is safe and effective in labor and during birth, though animal studies have shown an increased rate of maternal bleedingLabel. Animal studies in rats show apixaban excreted in milk, though it is not know if this also applies to humansLabel. Nursing mothers should either stop breastfeeding or stop taking apixaban depending on the risk and benefit of each optionLabel. Studies to determine safety and effectiveness in pediatric patients have yet to be performedLabel. Studies that involved geriatric patients (at least 75 years old) saw no difference in safety or effectiveness compared to younger patients, though geriatric patients at an especially advanced age may be more susceptible to adverse effectsLabel. Dosage adjustments for patients with end stage renal disease(ESRD) are based on estimates of pharmacokinetic principles and not clinical studyLabel. Patients with ESRD may experience pharmacodynamics similar to those seen in well controlled studies but it may not lead to the same clinical effectsLabel. Dosage adjustments are not necessary in mild hepatic impairmentLabel. In moderate hepatic impairment patients may already experience abnormalities in coagulation and so no dose recommendations are possibleLabel. Apixaban is not recommended for patients with severe hepatic impairmentLabel.'],
             'onglyza': ['Adverse reactions reported in ≥5% of patients treated with saxagliptin and more commonly than in patients treated with placebo are: upper respiratory tract infection, urinary tract infection, and headache.',
              'Adverse reactions reported in ≥5% of patients treated with saxagliptin and more commonly than in patients treated with placebo are: upper respiratory tract infection, urinary tract infection, and headache.'],
             'kombiglyze': ['Adverse reactions reported in ≥5% of patients treated with saxagliptin and more commonly than in patients treated with placebo are: upper respiratory tract infection, urinary tract infection, and headache.',
              'Adverse reactions reported in ≥5% of patients treated with saxagliptin and more commonly than in patients treated with placebo are: upper respiratory tract infection, urinary tract infection, and headache.'],
             'cellcept': ['LD50', 'LD50'],
             'tamiflu': ['Reports of overdoses with oseltamivir have been received from clinical trials and during postmarketing experience. In the majority of cases reporting overdose, no adverse reactions were reported. Adverse reactions reported following overdose were similar in nature to those observed with therapeutic doses of oseltamivir.',
              'Reports of overdoses with oseltamivir have been received from clinical trials and during postmarketing experience. In the majority of cases reporting overdose, no adverse reactions were reported. Adverse reactions reported following overdose were similar in nature to those observed with therapeutic doses of oseltamivir.'],
             'xeloda (eciine)': ['Not Available', 'Not Available'],
             'stromectol': ['Used appropriately, topically applied ivermectin lotions and creams are unlikely to cause significant toxicity. With accidental or intentional significant exposure to unknown quantities of veterinary formulations of ivermectin (by ingestion, inhalation, injection, or significant body surface area exposure) patients have reported rash, edema, headache, dizziness, asthenia, nausea, vomiting, and diarrhea. Other less common effects include seizure, ataxia, dyspnea, abdominal pain, paresthesia, urticaria, and contact dermatitis.8 Overdosage with orally administered ivermectin was lethal in mice and rats (at doses several-fold higher than the recommended dose), with death preceded by significant ataxia, bradypnea, tremors, ptosis, decreased activity, emesis, and mydriasis.8',
              'Used appropriately, topically applied ivermectin lotions and creams are unlikely to cause significant toxicity. With accidental or intentional significant exposure to unknown quantities of veterinary formulations of ivermectin (by ingestion, inhalation, injection, or significant body surface area exposure) patients have reported rash, edema, headache, dizziness, asthenia, nausea, vomiting, and diarrhea. Other less common effects include seizure, ataxia, dyspnea, abdominal pain, paresthesia, urticaria, and contact dermatitis.8 Overdosage with orally administered ivermectin was lethal in mice and rats (at doses several-fold higher than the recommended dose), with death preceded by significant ataxia, bradypnea, tremors, ptosis, decreased activity, emesis, and mydriasis.8'],
             'janumet xr': ['Animal studies in pregnancy have shown no adverse effects on the mother or offspring at normal doses, however these results are not always applicable to humansLabel. There is currently a voluntary registry of fetal exposureLabel,7. Animal studies at 100 times the maximum recommended human dose resulted in an increase in rib malformationsLabel. Sitagliptin is excreted in the milk of rats but it is not known if it would also be expressed in human breast milkLabel. Because many drugs are expressed in human breast milk, the risk and benefit of prescribing the medication must be consideredLabel. There is currently a lack of safety and effectiveness data in pediatric patientsLabel. No differences in safety and efficacy were observed in geriatric patients compared to younger patients, however caution should be used in this population as they are more likely to have reduced renal functionLabel. Sitagliptin has also been associated with a 34% relative risk increase for all cause infection4. There was no significant difference in patient response across sex, age, race, ethnicity, and BMI3.',
              'Animal studies in pregnancy have shown no adverse effects on the mother or offspring at normal doses, however these results are not always applicable to humansLabel. There is currently a voluntary registry of fetal exposureLabel,7. Animal studies at 100 times the maximum recommended human dose resulted in an increase in rib malformationsLabel. Sitagliptin is excreted in the milk of rats but it is not known if it would also be expressed in human breast milkLabel. Because many drugs are expressed in human breast milk, the risk and benefit of prescribing the medication must be consideredLabel. There is currently a lack of safety and effectiveness data in pediatric patientsLabel. No differences in safety and efficacy were observed in geriatric patients compared to younger patients, however caution should be used in this population as they are more likely to have reduced renal functionLabel. Sitagliptin has also been associated with a 34% relative risk increase for all cause infection4. There was no significant difference in patient response across sex, age, race, ethnicity, and BMI3.'],
             'januvia': ['Animal studies in pregnancy have shown no adverse effects on the mother or offspring at normal doses, however these results are not always applicable to humansLabel. There is currently a voluntary registry of fetal exposureLabel,7. Animal studies at 100 times the maximum recommended human dose resulted in an increase in rib malformationsLabel. Sitagliptin is excreted in the milk of rats but it is not known if it would also be expressed in human breast milkLabel. Because many drugs are expressed in human breast milk, the risk and benefit of prescribing the medication must be consideredLabel. There is currently a lack of safety and effectiveness data in pediatric patientsLabel. No differences in safety and efficacy were observed in geriatric patients compared to younger patients, however caution should be used in this population as they are more likely to have reduced renal functionLabel. Sitagliptin has also been associated with a 34% relative risk increase for all cause infection4. There was no significant difference in patient response across sex, age, race, ethnicity, and BMI3.',
              'Animal studies in pregnancy have shown no adverse effects on the mother or offspring at normal doses, however these results are not always applicable to humansLabel. There is currently a voluntary registry of fetal exposureLabel,7. Animal studies at 100 times the maximum recommended human dose resulted in an increase in rib malformationsLabel. Sitagliptin is excreted in the milk of rats but it is not known if it would also be expressed in human breast milkLabel. Because many drugs are expressed in human breast milk, the risk and benefit of prescribing the medication must be consideredLabel. There is currently a lack of safety and effectiveness data in pediatric patientsLabel. No differences in safety and efficacy were observed in geriatric patients compared to younger patients, however caution should be used in this population as they are more likely to have reduced renal functionLabel. Sitagliptin has also been associated with a 34% relative risk increase for all cause infection4. There was no significant difference in patient response across sex, age, race, ethnicity, and BMI3.'],
             'isentress': ['Not Available', 'Not Available'],
             'emend (aprepitant)': ['Not Available', 'Not Available'],
             'emend (aprepitant) sule': ['Not Available', 'Not Available'],
             'janumet': ['Animal studies in pregnancy have shown no adverse effects on the mother or offspring at normal doses, however these results are not always applicable to humansLabel. There is currently a voluntary registry of fetal exposureLabel,7. Animal studies at 100 times the maximum recommended human dose resulted in an increase in rib malformationsLabel. Sitagliptin is excreted in the milk of rats but it is not known if it would also be expressed in human breast milkLabel. Because many drugs are expressed in human breast milk, the risk and benefit of prescribing the medication must be consideredLabel. There is currently a lack of safety and effectiveness data in pediatric patientsLabel. No differences in safety and efficacy were observed in geriatric patients compared to younger patients, however caution should be used in this population as they are more likely to have reduced renal functionLabel. Sitagliptin has also been associated with a 34% relative risk increase for all cause infection4. There was no significant difference in patient response across sex, age, race, ethnicity, and BMI3.',
              'Animal studies in pregnancy have shown no adverse effects on the mother or offspring at normal doses, however these results are not always applicable to humansLabel. There is currently a voluntary registry of fetal exposureLabel,7. Animal studies at 100 times the maximum recommended human dose resulted in an increase in rib malformationsLabel. Sitagliptin is excreted in the milk of rats but it is not known if it would also be expressed in human breast milkLabel. Because many drugs are expressed in human breast milk, the risk and benefit of prescribing the medication must be consideredLabel. There is currently a lack of safety and effectiveness data in pediatric patientsLabel. No differences in safety and efficacy were observed in geriatric patients compared to younger patients, however caution should be used in this population as they are more likely to have reduced renal functionLabel. Sitagliptin has also been associated with a 34% relative risk increase for all cause infection4. There was no significant difference in patient response across sex, age, race, ethnicity, and BMI3.'],
             'hyzaar': ['The oral LD50 of hydrochlorothiazide is >10g/kg in mice and rats.10,11',
              'The oral LD50 of hydrochlorothiazide is >10g/kg in mice and rats.10,11'],
             'zocor': ['Not Available', 'Not Available'],
             'cozaar': ['The oral TDLO in mice is 1000mg/kg and in rats is 2000mg/kg.3,4 In humans the TDLO for men is 10mg/kg/2W and for women is 1mg/kg/1D.5',
              'The oral TDLO in mice is 1000mg/kg and in rats is 2000mg/kg.3,4 In humans the TDLO for men is 10mg/kg/2W and for women is 1mg/kg/1D.5'],
             'sinemet': ['The LD50 of carbidopa is reported to be in the rat of 4810 mg/kg.MSDS In animal studies, carbidopa showed no incidences on neoplasia and showed no effect on the fertility status and development.Label',
              'The LD50 of carbidopa is reported to be in the rat of 4810 mg/kg.MSDS In animal studies, carbidopa showed no incidences on neoplasia and showed no effect on the fertility status and development.Label'],
             'singulair': ['The adverse effects associated with overdosage of montelukast include abdominal pain, somnolence, thirst, headache, vomiting, psychomotor hyperactivity, and less frequently, convulsion.3,4,5,6,7,8,9',
              'The adverse effects associated with overdosage of montelukast include abdominal pain, somnolence, thirst, headache, vomiting, psychomotor hyperactivity, and less frequently, convulsion.3,4,5,6,7,8,9'],
             'avandaryl': ['The oral LD50 value in rats is > 10000 mg/kg.MSDS The intraperitoneal LD50 value in rats is reported to be 3950 mg/kg MSDS. Although glimepiride is reported to have fewer risks of hypoglycemia compared to other sulfonylureas such as glyburide, overdosage of glimepiride may result in severe hypoglycemia with coma, seizure, or other neurological impairment may occur. This can be treated with glucagon or intravenous glucose. Continued observation and additional carbohydrate intake may be necessary since hypoglycemia may recur after apparent clinical recovery.8',
              'The oral LD50 value in rats is > 10000 mg/kg.MSDS The intraperitoneal LD50 value in rats is reported to be 3950 mg/kg MSDS. Although glimepiride is reported to have fewer risks of hypoglycemia compared to other sulfonylureas such as glyburide, overdosage of glimepiride may result in severe hypoglycemia with coma, seizure, or other neurological impairment may occur. This can be treated with glucagon or intravenous glucose. Continued observation and additional carbohydrate intake may be necessary since hypoglycemia may recur after apparent clinical recovery.8'],
             'avandamet': ['Side effects include fluid retention, congestive heart failure (CHF), liver disease',
              'Side effects include fluid retention, congestive heart failure (CHF), liver disease'],
             'coreg cr': ['Patients experiencing an overdose may present with hypotension, bradycardia, cardiac insufficiency, cardiogenic shock, and cardiac arrest.5,6 Patients should remain in a supine position and may be given atropine for bradycardia and glucagon followed by sympathomimetics to support cardiovascular function.5,6',
              'Patients experiencing an overdose may present with hypotension, bradycardia, cardiac insufficiency, cardiogenic shock, and cardiac arrest.5,6 Patients should remain in a supine position and may be given atropine for bradycardia and glucagon followed by sympathomimetics to support cardiovascular function.5,6'],
             'dyazide': ['The oral LD50 of hydrochlorothiazide is >10g/kg in mice and rats.10,11',
              'The oral LD50 of hydrochlorothiazide is >10g/kg in mice and rats.10,11'],
             'dyazide s': ['The oral LD50 of hydrochlorothiazide is >10g/kg in mice and rats.10,11',
              'The oral LD50 of hydrochlorothiazide is >10g/kg in mice and rats.10,11'],
             'coreg': ['Patients experiencing an overdose may present with hypotension, bradycardia, cardiac insufficiency, cardiogenic shock, and cardiac arrest.5,6 Patients should remain in a supine position and may be given atropine for bradycardia and glucagon followed by sympathomimetics to support cardiovascular function.5,6',
              'Patients experiencing an overdose may present with hypotension, bradycardia, cardiac insufficiency, cardiogenic shock, and cardiac arrest.5,6 Patients should remain in a supine position and may be given atropine for bradycardia and glucagon followed by sympathomimetics to support cardiovascular function.5,6'],
             'hycamtin': ['The primary anticipated complication of overdosage would consist of bone marrow suppression.',
              'The primary anticipated complication of overdosage would consist of bone marrow suppression.'],
             'parnate': ['In overdosage, some patients exhibit insomnia, restlessness and anxiety, progressing in severe cases to agitation, mental confusion and incoherence. Hypotension, dizziness, weakness and drowsiness may occur, progressing in severe cases to extreme dizziness and shock. A few patients have displayed hypertension with severe headache and other symptoms. Rare instances have been reported in which hypertension was accompanied by twitching or myoclonic fibrillation of skeletal muscles with hyperpyrexia, sometimes progressing to generalized rigidity and coma.',
              'In overdosage, some patients exhibit insomnia, restlessness and anxiety, progressing in severe cases to agitation, mental confusion and incoherence. Hypotension, dizziness, weakness and drowsiness may occur, progressing in severe cases to extreme dizziness and shock. A few patients have displayed hypertension with severe headache and other symptoms. Rare instances have been reported in which hypertension was accompanied by twitching or myoclonic fibrillation of skeletal muscles with hyperpyrexia, sometimes progressing to generalized rigidity and coma.'],
             'promacta': ['Eltrombopag may cause hepatotoxicity, especially if administered in combination with interferon and ribavirin in patients with chronic hepatitis C (may increase the risk of hepatic decompensation).',
              'Eltrombopag may cause hepatotoxicity, especially if administered in combination with interferon and ribavirin in patients with chronic hepatitis C (may increase the risk of hepatic decompensation).'],
             'requip xl': ['Overdose', 'Overdose'],
             'requip': ['Overdose', 'Overdose'],
             'pantoprazole': ['Rat Oral LD 50 747 mg/kg17 ',
              'Rat Oral LD 50 747 mg/kg17 '],
             'effexor xr': ['Oral LD50 was 350 mg/kg in female rats and 700 mg/kg in male rats.16',
              'Oral LD50 was 350 mg/kg in female rats and 700 mg/kg in male rats.16'],
             'effexor xr brown s': ['Oral LD50 was 350 mg/kg in female rats and 700 mg/kg in male rats.16',
              'Oral LD50 was 350 mg/kg in female rats and 700 mg/kg in male rats.16'],
             'effexor xr greypink': ['Oral LD50 was 350 mg/kg in female rats and 700 mg/kg in male rats.16',
              'Oral LD50 was 350 mg/kg in female rats and 700 mg/kg in male rats.16'],
             'pristiq': ['Desvenlafaxine is excreted in breast milk and as a result, nursing mothers must either stop the drug or stop breast feeding depending on the risks and benefit to mother and childLabel,2.',
              'Desvenlafaxine is excreted in breast milk and as a result, nursing mothers must either stop the drug or stop breast feeding depending on the risks and benefit to mother and childLabel,2.'],
             'cortef': ['Not Available', 'Not Available'],
             'xanax': ['Alprazolam overdose can present as sleepiness, confusion, poor coordination, slow reflexes, coma, and death.18,19 Taking alprazolam with alcohol lowers the threshold for overdose.18,19 Patients should have their respiration, pulse, and blood pressure monitored.18,19 Patients can be treated by gastric lavage and intravenous fluids.18,19. If hypotension occurs, patients may be treated with vasopressors.18,19 In known, or suspected overdoses, patients can be given the benzodiazepine receptor antagonist flumazenil in addition to other methods of management.18,19',
              'Alprazolam overdose can present as sleepiness, confusion, poor coordination, slow reflexes, coma, and death.18,19 Taking alprazolam with alcohol lowers the threshold for overdose.18,19 Patients should have their respiration, pulse, and blood pressure monitored.18,19 Patients can be treated by gastric lavage and intravenous fluids.18,19. If hypotension occurs, patients may be treated with vasopressors.18,19 In known, or suspected overdoses, patients can be given the benzodiazepine receptor antagonist flumazenil in addition to other methods of management.18,19'],
             'cleocin s': ['The oral LD50 in mice and rats is 2540 mg/kg and 2190 mg/kg, respectively.19',
              'The oral LD50 in mice and rats is 2540 mg/kg and 2190 mg/kg, respectively.19'],
             'detrol': ['Not Available', 'Not Available'],
             'glyset': ['Unlike sulfonylureas or insulin, an overdose will not result in hypoglycemia. An overdose may result in transient increases in flatulence, diarrhea, and abdomi-nal discomfort. Because of the lack of extra-intestinal effects seen with miglitol, no serious systemic reactions are expected in the event of an overdose.',
              'Unlike sulfonylureas or insulin, an overdose will not result in hypoglycemia. An overdose may result in transient increases in flatulence, diarrhea, and abdomi-nal discomfort. Because of the lack of extra-intestinal effects seen with miglitol, no serious systemic reactions are expected in the event of an overdose.'],
             'zyvox (linezolid)': ['Clinical signs of overdosage observed in rats were decreased activity and ataxia (2000 mg/kg/day) and in dogs were vomiting and tremors (3000 mg/kg/day).9 Treatment of overdose should involve symptomatic and supportive measures and may include hemodialysis if clinically necessary.',
              'Clinical signs of overdosage observed in rats were decreased activity and ataxia (2000 mg/kg/day) and in dogs were vomiting and tremors (3000 mg/kg/day).9 Treatment of overdose should involve symptomatic and supportive measures and may include hemodialysis if clinically necessary.'],
             'detrol la': ['Not Available', 'Not Available'],
             'aromasin': ['Convulsions', 'Convulsions'],
             'mycobutin': ['LD50 = 4.8 g/kg (mouse, male)',
              'LD50 = 4.8 g/kg (mouse, male)'],
             'sanctura xr': ['Not Available', 'Not Available'],
             'multaq': ['In an acute toxicity study, the oral LD50 in rat was >2,000 mg/kg.10 ',
              'In an acute toxicity study, the oral LD50 in rat was >2,000 mg/kg.10 '],
             'uroxatral': ['l LD50 of alfuzosin is 2300 mg/kg in male mice and 1950 mg/kg in female mice.14\nAn overdose of alfuzosin can cause hypotension. Cardiovascular support should be initiated immediately. The patient should be kept in the supine position to aid in restoring pressure and managing heart rate. Fluid resuscitation should also be considered in severe case',
              'mes, vasopressors are required. Renal function should be monitored frequently. Dialysis may not be of benefit to alfuzosin protein binding of up to 90%.'],
             'ambien': ['Oral (male rat) LD50 = 695 mg/kg MSDS. ',
              'Oral (male rat) LD50 = 695 mg/kg MSDS. '],
             'ambien cr s': ['Oral (male rat) LD50 = 695 mg/kg MSDS. ',
              'Oral (male rat) LD50 = 695 mg/kg MSDS. '],
             'ambien cr': ['Oral (male rat) LD50 = 695 mg/kg MSDS. ',
              'Oral (male rat) LD50 = 695 mg/kg MSDS. '],
             'xyzal': ['Patients experiencing an overdose may present with drowsiness.4 Children may become agitated and restless before drowsiness.4 Patients should be treated with supportive measures.4 Dialysis will not assist in removing the drug from the body.4',
              'Patients experiencing an overdose may present with drowsiness.4 Children may become agitated and restless before drowsiness.4 Patients should be treated with supportive measures.4 Dialysis will not assist in removing the drug from the body.4'],
             'aldactazide': ['The oral LD50 of hydrochlorothiazide is >10g/kg in mice and rats.10,11',
              'The oral LD50 of hydrochlorothiazide is >10g/kg in mice and rats.10,11'],
             'daypro': ['Oral, mouse: LD50 = 1210 mg/kg; Oral, rabbit: LD50 = 172 mg/kg; Oral, rat: LD50 = 4470 mg/kg',
              'Oral, mouse: LD50 = 1210 mg/kg; Oral, rabbit: LD50 = 172 mg/kg; Oral, rat: LD50 = 4470 mg/kg'],
             'arthrotec': ['The oral LD50 in rats is 81mg/kg and in mice is 27mg/kg.12 The intraperitoneal LD50 in rats is 40mg/kg and in mice is 70mg/kg.12',
              'The oral LD50 in rats is 81mg/kg and in mice is 27mg/kg.12 The intraperitoneal LD50 in rats is 40mg/kg and in mice is 70mg/kg.12'],
             'celebrex celecoxib': ['The oral TDLo in humans 5.71 mg/kg.29',
              'The oral TDLo in humans 5.71 mg/kg.29'],
             'calan sr': ["Verapamil's reported oral TDLo is 14.4 mg/kg in women and 3.429 mg/kg in men.21 The oral LD50 is 150 mg/kg in rats and 163 mg/kg in mice.21",
              "Verapamil's reported oral TDLo is 14.4 mg/kg in women and 3.429 mg/kg in men.21 The oral LD50 is 150 mg/kg in rats and 163 mg/kg in mice.21"],
             'avandia': ['Side effects include fluid retention, congestive heart failure (CHF), liver disease',
              'Side effects include fluid retention, congestive heart failure (CHF), liver disease'],
             'creon (pancrelipase)': ['The studies of the toxicology of pancrelipase are not needed as this drug has been used clinically for a long time.8 Clinical overdose studies proved no effect on lungs, pancreas, liver and kidneys but it can produce symptoms such as diarrhea or stomach upset. Carcinogenicity studies have not shown any increased incidence with the use of pancrelipase. As pancrelipase is not absorbed, the effect on fetal development or reproduction is not expected.9',
              'The studies of the toxicology of pancrelipase are not needed as this drug has been used clinically for a long time.8 Clinical overdose studies proved no effect on lungs, pancreas, liver and kidneys but it can produce symptoms such as diarrhea or stomach upset. Carcinogenicity studies have not shown any increased incidence with the use of pancrelipase. As pancrelipase is not absorbed, the effect on fetal development or reproduction is not expected.9'],
             'creon': ['The studies of the toxicology of pancrelipase are not needed as this drug has been used clinically for a long time.8 Clinical overdose studies proved no effect on lungs, pancreas, liver and kidneys but it can produce symptoms such as diarrhea or stomach upset. Carcinogenicity studies have not shown any increased incidence with the use of pancrelipase. As pancrelipase is not absorbed, the effect on fetal development or reproduction is not expected.9',
              'The studies of the toxicology of pancrelipase are not needed as this drug has been used clinically for a long time.8 Clinical overdose studies proved no effect on lungs, pancreas, liver and kidneys but it can produce symptoms such as diarrhea or stomach upset. Carcinogenicity studies have not shown any increased incidence with the use of pancrelipase. As pancrelipase is not absorbed, the effect on fetal development or reproduction is not expected.9'],
             'creon unt': ['The studies of the toxicology of pancrelipase are not needed as this drug has been used clinically for a long time.8 Clinical overdose studies proved no effect on lungs, pancreas, liver and kidneys but it can produce symptoms such as diarrhea or stomach upset. Carcinogenicity studies have not shown any increased incidence with the use of pancrelipase. As pancrelipase is not absorbed, the effect on fetal development or reproduction is not expected.9',
              'The studies of the toxicology of pancrelipase are not needed as this drug has been used clinically for a long time.8 Clinical overdose studies proved no effect on lungs, pancreas, liver and kidneys but it can produce symptoms such as diarrhea or stomach upset. Carcinogenicity studies have not shown any increased incidence with the use of pancrelipase. As pancrelipase is not absorbed, the effect on fetal development or reproduction is not expected.9'],
             'prometrium': ['Intraperitoneal LD50 (rat): 327 mg/kg MSDS. ',
              'Intraperitoneal LD50 (rat): 327 mg/kg MSDS. '],
             'soma': ['Subcutaneous doses of somatrogon higher than 0.66 mg/kg per week have not been studied. Based on existing clinical data of other growth hormone products, short-term overdosage from somatrogon could lead initially to hypoglycemia and subsequently to hyperglycemia. Long-term overdosage could result in signs and symptoms of gigantism or acromegaly consistent with the effects of excess growth hormone. Overdose with somatrogon should be treated with general supportive measures.6',
              'Subcutaneous doses of somatrogon higher than 0.66 mg/kg per week have not been studied. Based on existing clinical data of other growth hormone products, short-term overdosage from somatrogon could lead initially to hypoglycemia and subsequently to hyperglycemia. Long-term overdosage could result in signs and symptoms of gigantism or acromegaly consistent with the effects of excess growth hormone. Overdose with somatrogon should be treated with general supportive measures.6'],
             'lasix': ['Clinical consequences from overdose depend on the extent of electrolyte and fluid loss and include dehydration, blood volume reduction, hypotension, electrolyte imbalance, hypokalemia, hypochloremic alkalosis,9 hemoconcentration, cardiac arrhythmias (including A-V block and ventricular fibrillation).12 Symptoms of overdose include acute renal failure, thrombosis, delirious states, flaccid paralysis, apathy and confusion. In cirrhotic patients, overdosage might precipitate hepatic coma.12',
              'Clinical consequences from overdose depend on the extent of electrolyte and fluid loss and include dehydration, blood volume reduction, hypotension, electrolyte imbalance, hypokalemia, hypochloremic alkalosis,9 hemoconcentration, cardiac arrhythmias (including A-V block and ventricular fibrillation).12 Symptoms of overdose include acute renal failure, thrombosis, delirious states, flaccid paralysis, apathy and confusion. In cirrhotic patients, overdosage might precipitate hepatic coma.12'],
             'amaryl': ['The oral LD50 value in rats is > 10000 mg/kg.MSDS The intraperitoneal LD50 value in rats is reported to be 3950 mg/kg MSDS. Although glimepiride is reported to have fewer risks of hypoglycemia compared to other sulfonylureas such as glyburide, overdosage of glimepiride may result in severe hypoglycemia with coma, seizure, or other neurological impairment may occur. This can be treated with glucagon or intravenous glucose. Continued observation and additional carbohydrate intake may be necessary since hypoglycemia may recur after apparent clinical recovery.8',
              'The oral LD50 value in rats is > 10000 mg/kg.MSDS The intraperitoneal LD50 value in rats is reported to be 3950 mg/kg MSDS. Although glimepiride is reported to have fewer risks of hypoglycemia compared to other sulfonylureas such as glyburide, overdosage of glimepiride may result in severe hypoglycemia with coma, seizure, or other neurological impairment may occur. This can be treated with glucagon or intravenous glucose. Continued observation and additional carbohydrate intake may be necessary since hypoglycemia may recur after apparent clinical recovery.8'],
             'ultracet': ['The reported LD50 for tramadol, when administered orally in mice, is 350 mg/kg.11 ',
              'The reported LD50 for tramadol, when administered orally in mice, is 350 mg/kg.11 '],
             'premarin': ['The reported oral LD50 in the rat is of more than 5000 mg/kg.13 Serious overdosage symptoms have not been reported. There have been only reports of nausea, vomiting, and withdrawal in bleeding in females.Label',
              'The reported oral LD50 in the rat is of more than 5000 mg/kg.13 Serious overdosage symptoms have not been reported. There have been only reports of nausea, vomiting, and withdrawal in bleeding in females.Label'],
             'relpax': ['Based on the pharmacology of the 5-HT1B/1D agonists, hypertension or other more serious cardiovascular symptoms could occur on overdose.',
              'Based on the pharmacology of the 5-HT1B/1D agonists, hypertension or other more serious cardiovascular symptoms could occur on overdose.'],
             'geodon sule': ['The most common adverse reactions reported with ziprasidone include somnolence, respiratory tract infections, extrapyramidal symptoms, dizziness, akathisia, abnormal vision, asthenia, vomiting, headache and nausea. 13',
              'The most common adverse reactions reported with ziprasidone include somnolence, respiratory tract infections, extrapyramidal symptoms, dizziness, akathisia, abnormal vision, asthenia, vomiting, headache and nausea. 13'],
             'geodon blue': ['The most common adverse reactions reported with ziprasidone include somnolence, respiratory tract infections, extrapyramidal symptoms, dizziness, akathisia, abnormal vision, asthenia, vomiting, headache and nausea. 13',
              'The most common adverse reactions reported with ziprasidone include somnolence, respiratory tract infections, extrapyramidal symptoms, dizziness, akathisia, abnormal vision, asthenia, vomiting, headache and nausea. 13'],
             'geodon whhite': ['The most common adverse reactions reported with ziprasidone include somnolence, respiratory tract infections, extrapyramidal symptoms, dizziness, akathisia, abnormal vision, asthenia, vomiting, headache and nausea. 13',
              'The most common adverse reactions reported with ziprasidone include somnolence, respiratory tract infections, extrapyramidal symptoms, dizziness, akathisia, abnormal vision, asthenia, vomiting, headache and nausea. 13'],
             'geodon ziprasidone hcl': ['The most common adverse reactions reported with ziprasidone include somnolence, respiratory tract infections, extrapyramidal symptoms, dizziness, akathisia, abnormal vision, asthenia, vomiting, headache and nausea. 13',
              'The most common adverse reactions reported with ziprasidone include somnolence, respiratory tract infections, extrapyramidal symptoms, dizziness, akathisia, abnormal vision, asthenia, vomiting, headache and nausea. 13'],
             'zoloft': ['The LD50 of sertraline is >2000 mg/kg in rats according to the FDA label.25 One other references indicates an oral LD50 of in mice and rats of 419 - 548 mg/kg and 1327 - 1591mg/kg, respectively.MSDS',
              'The LD50 of sertraline is >2000 mg/kg in rats according to the FDA label.25 One other references indicates an oral LD50 of in mice and rats of 419 - 548 mg/kg and 1327 - 1591mg/kg, respectively.MSDS'],
             'calcitriol': ['', ''],
             'flecainide': ['The oral LD50 in rats is 1346mg/kg and in mice is 170mg/kg.14 The subcutaneous LD50 in rats is 215mg/kg and in mice is 188mg/kg.14 The oral TDLO in women is 20mg/kg and in men is 40mg/kg/2W.14',
              'The oral LD50 in rats is 1346mg/kg and in mice is 170mg/kg.14 The subcutaneous LD50 in rats is 215mg/kg and in mice is 188mg/kg.14 The oral TDLO in women is 20mg/kg and in men is 40mg/kg/2W.14'],
             'prednisone': ['Data regarding acute overdoses of prednisone are rare but prolonged high doses of prednisone can lead to a higher incidence and severity of adverse effects such as mental symptoms, moon face, abnormal fat deposits, fluid retention, excessive appetite, weight gain, hypertrichosis, acne, striae, ecchymosis, increased sweating, pigmentation, dry scaly skin, thinning scalp hair, increased blood pressure, tachycardia, thrombophlebitis, decreased resistance to infection, negative nitrogen balance with delayed bone and wound healing, headache, weakness, menstrual disorders, accentuated menopausal symptoms, neuropathy, fractures, osteoporosis, peptic ulcer, decreased glucose tolerance, hypokalemia, and adrenal insufficiency.9',
              'Data regarding acute overdoses of prednisone are rare but prolonged high doses of prednisone can lead to a higher incidence and severity of adverse effects such as mental symptoms, moon face, abnormal fat deposits, fluid retention, excessive appetite, weight gain, hypertrichosis, acne, striae, ecchymosis, increased sweating, pigmentation, dry scaly skin, thinning scalp hair, increased blood pressure, tachycardia, thrombophlebitis, decreased resistance to infection, negative nitrogen balance with delayed bone and wound healing, headache, weakness, menstrual disorders, accentuated menopausal symptoms, neuropathy, fractures, osteoporosis, peptic ulcer, decreased glucose tolerance, hypokalemia, and adrenal insufficiency.9'],
             'lithium carb er': ['Not Available', 'Not Available'],
             'lithium carbonate er': ['In rats, the oral LD50 is 525mg/kg and the inhalation LC50 is >2.17mg/L over 4 hoursMSDS.',
              'In rats, the oral LD50 is 525mg/kg and the inhalation LC50 is >2.17mg/L over 4 hoursMSDS.'],
             'cilostazol': ['Information on acute overdosage with cilostazol in humans is limited. The signs and symptoms of an acute overdose can be anticipated to be those of excessive pharmacologic effect: severe headache, diarrhea, hypotension, tachycardia, and possibly cardiac arrhythmias. The oral LD50 of cilostazol is >5.0 g/kg in mice and rats and >2.0 g/kg in dogs.',
              'Information on acute overdosage with cilostazol in humans is limited. The signs and symptoms of an acute overdose can be anticipated to be those of excessive pharmacologic effect: severe headache, diarrhea, hypotension, tachycardia, and possibly cardiac arrhythmias. The oral LD50 of cilostazol is >5.0 g/kg in mice and rats and >2.0 g/kg in dogs.'],
             'clarithromycin': ['Symptoms of toxicity include diarrhea, nausea, abnormal taste, dyspepsia, and abdominal discomfort. Transient hearing loss with high doses has been observed. Pseudomembraneous colitis has been reported with clarithromycin use. Allergic reactions ranging from urticaria and mild skin eruptions to rare cases of anaphylaxis and Stevens-Johnson syndrome have also occurred. Rare cases of severe hepatic dysfunctions also have been reported. Hepatic failure is usually reversible, but fatalities have been reported. Clarithromycin may also cause tooth decolouration which may be removed by dental cleaning. Fetal abnormalities, such as cardiovascular defects, cleft palate and fetal growth retardation, have been observed in animals. Clarithromycin may cause QT prolongation.',
              'Symptoms of toxicity include diarrhea, nausea, abnormal taste, dyspepsia, and abdominal discomfort. Transient hearing loss with high doses has been observed. Pseudomembraneous colitis has been reported with clarithromycin use. Allergic reactions ranging from urticaria and mild skin eruptions to rare cases of anaphylaxis and Stevens-Johnson syndrome have also occurred. Rare cases of severe hepatic dysfunctions also have been reported. Hepatic failure is usually reversible, but fatalities have been reported. Clarithromycin may also cause tooth decolouration which may be removed by dental cleaning. Fetal abnormalities, such as cardiovascular defects, cleft palate and fetal growth retardation, have been observed in animals. Clarithromycin may cause QT prolongation.'],
             'clarithromyc': ['Symptoms of toxicity include diarrhea, nausea, abnormal taste, dyspepsia, and abdominal discomfort. Transient hearing loss with high doses has been observed. Pseudomembraneous colitis has been reported with clarithromycin use. Allergic reactions ranging from urticaria and mild skin eruptions to rare cases of anaphylaxis and Stevens-Johnson syndrome have also occurred. Rare cases of severe hepatic dysfunctions also have been reported. Hepatic failure is usually reversible, but fatalities have been reported. Clarithromycin may also cause tooth decolouration which may be removed by dental cleaning. Fetal abnormalities, such as cardiovascular defects, cleft palate and fetal growth retardation, have been observed in animals. Clarithromycin may cause QT prolongation.',
              'Symptoms of toxicity include diarrhea, nausea, abnormal taste, dyspepsia, and abdominal discomfort. Transient hearing loss with high doses has been observed. Pseudomembraneous colitis has been reported with clarithromycin use. Allergic reactions ranging from urticaria and mild skin eruptions to rare cases of anaphylaxis and Stevens-Johnson syndrome have also occurred. Rare cases of severe hepatic dysfunctions also have been reported. Hepatic failure is usually reversible, but fatalities have been reported. Clarithromycin may also cause tooth decolouration which may be removed by dental cleaning. Fetal abnormalities, such as cardiovascular defects, cleft palate and fetal growth retardation, have been observed in animals. Clarithromycin may cause QT prolongation.'],
             'torsemide': ['The oral LD50 of torasemide in the rat is 5 g/kg. When overdose occurs, there is a marked diuresis with the danger of loss of fluid and electrolytes which has been seen to lead to somnolence, confusion, hypotension, hyponatremia, hypokalemia, hypochloremic alkalosis, hemoconcentration dehydration and circulatory collapse. This effects can include some gastrointestinal disturbances.13',
              'The oral LD50 of torasemide in the rat is 5 g/kg. When overdose occurs, there is a marked diuresis with the danger of loss of fluid and electrolytes which has been seen to lead to somnolence, confusion, hypotension, hyponatremia, hypokalemia, hypochloremic alkalosis, hemoconcentration dehydration and circulatory collapse. This effects can include some gastrointestinal disturbances.13'],
             'balsalazide disodium': ['A single oral dose of balsalazide disodium at 5 grams/kg or 4-aminobenzoyl-(beta)-alanine, a metabolite of balsalazide disodium, at 1 gram/kg was non-lethal in mice and rats. No symptoms of acute toxicity were seen at these doses.',
              'A single oral dose of balsalazide disodium at 5 grams/kg or 4-aminobenzoyl-(beta)-alanine, a metabolite of balsalazide disodium, at 1 gram/kg was non-lethal in mice and rats. No symptoms of acute toxicity were seen at these doses.'],
             'calcium acetate': ['Oral, rat: LD50 = 4280 mg/kg. Symptoms of overdose include mild hypercalcemia (constipation; loss of appetite; nausea and vomiting), and severe hypercalcemia (confusion; full or partial loss of consciousness; incoherent speech).',
              'Oral, rat: LD50 = 4280 mg/kg. Symptoms of overdose include mild hypercalcemia (constipation; loss of appetite; nausea and vomiting), and severe hypercalcemia (confusion; full or partial loss of consciousness; incoherent speech).'],
             'oxcarbazepine': ['The oral LD50 of oxcarbazepine in mammals is 1240 mg/kg and the oral TDLo in children has been reported to be 73 mg/kg.13 Isolated cases of oxcarbazepine overdose have been reported - patients who ingested up to 24,000mg recovered with symptomatic treatment.10,11 Symptoms may include respiratory and CNS depression, movement-related disorders (e.g. dyskinesia, ataxia), nausea/vomiting, hyponatremia, or QTc prolongation. There is no antidote for oxcarbazepine overdose - management should consist of supportive and symptomatic treatment, and consideration should be given to the use of gastric lavage or activated charcoal.10,11',
              'The oral LD50 of oxcarbazepine in mammals is 1240 mg/kg and the oral TDLo in children has been reported to be 73 mg/kg.13 Isolated cases of oxcarbazepine overdose have been reported - patients who ingested up to 24,000mg recovered with symptomatic treatment.10,11 Symptoms may include respiratory and CNS depression, movement-related disorders (e.g. dyskinesia, ataxia), nausea/vomiting, hyponatremia, or QTc prolongation. There is no antidote for oxcarbazepine overdose - management should consist of supportive and symptomatic treatment, and consideration should be given to the use of gastric lavage or activated charcoal.10,11'],
             'ramipril': ['Symptoms of overdose may include excessive peripheral vasodilation (with marked hypotension and shock), bradycardia, electrolyte disturbances, and renal failure. Cases of ACE inhibitor induced hepatotoxicity have been reported in humans and presented as acute jaundice and elevated liver enzymes.3 Removal of the ACE inhbitor resulted in a decline in liver enzymes and re-challenge produced a subsequent increase. ',
              'Symptoms of overdose may include excessive peripheral vasodilation (with marked hypotension and shock), bradycardia, electrolyte disturbances, and renal failure. Cases of ACE inhibitor induced hepatotoxicity have been reported in humans and presented as acute jaundice and elevated liver enzymes.3 Removal of the ACE inhbitor resulted in a decline in liver enzymes and re-challenge produced a subsequent increase. '],
             'perindopril erbumine': ['The most likely symptom of overdose is severe hypotension. The most common adverse effects observed in controlled clinical trials include cough, digestive symptoms, fatigue, headache, and dizziness. ',
              'The most likely symptom of overdose is severe hypotension. The most common adverse effects observed in controlled clinical trials include cough, digestive symptoms, fatigue, headache, and dizziness. '],
             'acarbose': ['The symptoms of acarbose overdose are likely to be consistent with its adverse effect profile and may therefore include significant gastrointestinal (GI) symptoms (flatulence, distension, etc), although an overdose on an empty stomach (i.e. when not co-administered with food) is less likely to result in these GI symptoms.6 In the event of an overdose, patients should be instructed to avoid carbohydrate-containing foods for 4-6 hours following administration as these can precipitate the aforementioned GI symptoms.6',
              'The symptoms of acarbose overdose are likely to be consistent with its adverse effect profile and may therefore include significant gastrointestinal (GI) symptoms (flatulence, distension, etc), although an overdose on an empty stomach (i.e. when not co-administered with food) is less likely to result in these GI symptoms.6 In the event of an overdose, patients should be instructed to avoid carbohydrate-containing foods for 4-6 hours following administration as these can precipitate the aforementioned GI symptoms.6'],
             'mycophenolate mofetil': ['LD50', 'LD50'],
             'mycophenolat': ['LD50', 'LD50'],
             'buprenorphine sub': ['Manifestations of acute overdose include pinpoint pupils, sedation, hypotension, respiratory depression and death.',
              'Manifestations of acute overdose include pinpoint pupils, sedation, hypotension, respiratory depression and death.'],
             'famciclovir': ['Symptoms of overdose include constipation, diarrhea, dizziness, fatigue, fever, headache, nausea, and vomiting.',
              'Symptoms of overdose include constipation, diarrhea, dizziness, fatigue, fever, headache, nausea, and vomiting.'],
             'quetiapine fumarate': ['The oral LD50 if quetiapine in rats is 2000 mg/kg.20',
              'The oral LD50 if quetiapine in rats is 2000 mg/kg.20'],
             'irbesartan and hydrochlorothiazide': ['The oral TDLO in humans is 30mg/kg/6W.9',
              'The oral TDLO in humans is 30mg/kg/6W.9'],
             'letrozole': ['Overdose data in humans is not readily available, however 1 reported case was not associated with serious adverse reactions.9 Animal studies do not report serious adverse effects with high dose treatment.6 Patients experiencing and overdose should be treated with symptomatic and supportive measures.9',
              'Overdose data in humans is not readily available, however 1 reported case was not associated with serious adverse reactions.9 Animal studies do not report serious adverse effects with high dose treatment.6 Patients experiencing and overdose should be treated with symptomatic and supportive measures.9'],
             'naratriptan': ['Symptoms of overdose include light-headedness, loss of coordination, tension in the neck, and tiredness.',
              'Symptoms of overdose include light-headedness, loss of coordination, tension in the neck, and tiredness.'],
             'montelukast sodium chewable': ['The adverse effects associated with overdosage of montelukast include abdominal pain, somnolence, thirst, headache, vomiting, psychomotor hyperactivity, and less frequently, convulsion.3,4,5,6,7,8,9',
              'The adverse effects associated with overdosage of montelukast include abdominal pain, somnolence, thirst, headache, vomiting, psychomotor hyperactivity, and less frequently, convulsion.3,4,5,6,7,8,9'],
             'cevimeline hydrochloride': ['Not Available', 'Not Available'],
             'tinidazole': ['There are no reported overdoses with tinidazole in humans. In acute studies with mice and rats, the LD 50 for mice was generally > 3,600 mg/kg for oral administration and was > 2,300 mg/kg for intraperitoneal administration. In rats, the LD 50 was > 2,000 mg/kg for both oral and intraperitoneal administration.',
              'There are no reported overdoses with tinidazole in humans. In acute studies with mice and rats, the LD 50 for mice was generally > 3,600 mg/kg for oral administration and was > 2,300 mg/kg for intraperitoneal administration. In rats, the LD 50 was > 2,000 mg/kg for both oral and intraperitoneal administration.'],
             'telmisartan and hydrochlorothiazide': ['nous LD50 in rats is 150-200 mg/kg in males and 200 to 250 mg/kg in females. Acute oral toxicity is low: no deaths and no changes occurred in rats or dogs at 2000 mg/kg, the highest dose tested. Limited data are available with regard to overdosage in humans. The most likely manifestations of overdosage with telmisartan would be hypotension, dizziness and tachycardi',
              'ardia could occur from parasympathetic'],
             'lithium carbonate s': ['Not Available', 'Not Available'],
             'lithium carb s': ['Not Available', 'Not Available'],
             'lithium carbonate': ['In rats, the oral LD50 is 525mg/kg and the inhalation LC50 is >2.17mg/L over 4 hoursMSDS.',
              'In rats, the oral LD50 is 525mg/kg and the inhalation LC50 is >2.17mg/L over 4 hoursMSDS.'],
             'azathioprine': ['The oral LD50 in mice is 2500mg/kg and in rats is 400mg/kg.12',
              'The oral LD50 in mice is 2500mg/kg and in rats is 400mg/kg.12'],
             'cyclophosphamide': ['Adverse reactions reported most often include neutropenia, febrile neutropenia, fever, alopecia, nausea, vomiting, and diarrhea.',
              'Adverse reactions reported most often include neutropenia, febrile neutropenia, fever, alopecia, nausea, vomiting, and diarrhea.'],
             'dexamethasone s': ['The oral LD50 in female mice was 6.5g/kg and 794mg/kg via the intravenous route.13',
              'The oral LD50 in female mice was 6.5g/kg and 794mg/kg via the intravenous route.13'],
             'dexamethasone': ['The oral LD50 in female mice was 6.5g/kg and 794mg/kg via the intravenous route.13',
              'The oral LD50 in female mice was 6.5g/kg and 794mg/kg via the intravenous route.13'],
             'furosemide': ['Clinical consequences from overdose depend on the extent of electrolyte and fluid loss and include dehydration, blood volume reduction, hypotension, electrolyte imbalance, hypokalemia, hypochloremic alkalosis,9 hemoconcentration, cardiac arrhythmias (including A-V block and ventricular fibrillation).12 Symptoms of overdose include acute renal failure, thrombosis, delirious states, flaccid paralysis, apathy and confusion. In cirrhotic patients, overdosage might precipitate hepatic coma.12',
              'Clinical consequences from overdose depend on the extent of electrolyte and fluid loss and include dehydration, blood volume reduction, hypotension, electrolyte imbalance, hypokalemia, hypochloremic alkalosis,9 hemoconcentration, cardiac arrhythmias (including A-V block and ventricular fibrillation).12 Symptoms of overdose include acute renal failure, thrombosis, delirious states, flaccid paralysis, apathy and confusion. In cirrhotic patients, overdosage might precipitate hepatic coma.12'],
             'leucovorin calcium': ['LD50>8000 mg/kg (orally in rats). Excessive amounts of leucovorin may nullify the chemotherapeutic effect of folic acid antagonists.',
              'LD50>8000 mg/kg (orally in rats). Excessive amounts of leucovorin may nullify the chemotherapeutic effect of folic acid antagonists.'],
             'lithium carb': ['Not Available', 'Not Available'],
             'methotrexate s': ['The oral LD50 in rats is 135mg/kg and in mice is 146mg/kg.9',
              'The oral LD50 in rats is 135mg/kg and in mice is 146mg/kg.9'],
             'mertopurine': ['Not Available', 'Not Available'],
             'coumadin blue s': ['LD50 Values', 'LD50 Values'],
             'coumadin pink s': ['LD50 Values', 'LD50 Values'],
             'coumadin lavender s': ['LD50 Values', 'LD50 Values'],
             'coumadin peach s': ['LD50 Values', 'LD50 Values'],
             'coumadin yellow s': ['LD50 Values', 'LD50 Values'],
             'coumadin green s': ['LD50 Values', 'LD50 Values'],
             'coumadin tan s': ['LD50 Values', 'LD50 Values'],
             'sustiva': ['Not Available', 'Not Available'],
             'lodosyn': ['The LD50 of carbidopa is reported to be in the rat of 4810 mg/kg.MSDS In animal studies, carbidopa showed no incidences on neoplasia and showed no effect on the fertility status and development.Label',
              'The LD50 of carbidopa is reported to be in the rat of 4810 mg/kg.MSDS In animal studies, carbidopa showed no incidences on neoplasia and showed no effect on the fertility status and development.Label'],
             'grisfulvin v s': ['Side effects are minor: headaches, gastrointestinal reactions and cutaneous eruptions.',
              'Side effects are minor: headaches, gastrointestinal reactions and cutaneous eruptions.'],
             'axert &apos;s': ['Not Available', 'Not Available'],
             'chantix': ['Not Available', 'Not Available'],
             'norvasc': ['Acute oral toxicity (LD50): 37 mg/kg (mouse) MSDS. ',
              'Acute oral toxicity (LD50): 37 mg/kg (mouse) MSDS. '],
             'caduet': ['The reported LD50 of oral atorvastatin in mice is higher than 5000 mg/kg.MSDS In cases of overdose with atorvastatin, there is reported symptoms of complicated breathing, jaundice, liver damage, dark urine, muscle pain, and seizures.38 In case of overdose, symptomatic treatment is recommended and due to the high plasma protein binding, hemodialysis is not expected to generate significant improvement.Label',
              'The reported LD50 of oral atorvastatin in mice is higher than 5000 mg/kg.MSDS In cases of overdose with atorvastatin, there is reported symptoms of complicated breathing, jaundice, liver damage, dark urine, muscle pain, and seizures.38 In case of overdose, symptomatic treatment is recommended and due to the high plasma protein binding, hemodialysis is not expected to generate significant improvement.Label'],
             'procardia': ['The oral LD50 in rats is 1022mg/kg and in mice is 202mg/kg.16',
              'The oral LD50 in rats is 1022mg/kg and in mice is 202mg/kg.16'],
             'viagra': ['In single-dose volunteer studies of doses up to 800 mg, adverse reactions were similar to those seen at lower doses, but the incidence rates and severities were increased 11,12,13,14,15,16,8,9. Doses of 200 mg did not result in increased efficacy but the incidence of adverse reaction (headache, flushing, dizziness, dyspepsia, nasal congestion, altered vision) was increased 11,12,13,14,15,16,8,9.',
              'In single-dose volunteer studies of doses up to 800 mg, adverse reactions were similar to those seen at lower doses, but the incidence rates and severities were increased 11,12,13,14,15,16,8,9. Doses of 200 mg did not result in increased efficacy but the incidence of adverse reaction (headache, flushing, dizziness, dyspepsia, nasal congestion, altered vision) was increased 11,12,13,14,15,16,8,9.'],
             'tikosyn': ['Not Available', 'Not Available'],
             'lipitor': ['The reported LD50 of oral atorvastatin in mice is higher than 5000 mg/kg.MSDS In cases of overdose with atorvastatin, there is reported symptoms of complicated breathing, jaundice, liver damage, dark urine, muscle pain, and seizures.38 In case of overdose, symptomatic treatment is recommended and due to the high plasma protein binding, hemodialysis is not expected to generate significant improvement.Label',
              'The reported LD50 of oral atorvastatin in mice is higher than 5000 mg/kg.MSDS In cases of overdose with atorvastatin, there is reported symptoms of complicated breathing, jaundice, liver damage, dark urine, muscle pain, and seizures.38 In case of overdose, symptomatic treatment is recommended and due to the high plasma protein binding, hemodialysis is not expected to generate significant improvement.Label'],
             'lipitor let': ['The reported LD50 of oral atorvastatin in mice is higher than 5000 mg/kg.MSDS In cases of overdose with atorvastatin, there is reported symptoms of complicated breathing, jaundice, liver damage, dark urine, muscle pain, and seizures.38 In case of overdose, symptomatic treatment is recommended and due to the high plasma protein binding, hemodialysis is not expected to generate significant improvement.Label',
              'The reported LD50 of oral atorvastatin in mice is higher than 5000 mg/kg.MSDS In cases of overdose with atorvastatin, there is reported symptoms of complicated breathing, jaundice, liver damage, dark urine, muscle pain, and seizures.38 In case of overdose, symptomatic treatment is recommended and due to the high plasma protein binding, hemodialysis is not expected to generate significant improvement.Label'],
             'dilantin s': ['The experience of phenytoin toxicity is not limited to situations of acute ingestion, but may also occur due to drug interactions or due to physiological circumstances that impact serum albumin (ie. kidney disease) or drug metabolism.3 Other changes that may result in phenytoin toxicity include pregnancy, malnutrition and malignancy.3',
              'The experience of phenytoin toxicity is not limited to situations of acute ingestion, but may also occur due to drug interactions or due to physiological circumstances that impact serum albumin (ie. kidney disease) or drug metabolism.3 Other changes that may result in phenytoin toxicity include pregnancy, malnutrition and malignancy.3'],
             'nitrostat': ['The oral LD50 of nitroglycerin in rats is 105 mg/kg and the LD50 of the intravenous form in rats is 23.2 mg/kg.13',
              'The oral LD50 of nitroglycerin in rats is 105 mg/kg and the LD50 of the intravenous form in rats is 23.2 mg/kg.13'],
             'accupril': ['The oral LD50 in rats is 3541mg/kg and in mice is 1739mg/kg.8',
              'The oral LD50 in rats is 3541mg/kg and in mice is 1739mg/kg.8'],
             'lyrica': ['In a systematic review that included 38 randomized controlled trials, there were 20 identified adverse effects that were significantly associated with pregabalin, most of which involve the central nervous system and cognition. The identified adverse effects include vertigo, dizziness, balance disorder, incoordination, ataxia, blurred vision, diplopia, amblyopia, somnolence, confusional state, tremor, disturbance in attention, abnormal thinking, asthenia, fatigue, euphoria, edema, peripheral edema, dry mouth, and constipation 14.',
              'In a systematic review that included 38 randomized controlled trials, there were 20 identified adverse effects that were significantly associated with pregabalin, most of which involve the central nervous system and cognition. The identified adverse effects include vertigo, dizziness, balance disorder, incoordination, ataxia, blurred vision, diplopia, amblyopia, somnolence, confusional state, tremor, disturbance in attention, abnormal thinking, asthenia, fatigue, euphoria, edema, peripheral edema, dry mouth, and constipation 14.'],
             'dilantin': ['The experience of phenytoin toxicity is not limited to situations of acute ingestion, but may also occur due to drug interactions or due to physiological circumstances that impact serum albumin (ie. kidney disease) or drug metabolism.3 Other changes that may result in phenytoin toxicity include pregnancy, malnutrition and malignancy.3',
              'The experience of phenytoin toxicity is not limited to situations of acute ingestion, but may also occur due to drug interactions or due to physiological circumstances that impact serum albumin (ie. kidney disease) or drug metabolism.3 Other changes that may result in phenytoin toxicity include pregnancy, malnutrition and malignancy.3'],
             'vicodin es': ['Overdosage with hydrocodone presents as opioid intoxication including respiratory depression, somnolence, coma, skeletal muscle flaccidity, cold and clammy skin, constricted pupils, pulmonary edema, bradycardia, hypotension, partial or complete airway obstruction, atypical snoring, and death.15',
              'Overdosage with hydrocodone presents as opioid intoxication including respiratory depression, somnolence, coma, skeletal muscle flaccidity, cold and clammy skin, constricted pupils, pulmonary edema, bradycardia, hypotension, partial or complete airway obstruction, atypical snoring, and death.15'],
             'biaxin': ['Symptoms of toxicity include diarrhea, nausea, abnormal taste, dyspepsia, and abdominal discomfort. Transient hearing loss with high doses has been observed. Pseudomembraneous colitis has been reported with clarithromycin use. Allergic reactions ranging from urticaria and mild skin eruptions to rare cases of anaphylaxis and Stevens-Johnson syndrome have also occurred. Rare cases of severe hepatic dysfunctions also have been reported. Hepatic failure is usually reversible, but fatalities have been reported. Clarithromycin may also cause tooth decolouration which may be removed by dental cleaning. Fetal abnormalities, such as cardiovascular defects, cleft palate and fetal growth retardation, have been observed in animals. Clarithromycin may cause QT prolongation.',
              'Symptoms of toxicity include diarrhea, nausea, abnormal taste, dyspepsia, and abdominal discomfort. Transient hearing loss with high doses has been observed. Pseudomembraneous colitis has been reported with clarithromycin use. Allergic reactions ranging from urticaria and mild skin eruptions to rare cases of anaphylaxis and Stevens-Johnson syndrome have also occurred. Rare cases of severe hepatic dysfunctions also have been reported. Hepatic failure is usually reversible, but fatalities have been reported. Clarithromycin may also cause tooth decolouration which may be removed by dental cleaning. Fetal abnormalities, such as cardiovascular defects, cleft palate and fetal growth retardation, have been observed in animals. Clarithromycin may cause QT prolongation.'],
             'erythrom eth': ['LD50', 'LD50'],
             'advicor': ['ian lethal dose of lovastatin is higher than 15 g/m2. Five healthy human volunteers have received up to 200 mg of lovastatin as a single dose without clinically significant adverse experiences. A few cases of accidental overdosage have been reporte',
              'ients had any specific symptoms, and all patients recovered without sequelae. The maximum dose taken was 5 to 6 g.4'],
             'teveten hct': ['There was no mortality in rats and mice receiving oral doses of up to 3000 mg eprosartan/kg and in dogs receiving oral doses of up to 1000 mg eprosartan/kg.',
              'There was no mortality in rats and mice receiving oral doses of up to 3000 mg eprosartan/kg and in dogs receiving oral doses of up to 1000 mg eprosartan/kg.'],
             'teveten': ['There was no mortality in rats and mice receiving oral doses of up to 3000 mg eprosartan/kg and in dogs receiving oral doses of up to 1000 mg eprosartan/kg.',
              'There was no mortality in rats and mice receiving oral doses of up to 3000 mg eprosartan/kg and in dogs receiving oral doses of up to 1000 mg eprosartan/kg.'],
             'cardizem la': ['Clinical Toxicity and Overdose',
              'Clinical Toxicity and Overdose'],
             'cardizem la s': ['Clinical Toxicity and Overdose',
              'Clinical Toxicity and Overdose'],
             'niaspan er': ['Overdose of niacin may present with severe prolonged hypotension.1 Patients experiencing an overdose should be treated with supportive measures which may include intravenous fluids.1,6',
              'Overdose of niacin may present with severe prolonged hypotension.1 Patients experiencing an overdose should be treated with supportive measures which may include intravenous fluids.1,6'],
             'niaspan': ['Overdose of niacin may present with severe prolonged hypotension.1 Patients experiencing an overdose should be treated with supportive measures which may include intravenous fluids.1,6',
              'Overdose of niacin may present with severe prolonged hypotension.1 Patients experiencing an overdose should be treated with supportive measures which may include intravenous fluids.1,6'],
             'tarka': ['kely clinical manifestations of overdose are symptoms of severe hypotension. Most common adverse effects include cough, headache and dizziness. The oral LD50 of trandolapril in mice was 4875 mg/kg in males and 3990 mg/kg in females. In rats, an oral dose of 5000 mg/kg caused low mortality',
              'les). In dogs, an oral dose of 1000 mg/kg did not cause mortality and abnormal clinical signs were not observed'],
             'simcor': ['Overdose of niacin may present with severe prolonged hypotension.1 Patients experiencing an overdose should be treated with supportive measures which may include intravenous fluids.1,6',
              'Overdose of niacin may present with severe prolonged hypotension.1 Patients experiencing an overdose should be treated with supportive measures which may include intravenous fluids.1,6'],
             'synthroid mcg': ['LD50=20 mg/kg (orally in rat). Hypermetabolic state indistinguishable from thyrotoxicosis of endogenous origin. Symptoms of thyrotoxicosis include weight loss, increased appetite, palpitations, nervousness, diarrhea, abdominal cramps, sweating, tachycardia, increased pulse and blood pressure, cardiac arrhythmias, tremors, insomnia, heat intolerance, fever, and menstrual irregularities.',
              'LD50=20 mg/kg (orally in rat). Hypermetabolic state indistinguishable from thyrotoxicosis of endogenous origin. Symptoms of thyrotoxicosis include weight loss, increased appetite, palpitations, nervousness, diarrhea, abdominal cramps, sweating, tachycardia, increased pulse and blood pressure, cardiac arrhythmias, tremors, insomnia, heat intolerance, fever, and menstrual irregularities.'],
             'depakote xr white': ['LD50 Values', 'LD50 Values'],
             'synthroid orange': ['LD50=20 mg/kg (orally in rat). Hypermetabolic state indistinguishable from thyrotoxicosis of endogenous origin. Symptoms of thyrotoxicosis include weight loss, increased appetite, palpitations, nervousness, diarrhea, abdominal cramps, sweating, tachycardia, increased pulse and blood pressure, cardiac arrhythmias, tremors, insomnia, heat intolerance, fever, and menstrual irregularities.',
              'LD50=20 mg/kg (orally in rat). Hypermetabolic state indistinguishable from thyrotoxicosis of endogenous origin. Symptoms of thyrotoxicosis include weight loss, increased appetite, palpitations, nervousness, diarrhea, abdominal cramps, sweating, tachycardia, increased pulse and blood pressure, cardiac arrhythmias, tremors, insomnia, heat intolerance, fever, and menstrual irregularities.'],
             'synthroid': ['LD50=20 mg/kg (orally in rat). Hypermetabolic state indistinguishable from thyrotoxicosis of endogenous origin. Symptoms of thyrotoxicosis include weight loss, increased appetite, palpitations, nervousness, diarrhea, abdominal cramps, sweating, tachycardia, increased pulse and blood pressure, cardiac arrhythmias, tremors, insomnia, heat intolerance, fever, and menstrual irregularities.',
              'LD50=20 mg/kg (orally in rat). Hypermetabolic state indistinguishable from thyrotoxicosis of endogenous origin. Symptoms of thyrotoxicosis include weight loss, increased appetite, palpitations, nervousness, diarrhea, abdominal cramps, sweating, tachycardia, increased pulse and blood pressure, cardiac arrhythmias, tremors, insomnia, heat intolerance, fever, and menstrual irregularities.'],
             'synthroid violet': ['LD50=20 mg/kg (orally in rat). Hypermetabolic state indistinguishable from thyrotoxicosis of endogenous origin. Symptoms of thyrotoxicosis include weight loss, increased appetite, palpitations, nervousness, diarrhea, abdominal cramps, sweating, tachycardia, increased pulse and blood pressure, cardiac arrhythmias, tremors, insomnia, heat intolerance, fever, and menstrual irregularities.',
              'LD50=20 mg/kg (orally in rat). Hypermetabolic state indistinguishable from thyrotoxicosis of endogenous origin. Symptoms of thyrotoxicosis include weight loss, increased appetite, palpitations, nervousness, diarrhea, abdominal cramps, sweating, tachycardia, increased pulse and blood pressure, cardiac arrhythmias, tremors, insomnia, heat intolerance, fever, and menstrual irregularities.'],
             'depakote sprinkles': ['LD50 Values', 'LD50 Values'],
             'tricor yellow s': ['The oral LD50 in rats is >2g/kg and in mice is 1600mg/kg.13 The oral TDLO in rats is 9mg/kg.13',
              'The oral LD50 in rats is >2g/kg and in mice is 1600mg/kg.13 The oral TDLO in rats is 9mg/kg.13'],
             'tricor fenofibrate': ['The oral LD50 in rats is >2g/kg and in mice is 1600mg/kg.13 The oral TDLO in rats is 9mg/kg.13',
              'The oral LD50 in rats is >2g/kg and in mice is 1600mg/kg.13 The oral TDLO in rats is 9mg/kg.13'],
             'depakote peach s': ['LD50 Values', 'LD50 Values'],
             'depakote let': ['LD50 Values', 'LD50 Values'],
             'erythromycin base': ['LD50', 'LD50'],
             'ery ec': ['LD50', 'LD50'],
             'ery': ['LD50', 'LD50'],
             'synthroid olive': ['LD50=20 mg/kg (orally in rat). Hypermetabolic state indistinguishable from thyrotoxicosis of endogenous origin. Symptoms of thyrotoxicosis include weight loss, increased appetite, palpitations, nervousness, diarrhea, abdominal cramps, sweating, tachycardia, increased pulse and blood pressure, cardiac arrhythmias, tremors, insomnia, heat intolerance, fever, and menstrual irregularities.',
              'LD50=20 mg/kg (orally in rat). Hypermetabolic state indistinguishable from thyrotoxicosis of endogenous origin. Symptoms of thyrotoxicosis include weight loss, increased appetite, palpitations, nervousness, diarrhea, abdominal cramps, sweating, tachycardia, increased pulse and blood pressure, cardiac arrhythmias, tremors, insomnia, heat intolerance, fever, and menstrual irregularities.'],
             'synthroid yellow': ['LD50=20 mg/kg (orally in rat). Hypermetabolic state indistinguishable from thyrotoxicosis of endogenous origin. Symptoms of thyrotoxicosis include weight loss, increased appetite, palpitations, nervousness, diarrhea, abdominal cramps, sweating, tachycardia, increased pulse and blood pressure, cardiac arrhythmias, tremors, insomnia, heat intolerance, fever, and menstrual irregularities.',
              'LD50=20 mg/kg (orally in rat). Hypermetabolic state indistinguishable from thyrotoxicosis of endogenous origin. Symptoms of thyrotoxicosis include weight loss, increased appetite, palpitations, nervousness, diarrhea, abdominal cramps, sweating, tachycardia, increased pulse and blood pressure, cardiac arrhythmias, tremors, insomnia, heat intolerance, fever, and menstrual irregularities.'],
             'depakote xr': ['LD50 Values', 'LD50 Values'],
             'synthroid pink': ['LD50=20 mg/kg (orally in rat). Hypermetabolic state indistinguishable from thyrotoxicosis of endogenous origin. Symptoms of thyrotoxicosis include weight loss, increased appetite, palpitations, nervousness, diarrhea, abdominal cramps, sweating, tachycardia, increased pulse and blood pressure, cardiac arrhythmias, tremors, insomnia, heat intolerance, fever, and menstrual irregularities.',
              'LD50=20 mg/kg (orally in rat). Hypermetabolic state indistinguishable from thyrotoxicosis of endogenous origin. Symptoms of thyrotoxicosis include weight loss, increased appetite, palpitations, nervousness, diarrhea, abdominal cramps, sweating, tachycardia, increased pulse and blood pressure, cardiac arrhythmias, tremors, insomnia, heat intolerance, fever, and menstrual irregularities.'],
             'k s meq s': ['The oral LD50 of potassium chloride in rats is 2600 mg/kg.MSDS',
              'The oral LD50 of potassium chloride in rats is 2600 mg/kg.MSDS'],
             'trilipix': ['Oral LD50 (rat): 1242 mg/kg, Oral LD50 (mouse): 100 mg/kg, lntraperitoneal LD50 (mouse): 500 mg/kg MSDS',
              'Oral LD50 (rat): 1242 mg/kg, Oral LD50 (mouse): 100 mg/kg, lntraperitoneal LD50 (mouse): 500 mg/kg MSDS'],
             'rilutek': ['Not Available', 'Not Available'],
             'uribel': ['Oral LD50 in the rat is 3000 mg/kg 13',
              'Oral LD50 in the rat is 3000 mg/kg 13'],
             'hyophen': ['Oral LD50 in the rat is 3000 mg/kg 13',
              'Oral LD50 in the rat is 3000 mg/kg 13'],
             'methergine s': ['Signs and symptoms of overexposure: hypertension, seizures, headache, hypotension, nausea, and vomiting.',
              'Signs and symptoms of overexposure: hypertension, seizures, headache, hypotension, nausea, and vomiting.'],
             'lescol': ['Generally well-tolerated. May cause gastrointestinal upset (diarrhea, nausea, constipation, gas, abdominal pain), myotoxicity (mypothy, myositis, rhabdomyolysis), and hepatotoxicity.',
              'Generally well-tolerated. May cause gastrointestinal upset (diarrhea, nausea, constipation, gas, abdominal pain), myotoxicity (mypothy, myositis, rhabdomyolysis), and hepatotoxicity.'],
             'diovan hct': ['Approximate LD50 >2000 mg/kg (Gavage, rat) 24',
              'Approximate LD50 >2000 mg/kg (Gavage, rat) 24'],
             'diovanhctz': ['', ''],
             'exelon s': ['Not Available', 'Not Available'],
             'comtan': ['Side effect include increase the occurrence of orthostatic hypotension, severe rhabdomyolysis, dyskinesia, hallucinations, hyperkinesia, hypokinesia, dizziness, fatigu,e gastrointestinal effects including abdominal pain constipation diarrhea nausea',
              'Side effect include increase the occurrence of orthostatic hypotension, severe rhabdomyolysis, dyskinesia, hallucinations, hyperkinesia, hypokinesia, dizziness, fatigu,e gastrointestinal effects including abdominal pain constipation diarrhea nausea'],
             'trileptal yellow s': ['The oral LD50 of oxcarbazepine in mammals is 1240 mg/kg and the oral TDLo in children has been reported to be 73 mg/kg.13 Isolated cases of oxcarbazepine overdose have been reported - patients who ingested up to 24,000mg recovered with symptomatic treatment.10,11 Symptoms may include respiratory and CNS depression, movement-related disorders (e.g. dyskinesia, ataxia), nausea/vomiting, hyponatremia, or QTc prolongation. There is no antidote for oxcarbazepine overdose - management should consist of supportive and symptomatic treatment, and consideration should be given to the use of gastric lavage or activated charcoal.10,11',
              'The oral LD50 of oxcarbazepine in mammals is 1240 mg/kg and the oral TDLo in children has been reported to be 73 mg/kg.13 Isolated cases of oxcarbazepine overdose have been reported - patients who ingested up to 24,000mg recovered with symptomatic treatment.10,11 Symptoms may include respiratory and CNS depression, movement-related disorders (e.g. dyskinesia, ataxia), nausea/vomiting, hyponatremia, or QTc prolongation. There is no antidote for oxcarbazepine overdose - management should consist of supportive and symptomatic treatment, and consideration should be given to the use of gastric lavage or activated charcoal.10,11'],
             'starlix': ['An overdose may result in an exaggerated glucose-lowering effect with the development of hypoglycemic symptoms.',
              'An overdose may result in an exaggerated glucose-lowering effect with the development of hypoglycemic symptoms.'],
             'lescol xl': ['Generally well-tolerated. May cause gastrointestinal upset (diarrhea, nausea, constipation, gas, abdominal pain), myotoxicity (mypothy, myositis, rhabdomyolysis), and hepatotoxicity.',
              'Generally well-tolerated. May cause gastrointestinal upset (diarrhea, nausea, constipation, gas, abdominal pain), myotoxicity (mypothy, myositis, rhabdomyolysis), and hepatotoxicity.'],
             'diovan': ['Approximate LD50 >2000 mg/kg (Gavage, rat) 24',
              'Approximate LD50 >2000 mg/kg (Gavage, rat) 24'],
             'lotrel': ['Common gastrointestinal adverse effects include dry mouth, foul taste, diarrhea, dyspepsia, epigastric pain, nausea and constipation. Some CNS effects include drowsiness, fatigue, dizziness, hot/cold sensations and headaches. In case of overdosage, gastric lavage is recommended. Oral LD50 in mouse and rats is >5000 mg/kg and 2500 mg/kg in rabbits, respectively. Trimebutine is not reported to display teratogenic, mutagenic or carcinogenic potential 8.',
              'Common gastrointestinal adverse effects include dry mouth, foul taste, diarrhea, dyspepsia, epigastric pain, nausea and constipation. Some CNS effects include drowsiness, fatigue, dizziness, hot/cold sensations and headaches. In case of overdosage, gastric lavage is recommended. Oral LD50 in mouse and rats is >5000 mg/kg and 2500 mg/kg in rabbits, respectively. Trimebutine is not reported to display teratogenic, mutagenic or carcinogenic potential 8.'],
             'myfortic': ['re anecdotal reports of deliberate or accidental overdoses with mycophenolic aci',
              'r, not all patients have experienced related adverse reactions. In those cases where adverse reactions have been reported, reactions fall within the safety profile of its class of drugs. A mycophenolic acid overdose could lead to the oversuppression of the immune system and increase the susceptibility to infection.8 It may be appropriate to interrupt or discontinue mycophenolic acid if blood dyscrasias occur. Some of the signs and symptoms associated with mycophenolic acid overdose are hematological abnormalities, such as leukopenia and neutropenia, and gastrointestinal symptoms, such as abdominal pain, diarrhea, nausea and vomiting, and dyspepsia.'],
             'stalevo': ['The LD50 of carbidopa is reported to be in the rat of 4810 mg/kg.MSDS In animal studies, carbidopa showed no incidences on neoplasia and showed no effect on the fertility status and development.Label',
              'The LD50 of carbidopa is reported to be in the rat of 4810 mg/kg.MSDS In animal studies, carbidopa showed no incidences on neoplasia and showed no effect on the fertility status and development.Label'],
             'trileptal grey s': ['The oral LD50 of oxcarbazepine in mammals is 1240 mg/kg and the oral TDLo in children has been reported to be 73 mg/kg.13 Isolated cases of oxcarbazepine overdose have been reported - patients who ingested up to 24,000mg recovered with symptomatic treatment.10,11 Symptoms may include respiratory and CNS depression, movement-related disorders (e.g. dyskinesia, ataxia), nausea/vomiting, hyponatremia, or QTc prolongation. There is no antidote for oxcarbazepine overdose - management should consist of supportive and symptomatic treatment, and consideration should be given to the use of gastric lavage or activated charcoal.10,11',
              'The oral LD50 of oxcarbazepine in mammals is 1240 mg/kg and the oral TDLo in children has been reported to be 73 mg/kg.13 Isolated cases of oxcarbazepine overdose have been reported - patients who ingested up to 24,000mg recovered with symptomatic treatment.10,11 Symptoms may include respiratory and CNS depression, movement-related disorders (e.g. dyskinesia, ataxia), nausea/vomiting, hyponatremia, or QTc prolongation. There is no antidote for oxcarbazepine overdose - management should consist of supportive and symptomatic treatment, and consideration should be given to the use of gastric lavage or activated charcoal.10,11'],
             'trileptal pink s': ['The oral LD50 of oxcarbazepine in mammals is 1240 mg/kg and the oral TDLo in children has been reported to be 73 mg/kg.13 Isolated cases of oxcarbazepine overdose have been reported - patients who ingested up to 24,000mg recovered with symptomatic treatment.10,11 Symptoms may include respiratory and CNS depression, movement-related disorders (e.g. dyskinesia, ataxia), nausea/vomiting, hyponatremia, or QTc prolongation. There is no antidote for oxcarbazepine overdose - management should consist of supportive and symptomatic treatment, and consideration should be given to the use of gastric lavage or activated charcoal.10,11',
              'The oral LD50 of oxcarbazepine in mammals is 1240 mg/kg and the oral TDLo in children has been reported to be 73 mg/kg.13 Isolated cases of oxcarbazepine overdose have been reported - patients who ingested up to 24,000mg recovered with symptomatic treatment.10,11 Symptoms may include respiratory and CNS depression, movement-related disorders (e.g. dyskinesia, ataxia), nausea/vomiting, hyponatremia, or QTc prolongation. There is no antidote for oxcarbazepine overdose - management should consist of supportive and symptomatic treatment, and consideration should be given to the use of gastric lavage or activated charcoal.10,11'],
             'lopressor': ['Oral administration of metoprolol to rats presents an LD50 in the range of 3090 to 4670 mg/kg. Cases of overdose have reported bradycardia, hypotension, bronchospasm, and cardiac failure. In the case of an overdose, gastric lavage is recommended followed by specific treatment according to symptoms.Label',
              'Oral administration of metoprolol to rats presents an LD50 in the range of 3090 to 4670 mg/kg. Cases of overdose have reported bradycardia, hypotension, bronchospasm, and cardiac failure. In the case of an overdose, gastric lavage is recommended followed by specific treatment according to symptoms.Label'],
             'tekturna': ['The oral LD50 of aliskiren in rats is >2000 mg/kg.14 Overdose information is limited in the literature, however, an overdose with aliskiren is likely to result in hypotension. Supportive treatment should be initiated in the case of an overdose.9,15',
              'The oral LD50 of aliskiren in rats is >2000 mg/kg.14 Overdose information is limited in the literature, however, an overdose with aliskiren is likely to result in hypotension. Supportive treatment should be initiated in the case of an overdose.9,15'],
             'exforge': ['The oral LD50 of hydrochlorothiazide is >10g/kg in mice and rats.10,11',
              'The oral LD50 of hydrochlorothiazide is >10g/kg in mice and rats.10,11'],
             'tegretol': ['y information\nOral LDLO', 'DLO'],
             'tegretol xr': ['y information\nOral LDLO', 'DLO'],
             'tekturna hct': ['The oral LD50 of aliskiren in rats is >2000 mg/kg.14 Overdose information is limited in the literature, however, an overdose with aliskiren is likely to result in hypotension. Supportive treatment should be initiated in the case of an overdose.9,15',
              'The oral LD50 of aliskiren in rats is >2000 mg/kg.14 Overdose information is limited in the literature, however, an overdose with aliskiren is likely to result in hypotension. Supportive treatment should be initiated in the case of an overdose.9,15'],
             'exforge xct': ['The oral LD50 of hydrochlorothiazide is >10g/kg in mice and rats.10,11',
              'The oral LD50 of hydrochlorothiazide is >10g/kg in mice and rats.10,11'],
             'exforge hct': ['The oral LD50 of hydrochlorothiazide is >10g/kg in mice and rats.10,11',
              'The oral LD50 of hydrochlorothiazide is >10g/kg in mice and rats.10,11'],
             'coartem': ['Not Available', 'Not Available'],
             'valturna': ['The oral LD50 of aliskiren in rats is >2000 mg/kg.14 Overdose information is limited in the literature, however, an overdose with aliskiren is likely to result in hypotension. Supportive treatment should be initiated in the case of an overdose.9,15',
              'The oral LD50 of aliskiren in rats is >2000 mg/kg.14 Overdose information is limited in the literature, however, an overdose with aliskiren is likely to result in hypotension. Supportive treatment should be initiated in the case of an overdose.9,15'],
             'fanapt iloperidone': ['Commonly observed adverse reactions (incidence ≥5% and two-fold greater than placebo) were: dizziness, dry mouth, fatigue, nasal congestion, orthostatic hypotension, somnolence, tachycardia, and weight increased.',
              'Commonly observed adverse reactions (incidence ≥5% and two-fold greater than placebo) were: dizziness, dry mouth, fatigue, nasal congestion, orthostatic hypotension, somnolence, tachycardia, and weight increased.'],
             'fanapt': ['Commonly observed adverse reactions (incidence ≥5% and two-fold greater than placebo) were: dizziness, dry mouth, fatigue, nasal congestion, orthostatic hypotension, somnolence, tachycardia, and weight increased.',
              'Commonly observed adverse reactions (incidence ≥5% and two-fold greater than placebo) were: dizziness, dry mouth, fatigue, nasal congestion, orthostatic hypotension, somnolence, tachycardia, and weight increased.'],
             'clarinex': ['tion regarding desloratadine overdose is limited, although somnolence has been reported. In case of overdose, symptomatic and supportive treatment, including removing the unabsorbed drug, is recommende',
              'however, that desloratadine and its active metabolite 3-hydroxydesloratadine cannot be eliminated by hemodialysis.'],
             'clarinexd': ['tion regarding desloratadine overdose is limited, although somnolence has been reported. In case of overdose, symptomatic and supportive treatment, including removing the unabsorbed drug, is recommende',
              'however, that desloratadine and its active metabolite 3-hydroxydesloratadine cannot be eliminated by hemodialysis.'],
             'potassium chloride': ['The administration of oral potassium salts to persons with normal excretory mechanisms for potassium rarely causes serious hyperkalemia. However, if excretory mechanisms are impaired, of if potassium is administered too rapidly intravenously, potentially fatal hyperkalemia can result. It is important to recognize that hyperkalemia is usually asymptomatic and may be manifested only by an increased serum potassium concentration (6.5-8.0 mEq/L) and characteristic electrocardiographic changes (peaking of T-waves, loss of P-wave, depression of S-T segment, and prolongation of the QT interval). Late manifestations include muscle paralysis and cardiovascular collapse from cardiac arrest (9-12 mEq/L).',
              'The administration of oral potassium salts to persons with normal excretory mechanisms for potassium rarely causes serious hyperkalemia. However, if excretory mechanisms are impaired, of if potassium is administered too rapidly intravenously, potentially fatal hyperkalemia can result. It is important to recognize that hyperkalemia is usually asymptomatic and may be manifested only by an increased serum potassium concentration (6.5-8.0 mEq/L) and characteristic electrocardiographic changes (peaking of T-waves, loss of P-wave, depression of S-T segment, and prolongation of the QT interval). Late manifestations include muscle paralysis and cardiovascular collapse from cardiac arrest (9-12 mEq/L).'],
             'adalat cc': ['The oral LD50 in rats is 1022mg/kg and in mice is 202mg/kg.16',
              'The oral LD50 in rats is 1022mg/kg and in mice is 202mg/kg.16'],
             'avelox': ['Symptoms of overdose include CNS and gastrointestinal effects such as decreased activity, somnolence, tremor, convulsions, vomiting, and diarrhea. The minimal lethal intravenous dose in mice and rats is 100 mg/kg.',
              'Symptoms of overdose include CNS and gastrointestinal effects such as decreased activity, somnolence, tremor, convulsions, vomiting, and diarrhea. The minimal lethal intravenous dose in mice and rats is 100 mg/kg.'],
             'cipro': ['Not Available', 'Not Available'],
             'noxafil': ['During the clinical trials, some patients received posaconazole up to 1600 mg/day with no adverse events noted that were different from the lower doses. In addition, accidental overdose was noted in one patient who took 1200 mg BID for 3 days. No related adverse events were noted by the investigator.',
              'During the clinical trials, some patients received posaconazole up to 1600 mg/day with no adverse events noted that were different from the lower doses. In addition, accidental overdose was noted in one patient who took 1200 mg BID for 3 days. No related adverse events were noted by the investigator.'],
             'avapro': ['The oral TDLO in humans is 30mg/kg/6W.9',
              'The oral TDLO in humans is 30mg/kg/6W.9'],
             'avalide': ['The oral LD50 of hydrochlorothiazide is >10g/kg in mice and rats.10,11',
              'The oral LD50 of hydrochlorothiazide is >10g/kg in mice and rats.10,11'],
             'glucophage': ['Metformin (hydrochloride) toxicity data: ',
              'Metformin (hydrochloride) toxicity data: '],
             'glucophage xr': ['Metformin (hydrochloride) toxicity data: ',
              'Metformin (hydrochloride) toxicity data: '],
             'allegrad hour': ["No deaths were observed following the oral administration of up to 5000 mg/kg in both mice and rats (equivalent to approximately 100-200x the recommended human dose). Single doses of up to 800 mg and chronic exposure of up to 690 mg twice daily for 1 month in humans did not result in clinically significant adverse events. Symptoms of overdosage are consistent with fexofenadine's adverse effect profile and are likely to include dizziness, drowsiness, and dry mouth.10",
              "No deaths were observed following the oral administration of up to 5000 mg/kg in both mice and rats (equivalent to approximately 100-200x the recommended human dose). Single doses of up to 800 mg and chronic exposure of up to 690 mg twice daily for 1 month in humans did not result in clinically significant adverse events. Symptoms of overdosage are consistent with fexofenadine's adverse effect profile and are likely to include dizziness, drowsiness, and dry mouth.10"],
             'allegra odt': ["No deaths were observed following the oral administration of up to 5000 mg/kg in both mice and rats (equivalent to approximately 100-200x the recommended human dose). Single doses of up to 800 mg and chronic exposure of up to 690 mg twice daily for 1 month in humans did not result in clinically significant adverse events. Symptoms of overdosage are consistent with fexofenadine's adverse effect profile and are likely to include dizziness, drowsiness, and dry mouth.10",
              "No deaths were observed following the oral administration of up to 5000 mg/kg in both mice and rats (equivalent to approximately 100-200x the recommended human dose). Single doses of up to 800 mg and chronic exposure of up to 690 mg twice daily for 1 month in humans did not result in clinically significant adverse events. Symptoms of overdosage are consistent with fexofenadine's adverse effect profile and are likely to include dizziness, drowsiness, and dry mouth.10"],
             'dilatrate sr': ['Symptoms of overdose include reduced cardiac output and hypotension.',
              'Symptoms of overdose include reduced cardiac output and hypotension.'],
             'tolterodine tartrate': ['Not Available', 'Not Available'],
             'pantoprazole sodium': ['Rat Oral LD 50 747 mg/kg17 ',
              'Rat Oral LD 50 747 mg/kg17 '],
             'moexipril hydrochloride': ['Human overdoses of moexipril have not been reported. In case reports of overdoses with other ACE inhibitors, hypotension has been the principal adverse effect noted. Single oral doses of 2 g/kg moexipril were associated with significant lethality in mice. Rats, however, tolerated single oral doses of up to 3 g/kg. Common adverse effects include cough, dizziness, diarrhea, flu syndrome, fatigue, pharyngitis, flushing, rash, and myalgia',
              'Human overdoses of moexipril have not been reported. In case reports of overdoses with other ACE inhibitors, hypotension has been the principal adverse effect noted. Single oral doses of 2 g/kg moexipril were associated with significant lethality in mice. Rats, however, tolerated single oral doses of up to 3 g/kg. Common adverse effects include cough, dizziness, diarrhea, flu syndrome, fatigue, pharyngitis, flushing, rash, and myalgia'],
             'enalapril maleate': ['LD50 and Overdose', 'LD50 and Overdose'],
             'lamotrigine': ['The oral LD50 in mouse and rat is 205 mg/kg and 245 mg/kg, respectively.MSDS',
              'The oral LD50 in mouse and rat is 205 mg/kg and 245 mg/kg, respectively.MSDS'],
             'clomiphene citrate': ['The acute oral LD50 of clomifene is 1700 mg/kg in mice and 5750 mg/kg in rats. The toxic dose in humans is not known. Toxic effects accompanying acute overdosage of clomifene have not been reported. Signs and symptoms of overdosage as a result of the use of more than the recommended dose during clomifene therapy include nausea, vomiting, vasomotor flushes, visual blurring, spots or flashes, scotomata, ovarian enlargement with pelvic or abdominal pain.',
              'The acute oral LD50 of clomifene is 1700 mg/kg in mice and 5750 mg/kg in rats. The toxic dose in humans is not known. Toxic effects accompanying acute overdosage of clomifene have not been reported. Signs and symptoms of overdosage as a result of the use of more than the recommended dose during clomifene therapy include nausea, vomiting, vasomotor flushes, visual blurring, spots or flashes, scotomata, ovarian enlargement with pelvic or abdominal pain.'],
             'acetaminophen and codeine phosphate': ['338 mg/kg',
              '1944 mg/kg'],
             'carvedilol': ['Patients experiencing an overdose may present with hypotension, bradycardia, cardiac insufficiency, cardiogenic shock, and cardiac arrest.5,6 Patients should remain in a supine position and may be given atropine for bradycardia and glucagon followed by sympathomimetics to support cardiovascular function.5,6',
              'Patients experiencing an overdose may present with hypotension, bradycardia, cardiac insufficiency, cardiogenic shock, and cardiac arrest.5,6 Patients should remain in a supine position and may be given atropine for bradycardia and glucagon followed by sympathomimetics to support cardiovascular function.5,6'],
             'birone hydrochloride': ['Not Available', 'Not Available'],
             'birone': ['Not Available', 'Not Available'],
             'tramadol hydrochloride': ['The reported LD50 for tramadol, when administered orally in mice, is 350 mg/kg.11 ',
              'The reported LD50 for tramadol, when administered orally in mice, is 350 mg/kg.11 '],
             'rabeprazole sodium': ['Not Available', 'Not Available'],
             'zolpidem tartrate': ['Oral (male rat) LD50 = 695 mg/kg MSDS. ',
              'Oral (male rat) LD50 = 695 mg/kg MSDS. '],
             'isosorbide mononitrate': ['The oral LD50 is 2010 mg/kg in rats and 1771 mg/kg in mice.5 ',
              'The oral LD50 is 2010 mg/kg in rats and 1771 mg/kg in mice.5 '],
             'amlodipine besylate': ['Acute oral toxicity (LD50): 37 mg/kg (mouse) MSDS. ',
              'Acute oral toxicity (LD50): 37 mg/kg (mouse) MSDS. '],
             'epitol (carbamazepine )': ['y information\nOral LDLO', 'DLO'],
             'carbamazepine': ['y information\nOral LDLO', 'DLO'],
             'estazolam': ['Symptoms of overdose include confusion, depressed breathing, drowsiness and eventually coma, lack of coordination, and slurred speech.',
              'Symptoms of overdose include confusion, depressed breathing, drowsiness and eventually coma, lack of coordination, and slurred speech.'],
             'lamotrigine ()': ['The oral LD50 in mouse and rat is 205 mg/kg and 245 mg/kg, respectively.MSDS',
              'The oral LD50 in mouse and rat is 205 mg/kg and 245 mg/kg, respectively.MSDS'],
             'naproxen': ['Although the over-the-counter (OTC) availability of naproxen provides convenience to patients, it also increases the likelihood of overdose.3 Thankfully, the extent of overdose is typically mild with adverse effects normally limited to drowsiness, lethargy, epigastric pain, nausea and vomiting.31213 Although there is no antidote for naproxen overdose, symptoms will typically subside with appropriate supportive care.12133',
              'Although the over-the-counter (OTC) availability of naproxen provides convenience to patients, it also increases the likelihood of overdose.3 Thankfully, the extent of overdose is typically mild with adverse effects normally limited to drowsiness, lethargy, epigastric pain, nausea and vomiting.31213 Although there is no antidote for naproxen overdose, symptoms will typically subside with appropriate supportive care.12133'],
             'apapcodeine': ['', ''],
             'ticlopidine hydrochloride': ["Single oral doses of ticlopidine at 1600 mg/kg and 500 mg/kg were lethal to rats and mice, respectively. Symptoms of acute toxicity were GI hemorrhage, convulsions, hypothermia, dyspnea, loss of equilibrium and abnormal gait. The FDA label includes a black-box warning of neutropenia, aplastic anemia, thrombotic thrombocytopenia purpura, and agranulocytosis, so it is necessary to monitor patients' WBC and platelets when they are taking ticlopidine.",
              "Single oral doses of ticlopidine at 1600 mg/kg and 500 mg/kg were lethal to rats and mice, respectively. Symptoms of acute toxicity were GI hemorrhage, convulsions, hypothermia, dyspnea, loss of equilibrium and abnormal gait. The FDA label includes a black-box warning of neutropenia, aplastic anemia, thrombotic thrombocytopenia purpura, and agranulocytosis, so it is necessary to monitor patients' WBC and platelets when they are taking ticlopidine."],
             'ticlopidine': ["Single oral doses of ticlopidine at 1600 mg/kg and 500 mg/kg were lethal to rats and mice, respectively. Symptoms of acute toxicity were GI hemorrhage, convulsions, hypothermia, dyspnea, loss of equilibrium and abnormal gait. The FDA label includes a black-box warning of neutropenia, aplastic anemia, thrombotic thrombocytopenia purpura, and agranulocytosis, so it is necessary to monitor patients' WBC and platelets when they are taking ticlopidine.",
              "Single oral doses of ticlopidine at 1600 mg/kg and 500 mg/kg were lethal to rats and mice, respectively. Symptoms of acute toxicity were GI hemorrhage, convulsions, hypothermia, dyspnea, loss of equilibrium and abnormal gait. The FDA label includes a black-box warning of neutropenia, aplastic anemia, thrombotic thrombocytopenia purpura, and agranulocytosis, so it is necessary to monitor patients' WBC and platelets when they are taking ticlopidine."],
             'topiramate': ['The LD50 of intraperitoneal topiramate in the rat is above 1500 mg/kg.20',
              'The LD50 of intraperitoneal topiramate in the rat is above 1500 mg/kg.20'],
             'leflunomide': ['LD50=100-250 mg/kg (acute oral toxicity)',
              'LD50=100-250 mg/kg (acute oral toxicity)'],
             'venlafaxine hydrochloride': ['Oral LD50 was 350 mg/kg in female rats and 700 mg/kg in male rats.16',
              'Oral LD50 was 350 mg/kg in female rats and 700 mg/kg in male rats.16'],
             'bicalutamide': ['Not Available', 'Not Available'],
             'risperidone': ['Symptoms of overdose include lethargy, dystonia/spasm, tachycardia, bradycardia, and seizures. LD50=57.7 mg/kg (rat, oral) and 34 mg/kg (rat, intravenous).5,6,7',
              'Symptoms of overdose include lethargy, dystonia/spasm, tachycardia, bradycardia, and seizures. LD50=57.7 mg/kg (rat, oral) and 34 mg/kg (rat, intravenous).5,6,7'],
             'ondansetron hydrochloride': ['At present, there is little information concerning overdosage with ondansetron 8,9,10. Nevertheless, there have been certain cases of somewhat idiosyncratic adverse effects associated with particular dosages of ondansetron used 8,9,10.',
              'At present, there is little information concerning overdosage with ondansetron 8,9,10. Nevertheless, there have been certain cases of somewhat idiosyncratic adverse effects associated with particular dosages of ondansetron used 8,9,10.'],
             'bupropion': ['Symptoms of overdose include seizures, hallucinations, loss of consciousness, tachycardia, and cardiac arrest.',
              'Symptoms of overdose include seizures, hallucinations, loss of consciousness, tachycardia, and cardiac arrest.'],
             'carbidopa and levodopa': ['The LD50 of carbidopa is reported to be in the rat of 4810 mg/kg.MSDS In animal studies, carbidopa showed no incidences on neoplasia and showed no effect on the fertility status and development.Label',
              'The LD50 of carbidopa is reported to be in the rat of 4810 mg/kg.MSDS In animal studies, carbidopa showed no incidences on neoplasia and showed no effect on the fertility status and development.Label'],
             'clemastine fumarate': ['Oral LD50 in rat and mouse is 3550 mg/kg and 730 mg/kg, respectively. Antihistamine overdosage reactions may vary from central nervous system depression to stimulation. In children, stimulation predominates initially in a syndrome which may include excitement, hallucinations, ataxia, incoordination, muscle twitching, athetosis, hyperthermia, cyanosis convulsions, tremors, and hyperreflexia followed by postictal depression and cardio-respiratory arrest. Convulsions in children may be preceded by mild depression. Dry mouth, fixed dilated pupils, flushing of the face, and fever are common. In adults, CNS depression, ranging from drowsiness to coma, is more common.',
              'Oral LD50 in rat and mouse is 3550 mg/kg and 730 mg/kg, respectively. Antihistamine overdosage reactions may vary from central nervous system depression to stimulation. In children, stimulation predominates initially in a syndrome which may include excitement, hallucinations, ataxia, incoordination, muscle twitching, athetosis, hyperthermia, cyanosis convulsions, tremors, and hyperreflexia followed by postictal depression and cardio-respiratory arrest. Convulsions in children may be preceded by mild depression. Dry mouth, fixed dilated pupils, flushing of the face, and fever are common. In adults, CNS depression, ranging from drowsiness to coma, is more common.'],
             'loperamide hydrochloride': ['Oral LD50 is 185 mg/kg in rats.6',
              'Oral LD50 is 185 mg/kg in rats.6'],
             'loperamide': ['Oral LD50 is 185 mg/kg in rats.6',
              'Oral LD50 is 185 mg/kg in rats.6'],
             'ketorolac tromethamine': ['The rate of adverse effects increases with higher doses of ketorolac. The most frequently observed adverse effects in patients occurring with an incidence of greater than 10% include: abdominal pain, dyspepsia, nausea, and headaches.13 Most adverse effects associated with short term use are mild in nature, related to the gastrointestinal tract and nervous system, and occur in roughly 39% of patients.2 Common symptoms of ketorolac overdose include nausea, vomiting, epigastric pain, gastrointestinal bleeding, lethargy and drowsiness. More rare symptoms of overdose include acute renal failure, hypertension, respiratory depression, and coma.14',
              'The rate of adverse effects increases with higher doses of ketorolac. The most frequently observed adverse effects in patients occurring with an incidence of greater than 10% include: abdominal pain, dyspepsia, nausea, and headaches.13 Most adverse effects associated with short term use are mild in nature, related to the gastrointestinal tract and nervous system, and occur in roughly 39% of patients.2 Common symptoms of ketorolac overdose include nausea, vomiting, epigastric pain, gastrointestinal bleeding, lethargy and drowsiness. More rare symptoms of overdose include acute renal failure, hypertension, respiratory depression, and coma.14'],
             'ketorolac': ['The rate of adverse effects increases with higher doses of ketorolac. The most frequently observed adverse effects in patients occurring with an incidence of greater than 10% include: abdominal pain, dyspepsia, nausea, and headaches.13 Most adverse effects associated with short term use are mild in nature, related to the gastrointestinal tract and nervous system, and occur in roughly 39% of patients.2 Common symptoms of ketorolac overdose include nausea, vomiting, epigastric pain, gastrointestinal bleeding, lethargy and drowsiness. More rare symptoms of overdose include acute renal failure, hypertension, respiratory depression, and coma.14',
              'The rate of adverse effects increases with higher doses of ketorolac. The most frequently observed adverse effects in patients occurring with an incidence of greater than 10% include: abdominal pain, dyspepsia, nausea, and headaches.13 Most adverse effects associated with short term use are mild in nature, related to the gastrointestinal tract and nervous system, and occur in roughly 39% of patients.2 Common symptoms of ketorolac overdose include nausea, vomiting, epigastric pain, gastrointestinal bleeding, lethargy and drowsiness. More rare symptoms of overdose include acute renal failure, hypertension, respiratory depression, and coma.14'],
             'diltiazem hydrochloride': ['Clinical Toxicity and Overdose',
              'Clinical Toxicity and Overdose'],
             'diltiazem': ['Clinical Toxicity and Overdose',
              'Clinical Toxicity and Overdose'],
             'naproxen sodium': ['Although the over-the-counter (OTC) availability of naproxen provides convenience to patients, it also increases the likelihood of overdose.3 Thankfully, the extent of overdose is typically mild with adverse effects normally limited to drowsiness, lethargy, epigastric pain, nausea and vomiting.31213 Although there is no antidote for naproxen overdose, symptoms will typically subside with appropriate supportive care.12133',
              'Although the over-the-counter (OTC) availability of naproxen provides convenience to patients, it also increases the likelihood of overdose.3 Thankfully, the extent of overdose is typically mild with adverse effects normally limited to drowsiness, lethargy, epigastric pain, nausea and vomiting.31213 Although there is no antidote for naproxen overdose, symptoms will typically subside with appropriate supportive care.12133'],
             'lovastatin': ['ian lethal dose of lovastatin is higher than 15 g/m2. Five healthy human volunteers have received up to 200 mg of lovastatin as a single dose without clinically significant adverse experiences. A few cases of accidental overdosage have been reporte',
              'ients had any specific symptoms, and all patients recovered without sequelae. The maximum dose taken was 5 to 6 g.4'],
             'calcitriol mcg': ['', ''],
             'gemfibrozil': ['The oral TDLO of gemfibrozil in humans is 18gm/kg/3Y.12 The oral LD50 in mice is 2218mg/kg and in rats is 1414mg/kg.12 The intraperitoneal LD50 in rats is 445mg/kg.12',
              'The oral TDLO of gemfibrozil in humans is 18gm/kg/3Y.12 The oral LD50 in mice is 2218mg/kg and in rats is 1414mg/kg.12 The intraperitoneal LD50 in rats is 445mg/kg.12'],
             'flurbiprofen': ['LD50=10 mg/kg (orally in dogs).\n',
              'LD50=10 mg/kg (orally in dogs).\n'],
             'metoprolol tartrate': ['Oral administration of metoprolol to rats presents an LD50 in the range of 3090 to 4670 mg/kg. Cases of overdose have reported bradycardia, hypotension, bronchospasm, and cardiac failure. In the case of an overdose, gastric lavage is recommended followed by specific treatment according to symptoms.Label',
              'Oral administration of metoprolol to rats presents an LD50 in the range of 3090 to 4670 mg/kg. Cases of overdose have reported bradycardia, hypotension, bronchospasm, and cardiac failure. In the case of an overdose, gastric lavage is recommended followed by specific treatment according to symptoms.Label'],
             'donepezil': ['LD50', 'LD50'],
             'atenolol': ['LD50 Values', 'LD50 Values'],
             'diflunisal': ['Oral LD50 in rat, mouse, and rabbit is 392 mg/kg, 439 mg/kg, and 603 mg/kg, respectively. Symptoms of overdose include drowsiness, nausea, vomiting, diarrhea, hyperventilation, tachycardia, sweating, tinnitus, disorientation, stupor, and coma. As a monotherapy, the smallest dosage capable of causing death was reported as 15 grams.\n',
              'Oral LD50 in rat, mouse, and rabbit is 392 mg/kg, 439 mg/kg, and 603 mg/kg, respectively. Symptoms of overdose include drowsiness, nausea, vomiting, diarrhea, hyperventilation, tachycardia, sweating, tinnitus, disorientation, stupor, and coma. As a monotherapy, the smallest dosage capable of causing death was reported as 15 grams.\n'],
             'piroxicam': ['Symptoms of overdose include drowsiness, nausea, stomach pain, and/or vomiting.',
              'Symptoms of overdose include drowsiness, nausea, stomach pain, and/or vomiting.'],
             'pravastatin sodium': ["The reported oral LD50 of pravastatin in mice is of 8939 mg/kg.MSDS There haven't been significant overdosage reports however, in the case of overdosage, symptomatic treatment is recommended along with laboratory monitoring and supportive measures.Label",
              "The reported oral LD50 of pravastatin in mice is of 8939 mg/kg.MSDS There haven't been significant overdosage reports however, in the case of overdosage, symptomatic treatment is recommended along with laboratory monitoring and supportive measures.Label"],
             'tamoxifen': ['High doses of tamoxifen in animals lead to respiratory difficulty and convulsions.15,16 High doses in advanced metastatic cancer patients resulted in acute neurotoxicity seen by tremor, hyperreflexia, unsteady gait, and dizziness.15,16 Patients experiencing and overdose should be given supportive treatment as no specific treatment for overdose is suggested.15,16',
              'High doses of tamoxifen in animals lead to respiratory difficulty and convulsions.15,16 High doses in advanced metastatic cancer patients resulted in acute neurotoxicity seen by tremor, hyperreflexia, unsteady gait, and dizziness.15,16 Patients experiencing and overdose should be given supportive treatment as no specific treatment for overdose is suggested.15,16'],
             'tamoxifen citrate': ['High doses of tamoxifen in animals lead to respiratory difficulty and convulsions.15,16 High doses in advanced metastatic cancer patients resulted in acute neurotoxicity seen by tremor, hyperreflexia, unsteady gait, and dizziness.15,16 Patients experiencing and overdose should be given supportive treatment as no specific treatment for overdose is suggested.15,16',
              'High doses of tamoxifen in animals lead to respiratory difficulty and convulsions.15,16 High doses in advanced metastatic cancer patients resulted in acute neurotoxicity seen by tremor, hyperreflexia, unsteady gait, and dizziness.15,16 Patients experiencing and overdose should be given supportive treatment as no specific treatment for overdose is suggested.15,16'],
             'tamoxifen citrate s': ['High doses of tamoxifen in animals lead to respiratory difficulty and convulsions.15,16 High doses in advanced metastatic cancer patients resulted in acute neurotoxicity seen by tremor, hyperreflexia, unsteady gait, and dizziness.15,16 Patients experiencing and overdose should be given supportive treatment as no specific treatment for overdose is suggested.15,16',
              'High doses of tamoxifen in animals lead to respiratory difficulty and convulsions.15,16 High doses in advanced metastatic cancer patients resulted in acute neurotoxicity seen by tremor, hyperreflexia, unsteady gait, and dizziness.15,16 Patients experiencing and overdose should be given supportive treatment as no specific treatment for overdose is suggested.15,16'],
             'nortriptyline hydrochloride': ['The oral LD50 of nortriptyline in the rat is 405 mg/kg.22Symptoms of overdose with nortriptyline include cardiac arrhythmias, severe hypotension, shock, congestive heart failure, pulmonary edema, convulsions, coma, and CNS depression.18,20 Changes in the electrocardiogram, particularly in QRS segment, may be indicative of tricyclic antidepressant toxicity.13 ',
              'The oral LD50 of nortriptyline in the rat is 405 mg/kg.22Symptoms of overdose with nortriptyline include cardiac arrhythmias, severe hypotension, shock, congestive heart failure, pulmonary edema, convulsions, coma, and CNS depression.18,20 Changes in the electrocardiogram, particularly in QRS segment, may be indicative of tricyclic antidepressant toxicity.13 '],
             'nortriptyline': ['The oral LD50 of nortriptyline in the rat is 405 mg/kg.22Symptoms of overdose with nortriptyline include cardiac arrhythmias, severe hypotension, shock, congestive heart failure, pulmonary edema, convulsions, coma, and CNS depression.18,20 Changes in the electrocardiogram, particularly in QRS segment, may be indicative of tricyclic antidepressant toxicity.13 ',
              'The oral LD50 of nortriptyline in the rat is 405 mg/kg.22Symptoms of overdose with nortriptyline include cardiac arrhythmias, severe hypotension, shock, congestive heart failure, pulmonary edema, convulsions, coma, and CNS depression.18,20 Changes in the electrocardiogram, particularly in QRS segment, may be indicative of tricyclic antidepressant toxicity.13 '],
             'nifedical xl (nifedipine)': ['The oral LD50 in rats is 1022mg/kg and in mice is 202mg/kg.16',
              'The oral LD50 in rats is 1022mg/kg and in mice is 202mg/kg.16'],
             'clonazepam': ['Benzodiazepines like clonazepam commonly cause drowsiness, ataxia, dysarthria, and nystagmus. Overdose with clonazepam is generally not life-threatening if the drug is taken alone, but may lead to areflexia, apnea, hypotension, cardiorespiratory depression, and coma. Coma, if it does occur, usually lasts a few hours but it can become more protracted and cyclical, especially in elderly patients. Increased frequency of seizures may occur in patients at supratherapeutic plasma concentrations. Benzodiazepine respiratory depressant effects are more serious when compounded in patients with respiratory disease.',
              'Benzodiazepines like clonazepam commonly cause drowsiness, ataxia, dysarthria, and nystagmus. Overdose with clonazepam is generally not life-threatening if the drug is taken alone, but may lead to areflexia, apnea, hypotension, cardiorespiratory depression, and coma. Coma, if it does occur, usually lasts a few hours but it can become more protracted and cyclical, especially in elderly patients. Increased frequency of seizures may occur in patients at supratherapeutic plasma concentrations. Benzodiazepine respiratory depressant effects are more serious when compounded in patients with respiratory disease.'],
             'metronidazole': ['LD50 information', 'LD50 information'],
             'etodolac': ['Selective COX-2 inhibitors have been associated with increased risk of serious cardiovascular events (e.g. myocardial infarction, stroke) in some patients. Current data is insufficient to assess the cardiovascular risk of etodolac. Etodolac may increase blood pressure and/or cause fluid retention and edema. Risk of GI toxicity including bleeding, ulceration and perforation. Risk of direct renal injury, including renal papillary necrosis. Anaphylactoid and serious skin reactions (e.g. exfoliative dermatitis, Stevens-Johnson syndrome, toxic epidermal necrolysis) have been reported. Common adverse events include abdominal pain, constipation, diarrhea, dyspepsia, flatulence, GI bleeding, GI perforation, nausea, peptic ulcer, vomiting, renal function abnormalities, anemia, dizziness, edema, liver function test abnormalities, headache, prolonged bleeding time, pruritus, rash, tinnitus. Symptoms of overdose include lethargy, drowsiness, nausea, vomiting, and epigastric pain.',
              'Selective COX-2 inhibitors have been associated with increased risk of serious cardiovascular events (e.g. myocardial infarction, stroke) in some patients. Current data is insufficient to assess the cardiovascular risk of etodolac. Etodolac may increase blood pressure and/or cause fluid retention and edema. Risk of GI toxicity including bleeding, ulceration and perforation. Risk of direct renal injury, including renal papillary necrosis. Anaphylactoid and serious skin reactions (e.g. exfoliative dermatitis, Stevens-Johnson syndrome, toxic epidermal necrolysis) have been reported. Common adverse events include abdominal pain, constipation, diarrhea, dyspepsia, flatulence, GI bleeding, GI perforation, nausea, peptic ulcer, vomiting, renal function abnormalities, anemia, dizziness, edema, liver function test abnormalities, headache, prolonged bleeding time, pruritus, rash, tinnitus. Symptoms of overdose include lethargy, drowsiness, nausea, vomiting, and epigastric pain.'],
             'ketoconazole': ['Symptoms of overdose include acute liver injury, which may include both hepatocellular and cholestatic injury, accompanied by anorexia, fatigue, nausea, and jaundice.14,7 In case of overdose, gastric lavage with activated charcoal may be used if within one hour of ketoconazole ingestion otherwise provide supportive care.Label,12 If the patient shows signs of adrenal insufficiency, administer 100 mg hydrocortisone once together with saline and glucose infusion and monitor the patient closely. Blood pressure and fluid and electrolyte balance should be monitored over the next few days.12',
              'Symptoms of overdose include acute liver injury, which may include both hepatocellular and cholestatic injury, accompanied by anorexia, fatigue, nausea, and jaundice.14,7 In case of overdose, gastric lavage with activated charcoal may be used if within one hour of ketoconazole ingestion otherwise provide supportive care.Label,12 If the patient shows signs of adrenal insufficiency, administer 100 mg hydrocortisone once together with saline and glucose infusion and monitor the patient closely. Blood pressure and fluid and electrolyte balance should be monitored over the next few days.12'],
             'oxaprozin': ['Oral, mouse: LD50 = 1210 mg/kg; Oral, rabbit: LD50 = 172 mg/kg; Oral, rat: LD50 = 4470 mg/kg',
              'Oral, mouse: LD50 = 1210 mg/kg; Oral, rabbit: LD50 = 172 mg/kg; Oral, rat: LD50 = 4470 mg/kg'],
             'diclofenac potassium': ['Symptoms of overdose include lethargy, drowsiness, nausea, vomiting, and epigastric pain, and gastrointestinal bleeding.Label Hypertension, acute renal failure, respiratory depression and coma occur rarely. In case of overdose, provide supportive care and consider inducing emesis and administering activated charcoal if overdose occurred less than 4 hours prior.',
              'Symptoms of overdose include lethargy, drowsiness, nausea, vomiting, and epigastric pain, and gastrointestinal bleeding.Label Hypertension, acute renal failure, respiratory depression and coma occur rarely. In case of overdose, provide supportive care and consider inducing emesis and administering activated charcoal if overdose occurred less than 4 hours prior.'],
             'nystatin': ['The oral LD50 in rats is 10 g/kg.11 There have been no reports of serious toxic effects following overdosage of nystatin - doses in excess of five million units daily have resulted in nausea and gastrointestinal upset with no other associated effects.8',
              'The oral LD50 in rats is 10 g/kg.11 There have been no reports of serious toxic effects following overdosage of nystatin - doses in excess of five million units daily have resulted in nausea and gastrointestinal upset with no other associated effects.8'],
             'brione hydrochloride': ['Not Available', 'Not Available'],
             'ecnaprosyn': ['', ''],
             'nabumetone': ['LD50 Values', 'LD50 Values'],
             'nifediac cc (nifedipine)': ['The oral LD50 in rats is 1022mg/kg and in mice is 202mg/kg.16',
              'The oral LD50 in rats is 1022mg/kg and in mice is 202mg/kg.16'],
             'nefazodone hydrochloride': ['Cases of life-threatening hepatic failure have been reported in patients treated with nefazodone.',
              'Cases of life-threatening hepatic failure have been reported in patients treated with nefazodone.'],
             'diclofenac sodium': ['Symptoms of overdose include lethargy, drowsiness, nausea, vomiting, and epigastric pain, and gastrointestinal bleeding.Label Hypertension, acute renal failure, respiratory depression and coma occur rarely. In case of overdose, provide supportive care and consider inducing emesis and administering activated charcoal if overdose occurred less than 4 hours prior.',
              'Symptoms of overdose include lethargy, drowsiness, nausea, vomiting, and epigastric pain, and gastrointestinal bleeding.Label Hypertension, acute renal failure, respiratory depression and coma occur rarely. In case of overdose, provide supportive care and consider inducing emesis and administering activated charcoal if overdose occurred less than 4 hours prior.'],
             'enalapril maleate and hydrochlorothiazide': ['LD50 and Overdose',
              'LD50 and Overdose'],
             'metformin': ['Metformin (hydrochloride) toxicity data: ',
              'Metformin (hydrochloride) toxicity data: '],
             'metformin hydrochloride': ['Metformin (hydrochloride) toxicity data: ',
              'Metformin (hydrochloride) toxicity data: '],
             'metformin hydrochloride leets': ['Metformin (hydrochloride) toxicity data: ',
              'Metformin (hydrochloride) toxicity data: '],
             'quinapril': ['The oral LD50 in rats is 3541mg/kg and in mice is 1739mg/kg.8',
              'The oral LD50 in rats is 3541mg/kg and in mice is 1739mg/kg.8'],
             'sotalol hydrochloride': ['Patients experiencing an overdose may present with bradycardia, congestive heart failure, hypotension, bronchospasm, and hypoglycemia.Label Larger intentional overdoses may present as hypotension, bradycardia, cardiac asystole, prolonged QT interval, torsade de pointes, ventricular tachycardia, and premature ventricular complexes.Label Stop administering sotalol and observe the patient until the QT interval returns to normal and the heart rate rises above 50 beats per minute.Label Hemodialysis may help lower plasma concentrations of sotalol as it is not bound to plasma proteins.Label Bradycardia and cardiac asystole may be treated with atropine, other anticholinergic drugs, beta adrenergic agonists, or transvenous cardiac pacing.Label. Second or third degree heart block may be treated with a transvenous cardiac pacemaker.Label Hypotension may be treated with epinephrine or norepinephrine.Label Bronchospasm may be treated with aminophylline or a beta-2 agonist, possibly at higher than normal doses.Label Torsade de pointes may be treated with DC cardioversion, transvenous cardiac pacing, epinephrine, or magnesium sulfate.Label',
              'Patients experiencing an overdose may present with bradycardia, congestive heart failure, hypotension, bronchospasm, and hypoglycemia.Label Larger intentional overdoses may present as hypotension, bradycardia, cardiac asystole, prolonged QT interval, torsade de pointes, ventricular tachycardia, and premature ventricular complexes.Label Stop administering sotalol and observe the patient until the QT interval returns to normal and the heart rate rises above 50 beats per minute.Label Hemodialysis may help lower plasma concentrations of sotalol as it is not bound to plasma proteins.Label Bradycardia and cardiac asystole may be treated with atropine, other anticholinergic drugs, beta adrenergic agonists, or transvenous cardiac pacing.Label. Second or third degree heart block may be treated with a transvenous cardiac pacemaker.Label Hypotension may be treated with epinephrine or norepinephrine.Label Bronchospasm may be treated with aminophylline or a beta-2 agonist, possibly at higher than normal doses.Label Torsade de pointes may be treated with DC cardioversion, transvenous cardiac pacing, epinephrine, or magnesium sulfate.Label'],
             'cefprozil': ['Single 5000 mg/kg oral doses of cefprozil caused no mortality or signs of toxicity in adult, weaning or neonatal rats, or adult mice. A single oral dose of 3000 mg/kg caused diarrhea and loss of appetite in cynomolgus monkeys, but no mortality.',
              'Single 5000 mg/kg oral doses of cefprozil caused no mortality or signs of toxicity in adult, weaning or neonatal rats, or adult mice. A single oral dose of 3000 mg/kg caused diarrhea and loss of appetite in cynomolgus monkeys, but no mortality.'],
             'cefaclor': ['Symptoms of overdose include diarrhea, nausea, stomach upset, and vomiting.',
              'Symptoms of overdose include diarrhea, nausea, stomach upset, and vomiting.'],
             'etodolac xr': ['Selective COX-2 inhibitors have been associated with increased risk of serious cardiovascular events (e.g. myocardial infarction, stroke) in some patients. Current data is insufficient to assess the cardiovascular risk of etodolac. Etodolac may increase blood pressure and/or cause fluid retention and edema. Risk of GI toxicity including bleeding, ulceration and perforation. Risk of direct renal injury, including renal papillary necrosis. Anaphylactoid and serious skin reactions (e.g. exfoliative dermatitis, Stevens-Johnson syndrome, toxic epidermal necrolysis) have been reported. Common adverse events include abdominal pain, constipation, diarrhea, dyspepsia, flatulence, GI bleeding, GI perforation, nausea, peptic ulcer, vomiting, renal function abnormalities, anemia, dizziness, edema, liver function test abnormalities, headache, prolonged bleeding time, pruritus, rash, tinnitus. Symptoms of overdose include lethargy, drowsiness, nausea, vomiting, and epigastric pain.',
              'Selective COX-2 inhibitors have been associated with increased risk of serious cardiovascular events (e.g. myocardial infarction, stroke) in some patients. Current data is insufficient to assess the cardiovascular risk of etodolac. Etodolac may increase blood pressure and/or cause fluid retention and edema. Risk of GI toxicity including bleeding, ulceration and perforation. Risk of direct renal injury, including renal papillary necrosis. Anaphylactoid and serious skin reactions (e.g. exfoliative dermatitis, Stevens-Johnson syndrome, toxic epidermal necrolysis) have been reported. Common adverse events include abdominal pain, constipation, diarrhea, dyspepsia, flatulence, GI bleeding, GI perforation, nausea, peptic ulcer, vomiting, renal function abnormalities, anemia, dizziness, edema, liver function test abnormalities, headache, prolonged bleeding time, pruritus, rash, tinnitus. Symptoms of overdose include lethargy, drowsiness, nausea, vomiting, and epigastric pain.'],
             'fexofenadinepse hr': ['', ''],
             'acitretin': ['Oral, rat: LD50 = >4000 mg/kg. Symptoms of overdose include headache and vertigo.',
              'Oral, rat: LD50 = >4000 mg/kg. Symptoms of overdose include headache and vertigo.'],
             'penicillin v potassium': ['The oral LD50 is >1040 mg/kg in rats. Nausea, vomiting, black hairy tongue, and epigastric distress are common reactions to oral penicillins.Label In rare cases, neuromuscular sensitivity and seizures may be seen with antibiotics and supportive treatments are advised and further drug absorption should be limited through induced emesis or gastric lavage, followed by administration of activated charcoal.10 Severe hypersensitivity reactions, often leading to death, have been reported with penicillin therapies.Label Although phenoxymethylpenicillin was shown to be excreted in human breast milk, the use of this drug in pregnant or nursing women is regarded generally safe.9',
              'The oral LD50 is >1040 mg/kg in rats. Nausea, vomiting, black hairy tongue, and epigastric distress are common reactions to oral penicillins.Label In rare cases, neuromuscular sensitivity and seizures may be seen with antibiotics and supportive treatments are advised and further drug absorption should be limited through induced emesis or gastric lavage, followed by administration of activated charcoal.10 Severe hypersensitivity reactions, often leading to death, have been reported with penicillin therapies.Label Although phenoxymethylpenicillin was shown to be excreted in human breast milk, the use of this drug in pregnant or nursing women is regarded generally safe.9'],
             'neomycin sulfate': ['The oral LD50 of neomycin sulfate in mouse is > 8 g/kg. The subcutaneous LD50 is 200 mg/kg in rat and 190 mg/kg in mouse. The intraperitoneal LD50 in mouse is 305 mg/kg. The oral Lowest published toxic dose (TDLo) in woman is 12600 mg/kg/7D.8',
              'The oral LD50 of neomycin sulfate in mouse is > 8 g/kg. The subcutaneous LD50 is 200 mg/kg in rat and 190 mg/kg in mouse. The intraperitoneal LD50 in mouse is 305 mg/kg. The oral Lowest published toxic dose (TDLo) in woman is 12600 mg/kg/7D.8'],
             'warfarin sodium': ['LD50 Values', 'LD50 Values'],
             'pioglitazone hydrochloride': ['The oral TDLo observed in mice is 24 mg/kg for 4 days and for rats is 3 mg/kg for 6 days.8',
              'The oral TDLo observed in mice is 24 mg/kg for 4 days and for rats is 3 mg/kg for 6 days.8'],
             'nifedipine': ['The oral LD50 in rats is 1022mg/kg and in mice is 202mg/kg.16',
              'The oral LD50 in rats is 1022mg/kg and in mice is 202mg/kg.16'],
             'fenofibrate': ['The oral LD50 in rats is >2g/kg and in mice is 1600mg/kg.13 The oral TDLO in rats is 9mg/kg.13',
              'The oral LD50 in rats is >2g/kg and in mice is 1600mg/kg.13 The oral TDLO in rats is 9mg/kg.13'],
             'doxazosin': ['LD50 information', 'LD50 information'],
             'tolmetin sodium': ['Symptoms of overdose include lethargy, drowsiness, nausea, vomiting, and epigastric pain.',
              'Symptoms of overdose include lethargy, drowsiness, nausea, vomiting, and epigastric pain.'],
             'hydrochlorothiazide': ['The oral LD50 of hydrochlorothiazide is >10g/kg in mice and rats.10,11',
              'The oral LD50 of hydrochlorothiazide is >10g/kg in mice and rats.10,11'],
             'nitrofurantoin s': ['Symptoms of overdose include vomiting.8,9,10 In case of overdose, induce vomiting if it has not already occurred and increase fluid intake to promote urination.8,9,10 In extreme cases, nitrofurantoin can be removed from circulation by dialysis.8,9,10',
              'Symptoms of overdose include vomiting.8,9,10 In case of overdose, induce vomiting if it has not already occurred and increase fluid intake to promote urination.8,9,10 In extreme cases, nitrofurantoin can be removed from circulation by dialysis.8,9,10'],
             'nitrofurantoin': ['Symptoms of overdose include vomiting.8,9,10 In case of overdose, induce vomiting if it has not already occurred and increase fluid intake to promote urination.8,9,10 In extreme cases, nitrofurantoin can be removed from circulation by dialysis.8,9,10',
              'Symptoms of overdose include vomiting.8,9,10 In case of overdose, induce vomiting if it has not already occurred and increase fluid intake to promote urination.8,9,10 In extreme cases, nitrofurantoin can be removed from circulation by dialysis.8,9,10'],
             'trilosprintec triphasic regimen': ['', ''],
             'trimethoprim': ['The oral LD50 in mice and rats is 2764 mg/kg and >5300 mg/kg, respectively.18',
              'The oral LD50 in mice and rats is 2764 mg/kg and >5300 mg/kg, respectively.18'],
             'metoclopramide': ['The rat oral LD50 of metoclopramide is 750 mg/kg.20',
              'The rat oral LD50 of metoclopramide is 750 mg/kg.20'],
             'sucralfate': ['Overdose', 'Overdose'],
             'cephalexin': ['Symptoms of overdose include blood in the urine, diarrhea, nausea, upper abdominal pain, and vomiting.Label An overdose is generally managed through supportive treatment as diuresis, dialysis, hemodialysis, and charcoal hemoperfusion are not well studied in this case.Label',
              'Symptoms of overdose include blood in the urine, diarrhea, nausea, upper abdominal pain, and vomiting.Label An overdose is generally managed through supportive treatment as diuresis, dialysis, hemodialysis, and charcoal hemoperfusion are not well studied in this case.Label'],
             'amoxicillin': ['Patients experiencing an overdose may present with hematuria, oliguria, abdominal pain, acute renal failure, vomiting, diarrhea, rash, hyperactivity, and drowsiness.7,14 Treat overdose with symptomatic and supportive treatment, which may include emesis or hemodialysis.14',
              'Patients experiencing an overdose may present with hematuria, oliguria, abdominal pain, acute renal failure, vomiting, diarrhea, rash, hyperactivity, and drowsiness.7,14 Treat overdose with symptomatic and supportive treatment, which may include emesis or hemodialysis.14'],
             'amoxicillin (chewable)': ['Patients experiencing an overdose may present with hematuria, oliguria, abdominal pain, acute renal failure, vomiting, diarrhea, rash, hyperactivity, and drowsiness.7,14 Treat overdose with symptomatic and supportive treatment, which may include emesis or hemodialysis.14',
              'Patients experiencing an overdose may present with hematuria, oliguria, abdominal pain, acute renal failure, vomiting, diarrhea, rash, hyperactivity, and drowsiness.7,14 Treat overdose with symptomatic and supportive treatment, which may include emesis or hemodialysis.14'],
             'amoxicillin chw': ['Patients experiencing an overdose may present with hematuria, oliguria, abdominal pain, acute renal failure, vomiting, diarrhea, rash, hyperactivity, and drowsiness.7,14 Treat overdose with symptomatic and supportive treatment, which may include emesis or hemodialysis.14',
              'Patients experiencing an overdose may present with hematuria, oliguria, abdominal pain, acute renal failure, vomiting, diarrhea, rash, hyperactivity, and drowsiness.7,14 Treat overdose with symptomatic and supportive treatment, which may include emesis or hemodialysis.14'],
             'amoxicillin and clavulanate potassium': ['Patients experiencing an overdose may present with hematuria, oliguria, abdominal pain, acute renal failure, vomiting, diarrhea, rash, hyperactivity, and drowsiness.7,14 Treat overdose with symptomatic and supportive treatment, which may include emesis or hemodialysis.14',
              'Patients experiencing an overdose may present with hematuria, oliguria, abdominal pain, acute renal failure, vomiting, diarrhea, rash, hyperactivity, and drowsiness.7,14 Treat overdose with symptomatic and supportive treatment, which may include emesis or hemodialysis.14'],
             'amoxicilclavulanate': ['', ''],
             'amoxk clav': ['Patients experiencing an overdose may present with hematuria, oliguria, abdominal pain, acute renal failure, vomiting, diarrhea, rash, hyperactivity, and drowsiness.7,14 Treat overdose with symptomatic and supportive treatment, which may include emesis or hemodialysis.14',
              'Patients experiencing an overdose may present with hematuria, oliguria, abdominal pain, acute renal failure, vomiting, diarrhea, rash, hyperactivity, and drowsiness.7,14 Treat overdose with symptomatic and supportive treatment, which may include emesis or hemodialysis.14'],
             'famotidine': ['The oral LD50 is 4049 mg/kg in rats and 4686 mg/kg in mice. The subcutaneous LD50 is 800 mg/kg in rats and mice. The lowest published toxic dose (TDLo) in man following oral administration is 4 mg/kg/7D.7 ',
              'The oral LD50 is 4049 mg/kg in rats and 4686 mg/kg in mice. The subcutaneous LD50 is 800 mg/kg in rats and mice. The lowest published toxic dose (TDLo) in man following oral administration is 4 mg/kg/7D.7 '],
             'cyproheptadine hcl': ['Overdosage with cyproheptadine is likely to result in significant sedation - although paradoxical stimulation has been noted in pediatric patients - and anticholinergic adverse effects such as dry mouth and flushing.8 Most patients appear to recover without incident, as a review of cyproheptadine overdose cases in Hong Kong found the majority of patients had no or mild symptoms following intentional overdose.6',
              'Overdosage with cyproheptadine is likely to result in significant sedation - although paradoxical stimulation has been noted in pediatric patients - and anticholinergic adverse effects such as dry mouth and flushing.8 Most patients appear to recover without incident, as a review of cyproheptadine overdose cases in Hong Kong found the majority of patients had no or mild symptoms following intentional overdose.6'],
             'methyldopa': ['The lowest published toxic dose via oral route is 44 gm/kg/3Y (intermittent) in a female. Oral LD50 is 5000 mg/kg in rats and 5300 mg/kg in mice. Intraperitoneal LD50 is 300 mg/kg in rats and 150 mg/kg in mice.14',
              'The lowest published toxic dose via oral route is 44 gm/kg/3Y (intermittent) in a female. Oral LD50 is 5000 mg/kg in rats and 5300 mg/kg in mice. Intraperitoneal LD50 is 300 mg/kg in rats and 150 mg/kg in mice.14'],
             'quinine sulfate': ['Quinine is a documented causative agent of drug induced thrombocytopenia (DIT). Thrombocytopenia is a low amount of platelets in the blood. Quinine induces production of antibodies against glycoprotein (GP) Ib-IX complex in the majority of cases of DIT, or more rarely, the platelet-glycoprotein complex GPIIb-IIIa. Increased antibodies against these complexes increases platelet clearance, leading to the observed thrombocytopenia. ',
              'Quinine is a documented causative agent of drug induced thrombocytopenia (DIT). Thrombocytopenia is a low amount of platelets in the blood. Quinine induces production of antibodies against glycoprotein (GP) Ib-IX complex in the majority of cases of DIT, or more rarely, the platelet-glycoprotein complex GPIIb-IIIa. Increased antibodies against these complexes increases platelet clearance, leading to the observed thrombocytopenia. '],
             'clozapine': ['Clozapine carries a black-box warning for agranulocytosis.',
              'Clozapine carries a black-box warning for agranulocytosis.'],
             'aspirin and dipyridamole': ['Lethal doses', 'Lethal doses'],
             'verapamil hcl': ["Verapamil's reported oral TDLo is 14.4 mg/kg in women and 3.429 mg/kg in men.21 The oral LD50 is 150 mg/kg in rats and 163 mg/kg in mice.21",
              "Verapamil's reported oral TDLo is 14.4 mg/kg in women and 3.429 mg/kg in men.21 The oral LD50 is 150 mg/kg in rats and 163 mg/kg in mice.21"],
             'rasagiline': ['Signs and symptoms of overdosage may include, alone or in combination, any of the following: drowsiness, dizziness, faintness, irritability, hyperactivity, agitation, severe headache, hallucinations, trismus, opisthotonos, convulsions, and coma; rapid and irregular pulse, hypertension, hypotension and vascular collapse; precordial pain, respiratory depression and failure, hyperpyrexia, diaphoresis, and cool, clammy skin.',
              'Signs and symptoms of overdosage may include, alone or in combination, any of the following: drowsiness, dizziness, faintness, irritability, hyperactivity, agitation, severe headache, hallucinations, trismus, opisthotonos, convulsions, and coma; rapid and irregular pulse, hypertension, hypotension and vascular collapse; precordial pain, respiratory depression and failure, hyperpyrexia, diaphoresis, and cool, clammy skin.'],
             'clozapine orally disintegrating': ['Clozapine carries a black-box warning for agranulocytosis.',
              'Clozapine carries a black-box warning for agranulocytosis.'],
             'onceaweek risedronate sodium': ['', ''],
             'risedronate sodium': ['In clinical trials, over 10% of patients experienced back pain, arthralgia, abdominal pain, and dyspepsiaLabel. Less commonly, patients experience angioedema, generalized rash, bullous skin reactions, iritis, and uveitisLabel.',
              'In clinical trials, over 10% of patients experienced back pain, arthralgia, abdominal pain, and dyspepsiaLabel. Less commonly, patients experience angioedema, generalized rash, bullous skin reactions, iritis, and uveitisLabel.'],
             'jinteli mcg': ['Female patients experiencing and overdose may present with withdrawal bleeding, nausea, vomiting, breast tenderness, abdominal pain, drowsiness, and fatigue.15,25,16,17,24,18,19,20,21,22,23 Overdose should be treated with symptomatic and supportive care including monitoring for potassium concentrations, sodium concentrations, and signs of metabolic acidosis.15,25,16,17,24,18,19,20,21,22,23',
              'Female patients experiencing and overdose may present with withdrawal bleeding, nausea, vomiting, breast tenderness, abdominal pain, drowsiness, and fatigue.15,25,16,17,24,18,19,20,21,22,23 Overdose should be treated with symptomatic and supportive care including monitoring for potassium concentrations, sodium concentrations, and signs of metabolic acidosis.15,25,16,17,24,18,19,20,21,22,23'],
             'dicloxacillin sodium': ['Oral LD50 in rat is 3579 mg/kg. Symptoms of overexposure include irritation, rash, labored breathing, hives, itching, wheezing, nausea, chills, and fever.',
              'Oral LD50 in rat is 3579 mg/kg. Symptoms of overexposure include irritation, rash, labored breathing, hives, itching, wheezing, nausea, chills, and fever.'],
             'dicloxacillin sod s': ['Oral LD50 in rat is 3579 mg/kg. Symptoms of overexposure include irritation, rash, labored breathing, hives, itching, wheezing, nausea, chills, and fever.',
              'Oral LD50 in rat is 3579 mg/kg. Symptoms of overexposure include irritation, rash, labored breathing, hives, itching, wheezing, nausea, chills, and fever.'],
             'disopyramide phosphate': ['LD50=580 mg/kg in rats',
              'LD50=580 mg/kg in rats'],
             'camrese': ['Female patients experiencing and overdose may present with withdrawal bleeding, nausea, vomiting, breast tenderness, abdominal pain, drowsiness, and fatigue.15,25,16,17,24,18,19,20,21,22,23 Overdose should be treated with symptomatic and supportive care including monitoring for potassium concentrations, sodium concentrations, and signs of metabolic acidosis.15,25,16,17,24,18,19,20,21,22,23',
              'Female patients experiencing and overdose may present with withdrawal bleeding, nausea, vomiting, breast tenderness, abdominal pain, drowsiness, and fatigue.15,25,16,17,24,18,19,20,21,22,23 Overdose should be treated with symptomatic and supportive care including monitoring for potassium concentrations, sodium concentrations, and signs of metabolic acidosis.15,25,16,17,24,18,19,20,21,22,23'],
             'cefdinir': ['LD50 information\nOral LD50 of cefdinir in the rat is >2000 mg/kg.15',
              'LD50 information\nOral LD50 of cefdinir in the rat is >2000 mg/kg.15'],
             'minocycline hydrochloride': ['The oral LD50 in rats is 2380mg/kg and in mice is 3600mg/kg.17 The intraperitoneal LD50 in rats is 331mg/kg and in mice is 299mg/kg.17 The subcutaneous LD50 in rats is 1700mg/kg and in mice is 2290mg/kg.17',
              'The oral LD50 in rats is 2380mg/kg and in mice is 3600mg/kg.17 The intraperitoneal LD50 in rats is 331mg/kg and in mice is 299mg/kg.17 The subcutaneous LD50 in rats is 1700mg/kg and in mice is 2290mg/kg.17'],
             'clindamycin hydrochloride': ['The oral LD50 in mice and rats is 2540 mg/kg and 2190 mg/kg, respectively.19',
              'The oral LD50 in mice and rats is 2540 mg/kg and 2190 mg/kg, respectively.19'],
             'ketoprofen': ['LD50=62.4 mg/kg (rat, oral).\n',
              'LD50=62.4 mg/kg (rat, oral).\n'],
             'cefadroxil': ['Nausea, vomiting, diarrhoea, allergic rashes may occur',
              'Nausea, vomiting, diarrhoea, allergic rashes may occur'],
             'reclipsen': ['Female patients experiencing and overdose may present with withdrawal bleeding, nausea, vomiting, breast tenderness, abdominal pain, drowsiness, and fatigue.15,25,16,17,24,18,19,20,21,22,23 Overdose should be treated with symptomatic and supportive care including monitoring for potassium concentrations, sodium concentrations, and signs of metabolic acidosis.15,25,16,17,24,18,19,20,21,22,23',
              'Female patients experiencing and overdose may present with withdrawal bleeding, nausea, vomiting, breast tenderness, abdominal pain, drowsiness, and fatigue.15,25,16,17,24,18,19,20,21,22,23 Overdose should be treated with symptomatic and supportive care including monitoring for potassium concentrations, sodium concentrations, and signs of metabolic acidosis.15,25,16,17,24,18,19,20,21,22,23'],
             'zenchent': ['Female patients experiencing and overdose may present with withdrawal bleeding, nausea, vomiting, breast tenderness, abdominal pain, drowsiness, and fatigue.15,25,16,17,24,18,19,20,21,22,23 Overdose should be treated with symptomatic and supportive care including monitoring for potassium concentrations, sodium concentrations, and signs of metabolic acidosis.15,25,16,17,24,18,19,20,21,22,23',
              'Female patients experiencing and overdose may present with withdrawal bleeding, nausea, vomiting, breast tenderness, abdominal pain, drowsiness, and fatigue.15,25,16,17,24,18,19,20,21,22,23 Overdose should be treated with symptomatic and supportive care including monitoring for potassium concentrations, sodium concentrations, and signs of metabolic acidosis.15,25,16,17,24,18,19,20,21,22,23'],
             'atomoxetine': ['There is limited clinical trial experience with atomoxetine overdose. During postmarketing, there have been fatalities reported involving a mixed ingestion overdose of atomoxetine capsules and at least one other drug. There have been no reports of death involving overdose of atomoxetine capsules alone, including intentional overdoses at amounts up to 1400 mg. In some cases of overdose involving atomoxetine, seizures have been reported. The most commonly reported symptoms accompanying acute and chronic overdoses of atomoxetine capsules were gastrointestinal symptoms, somnolence, dizziness, tremor, and abnormal behavior. Hyperactivity and agitation have also been reported. Signs and symptoms consistent with mild to moderate sympathetic nervous system activation (e.g., tachycardia, blood pressure increased, mydriasis, dry mouth) have also been observed. Most events were mild to moderate. Less commonly, there have been reports of QT prolongation and mental changes, including disorientation and hallucinations. If symptoms of overdose are suspected, a Certified Poison Control Center should be consulted for up to date guidance and advice. Because atomoxetine is highly protein-bound, dialysis is not likely to be useful in the treatment of overdose.18',
              'There is limited clinical trial experience with atomoxetine overdose. During postmarketing, there have been fatalities reported involving a mixed ingestion overdose of atomoxetine capsules and at least one other drug. There have been no reports of death involving overdose of atomoxetine capsules alone, including intentional overdoses at amounts up to 1400 mg. In some cases of overdose involving atomoxetine, seizures have been reported. The most commonly reported symptoms accompanying acute and chronic overdoses of atomoxetine capsules were gastrointestinal symptoms, somnolence, dizziness, tremor, and abnormal behavior. Hyperactivity and agitation have also been reported. Signs and symptoms consistent with mild to moderate sympathetic nervous system activation (e.g., tachycardia, blood pressure increased, mydriasis, dry mouth) have also been observed. Most events were mild to moderate. Less commonly, there have been reports of QT prolongation and mental changes, including disorientation and hallucinations. If symptoms of overdose are suspected, a Certified Poison Control Center should be consulted for up to date guidance and advice. Because atomoxetine is highly protein-bound, dialysis is not likely to be useful in the treatment of overdose.18'],
             'indomethacin': ['Acute oral LD50 is 2.42 mg/kg in rats and 13 mg/kg in mice.MSDS The oral LD50 of indomethacin in mice and rats (based on 14-day mortality response) was 50 and 12 mg/kg, respectively.13 ',
              'Acute oral LD50 is 2.42 mg/kg in rats and 13 mg/kg in mice.MSDS The oral LD50 of indomethacin in mice and rats (based on 14-day mortality response) was 50 and 12 mg/kg, respectively.13 '],
             'prazosin hydrochloride': ['D50: Oral TDLO', 'DLO'],
             'prazosin hcl': ['D50: Oral TDLO', 'DLO'],
             'prazosin s': ['D50: Oral TDLO', 'DLO'],
             'bumetanide': ['Overdosage can lead to acute profound water loss, volume and electrolyte depletion, dehydration, reduction of blood volume and circulatory collapse with a possibility of vascular thrombosis and embolism. Electrolyte depletion may be manifested by weakness, dizziness, mental confusion, anorexia, lethargy, vomiting and cramps. Treatment consists of replacement of fluid and electrolyte losses by careful monitoring of the urine and electrolyte output and serum electrolyte levels.',
              'Overdosage can lead to acute profound water loss, volume and electrolyte depletion, dehydration, reduction of blood volume and circulatory collapse with a possibility of vascular thrombosis and embolism. Electrolyte depletion may be manifested by weakness, dizziness, mental confusion, anorexia, lethargy, vomiting and cramps. Treatment consists of replacement of fluid and electrolyte losses by careful monitoring of the urine and electrolyte output and serum electrolyte levels.'],
             'nadolol': ['The oral LD50 in mice is 4500mg/kg.10',
              'The oral LD50 in mice is 4500mg/kg.10'],
             'terazosin hydrochloride': ['In the event of an overdose, patients may experience hypotensionLabel. Blood pressure and heart rate should be controlled by having the patient lie down or by treating with volume expanders or if necessary vasopressorsLabel. Patients should be monitored for renal functionLabel. Because terazosin is highly protein bound, dialysis is unlikely to provide benefit to overdosing patientsLabel.',
              'In the event of an overdose, patients may experience hypotensionLabel. Blood pressure and heart rate should be controlled by having the patient lie down or by treating with volume expanders or if necessary vasopressorsLabel. Patients should be monitored for renal functionLabel. Because terazosin is highly protein bound, dialysis is unlikely to provide benefit to overdosing patientsLabel.'],
             'fluoxetine': ['In a report that included 234 fluoxetine overdose cases, it was concluded that symptoms resulting from fluoxetine overdose were generally minor and short in duration.5 The most common overdose adverse effects included drowsiness, tremor, tachycardia, nausea and vomiting, and providing the patient with aggressive supportive care was the recommended intervention.45',
              'In a report that included 234 fluoxetine overdose cases, it was concluded that symptoms resulting from fluoxetine overdose were generally minor and short in duration.5 The most common overdose adverse effects included drowsiness, tremor, tachycardia, nausea and vomiting, and providing the patient with aggressive supportive care was the recommended intervention.45'],
             'fluoxetine hydrochloride': ['In a report that included 234 fluoxetine overdose cases, it was concluded that symptoms resulting from fluoxetine overdose were generally minor and short in duration.5 The most common overdose adverse effects included drowsiness, tremor, tachycardia, nausea and vomiting, and providing the patient with aggressive supportive care was the recommended intervention.45',
              'In a report that included 234 fluoxetine overdose cases, it was concluded that symptoms resulting from fluoxetine overdose were generally minor and short in duration.5 The most common overdose adverse effects included drowsiness, tremor, tachycardia, nausea and vomiting, and providing the patient with aggressive supportive care was the recommended intervention.45'],
             'gabapentin': ["The oral TDLo of gabapentin in humans is 2.86 mg/kg and the LD50 in rats has been found to be >8000 mg/kg.21 Symptoms of overdose are consistent with the drug's adverse effect profile and involve CNS depression (e.g. dizziness, drowsiness, slurred speech, lethargy, loss of consciousness) and gastrointestinal symptoms such as diarrhea.18,17 Management of overdose should involve symptomatic and supportive treatment. Gabapentin can be removed by hemodialysis - this may be of benefit in some patients, such as those with impaired renal function.20",
              "The oral TDLo of gabapentin in humans is 2.86 mg/kg and the LD50 in rats has been found to be >8000 mg/kg.21 Symptoms of overdose are consistent with the drug's adverse effect profile and involve CNS depression (e.g. dizziness, drowsiness, slurred speech, lethargy, loss of consciousness) and gastrointestinal symptoms such as diarrhea.18,17 Management of overdose should involve symptomatic and supportive treatment. Gabapentin can be removed by hemodialysis - this may be of benefit in some patients, such as those with impaired renal function.20"],
             'citalopram': ['Oral (Human) LD: 56 mg/kg\nIntraperitoneal (Mouse) LD50: 179 mg/kg',
              'Oral (Human) LD: 56 mg/kg\nIntraperitoneal (Mouse) LD50: 179 mg/kg'],
             'olmesartan medoxomil amlodipine and hydrochlorothiazide': ['The reported LD50 of olmesartan in dogs was reported to be greater of 1500 mg/kg. Overdose is expressed as hypotension, tachycardia, and bradycardia when there is parasympathetic stimulation. In case of overdose, supportive treatment is recommended.Label',
              'The reported LD50 of olmesartan in dogs was reported to be greater of 1500 mg/kg. Overdose is expressed as hypotension, tachycardia, and bradycardia when there is parasympathetic stimulation. In case of overdose, supportive treatment is recommended.Label'],
             'tiagabine hydrochloride': ['mptoms most often accompanying tiagabine overdose, alone or in combination with other drugs, have included: seizures including status epilepticus in patients with and without underlying seizure disorders, nonconvulsive status epilepticus, coma, ataxia, confusion, somnolence, drowsiness, impaired speech, agitation, lethargy, myoclonus, spike wave stupor, tremors, disorientation, vomiting, hostility, and temporary paralysis. Respiratory depression was seen in a number of patients, including children, in the context of seizures.',
              'mptoms most often accompanying tiagabine overdose, alone or in combination with other drugs, have included: seizures including status epilepticus in patients with and without underlying seizure disorders, nonconvulsive status epilepticus, coma, ataxia, confusion, somnolence, drowsiness, impaired speech, agitation, lethargy, myoclonus, spike wave stupor, tremors, disorientation, vomiting, hostility, and temporary paralysis. Respiratory depression was seen in a number of patients, including children, in the context of seizures.'],
             'disulfiram': ['LD50=8.6g/kg (orally in rats). Symptoms of overdose include irritation, slight drowsiness, unpleasant taste, mild GI disturbances, and orthostatic hypotension.',
              'LD50=8.6g/kg (orally in rats). Symptoms of overdose include irritation, slight drowsiness, unpleasant taste, mild GI disturbances, and orthostatic hypotension.'],
             'pioglitazone hcl and metformin hcl': ['The oral TDLo observed in mice is 24 mg/kg for 4 days and for rats is 3 mg/kg for 6 days.8',
              'The oral TDLo observed in mice is 24 mg/kg for 4 days and for rats is 3 mg/kg for 6 days.8'],
             'atorvastatin calcium': ['The reported LD50 of oral atorvastatin in mice is higher than 5000 mg/kg.MSDS In cases of overdose with atorvastatin, there is reported symptoms of complicated breathing, jaundice, liver damage, dark urine, muscle pain, and seizures.38 In case of overdose, symptomatic treatment is recommended and due to the high plasma protein binding, hemodialysis is not expected to generate significant improvement.Label',
              'The reported LD50 of oral atorvastatin in mice is higher than 5000 mg/kg.MSDS In cases of overdose with atorvastatin, there is reported symptoms of complicated breathing, jaundice, liver damage, dark urine, muscle pain, and seizures.38 In case of overdose, symptomatic treatment is recommended and due to the high plasma protein binding, hemodialysis is not expected to generate significant improvement.Label'],
             'hydroxyzine hydrochloride': ['The oral LD50 is 840 mg/kg in rats and 400 mg/kg in mice.16 ',
              'The oral LD50 is 840 mg/kg in rats and 400 mg/kg in mice.16 '],
             'hydroxyzine hydrochlorinde': ['The oral LD50 is 840 mg/kg in rats and 400 mg/kg in mice.16 ',
              'The oral LD50 is 840 mg/kg in rats and 400 mg/kg in mice.16 '],
             'olanzapine': ['The toxicity symptoms of olanzapine are known to include somnolence, mydriasis, blurred vision, respiratory depression, hypotension, extrapyramidal symptoms and anticholinergic effects. The overdosage effects in children are generally associated with more significant side effects.7',
              'The toxicity symptoms of olanzapine are known to include somnolence, mydriasis, blurred vision, respiratory depression, hypotension, extrapyramidal symptoms and anticholinergic effects. The overdosage effects in children are generally associated with more significant side effects.7'],
             'pentoxifylline': ['Overdoses of pentoxifylline have been reported with symptoms including agitation, fever, flushing, hypotension, convulsions, somnolence, and loss of consciousness beginning 4-5 hours following ingestion and lasting up to 12 hours. Symptomatic treatment is recommended, specifically pertaining to maintaining proper respiration, blood pressure, and controlling convulsions. Activated charcoal may prove useful in absorbing excess pentoxifylline in overdose cases. Patients have recovered from overdose even at doses as high as 80 mg/kg.29',
              'Overdoses of pentoxifylline have been reported with symptoms including agitation, fever, flushing, hypotension, convulsions, somnolence, and loss of consciousness beginning 4-5 hours following ingestion and lasting up to 12 hours. Symptomatic treatment is recommended, specifically pertaining to maintaining proper respiration, blood pressure, and controlling convulsions. Activated charcoal may prove useful in absorbing excess pentoxifylline in overdose cases. Patients have recovered from overdose even at doses as high as 80 mg/kg.29'],
             'diltiazem er': ['Clinical Toxicity and Overdose',
              'Clinical Toxicity and Overdose'],
             'diltiazem xr': ['Clinical Toxicity and Overdose',
              'Clinical Toxicity and Overdose'],
             'benazepril hydrochloride': ['The most common adverse effects include headache, dizziness, fatigue, somnolence, postural dizziness, nausea, and coughLabel,4.',
              'The most common adverse effects include headache, dizziness, fatigue, somnolence, postural dizziness, nausea, and coughLabel,4.'],
             'alendronate sodium': ['In clinical studies, ≥3% of patients experience abdominal pain, acid regurgitation, constipation, diarrhea, dyspepsia, musculoskeletal pain, and nauseaLabel.',
              'In clinical studies, ≥3% of patients experience abdominal pain, acid regurgitation, constipation, diarrhea, dyspepsia, musculoskeletal pain, and nauseaLabel.'],
             'alendronate': ['In clinical studies, ≥3% of patients experience abdominal pain, acid regurgitation, constipation, diarrhea, dyspepsia, musculoskeletal pain, and nauseaLabel.',
              'In clinical studies, ≥3% of patients experience abdominal pain, acid regurgitation, constipation, diarrhea, dyspepsia, musculoskeletal pain, and nauseaLabel.'],
             'hydrocodone bitartrate and ibuprofen': ['Overdosage with hydrocodone presents as opioid intoxication including respiratory depression, somnolence, coma, skeletal muscle flaccidity, cold and clammy skin, constricted pupils, pulmonary edema, bradycardia, hypotension, partial or complete airway obstruction, atypical snoring, and death.15',
              'Overdosage with hydrocodone presents as opioid intoxication including respiratory depression, somnolence, coma, skeletal muscle flaccidity, cold and clammy skin, constricted pupils, pulmonary edema, bradycardia, hypotension, partial or complete airway obstruction, atypical snoring, and death.15'],
             'oxybutynin chloride': ['The acute oral LD50 in rats is 460 mg/kg.13',
              'The acute oral LD50 in rats is 460 mg/kg.13'],
             'moexipril hydrochloride and hydrochlorothiazide': ['Human overdoses of moexipril have not been reported. In case reports of overdoses with other ACE inhibitors, hypotension has been the principal adverse effect noted. Single oral doses of 2 g/kg moexipril were associated with significant lethality in mice. Rats, however, tolerated single oral doses of up to 3 g/kg. Common adverse effects include cough, dizziness, diarrhea, flu syndrome, fatigue, pharyngitis, flushing, rash, and myalgia',
              'Human overdoses of moexipril have not been reported. In case reports of overdoses with other ACE inhibitors, hypotension has been the principal adverse effect noted. Single oral doses of 2 g/kg moexipril were associated with significant lethality in mice. Rats, however, tolerated single oral doses of up to 3 g/kg. Common adverse effects include cough, dizziness, diarrhea, flu syndrome, fatigue, pharyngitis, flushing, rash, and myalgia'],
             'olanzapine orally disintegrating': ['The toxicity symptoms of olanzapine are known to include somnolence, mydriasis, blurred vision, respiratory depression, hypotension, extrapyramidal symptoms and anticholinergic effects. The overdosage effects in children are generally associated with more significant side effects.7',
              'The toxicity symptoms of olanzapine are known to include somnolence, mydriasis, blurred vision, respiratory depression, hypotension, extrapyramidal symptoms and anticholinergic effects. The overdosage effects in children are generally associated with more significant side effects.7'],
             'olanzapine orally disintegrating let': ['The toxicity symptoms of olanzapine are known to include somnolence, mydriasis, blurred vision, respiratory depression, hypotension, extrapyramidal symptoms and anticholinergic effects. The overdosage effects in children are generally associated with more significant side effects.7',
              'The toxicity symptoms of olanzapine are known to include somnolence, mydriasis, blurred vision, respiratory depression, hypotension, extrapyramidal symptoms and anticholinergic effects. The overdosage effects in children are generally associated with more significant side effects.7'],
             'almotriptan malate': ['Not Available', 'Not Available'],
             'zaleplon': ['Side effects include abdominal pain, amnesia, dizziness, drowsiness, eye pain, headache, memory loss, menstrual pain, nausea, sleepiness, tingling, weakness',
              'Side effects include abdominal pain, amnesia, dizziness, drowsiness, eye pain, headache, memory loss, menstrual pain, nausea, sleepiness, tingling, weakness'],
             'bisoprolol fumarate': ['LD50 information\nOral LD50 of bisoprolol in the mouse was 730 mg/kg.20',
              'LD50 information\nOral LD50 of bisoprolol in the mouse was 730 mg/kg.20'],
             'dexmethylphenidate hydrochloride': ['There is no difference in effect across genders3,Label. The difference in effect across racial groups, patients under 6 years, renal impairment, hepatic impairment, pregnancy, lactation, and geriatric patients has not been well studied. Patients with renal impairment are not expected to need dose adjustment as the drug is not mainly cleared renallyLabel. Animal studies in pregnant and lactating rats showed delayed fetal skeletal ossification, and reduced weight gain in male offspringLabel. Due to these studies, caution must be exercised and the benefits and risks of taking this drug must be weighedLabel. It is unlikely that dexmethylphenidate is carcinogenic but B6C3F1 mice, which are sensitive to the development of hepatic tumours, developed hepatoblastomas at 2 times the maximum recommended human doseLabel. Methylpheidate was not found to be mutagenic but is weakly clastogenic in Chinese Hamster Ovary cellsLabel. Methylphenidate does not impair fertility in animal studiesLabel.',
              'There is no difference in effect across genders3,Label. The difference in effect across racial groups, patients under 6 years, renal impairment, hepatic impairment, pregnancy, lactation, and geriatric patients has not been well studied. Patients with renal impairment are not expected to need dose adjustment as the drug is not mainly cleared renallyLabel. Animal studies in pregnant and lactating rats showed delayed fetal skeletal ossification, and reduced weight gain in male offspringLabel. Due to these studies, caution must be exercised and the benefits and risks of taking this drug must be weighedLabel. It is unlikely that dexmethylphenidate is carcinogenic but B6C3F1 mice, which are sensitive to the development of hepatic tumours, developed hepatoblastomas at 2 times the maximum recommended human doseLabel. Methylpheidate was not found to be mutagenic but is weakly clastogenic in Chinese Hamster Ovary cellsLabel. Methylphenidate does not impair fertility in animal studiesLabel.'],
             'voriconazole': ['Symptoms of overdose include photophobia and possible QTc prolongation.13,14 In case of overdose, supportive care and ECG monitoring are recommended. Activated charcoal may aid in the removal of unabsorbed drug. Voriconazole is cleared by hemodialysis at a rate of 121 mL/min which may be helpful in removing absorbed drug. Carcinogenicity studies found hepatocellular adenomas in female rats at doses of 50 mg/kg and hepatocellular carcinomas found in male rats at doses of 6 and 50 mg/kg. These doses are equivalent to 0.2 and 1.6 times the recommended maintenance dose (RMD). Studies in mice detected hepatocellular carcinomas in males at doses of 100 mg/kg or 1.4 times the RMD. Hepatocellular adenomas were detected in both male and female mice.',
              'Symptoms of overdose include photophobia and possible QTc prolongation.13,14 In case of overdose, supportive care and ECG monitoring are recommended. Activated charcoal may aid in the removal of unabsorbed drug. Voriconazole is cleared by hemodialysis at a rate of 121 mL/min which may be helpful in removing absorbed drug. Carcinogenicity studies found hepatocellular adenomas in female rats at doses of 50 mg/kg and hepatocellular carcinomas found in male rats at doses of 6 and 50 mg/kg. These doses are equivalent to 0.2 and 1.6 times the recommended maintenance dose (RMD). Studies in mice detected hepatocellular carcinomas in males at doses of 100 mg/kg or 1.4 times the RMD. Hepatocellular adenomas were detected in both male and female mice.'],
             'methylphenidate hydrochloride (cd)': ['Symptoms of overdose include vomiting, agitation, tremors, hyperreflexia, muscle twitching, convulsions (may be followed by coma), euphoria, confusion, hallucinations, delirium, sweating, flushing, headache, hyperpyrexia, tachycardia, palpitations, cardiac arrhythmias, hypertension, mydriasis, and dryness of mucous membranes. LD50=190mg/kg (orally in mice)',
              'Symptoms of overdose include vomiting, agitation, tremors, hyperreflexia, muscle twitching, convulsions (may be followed by coma), euphoria, confusion, hallucinations, delirium, sweating, flushing, headache, hyperpyrexia, tachycardia, palpitations, cardiac arrhythmias, hypertension, mydriasis, and dryness of mucous membranes. LD50=190mg/kg (orally in mice)'],
             'junel fe': ['Female patients experiencing and overdose may present with withdrawal bleeding, nausea, vomiting, breast tenderness, abdominal pain, drowsiness, and fatigue.15,25,16,17,24,18,19,20,21,22,23 Overdose should be treated with symptomatic and supportive care including monitoring for potassium concentrations, sodium concentrations, and signs of metabolic acidosis.15,25,16,17,24,18,19,20,21,22,23',
              'Female patients experiencing and overdose may present with withdrawal bleeding, nausea, vomiting, breast tenderness, abdominal pain, drowsiness, and fatigue.15,25,16,17,24,18,19,20,21,22,23 Overdose should be treated with symptomatic and supportive care including monitoring for potassium concentrations, sodium concentrations, and signs of metabolic acidosis.15,25,16,17,24,18,19,20,21,22,23'],
             'budeprion xl': ['Symptoms of overdose include seizures, hallucinations, loss of consciousness, tachycardia, and cardiac arrest.',
              'Symptoms of overdose include seizures, hallucinations, loss of consciousness, tachycardia, and cardiac arrest.'],
             'acamprosate calcium': ['The intraperitoneal LD50 in male mice is 1.87 g/kg.14 In reported cases of acute overdosage with acamprosate (doses of up to 56 grams of acamprosate calcium) diarrhea was the only reported symptom attributable to acamprosate. In the case of an overdose, supportive and symptomatic treatment is recommended.11',
              'The intraperitoneal LD50 in male mice is 1.87 g/kg.14 In reported cases of acute overdosage with acamprosate (doses of up to 56 grams of acamprosate calcium) diarrhea was the only reported symptom attributable to acamprosate. In the case of an overdose, supportive and symptomatic treatment is recommended.11'],
             'progesterone': ['Intraperitoneal LD50 (rat): 327 mg/kg MSDS. ',
              'Intraperitoneal LD50 (rat): 327 mg/kg MSDS. '],
             'ursodiol': ['Neither accidental nor intentional overdosing with ursodeoxycholic acid has been reported. Doses of ursodeoxycholic acid in the range of 16-20 mg/kg/day have been tolerated for 6-37 months without symptoms by 7 patients. The LD50 for ursodeoxycholic acid in rats is over 5000 mg/kg given over 7-10 days and over 7500 mg/kg for mice. The most likely manifestation of severe overdose with ursodeoxycholic acid would probably be diarrhea, which should be treated symptomatically.',
              'Neither accidental nor intentional overdosing with ursodeoxycholic acid has been reported. Doses of ursodeoxycholic acid in the range of 16-20 mg/kg/day have been tolerated for 6-37 months without symptoms by 7 patients. The LD50 for ursodeoxycholic acid in rats is over 5000 mg/kg given over 7-10 days and over 7500 mg/kg for mice. The most likely manifestation of severe overdose with ursodeoxycholic acid would probably be diarrhea, which should be treated symptomatically.'],
             'ursodiol forte': ['Neither accidental nor intentional overdosing with ursodeoxycholic acid has been reported. Doses of ursodeoxycholic acid in the range of 16-20 mg/kg/day have been tolerated for 6-37 months without symptoms by 7 patients. The LD50 for ursodeoxycholic acid in rats is over 5000 mg/kg given over 7-10 days and over 7500 mg/kg for mice. The most likely manifestation of severe overdose with ursodeoxycholic acid would probably be diarrhea, which should be treated symptomatically.',
              'Neither accidental nor intentional overdosing with ursodeoxycholic acid has been reported. Doses of ursodeoxycholic acid in the range of 16-20 mg/kg/day have been tolerated for 6-37 months without symptoms by 7 patients. The LD50 for ursodeoxycholic acid in rats is over 5000 mg/kg given over 7-10 days and over 7500 mg/kg for mice. The most likely manifestation of severe overdose with ursodeoxycholic acid would probably be diarrhea, which should be treated symptomatically.'],
             'buprenorphione hydrochloride sublingual': ['Manifestations of acute overdose include pinpoint pupils, sedation, hypotension, respiratory depression and death.',
              'Manifestations of acute overdose include pinpoint pupils, sedation, hypotension, respiratory depression and death.'],
             'buprenorphine hydrochloride sublingual': ['Manifestations of acute overdose include pinpoint pupils, sedation, hypotension, respiratory depression and death.',
              'Manifestations of acute overdose include pinpoint pupils, sedation, hypotension, respiratory depression and death.'],
             'abacavir and lamivudine': ['Some myocardial degeneration has been noticed in rats and mice. The most commonly reported adverse reactions of at least moderate intensity (incidence ≥10%) in adult HIV-1 clinical trials were nausea, headache, malaise and fatigue, nausea and vomiting, and dreams/sleep disorders. Serious hypersensitivity reactions have been associated with abacavir which has been strongly linked to the presence of the HLA-B*57:01 allele. This reaction manifests itself in patients within the first 6 weeks of treatment. Patients should be tested for the presence of this allele as recommended by the U.S Food and Drug Administration (FDA). ',
              'Some myocardial degeneration has been noticed in rats and mice. The most commonly reported adverse reactions of at least moderate intensity (incidence ≥10%) in adult HIV-1 clinical trials were nausea, headache, malaise and fatigue, nausea and vomiting, and dreams/sleep disorders. Serious hypersensitivity reactions have been associated with abacavir which has been strongly linked to the presence of the HLA-B*57:01 allele. This reaction manifests itself in patients within the first 6 weeks of treatment. Patients should be tested for the presence of this allele as recommended by the U.S Food and Drug Administration (FDA). '],
             'lamivudine and zidovudine': ['The most common reported adverse reactions (incidence ≥15%) in adults were headache, nausea, malaise and fatigue, nasal signs and symptoms, diarrhea, and cough. ',
              'The most common reported adverse reactions (incidence ≥15%) in adults were headache, nausea, malaise and fatigue, nasal signs and symptoms, diarrhea, and cough. '],
             'omegaacid ethyl esters': ['', ''],
             'cabergoline': ['Overdosage might be expected to produce nasal congestion, syncope, or hallucinations.',
              'Overdosage might be expected to produce nasal congestion, syncope, or hallucinations.'],
             'gianvi': ['The oral LD50 of drospirenone in rats is >2000 mg/kg.22',
              'The oral LD50 of drospirenone in rats is >2000 mg/kg.22'],
             'mimvey lo': ['The NOAEL (no-observed-adverse-effect-level) oral toxicity of estradiol after 90 day in rats was 0.003 mg/kg/day for blood, female reproductive, and male reproductive, endocrine, and liver toxicity.36 Oral TDLO of ethinyl estradiol is 21 mg/kg/21D intermittent, woman) with an oral LD50 of 960 mg/kg in the rat.37 ',
              'The NOAEL (no-observed-adverse-effect-level) oral toxicity of estradiol after 90 day in rats was 0.003 mg/kg/day for blood, female reproductive, and male reproductive, endocrine, and liver toxicity.36 Oral TDLO of ethinyl estradiol is 21 mg/kg/21D intermittent, woman) with an oral LD50 of 960 mg/kg in the rat.37 '],
             'mimvey': ['The NOAEL (no-observed-adverse-effect-level) oral toxicity of estradiol after 90 day in rats was 0.003 mg/kg/day for blood, female reproductive, and male reproductive, endocrine, and liver toxicity.36 Oral TDLO of ethinyl estradiol is 21 mg/kg/21D intermittent, woman) with an oral LD50 of 960 mg/kg in the rat.37 ',
              'The NOAEL (no-observed-adverse-effect-level) oral toxicity of estradiol after 90 day in rats was 0.003 mg/kg/day for blood, female reproductive, and male reproductive, endocrine, and liver toxicity.36 Oral TDLO of ethinyl estradiol is 21 mg/kg/21D intermittent, woman) with an oral LD50 of 960 mg/kg in the rat.37 '],
             'budeprion sr': ['Symptoms of overdose include seizures, hallucinations, loss of consciousness, tachycardia, and cardiac arrest.',
              'Symptoms of overdose include seizures, hallucinations, loss of consciousness, tachycardia, and cardiac arrest.'],
             'olanzapine and fluoxetine': ['The toxicity symptoms of olanzapine are known to include somnolence, mydriasis, blurred vision, respiratory depression, hypotension, extrapyramidal symptoms and anticholinergic effects. The overdosage effects in children are generally associated with more significant side effects.7',
              'The toxicity symptoms of olanzapine are known to include somnolence, mydriasis, blurred vision, respiratory depression, hypotension, extrapyramidal symptoms and anticholinergic effects. The overdosage effects in children are generally associated with more significant side effects.7'],
             'sildenafil': ['In single-dose volunteer studies of doses up to 800 mg, adverse reactions were similar to those seen at lower doses, but the incidence rates and severities were increased 11,12,13,14,15,16,8,9. Doses of 200 mg did not result in increased efficacy but the incidence of adverse reaction (headache, flushing, dizziness, dyspepsia, nasal congestion, altered vision) was increased 11,12,13,14,15,16,8,9.',
              'In single-dose volunteer studies of doses up to 800 mg, adverse reactions were similar to those seen at lower doses, but the incidence rates and severities were increased 11,12,13,14,15,16,8,9. Doses of 200 mg did not result in increased efficacy but the incidence of adverse reaction (headache, flushing, dizziness, dyspepsia, nasal congestion, altered vision) was increased 11,12,13,14,15,16,8,9.'],
             'eszopiclone': ["The oral LD50 of eszopiclone in rats is 980 mg/kg and 3200 mg/kg in rabbits.13 Symptoms of overdose may include mental status changes and somnolence, demonstrating general exaggeration of the drug's pharmacological effects. Perform gastric lavage and offer supportive treatment if an overdose is suspected, including intravenous fluids as required. Flumazenil may be used. Vital signs should be closely monitored in addition to patient symptoms. Appropriate medical interventions should be employed. The possibility of an overdose with multiple drugs should be considered. Ensure to contact the local poison control center for the most updated management of hypnotic drug overdose.10",
              "The oral LD50 of eszopiclone in rats is 980 mg/kg and 3200 mg/kg in rabbits.13 Symptoms of overdose may include mental status changes and somnolence, demonstrating general exaggeration of the drug's pharmacological effects. Perform gastric lavage and offer supportive treatment if an overdose is suspected, including intravenous fluids as required. Flumazenil may be used. Vital signs should be closely monitored in addition to patient symptoms. Appropriate medical interventions should be employed. The possibility of an overdose with multiple drugs should be considered. Ensure to contact the local poison control center for the most updated management of hypnotic drug overdose.10"],
             'dutasteride': ['LD50 values', 'LD50 values'],
             'buproban': ['Symptoms of overdose include seizures, hallucinations, loss of consciousness, tachycardia, and cardiac arrest.',
              'Symptoms of overdose include seizures, hallucinations, loss of consciousness, tachycardia, and cardiac arrest.'],
             'glyburide and metformin hydrochloride': ['The oral LD50 in rats is >3200mg/kg, in mice is >1500mg/kg, in rabbits is >10,000mg/kg, and in guinea pigs is >1500mg/kg.13',
              'The oral LD50 in rats is >3200mg/kg, in mice is >1500mg/kg, in rabbits is >10,000mg/kg, and in guinea pigs is >1500mg/kg.13'],
             'glyburide and metformin': ['The oral LD50 in rats is >3200mg/kg, in mice is >1500mg/kg, in rabbits is >10,000mg/kg, and in guinea pigs is >1500mg/kg.13',
              'The oral LD50 in rats is >3200mg/kg, in mice is >1500mg/kg, in rabbits is >10,000mg/kg, and in guinea pigs is >1500mg/kg.13'],
             'oxycodone hydrochloride': ['Patients experiencing an overdose may present with respiratory depression, sleepiness, stupor, coma, skeletal muscle flaccidity, cold sweat, constricted pupils, bradycardia, hypotension, partial or complete airway obstruction, atypical snoring, and death.Label Overdose should be treated by maintaining airway, ventilation, and oxygenation.Label Oxygen and vasopressor treatment may be necessary to treat circulatory shock and pulmonary edema and defibrillation may be required for cardiac arrest of arrhythmia.Label Naloxone, nalmefene, or naltrexone may be used to counteract the effects of opioids but patients should be monitored in case further doses are required.Label',
              'Patients experiencing an overdose may present with respiratory depression, sleepiness, stupor, coma, skeletal muscle flaccidity, cold sweat, constricted pupils, bradycardia, hypotension, partial or complete airway obstruction, atypical snoring, and death.Label Overdose should be treated by maintaining airway, ventilation, and oxygenation.Label Oxygen and vasopressor treatment may be necessary to treat circulatory shock and pulmonary edema and defibrillation may be required for cardiac arrest of arrhythmia.Label Naloxone, nalmefene, or naltrexone may be used to counteract the effects of opioids but patients should be monitored in case further doses are required.Label'],
             'cyclosporine modified': ['The oral LD50 in rats is 1480 mg/kg and the TDLO in humans is 12 mg/kg.21 ',
              'The oral LD50 in rats is 1480 mg/kg and the TDLO in humans is 12 mg/kg.21 '],
             'entecavir': ['Healthy subjects who received single entecavir doses up to 40 mg or multiple doses up to 20 mg/day for up to 14 days had no increase in or unexpected adverse events. If overdose occurs, the patient must be monitored for evidence of toxicity, and standard supportive treatment applied as necessary.',
              'Healthy subjects who received single entecavir doses up to 40 mg or multiple doses up to 20 mg/day for up to 14 days had no increase in or unexpected adverse events. If overdose occurs, the patient must be monitored for evidence of toxicity, and standard supportive treatment applied as necessary.'],
             'escitalopram': ['Symptoms of overdose may include CNS effects (dizziness, convulsions, coma, somnolence), gastrointestinal distress (nausea, vomiting), and/or cardiac abnormalities (hypotension, tachycardia, ECG changes).18,19,20 There is no specific antidote for escitalopram overdose. Management of overdose should focus on monitoring for cardiac abnormalities and changes to vital signs as well as treatment with supportive measures as indicated. As escitalopram is highly distributed into tissue following oral administration, forced diuresis, dialysis, and other methods of extracting drug from plasma are unlikely to be beneficial.',
              'Symptoms of overdose may include CNS effects (dizziness, convulsions, coma, somnolence), gastrointestinal distress (nausea, vomiting), and/or cardiac abnormalities (hypotension, tachycardia, ECG changes).18,19,20 There is no specific antidote for escitalopram overdose. Management of overdose should focus on monitoring for cardiac abnormalities and changes to vital signs as well as treatment with supportive measures as indicated. As escitalopram is highly distributed into tissue following oral administration, forced diuresis, dialysis, and other methods of extracting drug from plasma are unlikely to be beneficial.'],
             'oxymorphone hydrochloride': ['Oxymorphone overdosage is characterized by respiratory depression, extreme somnolence progressing to stupor or coma, skeletal muscle flaccidity, cold and clammy skin, and sometimes bradycardia and hypotension. Patients experiencing an overdose may develop apnea, circulatory collapse, and cardiac arrest. Intravenous mouse LD50 is 172 mg/kg.',
              'Oxymorphone overdosage is characterized by respiratory depression, extreme somnolence progressing to stupor or coma, skeletal muscle flaccidity, cold and clammy skin, and sometimes bradycardia and hypotension. Patients experiencing an overdose may develop apnea, circulatory collapse, and cardiac arrest. Intravenous mouse LD50 is 172 mg/kg.'],
             'guanfacine': ['The oral LD50 in rats is 142mg/kg and 15.3mg/kg in mice.12 The subcutaneous LD50 in rats is 114mg/kg and 46mg/kg in mice.12',
              'The oral LD50 in rats is 142mg/kg and 15.3mg/kg in mice.12 The subcutaneous LD50 in rats is 114mg/kg and 46mg/kg in mice.12'],
             'rivelsa': ['Female patients experiencing and overdose may present with withdrawal bleeding, nausea, vomiting, breast tenderness, abdominal pain, drowsiness, and fatigue.15,25,16,17,24,18,19,20,21,22,23 Overdose should be treated with symptomatic and supportive care including monitoring for potassium concentrations, sodium concentrations, and signs of metabolic acidosis.15,25,16,17,24,18,19,20,21,22,23',
              'Female patients experiencing and overdose may present with withdrawal bleeding, nausea, vomiting, breast tenderness, abdominal pain, drowsiness, and fatigue.15,25,16,17,24,18,19,20,21,22,23 Overdose should be treated with symptomatic and supportive care including monitoring for potassium concentrations, sodium concentrations, and signs of metabolic acidosis.15,25,16,17,24,18,19,20,21,22,23'],
             'camrese lo': ['Female patients experiencing and overdose may present with withdrawal bleeding, nausea, vomiting, breast tenderness, abdominal pain, drowsiness, and fatigue.15,25,16,17,24,18,19,20,21,22,23 Overdose should be treated with symptomatic and supportive care including monitoring for potassium concentrations, sodium concentrations, and signs of metabolic acidosis.15,25,16,17,24,18,19,20,21,22,23',
              'Female patients experiencing and overdose may present with withdrawal bleeding, nausea, vomiting, breast tenderness, abdominal pain, drowsiness, and fatigue.15,25,16,17,24,18,19,20,21,22,23 Overdose should be treated with symptomatic and supportive care including monitoring for potassium concentrations, sodium concentrations, and signs of metabolic acidosis.15,25,16,17,24,18,19,20,21,22,23'],
             'esomeprazole magnesium': ['Blurred vision, confusion, drowsiness, dry mouth, flushing headache, nausea, rapid heartbeat, sweating',
              'Blurred vision, confusion, drowsiness, dry mouth, flushing headache, nausea, rapid heartbeat, sweating'],
             'amlodipine and olmesartan medoxomil': ['Acute oral toxicity (LD50): 37 mg/kg (mouse) MSDS. ',
              'Acute oral toxicity (LD50): 37 mg/kg (mouse) MSDS. '],
             'amlodipine valsartan and hydrochlorothiazide': ['Acute oral toxicity (LD50): 37 mg/kg (mouse) MSDS. ',
              'Acute oral toxicity (LD50): 37 mg/kg (mouse) MSDS. '],
             'nefazodone': ['Cases of life-threatening hepatic failure have been reported in patients treated with nefazodone.',
              'Cases of life-threatening hepatic failure have been reported in patients treated with nefazodone.'],
             'paroxetine': ['The acute LD50 in mice and rats is 350 mg/kg.35',
              'The acute LD50 in mice and rats is 350 mg/kg.35'],
             'azithromycin': ['Rat Oral LD50: >2000 mk/kg MSDS',
              'Rat Oral LD50: >2000 mk/kg MSDS'],
             'simvastatin': ['Not Available', 'Not Available'],
             'celecoxib': ['The oral TDLo in humans 5.71 mg/kg.29',
              'The oral TDLo in humans 5.71 mg/kg.29'],
             'ofloxacin': ['LD50=5450 mg/kg (orally in mice)',
              'LD50=5450 mg/kg (orally in mice)'],
             'pravastatin': ["The reported oral LD50 of pravastatin in mice is of 8939 mg/kg.MSDS There haven't been significant overdosage reports however, in the case of overdosage, symptomatic treatment is recommended along with laboratory monitoring and supportive measures.Label",
              "The reported oral LD50 of pravastatin in mice is of 8939 mg/kg.MSDS There haven't been significant overdosage reports however, in the case of overdosage, symptomatic treatment is recommended along with laboratory monitoring and supportive measures.Label"],
             'mirtazapine': ['LD50', 'LD50'],
             'fosinopril sodium': ['Human overdoses of fosinopril have not been reported, but the most common manifestation of human fosinopril overdosage is likely to be hypotension. Oral doses of fosinopril at 2600 mg/kg in rats were associated with significant lethality. The most common adverse effects include dizzines, cough, fatigue, and headache. ',
              'Human overdoses of fosinopril have not been reported, but the most common manifestation of human fosinopril overdosage is likely to be hypotension. Oral doses of fosinopril at 2600 mg/kg in rats were associated with significant lethality. The most common adverse effects include dizzines, cough, fatigue, and headache. '],
             'ribavirin': ['Rivabirin and PEG-Interferon Alfa-2A dual therapy is associated with flu-like symptoms, depression, suicide, insomnia, irritability, relapse of drug abuse/overdose, hepatic decompensation in 2% of HIV co-infected patients and bacterial infections each occurring at a frequency of less than 1%.\nRibavirin-induced anemia is a dose-dependent adverse effect where reduced hemoglobin levels can be seen within the first 1-2 weeks in therapy. The mechanism of ribavirin-induced anemia has been shown to involve reductions in reticulocyte counts and erythrocyte Na-K pump activity, and increases in K-Cl cotransport, membrane bound IgG, and C3, and erythrocyte band 3 6.\nOral LD50 in rats is 2700 mg/kg. Intraperitoneal LD50 in mouse is 1300 mg/kg. Potential carcinogenic effects of ribavirin to humans cannot be yet excluded as it demonstrates mutagenic activity in the in vitro mouse lymphoma assay. ',
              'Rivabirin and PEG-Interferon Alfa-2A dual therapy is associated with flu-like symptoms, depression, suicide, insomnia, irritability, relapse of drug abuse/overdose, hepatic decompensation in 2% of HIV co-infected patients and bacterial infections each occurring at a frequency of less than 1%.\nRibavirin-induced anemia is a dose-dependent adverse effect where reduced hemoglobin levels can be seen within the first 1-2 weeks in therapy. The mechanism of ribavirin-induced anemia has been shown to involve reductions in reticulocyte counts and erythrocyte Na-K pump activity, and increases in K-Cl cotransport, membrane bound IgG, and C3, and erythrocyte band 3 6.\nOral LD50 in rats is 2700 mg/kg. Intraperitoneal LD50 in mouse is 1300 mg/kg. Potential carcinogenic effects of ribavirin to humans cannot be yet excluded as it demonstrates mutagenic activity in the in vitro mouse lymphoma assay. '],
             'meloxicam': ['The oral LD50 in rats is 98 mg/kg.11 Signs and symptoms of overdose with meloxicam may include shallow breathing, seizure, decreased urine output, gastrointestinal irritation, nausea, vomiting, gastrointestinal bleeding, and black tarry stools.12 In the case of an overdose, offer supportive treatment and attempt to remove gastrointestinal contents. Cholestyramine has been shown to enhance the elimination of meloxicam.11',
              'The oral LD50 in rats is 98 mg/kg.11 Signs and symptoms of overdose with meloxicam may include shallow breathing, seizure, decreased urine output, gastrointestinal irritation, nausea, vomiting, gastrointestinal bleeding, and black tarry stools.12 In the case of an overdose, offer supportive treatment and attempt to remove gastrointestinal contents. Cholestyramine has been shown to enhance the elimination of meloxicam.11'],
             'irbesarhctz': ['', ''],
             'clarithromyc er': ['Symptoms of toxicity include diarrhea, nausea, abnormal taste, dyspepsia, and abdominal discomfort. Transient hearing loss with high doses has been observed. Pseudomembraneous colitis has been reported with clarithromycin use. Allergic reactions ranging from urticaria and mild skin eruptions to rare cases of anaphylaxis and Stevens-Johnson syndrome have also occurred. Rare cases of severe hepatic dysfunctions also have been reported. Hepatic failure is usually reversible, but fatalities have been reported. Clarithromycin may also cause tooth decolouration which may be removed by dental cleaning. Fetal abnormalities, such as cardiovascular defects, cleft palate and fetal growth retardation, have been observed in animals. Clarithromycin may cause QT prolongation.',
              'Symptoms of toxicity include diarrhea, nausea, abnormal taste, dyspepsia, and abdominal discomfort. Transient hearing loss with high doses has been observed. Pseudomembraneous colitis has been reported with clarithromycin use. Allergic reactions ranging from urticaria and mild skin eruptions to rare cases of anaphylaxis and Stevens-Johnson syndrome have also occurred. Rare cases of severe hepatic dysfunctions also have been reported. Hepatic failure is usually reversible, but fatalities have been reported. Clarithromycin may also cause tooth decolouration which may be removed by dental cleaning. Fetal abnormalities, such as cardiovascular defects, cleft palate and fetal growth retardation, have been observed in animals. Clarithromycin may cause QT prolongation.'],
             'fexofenadine': ["No deaths were observed following the oral administration of up to 5000 mg/kg in both mice and rats (equivalent to approximately 100-200x the recommended human dose). Single doses of up to 800 mg and chronic exposure of up to 690 mg twice daily for 1 month in humans did not result in clinically significant adverse events. Symptoms of overdosage are consistent with fexofenadine's adverse effect profile and are likely to include dizziness, drowsiness, and dry mouth.10",
              "No deaths were observed following the oral administration of up to 5000 mg/kg in both mice and rats (equivalent to approximately 100-200x the recommended human dose). Single doses of up to 800 mg and chronic exposure of up to 690 mg twice daily for 1 month in humans did not result in clinically significant adverse events. Symptoms of overdosage are consistent with fexofenadine's adverse effect profile and are likely to include dizziness, drowsiness, and dry mouth.10"],
             'glimepiride': ['The oral LD50 value in rats is > 10000 mg/kg.MSDS The intraperitoneal LD50 value in rats is reported to be 3950 mg/kg MSDS. Although glimepiride is reported to have fewer risks of hypoglycemia compared to other sulfonylureas such as glyburide, overdosage of glimepiride may result in severe hypoglycemia with coma, seizure, or other neurological impairment may occur. This can be treated with glucagon or intravenous glucose. Continued observation and additional carbohydrate intake may be necessary since hypoglycemia may recur after apparent clinical recovery.8',
              'The oral LD50 value in rats is > 10000 mg/kg.MSDS The intraperitoneal LD50 value in rats is reported to be 3950 mg/kg MSDS. Although glimepiride is reported to have fewer risks of hypoglycemia compared to other sulfonylureas such as glyburide, overdosage of glimepiride may result in severe hypoglycemia with coma, seizure, or other neurological impairment may occur. This can be treated with glucagon or intravenous glucose. Continued observation and additional carbohydrate intake may be necessary since hypoglycemia may recur after apparent clinical recovery.8'],
             'valacyclovir hydrochloride': ['LD50 Oral ', 'LD50 Oral '],
             'pioglitazone': ['The oral TDLo observed in mice is 24 mg/kg for 4 days and for rats is 3 mg/kg for 6 days.8',
              'The oral TDLo observed in mice is 24 mg/kg for 4 days and for rats is 3 mg/kg for 6 days.8'],
             'levetiracetam': ["The oral TDLO of levetiracetam in humans is 10 mg/kg.14 Symptoms of levetiracetam overdose are consistent with its adverse effect profile and may include agitation, aggression, somnolence, decreased level of consciousness, respiratory depression, or coma.11,13 There is no antidote for levetiracetam overdose, therefore management should involve general supportive measures and symptomatic treatment. Hemodialysis results in significant clearance of plasma levetiracetam (approximately 50% within 4 hours) and should be considered in cases of overdose as indicated by the patient's status.11,13",
              "The oral TDLO of levetiracetam in humans is 10 mg/kg.14 Symptoms of levetiracetam overdose are consistent with its adverse effect profile and may include agitation, aggression, somnolence, decreased level of consciousness, respiratory depression, or coma.11,13 There is no antidote for levetiracetam overdose, therefore management should involve general supportive measures and symptomatic treatment. Hemodialysis results in significant clearance of plasma levetiracetam (approximately 50% within 4 hours) and should be considered in cases of overdose as indicated by the patient's status.11,13"],
             'raloxifene hydrochloride': ['LD50 and Overdose',
              'LD50 and Overdose'],
             'levofloxacin': ['The LD50 following oral administration in mice and rats is 1803 mg/kg and 1478 mg/kg, respectively.14',
              'The LD50 following oral administration in mice and rats is 1803 mg/kg and 1478 mg/kg, respectively.14'],
             'terbinafine hydrochloride': ['The subcutaneous LD50 in rats and mice is >2g/kg.12 The TDLO for women is 210mg/kg/6W.12',
              'The subcutaneous LD50 in rats and mice is >2g/kg.12 The TDLO for women is 210mg/kg/6W.12'],
             'mirtazapine orally disintegrating': ['LD50', 'LD50'],
             'clopidogrel': ['A single dose of clopidogrel at 1500-2000mg/kg was lethal to mice and rats while 3000mg/kg was lethal to baboons.9 Symptoms of overdose include vomiting, breathing difficulty, gastrointestinal hemorrhage, and prostration.9 Clopidogrel is irreversibly bound to platelets for their lifetime, which is approximately 11 days.9 Overdoses of clopidogrel can be treated with platelet transfusions to restore clotting ability.9',
              'A single dose of clopidogrel at 1500-2000mg/kg was lethal to mice and rats while 3000mg/kg was lethal to baboons.9 Symptoms of overdose include vomiting, breathing difficulty, gastrointestinal hemorrhage, and prostration.9 Clopidogrel is irreversibly bound to platelets for their lifetime, which is approximately 11 days.9 Overdoses of clopidogrel can be treated with platelet transfusions to restore clotting ability.9'],
             'desmopressin acetate': ['Intravenous TDLo in humans is reported to be 0.3 µg/kg/10M MSDS. Desmopressin is associated with hyponatremia in case of overdose, which may require temporary or permanent discontinuation of the therapy depending on severity. The effects of hyponatremia include seizure, altered mental status (confusion, drowsiness or continuing headache), cardiac arrhythmias and worsening edema. Other signs of overdose may include oliguria and rapid weight gain due to fluid retention Label. In case of overdose, reduce the dose or frequency of drug administration, or discontinue use if appropriate. Assessment of serum sodium and initiation of appropriate medical treatment is recommended. ',
              'Intravenous TDLo in humans is reported to be 0.3 µg/kg/10M MSDS. Desmopressin is associated with hyponatremia in case of overdose, which may require temporary or permanent discontinuation of the therapy depending on severity. The effects of hyponatremia include seizure, altered mental status (confusion, drowsiness or continuing headache), cardiac arrhythmias and worsening edema. Other signs of overdose may include oliguria and rapid weight gain due to fluid retention Label. In case of overdose, reduce the dose or frequency of drug administration, or discontinue use if appropriate. Assessment of serum sodium and initiation of appropriate medical treatment is recommended. '],
             'trandolapril': ['kely clinical manifestations of overdose are symptoms of severe hypotension. Most common adverse effects include cough, headache and dizziness. The oral LD50 of trandolapril in mice was 4875 mg/kg in males and 3990 mg/kg in females. In rats, an oral dose of 5000 mg/kg caused low mortality',
              'les). In dogs, an oral dose of 1000 mg/kg did not cause mortality and abnormal clinical signs were not observed'],
             'mycophenolate mofetil sule': ['re anecdotal reports of deliberate or accidental overdoses with mycophenolic aci',
              'r, not all patients have experienced related adverse reactions. In those cases where adverse reactions have been reported, reactions fall within the safety profile of its class of drugs. A mycophenolic acid overdose could lead to the oversuppression of the immune system and increase the susceptibility to infection.8 It may be appropriate to interrupt or discontinue mycophenolic acid if blood dyscrasias occur. Some of the signs and symptoms associated with mycophenolic acid overdose are hematological abnormalities, such as leukopenia and neutropenia, and gastrointestinal symptoms, such as abdominal pain, diarrhea, nausea and vomiting, and dyspepsia.'],
             'topiramate (sprinkle)': ['The LD50 of intraperitoneal topiramate in the rat is above 1500 mg/kg.20',
              'The LD50 of intraperitoneal topiramate in the rat is above 1500 mg/kg.20'],
             'tamsulosin hydrochloride': ['In the event of overdose, patients may experience hypotension and should lie down in a supine position to maintain blood pressure and heart rate.Label If further measures are required intravenous fluids should be considered.Label If further progression is required, vasopressors may be used and renal function should be monitored.Label Dialysis is unlikely to assist in treating overdose because tamsulosin is extensively protein bound.Label',
              'In the event of overdose, patients may experience hypotension and should lie down in a supine position to maintain blood pressure and heart rate.Label If further measures are required intravenous fluids should be considered.Label If further progression is required, vasopressors may be used and renal function should be monitored.Label Dialysis is unlikely to assist in treating overdose because tamsulosin is extensively protein bound.Label'],
             'lansoprazole': ["The most commonly reported adverse events occurring more frequently in lansoprazole treated patients compared to placebo include abdominal pain, constipation, diarrhea, and nausea.Label There is a case report of toxic epidermal necrolysis (TEN), which is a rare but very serious cutaneous reaction, caused by lansoprazole.8 The previously healthy patient presented with symptoms of TEN 15 days after starting lansoprazole to manage peptic disease.8 Although the use of PPI's is rarely associated with TEN, causation should be considered if a patient presents with TEN shortly after newly commencing a PPI.8 ",
              "The most commonly reported adverse events occurring more frequently in lansoprazole treated patients compared to placebo include abdominal pain, constipation, diarrhea, and nausea.Label There is a case report of toxic epidermal necrolysis (TEN), which is a rare but very serious cutaneous reaction, caused by lansoprazole.8 The previously healthy patient presented with symptoms of TEN 15 days after starting lansoprazole to manage peptic disease.8 Although the use of PPI's is rarely associated with TEN, causation should be considered if a patient presents with TEN shortly after newly commencing a PPI.8 "],
             'finasteride': ['LD50', 'LD50'],
             'losartan potassium': ['The oral TDLO in mice is 1000mg/kg and in rats is 2000mg/kg.3,4 In humans the TDLO for men is 10mg/kg/2W and for women is 1mg/kg/1D.5',
              'The oral TDLO in mice is 1000mg/kg and in rats is 2000mg/kg.3,4 In humans the TDLO for men is 10mg/kg/2W and for women is 1mg/kg/1D.5'],
             'losartan pot': ['The oral TDLO in mice is 1000mg/kg and in rats is 2000mg/kg.3,4 In humans the TDLO for men is 10mg/kg/2W and for women is 1mg/kg/1D.5',
              'The oral TDLO in mice is 1000mg/kg and in rats is 2000mg/kg.3,4 In humans the TDLO for men is 10mg/kg/2W and for women is 1mg/kg/1D.5'],
             'losartan potassium and hydrochlorothiazide': ['The oral TDLO in mice is 1000mg/kg and in rats is 2000mg/kg.3,4 In humans the TDLO for men is 10mg/kg/2W and for women is 1mg/kg/1D.5',
              'The oral TDLO in mice is 1000mg/kg and in rats is 2000mg/kg.3,4 In humans the TDLO for men is 10mg/kg/2W and for women is 1mg/kg/1D.5'],
             'losartanhct': ['', ''],
             'amlodipine besylate and benazepril hydrochloride': ['Acute oral toxicity (LD50): 37 mg/kg (mouse) MSDS. ',
              'Acute oral toxicity (LD50): 37 mg/kg (mouse) MSDS. '],
             'venlafaxine er': ['Oral LD50 was 350 mg/kg in female rats and 700 mg/kg in male rats.16',
              'Oral LD50 was 350 mg/kg in female rats and 700 mg/kg in male rats.16'],
             'moxifloxacin hydrochloride': ['Symptoms of overdose include CNS and gastrointestinal effects such as decreased activity, somnolence, tremor, convulsions, vomiting, and diarrhea. The minimal lethal intravenous dose in mice and rats is 100 mg/kg.',
              'Symptoms of overdose include CNS and gastrointestinal effects such as decreased activity, somnolence, tremor, convulsions, vomiting, and diarrhea. The minimal lethal intravenous dose in mice and rats is 100 mg/kg.'],
             'niacin': ['Overdose of niacin may present with severe prolonged hypotension.1 Patients experiencing an overdose should be treated with supportive measures which may include intravenous fluids.1,6',
              'Overdose of niacin may present with severe prolonged hypotension.1 Patients experiencing an overdose should be treated with supportive measures which may include intravenous fluids.1,6'],
             'montelukast sodium': ['The adverse effects associated with overdosage of montelukast include abdominal pain, somnolence, thirst, headache, vomiting, psychomotor hyperactivity, and less frequently, convulsion.3,4,5,6,7,8,9',
              'The adverse effects associated with overdosage of montelukast include abdominal pain, somnolence, thirst, headache, vomiting, psychomotor hyperactivity, and less frequently, convulsion.3,4,5,6,7,8,9'],
             'valsartan': ['Approximate LD50 >2000 mg/kg (Gavage, rat) 24',
              'Approximate LD50 >2000 mg/kg (Gavage, rat) 24'],
             'divalproex sodium': ['LD50 Values', 'LD50 Values'],
             'fluvastatin': ['Generally well-tolerated. May cause gastrointestinal upset (diarrhea, nausea, constipation, gas, abdominal pain), myotoxicity (mypothy, myositis, rhabdomyolysis), and hepatotoxicity.',
              'Generally well-tolerated. May cause gastrointestinal upset (diarrhea, nausea, constipation, gas, abdominal pain), myotoxicity (mypothy, myositis, rhabdomyolysis), and hepatotoxicity.'],
             'budesonide (enteric coated)': ['Acute overdose of corticosteroids is rare, however prolonged high dosing of corticosteroids can lead to hypercorticism and adrenal axis suppression.9,10,11,12,14,15,16 In the case of overdose, reduce the dosage of corticosteroids temporarily.9,10,11,12,14,15,16',
              'Acute overdose of corticosteroids is rare, however prolonged high dosing of corticosteroids can lead to hypercorticism and adrenal axis suppression.9,10,11,12,14,15,16 In the case of overdose, reduce the dosage of corticosteroids temporarily.9,10,11,12,14,15,16'],
             'fluvastatin sodium': ['Generally well-tolerated. May cause gastrointestinal upset (diarrhea, nausea, constipation, gas, abdominal pain), myotoxicity (mypothy, myositis, rhabdomyolysis), and hepatotoxicity.',
              'Generally well-tolerated. May cause gastrointestinal upset (diarrhea, nausea, constipation, gas, abdominal pain), myotoxicity (mypothy, myositis, rhabdomyolysis), and hepatotoxicity.'],
             'glipizide and metformin hydrochloride': ['In rats, the oral LD50 is reported to be greater than 4000 mg/kg and the intraperitoneal LD50 is 1200 mg/kg. The lowest published toxic dose (TDLo) via oral route in child was 379 μg/kg.MSDS ',
              'In rats, the oral LD50 is reported to be greater than 4000 mg/kg and the intraperitoneal LD50 is 1200 mg/kg. The lowest published toxic dose (TDLo) via oral route in child was 379 μg/kg.MSDS '],
             'irbesartan': ['The oral TDLO in humans is 30mg/kg/6W.9',
              'The oral TDLO in humans is 30mg/kg/6W.9'],
             'irbesartan let': ['The oral TDLO in humans is 30mg/kg/6W.9',
              'The oral TDLO in humans is 30mg/kg/6W.9'],
             'rizatriptan benzoate': ['Symptoms of overdose include dizziness, fainting, heart and blood vessel problems, high blood pressure, loss of bowel and bladder control, slow heartbeat, and vomiting.',
              'Symptoms of overdose include dizziness, fainting, heart and blood vessel problems, high blood pressure, loss of bowel and bladder control, slow heartbeat, and vomiting.'],
             'eciine': ['Not Available', 'Not Available'],
             'granisetron hydrochloride': ['LD50>2000 mg/kg (rat, oral)',
              'LD50>2000 mg/kg (rat, oral)'],
             'linezolid': ['Clinical signs of overdosage observed in rats were decreased activity and ataxia (2000 mg/kg/day) and in dogs were vomiting and tremors (3000 mg/kg/day).9 Treatment of overdose should involve symptomatic and supportive measures and may include hemodialysis if clinically necessary.',
              'Clinical signs of overdosage observed in rats were decreased activity and ataxia (2000 mg/kg/day) and in dogs were vomiting and tremors (3000 mg/kg/day).9 Treatment of overdose should involve symptomatic and supportive measures and may include hemodialysis if clinically necessary.'],
             'anastrozole': ['The reported oral TDLo in a human woman is 1.68 mg/kg given intermittently over the course of 12 weeks.14 Knowledge of the signs and symptoms of anastrozole overdose is incomplete as there are no documented descriptions of a patient receiving more than 60mg,13 a dose which was administered to a healthy male volunteer and was well-tolerated.10,12 There is no antidote for anastrozole and treatment should be supportive and symptomatic, including close monitoring of patient vital signs. As anastrozole exhibits relatively low protein binding, dialysis may be helpful and should be considered in select cases.10,12',
              'The reported oral TDLo in a human woman is 1.68 mg/kg given intermittently over the course of 12 weeks.14 Knowledge of the signs and symptoms of anastrozole overdose is incomplete as there are no documented descriptions of a patient receiving more than 60mg,13 a dose which was administered to a healthy male volunteer and was well-tolerated.10,12 There is no antidote for anastrozole and treatment should be supportive and symptomatic, including close monitoring of patient vital signs. As anastrozole exhibits relatively low protein binding, dialysis may be helpful and should be considered in select cases.10,12'],
             'duloxetine': ['Overdose', 'Overdose'],
             'aripiprazole': ["Studies on the safety and effectiveness of aripiprazole in pregnancy have not been performed, though there is currently a national pregnancy registry for mothers currently taking aripiprazole in pregnancyLabel,6. In other studies of antipsychotic medication in pregnancy, children are at risk of extrapyramidal or withdrawal symptomsLabel. In animal studies of pregnancy, aripiprazole was associated with a number of malformations and fetal death at doses higher than the maximum recommended human doseLabel. Aripiprazole should only be prescribed in pregnancy if the benefits outweigh the risksLabel. Neonates with third trimester exposure to aripiprazole may show extrapyramidal or withdrawal symptoms of varying severityLabel. These symptoms may resolve in hours or require extended hospital careLabel. Aripiprazole's effect on labor and delivery has not been investigatedLabel. Aripiprazole is present in human breast milk and so patients should either stop breastfeeding or stop taking aripiprazole depending on the risk and benefit to mother and childLabel. Pharmacokinetic properties in patients 10-17 years of age are similar to that of adults once body weight has been corrected forLabel. No dosage adjustment is necessary in elderly patients however aripiprazole is not approved for Alzheimer's associated psychosisLabel. Patients calssified as CYP2D6 poor metabolizers should be prescribed half the regular dose of aripiprazoleLabel. Hepatic and renal function as well as sex, race, and smoking status do not affect dosage requirements for aripiprazoleLabel,3.",
              "Studies on the safety and effectiveness of aripiprazole in pregnancy have not been performed, though there is currently a national pregnancy registry for mothers currently taking aripiprazole in pregnancyLabel,6. In other studies of antipsychotic medication in pregnancy, children are at risk of extrapyramidal or withdrawal symptomsLabel. In animal studies of pregnancy, aripiprazole was associated with a number of malformations and fetal death at doses higher than the maximum recommended human doseLabel. Aripiprazole should only be prescribed in pregnancy if the benefits outweigh the risksLabel. Neonates with third trimester exposure to aripiprazole may show extrapyramidal or withdrawal symptoms of varying severityLabel. These symptoms may resolve in hours or require extended hospital careLabel. Aripiprazole's effect on labor and delivery has not been investigatedLabel. Aripiprazole is present in human breast milk and so patients should either stop breastfeeding or stop taking aripiprazole depending on the risk and benefit to mother and childLabel. Pharmacokinetic properties in patients 10-17 years of age are similar to that of adults once body weight has been corrected forLabel. No dosage adjustment is necessary in elderly patients however aripiprazole is not approved for Alzheimer's associated psychosisLabel. Patients calssified as CYP2D6 poor metabolizers should be prescribed half the regular dose of aripiprazoleLabel. Hepatic and renal function as well as sex, race, and smoking status do not affect dosage requirements for aripiprazoleLabel,3."],
             'temozolomide': ['The primary dose-limiting toxicity of temozolomide is myelosuppression, which can occur with any dose but is more severe at higher doses. Patients taking high doses experienced adverse reactions, including severe and prolonged myelosuppression, infections, and death. One patient who took 2000 mg/day for five days experienced pancytopenia, pyrexia, and multi-organ failure, which resulted in death. Patients experiencing an overdose should have complete blood counts monitored and provided with supportive care as necessary.17',
              'The primary dose-limiting toxicity of temozolomide is myelosuppression, which can occur with any dose but is more severe at higher doses. Patients taking high doses experienced adverse reactions, including severe and prolonged myelosuppression, infections, and death. One patient who took 2000 mg/day for five days experienced pancytopenia, pyrexia, and multi-organ failure, which resulted in death. Patients experiencing an overdose should have complete blood counts monitored and provided with supportive care as necessary.17'],
             'olmesartan medoxomil': ['The reported LD50 of olmesartan in dogs was reported to be greater of 1500 mg/kg. Overdose is expressed as hypotension, tachycardia, and bradycardia when there is parasympathetic stimulation. In case of overdose, supportive treatment is recommended.Label',
              'The reported LD50 of olmesartan in dogs was reported to be greater of 1500 mg/kg. Overdose is expressed as hypotension, tachycardia, and bradycardia when there is parasympathetic stimulation. In case of overdose, supportive treatment is recommended.Label'],
             'olmesartan medoxomil and hydrochlorothiazide': ['The reported LD50 of olmesartan in dogs was reported to be greater of 1500 mg/kg. Overdose is expressed as hypotension, tachycardia, and bradycardia when there is parasympathetic stimulation. In case of overdose, supportive treatment is recommended.Label',
              'The reported LD50 of olmesartan in dogs was reported to be greater of 1500 mg/kg. Overdose is expressed as hypotension, tachycardia, and bradycardia when there is parasympathetic stimulation. In case of overdose, supportive treatment is recommended.Label'],
             'imatinib mesylate': ['The most frequently reported adverse reactions (>30%) were edema, nausea, vomiting, muscle cramps, musculoskeletal pain, diarrhea, rash, fatigue and abdominal pain.14',
              'The most frequently reported adverse reactions (>30%) were edema, nausea, vomiting, muscle cramps, musculoskeletal pain, diarrhea, rash, fatigue and abdominal pain.14'],
             'paricalcitol': ['Not Available', 'Not Available'],
             'pioglitazone and metformin hydrochloride': ['The oral TDLo observed in mice is 24 mg/kg for 4 days and for rats is 3 mg/kg for 6 days.8',
              'The oral TDLo observed in mice is 24 mg/kg for 4 days and for rats is 3 mg/kg for 6 days.8'],
             'amlodipine and valsartan': ['Acute oral toxicity (LD50): 37 mg/kg (mouse) MSDS. ',
              'Acute oral toxicity (LD50): 37 mg/kg (mouse) MSDS. '],
             'levocetirizine dihydrochloride': ['Patients experiencing an overdose may present with drowsiness.4 Children may become agitated and restless before drowsiness.4 Patients should be treated with supportive measures.4 Dialysis will not assist in removing the drug from the body.4',
              'Patients experiencing an overdose may present with drowsiness.4 Children may become agitated and restless before drowsiness.4 Patients should be treated with supportive measures.4 Dialysis will not assist in removing the drug from the body.4'],
             'onceamonth risedronate sodium': ['', ''],
             'glyburide (micronized)': ['The oral LD50 in rats is >3200mg/kg, in mice is >1500mg/kg, in rabbits is >10,000mg/kg, and in guinea pigs is >1500mg/kg.13',
              'The oral LD50 in rats is >3200mg/kg, in mice is >1500mg/kg, in rabbits is >10,000mg/kg, and in guinea pigs is >1500mg/kg.13'],
             'glyburide': ['The oral LD50 in rats is >3200mg/kg, in mice is >1500mg/kg, in rabbits is >10,000mg/kg, and in guinea pigs is >1500mg/kg.13',
              'The oral LD50 in rats is >3200mg/kg, in mice is >1500mg/kg, in rabbits is >10,000mg/kg, and in guinea pigs is >1500mg/kg.13'],
             'lansoprazole and amoxicillin and clarithromycin': ["The most commonly reported adverse events occurring more frequently in lansoprazole treated patients compared to placebo include abdominal pain, constipation, diarrhea, and nausea.Label There is a case report of toxic epidermal necrolysis (TEN), which is a rare but very serious cutaneous reaction, caused by lansoprazole.8 The previously healthy patient presented with symptoms of TEN 15 days after starting lansoprazole to manage peptic disease.8 Although the use of PPI's is rarely associated with TEN, causation should be considered if a patient presents with TEN shortly after newly commencing a PPI.8 ",
              "The most commonly reported adverse events occurring more frequently in lansoprazole treated patients compared to placebo include abdominal pain, constipation, diarrhea, and nausea.Label There is a case report of toxic epidermal necrolysis (TEN), which is a rare but very serious cutaneous reaction, caused by lansoprazole.8 The previously healthy patient presented with symptoms of TEN 15 days after starting lansoprazole to manage peptic disease.8 Although the use of PPI's is rarely associated with TEN, causation should be considered if a patient presents with TEN shortly after newly commencing a PPI.8 "],
             'doxazosin mesylate': ['LD50 information', 'LD50 information'],
             'cimetidine': ['In the rare event of cimetidine overdose, it is vital to maintain the airway and cardiovascular status.5 The patient should be closely monitored and provided with symptomatic and supportive treatment as needed.5 Interventions such as gastric lavage and administration of activated charcoal may be initiated if deemed appropriate and necessary.5',
              'In the rare event of cimetidine overdose, it is vital to maintain the airway and cardiovascular status.5 The patient should be closely monitored and provided with symptomatic and supportive treatment as needed.5 Interventions such as gastric lavage and administration of activated charcoal may be initiated if deemed appropriate and necessary.5'],
             'eletriptan hydrobromide': ['Based on the pharmacology of the 5-HT1B/1D agonists, hypertension or other more serious cardiovascular symptoms could occur on overdose.',
              'Based on the pharmacology of the 5-HT1B/1D agonists, hypertension or other more serious cardiovascular symptoms could occur on overdose.'],
             'mexiletine hydrochloride': ['Symptoms of overdose include nausea, hypotension, sinus bradycardia, paresthesia, seizures, bundle branch block, AV heart block, asystole, ventricular tachyarrythmia, including ventricular fibrillation, cardiovascular collapse, and coma.',
              'Symptoms of overdose include nausea, hypotension, sinus bradycardia, paresthesia, seizures, bundle branch block, AV heart block, asystole, ventricular tachyarrythmia, including ventricular fibrillation, cardiovascular collapse, and coma.'],
             'acyclovir': ['Symptoms of overdose include agitation, coma, seizures, lethargy, and precipitation in renal tubules.11 These symptoms are more common in patients given high doses without monitoring of fluid and electrolyte balance or reduced kidney function.11,4,6 In the case of an overdose, treat with symptomatic and supportive care.13',
              'Symptoms of overdose include agitation, coma, seizures, lethargy, and precipitation in renal tubules.11 These symptoms are more common in patients given high doses without monitoring of fluid and electrolyte balance or reduced kidney function.11,4,6 In the case of an overdose, treat with symptomatic and supportive care.13'],
             'mebendazole': ['Acute oral toxicity (LD50): 620 mg/kg [Mouse]. Symptoms of overdose include elevated liver enzymes, headaches, hair loss, low levels of white blood cells (neutropenia), fever, and itching.',
              'Acute oral toxicity (LD50): 620 mg/kg [Mouse]. Symptoms of overdose include elevated liver enzymes, headaches, hair loss, low levels of white blood cells (neutropenia), fever, and itching.'],
             'amiodarone hydrochloride': ['The LD50 of oral amiodarone in mice and rats exceeds 3,000 mg/kg.18\nAn overdose with amiodarone can have a fatal outcome due to its potential to cause arrhythmia. Signs or symptoms of an overdose may include, hypotension, shock, bradycardia, AV block, and liver toxicity. In cases of an overdose, initiate supportive treatment and, if needed, use fluids, vasopressors, or positive inotropic agents. Temporary pacing may be required for heart block. Ensure to monitor liver function regularly. Amiodarone and its main metabolite, DEA, are not removable by dialysis.17',
              'The LD50 of oral amiodarone in mice and rats exceeds 3,000 mg/kg.18\nAn overdose with amiodarone can have a fatal outcome due to its potential to cause arrhythmia. Signs or symptoms of an overdose may include, hypotension, shock, bradycardia, AV block, and liver toxicity. In cases of an overdose, initiate supportive treatment and, if needed, use fluids, vasopressors, or positive inotropic agents. Temporary pacing may be required for heart block. Ensure to monitor liver function regularly. Amiodarone and its main metabolite, DEA, are not removable by dialysis.17'],
             'quinidine sulfate': ['Quinidine overdoses have been well described. The ingestion of 5 g of quinidine resulted in the death of a toddler, while an adolescent was reported to survive after ingesting 8 g of quinidine.5,6 A 16-month that ingested quinidine tablets developed a concretion of bezoar in the stomach, which led to non-declining toxic levels of quinidine. A gastric aspirate revealed that quinidine levels were 50 times higher than the ones detected in plasma. In cases of massive overdose, it may be appropriate to perform an endoscopy.5 Acute quinidine overdoses are characterized by ventricular arrhythmias and hypotension. Other signs and symptoms of quinidine overdose may include vomiting, diarrhea, tinnitus, high-frequency hearing loss, vertigo, blurred vision, diplopia, photophobia, headache, confusion and delirium.5,6 ',
              'Quinidine overdoses have been well described. The ingestion of 5 g of quinidine resulted in the death of a toddler, while an adolescent was reported to survive after ingesting 8 g of quinidine.5,6 A 16-month that ingested quinidine tablets developed a concretion of bezoar in the stomach, which led to non-declining toxic levels of quinidine. A gastric aspirate revealed that quinidine levels were 50 times higher than the ones detected in plasma. In cases of massive overdose, it may be appropriate to perform an endoscopy.5 Acute quinidine overdoses are characterized by ventricular arrhythmias and hypotension. Other signs and symptoms of quinidine overdose may include vomiting, diarrhea, tinnitus, high-frequency hearing loss, vertigo, blurred vision, diplopia, photophobia, headache, confusion and delirium.5,6 '],
             'prochlorperazine maleate': ['LD50 and Overdose',
              'LD50 and Overdose'],
             'hydroxychloroquine sulfate': ['Patients experiencing an overdose may present with headache, drowsiness, visual disturbances, cardiovascular collapse, convulsions, hypokalemia, rhythm and conduction disorders including QT prolongation, torsades de pointes, ventricular tachycardia, and ventricular fibrillation.13 This may progress to sudden respiratory and cardiac arrest.13 Overdose should be treated with immediate gastric lavage and activated charcoal at a dose of at least 5 times the hydroxychloroquine dose within 30 minutes.2,13 Parenteral diazepam may be given to treat cardiotoxicity, transfusion may reduce serum concentrations of drug, patients should be monitored for at least 6 hours, fluids should be given, and ammonium chloride should be given to acidify urine and promote urinary excretion.13 Patients may also be given epinephrine.11',
              'Patients experiencing an overdose may present with headache, drowsiness, visual disturbances, cardiovascular collapse, convulsions, hypokalemia, rhythm and conduction disorders including QT prolongation, torsades de pointes, ventricular tachycardia, and ventricular fibrillation.13 This may progress to sudden respiratory and cardiac arrest.13 Overdose should be treated with immediate gastric lavage and activated charcoal at a dose of at least 5 times the hydroxychloroquine dose within 30 minutes.2,13 Parenteral diazepam may be given to treat cardiotoxicity, transfusion may reduce serum concentrations of drug, patients should be monitored for at least 6 hours, fluids should be given, and ammonium chloride should be given to acidify urine and promote urinary excretion.13 Patients may also be given epinephrine.11'],
             'zolmitriptan': ['Toxicity information regarding zolmitriptan is not readily available. Patients experiencing an overdose are at an increased risk of severe adverse effects such as cardiovascular symptoms due to excessive vasoconstriction and activation of serotonergic receptors. Patients receiving a single 50 mg oral dose of zolmitriptan often experienced sedation. Symptomatic and supportive measures are recommended.14',
              'Toxicity information regarding zolmitriptan is not readily available. Patients experiencing an overdose are at an increased risk of severe adverse effects such as cardiovascular symptoms due to excessive vasoconstriction and activation of serotonergic receptors. Patients receiving a single 50 mg oral dose of zolmitriptan often experienced sedation. Symptomatic and supportive measures are recommended.14'],
             'promethazine hydrochloride': ['The intraperitoneal LD50 in rats is 170mg/kg and in mice is 160mg/kg.14 The subcutaneous LD50 in rats is 400mg/kg and in mice is 240mg/kg.14 The oral LD50 in mice is 255mg/kg.14',
              'The intraperitoneal LD50 in rats is 170mg/kg and in mice is 160mg/kg.14 The subcutaneous LD50 in rats is 400mg/kg and in mice is 240mg/kg.14 The oral LD50 in mice is 255mg/kg.14'],
             'dipyridamole': ['Hypotension, if it occurs, is likely to be of short duration, but a vasopressor drug may be used if necessary. The oral LD50 in rats is greater than 6,000 mg/kg while in the dogs, the oral LD50 is approximately 400 mg/kg. LD50=8.4g/kg (orally in rat)',
              'Hypotension, if it occurs, is likely to be of short duration, but a vasopressor drug may be used if necessary. The oral LD50 in rats is greater than 6,000 mg/kg while in the dogs, the oral LD50 is approximately 400 mg/kg. LD50=8.4g/kg (orally in rat)'],
             'fenofibric acid delayed release': ['Oral LD50 (rat): 1242 mg/kg, Oral LD50 (mouse): 100 mg/kg, lntraperitoneal LD50 (mouse): 500 mg/kg MSDS',
              'Oral LD50 (rat): 1242 mg/kg, Oral LD50 (mouse): 100 mg/kg, lntraperitoneal LD50 (mouse): 500 mg/kg MSDS'],
             'propranolol hydrochloride': ['Symptoms of overdose include hypotension, hypoglycemic seizure, restlessness, euphoria, insomnia.8 Patients with asthma may develop bronchospasm.8 In case of overdose, monitor vital signs, mental status, and blood glucose.8 Treat hypotension with intravenous fluids, bradycardia with atropine, and isoproterenol and aminophylline for bronchospasm.8 If patients do not respond to intravenous fluids, follow up with glucagon 50-150µg/kg intravenously, then 1-5mg/hour, followed by catecholamines.8,9,10 Dialysis will not be useful as propranolol is highly protein bound.8,9,10',
              'Symptoms of overdose include hypotension, hypoglycemic seizure, restlessness, euphoria, insomnia.8 Patients with asthma may develop bronchospasm.8 In case of overdose, monitor vital signs, mental status, and blood glucose.8 Treat hypotension with intravenous fluids, bradycardia with atropine, and isoproterenol and aminophylline for bronchospasm.8 If patients do not respond to intravenous fluids, follow up with glucagon 50-150µg/kg intravenously, then 1-5mg/hour, followed by catecholamines.8,9,10 Dialysis will not be useful as propranolol is highly protein bound.8,9,10'],
             'flavoxate': ['The oral LD50 for flavoxate HCl in rats is 4273 mg/kg. The oral LD50 for flavoxate HCl in mice is 1837 mg/kg. Symptoms of overdose include convulsions, decreased ability to sweat, (warm, red skin, dry mouth, and increased body temperature), hallucinations, increased heart rate and blood pressure, and mental confusion.',
              'The oral LD50 for flavoxate HCl in rats is 4273 mg/kg. The oral LD50 for flavoxate HCl in mice is 1837 mg/kg. Symptoms of overdose include convulsions, decreased ability to sweat, (warm, red skin, dry mouth, and increased body temperature), hallucinations, increased heart rate and blood pressure, and mental confusion.'],
             'orphenadrine citrate': ['ouse LD50 = 100 mg/k',
              'rat LD50 = 255 mg/k'],
             'riluzole': ['Not Available', 'Not Available'],
             'pyridostigmine let': ['Toxicity information regarding pyridostigmine is not readily available. Patients experiencing an overdose are at an increased risk of severe adverse effects such as bradycardia, excessive bronchial secretions, and cholinergic crisis. Symptomatic and supportive measures are recommended.10,11',
              'Toxicity information regarding pyridostigmine is not readily available. Patients experiencing an overdose are at an increased risk of severe adverse effects such as bradycardia, excessive bronchial secretions, and cholinergic crisis. Symptomatic and supportive measures are recommended.10,11'],
             'carblevo er': ['', ''],
             'midodrine hcl': ['Symptoms of overdose could include hypertension, piloerection (goosebumps), a sensation of coldness and urinary retention. The single doses that would be associated with symptoms of overdosage or would be potentially life- threatening are unknown. The oral LD50 is approximately 30 to 50 mg/kg in rats, 675 mg/kg in mice, and 125 to 160 mg/kg in dogs. Desglymidodrine is dialyzable.',
              'Symptoms of overdose could include hypertension, piloerection (goosebumps), a sensation of coldness and urinary retention. The single doses that would be associated with symptoms of overdosage or would be potentially life- threatening are unknown. The oral LD50 is approximately 30 to 50 mg/kg in rats, 675 mg/kg in mice, and 125 to 160 mg/kg in dogs. Desglymidodrine is dialyzable.'],
             'dantrolene sodium': ['Oral LD50 in rats is 7400 mg/kg. Symptoms which may occur in case of overdose include, but are not limited to, muscular weakness and alterations in the state of consciousness (e.g., lethargy, coma), vomiting, diarrhea, and crystalluria.',
              'Oral LD50 in rats is 7400 mg/kg. Symptoms which may occur in case of overdose include, but are not limited to, muscular weakness and alterations in the state of consciousness (e.g., lethargy, coma), vomiting, diarrhea, and crystalluria.'],
             'colestipol gm': ['Overdosage with colestipol has not been reported Label,1,5,7. Regardless, in the case of overdosage, the chief potential harm would be obstruction of the gastrointestinal tract Label,1,5,7. The location of such potential obstruction, the degree of obstruction and the presence or absence of normal gut motility would determine treatment Label,1,5,7.',
              'Overdosage with colestipol has not been reported Label,1,5,7. Regardless, in the case of overdosage, the chief potential harm would be obstruction of the gastrointestinal tract Label,1,5,7. The location of such potential obstruction, the degree of obstruction and the presence or absence of normal gut motility would determine treatment Label,1,5,7.'],
             'fludrocortisone acetate': ['The oral LD50 of fludrocortisone in rats is >1g/kg.17 Acute overdosage of fludrocortisone is likely to result in symptoms consistent with its adverse effect profile. Patients receiving a single large dose should be treated with plenty of water by mouth and should undergo monitoring of serum electrolytes, particularly potassium and sodium, and be treated appropriately for any developing imbalances.15,16 ',
              'The oral LD50 of fludrocortisone in rats is >1g/kg.17 Acute overdosage of fludrocortisone is likely to result in symptoms consistent with its adverse effect profile. Patients receiving a single large dose should be treated with plenty of water by mouth and should undergo monitoring of serum electrolytes, particularly potassium and sodium, and be treated appropriately for any developing imbalances.15,16 '],
             'digoxin': ['Oral TDLO (human female): 100 ug/kg, Oral TDLO (human male): 75 ug/kg, Oral LD50 (rat): 28270 ug/kgMSDS',
              'Oral TDLO (human female): 100 ug/kg, Oral TDLO (human male): 75 ug/kg, Oral LD50 (rat): 28270 ug/kgMSDS'],
             'vimpat': ['Not Available', 'Not Available'],
             'venlafaxine extended release': ['Oral LD50 was 350 mg/kg in female rats and 700 mg/kg in male rats.16',
              'Oral LD50 was 350 mg/kg in female rats and 700 mg/kg in male rats.16'],
             'aminophyllin': ['Not Available', 'Not Available'],
             'bella alkpb': ['Oral LD50 of atropine is 75 mg/kg in mouse. Clinical manifestations of anticholinergic syndrome include both central and peripheral effects. Central symptoms, which are dose-dependent and anticholinergic agent-specific, include ataxia, disorientation, short-term memory loss, confusion, hallucinations, psychosis, agitated delirium, seizures, coma, respiratory failure or cardiovascular collapse 1. Peripheral effects include mydriasis with cycloplegia, dry mucous membranes, hyperreflexia, flushed skin, diminished bowel sounds or ileus, urinary retention, tachycardia, and hypertension or hypotension 1. Management of anticholinergic intoxication should be symptomatic including gastrointestinal decontamination with activated charcoal 1. The antidote for belladonna poisoning is Physostigmine, which is the same as for atropine 1. Physosigmine crosses the blood-brain barrier and reversibly inhibits anticholinesterase. Benzodiazepines are frequently used for sedation to control anticholinergic effects including delirium and agitation 2.',
              'Oral LD50 of atropine is 75 mg/kg in mouse. Clinical manifestations of anticholinergic syndrome include both central and peripheral effects. Central symptoms, which are dose-dependent and anticholinergic agent-specific, include ataxia, disorientation, short-term memory loss, confusion, hallucinations, psychosis, agitated delirium, seizures, coma, respiratory failure or cardiovascular collapse 1. Peripheral effects include mydriasis with cycloplegia, dry mucous membranes, hyperreflexia, flushed skin, diminished bowel sounds or ileus, urinary retention, tachycardia, and hypertension or hypotension 1. Management of anticholinergic intoxication should be symptomatic including gastrointestinal decontamination with activated charcoal 1. The antidote for belladonna poisoning is Physostigmine, which is the same as for atropine 1. Physosigmine crosses the blood-brain barrier and reversibly inhibits anticholinesterase. Benzodiazepines are frequently used for sedation to control anticholinergic effects including delirium and agitation 2.'],
             'topril': ['Not Available', 'Not Available'],
             'colchicine': ['The oral LD50 of colchicine in mice is 5.87 mg/kg.15',
              'The oral LD50 of colchicine in mice is 5.87 mg/kg.15'],
             'cortisone s': ['Not Available', 'Not Available'],
             'chlorothiazide': ['Oral, rat LD50: > 10 g/kg. Signs of overdose include those caused by electrolyte depletion (hypokalemia, hypochloremia, hyponatremia) and dehydration resulting from excessive diuresis. If digitalis has also been administered hypokalemia may accentuate cardiac arrhythmias.',
              'Oral, rat LD50: > 10 g/kg. Signs of overdose include those caused by electrolyte depletion (hypokalemia, hypochloremia, hyponatremia) and dehydration resulting from excessive diuresis. If digitalis has also been administered hypokalemia may accentuate cardiac arrhythmias.'],
             'dicyclomine': ['Patients experiencing an overdose may present with headache, nausea, vomiting, blurred vision, dilated pupils, dizziness, dry mouth, difficulty swallowing, CNS stimulation, as well as hot, dry skin.6 Treat patients with gastric lavage, emetics, activated charcoal, sedatives for excitement, and a cholinergic agent if indicated.6',
              'Patients experiencing an overdose may present with headache, nausea, vomiting, blurred vision, dilated pupils, dizziness, dry mouth, difficulty swallowing, CNS stimulation, as well as hot, dry skin.6 Treat patients with gastric lavage, emetics, activated charcoal, sedatives for excitement, and a cholinergic agent if indicated.6'],
             'hydrocortisone': ['Data regarding acute overdoses of glucocorticoids are rare.14,15,16,17,18,19 Chronic high doses of glucocorticoids can lead to the development of cataract, glaucoma, hypertension, water retention, hyperlipidemia, peptic ulcer, pancreatitis, myopathy, osteoporosis, mood changes, psychosis, dermal atrophy, allergy, acne, hypertrichosis, immune suppression, decreased resistance to infection, moon face, hyperglycemia, hypocalcemia, hypophosphatemia, metabolic acidosis, growth suppression, and secondary adrenal insufficiency.8 Overdose may be treated by adjusting the dose or stopping the corticosteroid as well as initiating symptomatic and supportive treatment.8',
              'Data regarding acute overdoses of glucocorticoids are rare.14,15,16,17,18,19 Chronic high doses of glucocorticoids can lead to the development of cataract, glaucoma, hypertension, water retention, hyperlipidemia, peptic ulcer, pancreatitis, myopathy, osteoporosis, mood changes, psychosis, dermal atrophy, allergy, acne, hypertrichosis, immune suppression, decreased resistance to infection, moon face, hyperglycemia, hypocalcemia, hypophosphatemia, metabolic acidosis, growth suppression, and secondary adrenal insufficiency.8 Overdose may be treated by adjusting the dose or stopping the corticosteroid as well as initiating symptomatic and supportive treatment.8'],
             'lisinophctz': ['', ''],
             'lisinopril': ['The oral and subcutaneous LD50 in rats is >8500mg/kg and in mice is >9100mg/kg.11 The oral LDLO in women is 1200µg/kg/16D and in men is 43mg/kg/43W.11',
              'The oral and subcutaneous LD50 in rats is >8500mg/kg and in mice is >9100mg/kg.11 The oral LDLO in women is 1200µg/kg/16D and in men is 43mg/kg/43W.11'],
             'lithium carb xr': ['Not Available', 'Not Available'],
             'phenobarb': ['CNS and respiratory depression which may progress to Cheyne-Stokes respiration, areflexia, constriction of the pupils to a slight degree (though in severe poisoning they may wshow paralytic dilation), oliguria, tachycardia, hypotension, lowered body temperature, and coma. Typical shock syndrome (apnea, circulatory collapse, respiratory arrest, and death) may occur.',
              'CNS and respiratory depression which may progress to Cheyne-Stokes respiration, areflexia, constriction of the pupils to a slight degree (though in severe poisoning they may wshow paralytic dilation), oliguria, tachycardia, hypotension, lowered body temperature, and coma. Typical shock syndrome (apnea, circulatory collapse, respiratory arrest, and death) may occur.'],
             'phenobarbital': ['CNS and respiratory depression which may progress to Cheyne-Stokes respiration, areflexia, constriction of the pupils to a slight degree (though in severe poisoning they may wshow paralytic dilation), oliguria, tachycardia, hypotension, lowered body temperature, and coma. Typical shock syndrome (apnea, circulatory collapse, respiratory arrest, and death) may occur.',
              'CNS and respiratory depression which may progress to Cheyne-Stokes respiration, areflexia, constriction of the pupils to a slight degree (though in severe poisoning they may wshow paralytic dilation), oliguria, tachycardia, hypotension, lowered body temperature, and coma. Typical shock syndrome (apnea, circulatory collapse, respiratory arrest, and death) may occur.'],
             'isosorb din sub': ['Not Available', 'Not Available'],
             'isosorbide dinitrate': ['Symptoms of overdose include reduced cardiac output and hypotension.',
              'Symptoms of overdose include reduced cardiac output and hypotension.'],
             'isosorb din': ['Not Available', 'Not Available'],
             'butalbitalaspirincaffeine': ['', ''],
             'butapapcaf': ['', ''],
             'doxycycline': ['Oral LD50 is 2000 mg/kg in rats, 1870 mg/kg in mice, and 500 mg/kg in dog.14',
              'Oral LD50 is 2000 mg/kg in rats, 1870 mg/kg in mice, and 500 mg/kg in dog.14'],
             'chloroquine': ['Patients experiencing an overdose may present with headache, drowsiness, visual disturbances, nausea, vomiting, cardiovascular collapse, shock, convulsions, respiratory arrest, cardiac arrest, and hypokalemia.19 Overdose should be managed with symptomatic and supportive treatment which may include prompt emesis, gastric lavage, and activated charcoal.19',
              'Patients experiencing an overdose may present with headache, drowsiness, visual disturbances, nausea, vomiting, cardiovascular collapse, shock, convulsions, respiratory arrest, cardiac arrest, and hypokalemia.19 Overdose should be managed with symptomatic and supportive treatment which may include prompt emesis, gastric lavage, and activated charcoal.19'],
             'doxycycline hyc s': ['Oral LD50 is 2000 mg/kg in rats, 1870 mg/kg in mice, and 500 mg/kg in dog.14',
              'Oral LD50 is 2000 mg/kg in rats, 1870 mg/kg in mice, and 500 mg/kg in dog.14'],
             'doxycycline hyclate': ['Oral LD50 is 2000 mg/kg in rats, 1870 mg/kg in mice, and 500 mg/kg in dog.14',
              'Oral LD50 is 2000 mg/kg in rats, 1870 mg/kg in mice, and 500 mg/kg in dog.14'],
             'flurazepam hcl': ['Coma, confusion, low blood pressure, sleepiness',
              'Coma, confusion, low blood pressure, sleepiness'],
             'flurazepam': ['Coma, confusion, low blood pressure, sleepiness',
              'Coma, confusion, low blood pressure, sleepiness'],
             'gemfibrozil s': ['The oral TDLO of gemfibrozil in humans is 18gm/kg/3Y.12 The oral LD50 in mice is 2218mg/kg and in rats is 1414mg/kg.12 The intraperitoneal LD50 in rats is 445mg/kg.12',
              'The oral TDLO of gemfibrozil in humans is 18gm/kg/3Y.12 The oral LD50 in mice is 2218mg/kg and in rats is 1414mg/kg.12 The intraperitoneal LD50 in rats is 445mg/kg.12'],
             'sertraline hcl': ['The LD50 of sertraline is >2000 mg/kg in rats according to the FDA label.25 One other references indicates an oral LD50 of in mice and rats of 419 - 548 mg/kg and 1327 - 1591mg/kg, respectively.MSDS',
              'The LD50 of sertraline is >2000 mg/kg in rats according to the FDA label.25 One other references indicates an oral LD50 of in mice and rats of 419 - 548 mg/kg and 1327 - 1591mg/kg, respectively.MSDS'],
             'folic acid': ['IPR-MUS LD50 85 mg/kg,IVN-GPG LD50 120 mg/kg, IVN-MUS LD50 239 mg/kg, IVN-RAT LD50 500 mg/kg, IVN-RBT LD50 410 mg/kg',
              'IPR-MUS LD50 85 mg/kg,IVN-GPG LD50 120 mg/kg, IVN-MUS LD50 239 mg/kg, IVN-RAT LD50 500 mg/kg, IVN-RBT LD50 410 mg/kg'],
             'ciprofloxacin': ['Patients experiencing an overdose may present with nausea, vomiting, abdominal pain, crystalluria, nephrotoxicity, and oliguria.14,15,16 Ciprofloxacin overdose typically leads to acute renal failure.16 An overdose may progress over the next 6 days with rising serum creatinine and BUN, as well as anuria.15 Patients may require prednisone therapy, urgent hemodialysis, or supportive therapy.14,16 Depending on the degree of overdose, patients may recover normal kidney function or progress to chronic kidney failure.16',
              'Patients experiencing an overdose may present with nausea, vomiting, abdominal pain, crystalluria, nephrotoxicity, and oliguria.14,15,16 Ciprofloxacin overdose typically leads to acute renal failure.16 An overdose may progress over the next 6 days with rising serum creatinine and BUN, as well as anuria.15 Patients may require prednisone therapy, urgent hemodialysis, or supportive therapy.14,16 Depending on the degree of overdose, patients may recover normal kidney function or progress to chronic kidney failure.16'],
             'ciprofloxacin let': ['Patients experiencing an overdose may present with nausea, vomiting, abdominal pain, crystalluria, nephrotoxicity, and oliguria.14,15,16 Ciprofloxacin overdose typically leads to acute renal failure.16 An overdose may progress over the next 6 days with rising serum creatinine and BUN, as well as anuria.15 Patients may require prednisone therapy, urgent hemodialysis, or supportive therapy.14,16 Depending on the degree of overdose, patients may recover normal kidney function or progress to chronic kidney failure.16',
              'Patients experiencing an overdose may present with nausea, vomiting, abdominal pain, crystalluria, nephrotoxicity, and oliguria.14,15,16 Ciprofloxacin overdose typically leads to acute renal failure.16 An overdose may progress over the next 6 days with rising serum creatinine and BUN, as well as anuria.15 Patients may require prednisone therapy, urgent hemodialysis, or supportive therapy.14,16 Depending on the degree of overdose, patients may recover normal kidney function or progress to chronic kidney failure.16'],
             'soriatane': ['Oral, rat: LD50 = >4000 mg/kg. Symptoms of overdose include headache and vertigo.',
              'Oral, rat: LD50 = >4000 mg/kg. Symptoms of overdose include headache and vertigo.'],
             'macrodantin': ['Symptoms of overdose include vomiting.8,9,10 In case of overdose, induce vomiting if it has not already occurred and increase fluid intake to promote urination.8,9,10 In extreme cases, nitrofurantoin can be removed from circulation by dialysis.8,9,10',
              'Symptoms of overdose include vomiting.8,9,10 In case of overdose, induce vomiting if it has not already occurred and increase fluid intake to promote urination.8,9,10 In extreme cases, nitrofurantoin can be removed from circulation by dialysis.8,9,10'],
             'asacol dr': ['Oral, mouse: LD50 = 3370 mg/kg; Oral, rat: LD50 = 2800 mg/kg; Skin, rabbit: LD50 = >5 gm/kg MSDS.',
              'Oral, mouse: LD50 = 3370 mg/kg; Oral, rat: LD50 = 2800 mg/kg; Skin, rabbit: LD50 = >5 gm/kg MSDS.'],
             'prandin': ['LD50 >1 g/kg (rat) (W. Grell)',
              'LD50 >1 g/kg (rat) (W. Grell)'],
             'tetracycline s': ['LD50=808mg/kg (orally in mice)',
              'LD50=808mg/kg (orally in mice)'],
             'glipizide': ['In rats, the oral LD50 is reported to be greater than 4000 mg/kg and the intraperitoneal LD50 is 1200 mg/kg. The lowest published toxic dose (TDLo) via oral route in child was 379 μg/kg.MSDS ',
              'In rats, the oral LD50 is reported to be greater than 4000 mg/kg and the intraperitoneal LD50 is 1200 mg/kg. The lowest published toxic dose (TDLo) via oral route in child was 379 μg/kg.MSDS '],
             'diazepam': ['The symptoms of diazepam overdose are mainly an intensification of the therapeutic effects (ataxia, drowsiness, dysarthria, sedation, muscle weakness, profound sleep, hypotension, bradycardia, nystagmus) or paradoxical excitation 9,10,6. In most cases only observation of vital functions is required 9,10,6.',
              'The symptoms of diazepam overdose are mainly an intensification of the therapeutic effects (ataxia, drowsiness, dysarthria, sedation, muscle weakness, profound sleep, hypotension, bradycardia, nystagmus) or paradoxical excitation 9,10,6. In most cases only observation of vital functions is required 9,10,6.'],
             'baclofen': ['The oral LD50 in rats is 145 mg/kg.14',
              'The oral LD50 in rats is 145 mg/kg.14'],
             'verapamil hydrochloride': ["Verapamil's reported oral TDLo is 14.4 mg/kg in women and 3.429 mg/kg in men.21 The oral LD50 is 150 mg/kg in rats and 163 mg/kg in mice.21",
              "Verapamil's reported oral TDLo is 14.4 mg/kg in women and 3.429 mg/kg in men.21 The oral LD50 is 150 mg/kg in rats and 163 mg/kg in mice.21"],
             'verapamil sr': ["Verapamil's reported oral TDLo is 14.4 mg/kg in women and 3.429 mg/kg in men.21 The oral LD50 is 150 mg/kg in rats and 163 mg/kg in mice.21",
              "Verapamil's reported oral TDLo is 14.4 mg/kg in women and 3.429 mg/kg in men.21 The oral LD50 is 150 mg/kg in rats and 163 mg/kg in mice.21"],
             'ranitidine': ['Oral doses of 1,000 mg/kg in mice and rats were not found to be lethal. Intravenous LD50 values in mice and rats were 77 and 83 mg/kg, respectively.12',
              'Oral doses of 1,000 mg/kg in mice and rats were not found to be lethal. Intravenous LD50 values in mice and rats were 77 and 83 mg/kg, respectively.12'],
             'labetalol hydrochloride': ['The oral LD50 in mice is 600mg/kg and in rats is >2g/kg.7,8 The intravenous LD50 in mice and rats is 50-60mg/kg.7,8',
              'The oral LD50 in mice is 600mg/kg and in rats is >2g/kg.7,8 The intravenous LD50 in mice and rats is 50-60mg/kg.7,8'],
             'misoprostol mcg': ['The oral LD50 in rats is 81mg/kg and in mice is 27mg/kg.12 The intraperitoneal LD50 in rats is 40mg/kg and in mice is 70mg/kg.12',
              'The oral LD50 in rats is 81mg/kg and in mice is 27mg/kg.12 The intraperitoneal LD50 in rats is 40mg/kg and in mice is 70mg/kg.12'],
             'flutamide': ['In animal studies with flutamide alone, signs of overdose included hypoactivity, piloerection, slow respiration, ataxia, and/or lacrimation, anorexia, tranquilization, emesis, and methemoglobinemia.',
              'In animal studies with flutamide alone, signs of overdose included hypoactivity, piloerection, slow respiration, ataxia, and/or lacrimation, anorexia, tranquilization, emesis, and methemoglobinemia.'],
             'lisinopril and hydrochlorothiazide': ['The oral and subcutaneous LD50 in rats is >8500mg/kg and in mice is >9100mg/kg.11 The oral LDLO in women is 1200µg/kg/16D and in men is 43mg/kg/43W.11',
              'The oral and subcutaneous LD50 in rats is >8500mg/kg and in mice is >9100mg/kg.11 The oral LDLO in women is 1200µg/kg/16D and in men is 43mg/kg/43W.11'],
             'anagrelide hydrochloride': ['The oral LD50 of anagrelide as reported in rats and mice is >1500mg/kg and >2500mg/kg, respectively.10 Symptoms of overdose may include hypotension, sinus tachycardia, and vomiting. As the therapeutic effect of anagrelide (i.e. platelet reduction) is dose-related, significant thrombocytopenia is expected in instances of overdose.9 Treatment of overdose should involve careful monitoring of platelet counts and complications such as bleeding.9 Employ symptomatic and supportive measures if clinically indicated.',
              'The oral LD50 of anagrelide as reported in rats and mice is >1500mg/kg and >2500mg/kg, respectively.10 Symptoms of overdose may include hypotension, sinus tachycardia, and vomiting. As the therapeutic effect of anagrelide (i.e. platelet reduction) is dose-related, significant thrombocytopenia is expected in instances of overdose.9 Treatment of overdose should involve careful monitoring of platelet counts and complications such as bleeding.9 Employ symptomatic and supportive measures if clinically indicated.'],
             'benazepril and hydrochlorothiazide': ['The most common adverse effects include headache, dizziness, fatigue, somnolence, postural dizziness, nausea, and coughLabel,4.',
              'The most common adverse effects include headache, dizziness, fatigue, somnolence, postural dizziness, nausea, and coughLabel,4.'],
             'benazephctz': ['', ''],
             'fluconazole': ['Acute oral toxicity (LD50): 1271 mg/kg (rat) MSDS',
              'Acute oral toxicity (LD50): 1271 mg/kg (rat) MSDS'],
             'fluconazole let': ['Acute oral toxicity (LD50): 1271 mg/kg (rat) MSDS',
              'Acute oral toxicity (LD50): 1271 mg/kg (rat) MSDS'],
             'tramadol hcl and acetaminophen': ['The reported LD50 for tramadol, when administered orally in mice, is 350 mg/kg.11 ',
              'The reported LD50 for tramadol, when administered orally in mice, is 350 mg/kg.11 '],
             'tramadlapap': ['', ''],
             'wellbutrin sr': ['Symptoms of overdose include seizures, hallucinations, loss of consciousness, tachycardia, and cardiac arrest.',
              'Symptoms of overdose include seizures, hallucinations, loss of consciousness, tachycardia, and cardiac arrest.'],
             'wellbutrin': ['Symptoms of overdose include seizures, hallucinations, loss of consciousness, tachycardia, and cardiac arrest.',
              'Symptoms of overdose include seizures, hallucinations, loss of consciousness, tachycardia, and cardiac arrest.'],
             'lanoxin': ['Oral TDLO (human female): 100 ug/kg, Oral TDLO (human male): 75 ug/kg, Oral LD50 (rat): 28270 ug/kgMSDS',
              'Oral TDLO (human female): 100 ug/kg, Oral TDLO (human male): 75 ug/kg, Oral LD50 (rat): 28270 ug/kgMSDS'],
             'zantac': ['Oral doses of 1,000 mg/kg in mice and rats were not found to be lethal. Intravenous LD50 values in mice and rats were 77 and 83 mg/kg, respectively.12',
              'Oral doses of 1,000 mg/kg in mice and rats were not found to be lethal. Intravenous LD50 values in mice and rats were 77 and 83 mg/kg, respectively.12'],
             'ceftin': ['Allergic reactions might be expected, including rash, nasal congestion, cough, dry throat, eye irritation, or anaphylactic shock. Overdosage of cephalosporins can cause cerebral irritation leading to convulsions.',
              'Allergic reactions might be expected, including rash, nasal congestion, cough, dry throat, eye irritation, or anaphylactic shock. Overdosage of cephalosporins can cause cerebral irritation leading to convulsions.'],
             'lamictal': ['The oral LD50 in mouse and rat is 205 mg/kg and 245 mg/kg, respectively.MSDS',
              'The oral LD50 in mouse and rat is 205 mg/kg and 245 mg/kg, respectively.MSDS'],
             'zyban sr': ['Symptoms of overdose include seizures, hallucinations, loss of consciousness, tachycardia, and cardiac arrest.',
              'Symptoms of overdose include seizures, hallucinations, loss of consciousness, tachycardia, and cardiac arrest.'],
             'amerge': ['Symptoms of overdose include light-headedness, loss of coordination, tension in the neck, and tiredness.',
              'Symptoms of overdose include light-headedness, loss of coordination, tension in the neck, and tiredness.'],
             'valtrex gm': ['LD50 Oral ', 'LD50 Oral '],
             'zofran odt': ['At present, there is little information concerning overdosage with ondansetron 8,9,10. Nevertheless, there have been certain cases of somewhat idiosyncratic adverse effects associated with particular dosages of ondansetron used 8,9,10.',
              'At present, there is little information concerning overdosage with ondansetron 8,9,10. Nevertheless, there have been certain cases of somewhat idiosyncratic adverse effects associated with particular dosages of ondansetron used 8,9,10.'],
             'combivir': ['The most common reported adverse reactions (incidence ≥15%) in adults were headache, nausea, malaise and fatigue, nasal signs and symptoms, diarrhea, and cough. ',
              'The most common reported adverse reactions (incidence ≥15%) in adults were headache, nausea, malaise and fatigue, nasal signs and symptoms, diarrhea, and cough. '],
             'leukeran': ['Not Available', 'Not Available'],
             'epivirhbv': ['', ''],
             'malarone': ['Not Available', 'Not Available'],
             'malarone pediatric': ['Not Available', 'Not Available'],
             'avodart soft gelatin': ['LD50 values', 'LD50 values'],
             'avodart': ['LD50 values', 'LD50 values'],
             'myleran': ['Signs of overdose include allergic reaction, unusual bleeding or bruising, sudden weakness or unusual fatigue, persistent cough, congestion, or shortness of breath; flank, stomach or joint pain; pronounced nausea, vomiting, diarrhea, dizziness, confusion, or darkening of the skin, chills, fever, collapse, and loss of consciousness.',
              'Signs of overdose include allergic reaction, unusual bleeding or bruising, sudden weakness or unusual fatigue, persistent cough, congestion, or shortness of breath; flank, stomach or joint pain; pronounced nausea, vomiting, diarrhea, dizziness, confusion, or darkening of the skin, chills, fever, collapse, and loss of consciousness.'],
             'imitrex (sumatriptan succinate)': ['Symptoms of overdose include convulsions, tremor, paralysis, inactivity, ptosis, erythema of the extremities, abnormal respiration, cyanosis, ataxia, mydriasis, salivation, and lacrimation.5,6,7,8,9,10 Overdoses may be fatal and patients should be monitored for 3-5 half lives or while symptoms persist.5,6,7,8,9,10',
              'Symptoms of overdose include convulsions, tremor, paralysis, inactivity, ptosis, erythema of the extremities, abnormal respiration, cyanosis, ataxia, mydriasis, salivation, and lacrimation.5,6,7,8,9,10 Overdoses may be fatal and patients should be monitored for 3-5 half lives or while symptoms persist.5,6,7,8,9,10'],
             'tykerb': ['There has been a report of one patient who took 3,000 mg of lapatinib for 10 days. This patient had grade 3 diarrhea and vomiting on day 10.',
              'There has been a report of one patient who took 3,000 mg of lapatinib for 10 days. This patient had grade 3 diarrhea and vomiting on day 10.'],
             'lamictal xr': ['The oral LD50 in mouse and rat is 205 mg/kg and 245 mg/kg, respectively.MSDS',
              'The oral LD50 in mouse and rat is 205 mg/kg and 245 mg/kg, respectively.MSDS'],
             'lamictal xr (lamotrigine)': ['The oral LD50 in mouse and rat is 205 mg/kg and 245 mg/kg, respectively.MSDS',
              'The oral LD50 in mouse and rat is 205 mg/kg and 245 mg/kg, respectively.MSDS'],
             'lamictal odt': ['The oral LD50 in mouse and rat is 205 mg/kg and 245 mg/kg, respectively.MSDS',
              'The oral LD50 in mouse and rat is 205 mg/kg and 245 mg/kg, respectively.MSDS'],
             'lovaza gm': ['In rats, omega-3-acid ethyl esters are not mutagenic or clastogenic and do not lead to impairment of fertilityLabel.',
              'In rats, omega-3-acid ethyl esters are not mutagenic or clastogenic and do not lead to impairment of fertilityLabel.'],
             'lovaza': ['In rats, omega-3-acid ethyl esters are not mutagenic or clastogenic and do not lead to impairment of fertilityLabel.',
              'In rats, omega-3-acid ethyl esters are not mutagenic or clastogenic and do not lead to impairment of fertilityLabel.'],
             'dynacirc cr': ['Symptoms of overdose include lethargy, sinus tachycardia, and transient hypotension. Significant lethality was observed in mice given oral doses of over 200 mg/kg and rabbits given about 50 mg/kg of isradipine. Rats tolerated doses of over 2000 mg/kg without effects on survival.',
              'Symptoms of overdose include lethargy, sinus tachycardia, and transient hypotension. Significant lethality was observed in mice given oral doses of over 200 mg/kg and rabbits given about 50 mg/kg of isradipine. Rats tolerated doses of over 2000 mg/kg without effects on survival.'],
             'innopran xl er': ['Symptoms of overdose include hypotension, hypoglycemic seizure, restlessness, euphoria, insomnia.8 Patients with asthma may develop bronchospasm.8 In case of overdose, monitor vital signs, mental status, and blood glucose.8 Treat hypotension with intravenous fluids, bradycardia with atropine, and isoproterenol and aminophylline for bronchospasm.8 If patients do not respond to intravenous fluids, follow up with glucagon 50-150µg/kg intravenously, then 1-5mg/hour, followed by catecholamines.8,9,10 Dialysis will not be useful as propranolol is highly protein bound.8,9,10',
              'Symptoms of overdose include hypotension, hypoglycemic seizure, restlessness, euphoria, insomnia.8 Patients with asthma may develop bronchospasm.8 In case of overdose, monitor vital signs, mental status, and blood glucose.8 Treat hypotension with intravenous fluids, bradycardia with atropine, and isoproterenol and aminophylline for bronchospasm.8 If patients do not respond to intravenous fluids, follow up with glucagon 50-150µg/kg intravenously, then 1-5mg/hour, followed by catecholamines.8,9,10 Dialysis will not be useful as propranolol is highly protein bound.8,9,10'],
             'innopran xl': ['Symptoms of overdose include hypotension, hypoglycemic seizure, restlessness, euphoria, insomnia.8 Patients with asthma may develop bronchospasm.8 In case of overdose, monitor vital signs, mental status, and blood glucose.8 Treat hypotension with intravenous fluids, bradycardia with atropine, and isoproterenol and aminophylline for bronchospasm.8 If patients do not respond to intravenous fluids, follow up with glucagon 50-150µg/kg intravenously, then 1-5mg/hour, followed by catecholamines.8,9,10 Dialysis will not be useful as propranolol is highly protein bound.8,9,10',
              'Symptoms of overdose include hypotension, hypoglycemic seizure, restlessness, euphoria, insomnia.8 Patients with asthma may develop bronchospasm.8 In case of overdose, monitor vital signs, mental status, and blood glucose.8 Treat hypotension with intravenous fluids, bradycardia with atropine, and isoproterenol and aminophylline for bronchospasm.8 If patients do not respond to intravenous fluids, follow up with glucagon 50-150µg/kg intravenously, then 1-5mg/hour, followed by catecholamines.8,9,10 Dialysis will not be useful as propranolol is highly protein bound.8,9,10'],
             'rythmol': ['Symptoms of propafenone overdose (usually most severe within the first 3 hours) may include convulsions (rarely), heartbeat irregularities, low blood pressure, and sleepiness.',
              'Symptoms of propafenone overdose (usually most severe within the first 3 hours) may include convulsions (rarely), heartbeat irregularities, low blood pressure, and sleepiness.'],
             'horizant': ['Most common adverse reactions are headache, dizziness, and somnolence.',
              'Most common adverse reactions are headache, dizziness, and somnolence.'],
             'jalyn': ['LD50 values', 'LD50 values'],
             'staxyn': ['Symptoms of overdose include vision changes and back and muscle pain.',
              'Symptoms of overdose include vision changes and back and muscle pain.'],
             'rythmol sr er': ['Symptoms of propafenone overdose (usually most severe within the first 3 hours) may include convulsions (rarely), heartbeat irregularities, low blood pressure, and sleepiness.',
              'Symptoms of propafenone overdose (usually most severe within the first 3 hours) may include convulsions (rarely), heartbeat irregularities, low blood pressure, and sleepiness.'],
             'levitra': ['Symptoms of overdose include vision changes and back and muscle pain.',
              'Symptoms of overdose include vision changes and back and muscle pain.'],
             'loid': ['The oral TDLO of gemfibrozil in humans is 18gm/kg/3Y.12 The oral LD50 in mice is 2218mg/kg and in rats is 1414mg/kg.12 The intraperitoneal LD50 in rats is 445mg/kg.12',
              'The oral TDLO of gemfibrozil in humans is 18gm/kg/3Y.12 The oral LD50 in mice is 2218mg/kg and in rats is 1414mg/kg.12 The intraperitoneal LD50 in rats is 445mg/kg.12'],
             'valtrex': ['LD50 Oral ', 'LD50 Oral '],
             'zovirax': ['Symptoms of overdose include agitation, coma, seizures, lethargy, and precipitation in renal tubules.11 These symptoms are more common in patients given high doses without monitoring of fluid and electrolyte balance or reduced kidney function.11,4,6 In the case of an overdose, treat with symptomatic and supportive care.13',
              'Symptoms of overdose include agitation, coma, seizures, lethargy, and precipitation in renal tubules.11 These symptoms are more common in patients given high doses without monitoring of fluid and electrolyte balance or reduced kidney function.11,4,6 In the case of an overdose, treat with symptomatic and supportive care.13'],
             'ferralet': ['Acute iron overdosage can be divided into four stages. In the first stage, which occurs up to six hours after ingestion, the principal symptoms are vomiting and diarrhea. Other symptoms include hypotension, tachycardia and CNS depression ranging from lethargy to coma. The second phase may occur at 6-24 hours after ingestion and is characterized by a temporary remission. In the third phase, gastrointestinal symptoms recur accompanied by shock, metabolic acidosis, coma, hepatic necrosis and jaundice, hypoglycemia, renal failure and pulmonary edema. The fourth phase may occur several weeks after ingestion and is characterized by gastrointestinal obstruction and liver damage. In a young child, 75 milligrams per kilogram is considered extremely dangerous. A dose of 30 milligrams per kilogram can lead to symptoms of toxicity. Estimates of a lethal dosage range from 180 milligrams per kilogram and upwards. A peak serum iron concentration of five micrograms or more per ml is associated with moderate to severe poisoning in many.',
              'Acute iron overdosage can be divided into four stages. In the first stage, which occurs up to six hours after ingestion, the principal symptoms are vomiting and diarrhea. Other symptoms include hypotension, tachycardia and CNS depression ranging from lethargy to coma. The second phase may occur at 6-24 hours after ingestion and is characterized by a temporary remission. In the third phase, gastrointestinal symptoms recur accompanied by shock, metabolic acidosis, coma, hepatic necrosis and jaundice, hypoglycemia, renal failure and pulmonary edema. The fourth phase may occur several weeks after ingestion and is characterized by gastrointestinal obstruction and liver damage. In a young child, 75 milligrams per kilogram is considered extremely dangerous. A dose of 30 milligrams per kilogram can lead to symptoms of toxicity. Estimates of a lethal dosage range from 180 milligrams per kilogram and upwards. A peak serum iron concentration of five micrograms or more per ml is associated with moderate to severe poisoning in many.'],
             'potassium citrate': ['LD50 (dog): Intravenous 176 mg/kg.',
              'LD50 (dog): Intravenous 176 mg/kg.'],
             'aspirin': ['Lethal doses', 'Lethal doses'],
             'tetracycline hcl': ['LD50=808mg/kg (orally in mice)',
              'LD50=808mg/kg (orally in mice)'],
             'vitamin b': ['Not Available', 'Not Available'],
             'enteric aspirin': ['Lethal doses', 'Lethal doses'],
             'sennagen': ['Senna causes increased amounts of apoptosis in the large intestine shortly after use due to upregulated p53 activity3. This is normally reversed after 18 hours however chronic use has been shown to be associated with p53 resistance and potential carcinogenicity leading to colon cancer3. The LD50 value in rats was 5000mg/kg. Subacute studies in rats receiving 20mg/kg and dogs receiving 500mg/kg did not produce signs of toxicity5. Tests for mutagenicity and reproductive toxicity do not indicate toxic effects5.',
              'Senna causes increased amounts of apoptosis in the large intestine shortly after use due to upregulated p53 activity3. This is normally reversed after 18 hours however chronic use has been shown to be associated with p53 resistance and potential carcinogenicity leading to colon cancer3. The LD50 value in rats was 5000mg/kg. Subacute studies in rats receiving 20mg/kg and dogs receiving 500mg/kg did not produce signs of toxicity5. Tests for mutagenicity and reproductive toxicity do not indicate toxic effects5.'],
             'sennas': ['Not Available', 'Not Available'],
             'calcarb with vitamin d': ['Not Available', 'Not Available'],
             'labetalol hcl': ['The oral LD50 in mice is 600mg/kg and in rats is >2g/kg.7,8 The intravenous LD50 in mice and rats is 50-60mg/kg.7,8',
              'The oral LD50 in mice is 600mg/kg and in rats is >2g/kg.7,8 The intravenous LD50 in mice and rats is 50-60mg/kg.7,8'],
             'midodrin': ['Symptoms of overdose could include hypertension, piloerection (goosebumps), a sensation of coldness and urinary retention. The single doses that would be associated with symptoms of overdosage or would be potentially life- threatening are unknown. The oral LD50 is approximately 30 to 50 mg/kg in rats, 675 mg/kg in mice, and 125 to 160 mg/kg in dogs. Desglymidodrine is dialyzable.',
              'Symptoms of overdose could include hypertension, piloerection (goosebumps), a sensation of coldness and urinary retention. The single doses that would be associated with symptoms of overdosage or would be potentially life- threatening are unknown. The oral LD50 is approximately 30 to 50 mg/kg in rats, 675 mg/kg in mice, and 125 to 160 mg/kg in dogs. Desglymidodrine is dialyzable.'],
             'nitrofurantoin monomacro': ['Symptoms of overdose include vomiting.8,9,10 In case of overdose, induce vomiting if it has not already occurred and increase fluid intake to promote urination.8,9,10 In extreme cases, nitrofurantoin can be removed from circulation by dialysis.8,9,10',
              'Symptoms of overdose include vomiting.8,9,10 In case of overdose, induce vomiting if it has not already occurred and increase fluid intake to promote urination.8,9,10 In extreme cases, nitrofurantoin can be removed from circulation by dialysis.8,9,10'],
             'benazepril hydrochloride and hydrochlorothiazide': ['The most common adverse effects include headache, dizziness, fatigue, somnolence, postural dizziness, nausea, and coughLabel,4.',
              'The most common adverse effects include headache, dizziness, fatigue, somnolence, postural dizziness, nausea, and coughLabel,4.'],
             'midodrine': ['Symptoms of overdose could include hypertension, piloerection (goosebumps), a sensation of coldness and urinary retention. The single doses that would be associated with symptoms of overdosage or would be potentially life- threatening are unknown. The oral LD50 is approximately 30 to 50 mg/kg in rats, 675 mg/kg in mice, and 125 to 160 mg/kg in dogs. Desglymidodrine is dialyzable.',
              'Symptoms of overdose could include hypertension, piloerection (goosebumps), a sensation of coldness and urinary retention. The single doses that would be associated with symptoms of overdosage or would be potentially life- threatening are unknown. The oral LD50 is approximately 30 to 50 mg/kg in rats, 675 mg/kg in mice, and 125 to 160 mg/kg in dogs. Desglymidodrine is dialyzable.'],
             'sotalol hcl': ['Patients experiencing an overdose may present with bradycardia, congestive heart failure, hypotension, bronchospasm, and hypoglycemia.Label Larger intentional overdoses may present as hypotension, bradycardia, cardiac asystole, prolonged QT interval, torsade de pointes, ventricular tachycardia, and premature ventricular complexes.Label Stop administering sotalol and observe the patient until the QT interval returns to normal and the heart rate rises above 50 beats per minute.Label Hemodialysis may help lower plasma concentrations of sotalol as it is not bound to plasma proteins.Label Bradycardia and cardiac asystole may be treated with atropine, other anticholinergic drugs, beta adrenergic agonists, or transvenous cardiac pacing.Label. Second or third degree heart block may be treated with a transvenous cardiac pacemaker.Label Hypotension may be treated with epinephrine or norepinephrine.Label Bronchospasm may be treated with aminophylline or a beta-2 agonist, possibly at higher than normal doses.Label Torsade de pointes may be treated with DC cardioversion, transvenous cardiac pacing, epinephrine, or magnesium sulfate.Label',
              'Patients experiencing an overdose may present with bradycardia, congestive heart failure, hypotension, bronchospasm, and hypoglycemia.Label Larger intentional overdoses may present as hypotension, bradycardia, cardiac asystole, prolonged QT interval, torsade de pointes, ventricular tachycardia, and premature ventricular complexes.Label Stop administering sotalol and observe the patient until the QT interval returns to normal and the heart rate rises above 50 beats per minute.Label Hemodialysis may help lower plasma concentrations of sotalol as it is not bound to plasma proteins.Label Bradycardia and cardiac asystole may be treated with atropine, other anticholinergic drugs, beta adrenergic agonists, or transvenous cardiac pacing.Label. Second or third degree heart block may be treated with a transvenous cardiac pacemaker.Label Hypotension may be treated with epinephrine or norepinephrine.Label Bronchospasm may be treated with aminophylline or a beta-2 agonist, possibly at higher than normal doses.Label Torsade de pointes may be treated with DC cardioversion, transvenous cardiac pacing, epinephrine, or magnesium sulfate.Label'],
             'methimazole': ['The oral LD50 of methimazole in rats is 2250 mg/kg.17 Signs and symptoms of methimazole overdose may include gastrointestinal distress, headache, fever, joint pain, pruritus, and edema. More serious adverse effects, such as aplastic anemia or agranulocytosis, may manifest within hours to days.18,19 Hepatitis, nephrotic syndrome, exfoliative dermatitis, and CNS effects such as neuropathy or CNS depression/stimulation are also potential, albeit less frequent, results of overdose.18,19',
              'The oral LD50 of methimazole in rats is 2250 mg/kg.17 Signs and symptoms of methimazole overdose may include gastrointestinal distress, headache, fever, joint pain, pruritus, and edema. More serious adverse effects, such as aplastic anemia or agranulocytosis, may manifest within hours to days.18,19 Hepatitis, nephrotic syndrome, exfoliative dermatitis, and CNS effects such as neuropathy or CNS depression/stimulation are also potential, albeit less frequent, results of overdose.18,19'],
             'bupropion sr': ['Symptoms of overdose include seizures, hallucinations, loss of consciousness, tachycardia, and cardiac arrest.',
              'Symptoms of overdose include seizures, hallucinations, loss of consciousness, tachycardia, and cardiac arrest.'],
             'bupropion hydrochloride (sr)': ['Symptoms of overdose include seizures, hallucinations, loss of consciousness, tachycardia, and cardiac arrest.',
              'Symptoms of overdose include seizures, hallucinations, loss of consciousness, tachycardia, and cardiac arrest.'],
             'itraconazole': ['No significant lethality was observed when itraconazole was administered orally to mice and rats at dosage levels of 320 mg/kg or to dogs at 200 mg/kg.',
              'No significant lethality was observed when itraconazole was administered orally to mice and rats at dosage levels of 320 mg/kg or to dogs at 200 mg/kg.'],
             'hydoxyzine pamoate': ['The oral LD50 is 840 mg/kg in rats and 400 mg/kg in mice.16 ',
              'The oral LD50 is 840 mg/kg in rats and 400 mg/kg in mice.16 '],
             'phentermine': ['The reported LD50 after oral administration of phentermine in rats is reported to be of 151 mg/kg.12 Reports of acute overdose include restlessness, tremors, hyperreflexia, rapid respiration, confusion, assaultiveness, hallucinations and panic state followed by fatigue, and depression. In the cardiovascular system, there are reports of tachycardia, arrhythmia, hypertension, hypotension, circulatory collapse. In the GI tract, there are symptoms of nausea, vomiting, diarrhea and abdominal cramps. The management of acute overdosage includes symptomatic treatment as well as lavage and sedation with barbiturates.Label',
              'The reported LD50 after oral administration of phentermine in rats is reported to be of 151 mg/kg.12 Reports of acute overdose include restlessness, tremors, hyperreflexia, rapid respiration, confusion, assaultiveness, hallucinations and panic state followed by fatigue, and depression. In the cardiovascular system, there are reports of tachycardia, arrhythmia, hypertension, hypotension, circulatory collapse. In the GI tract, there are symptoms of nausea, vomiting, diarrhea and abdominal cramps. The management of acute overdosage includes symptomatic treatment as well as lavage and sedation with barbiturates.Label'],
             'hydroxyzine pamoate': ['The oral LD50 is 840 mg/kg in rats and 400 mg/kg in mice.16 ',
              'The oral LD50 is 840 mg/kg in rats and 400 mg/kg in mice.16 '],
             'bisoprolol fumarate and hydrochlorothiazide': ['LD50 information\nOral LD50 of bisoprolol in the mouse was 730 mg/kg.20',
              'LD50 information\nOral LD50 of bisoprolol in the mouse was 730 mg/kg.20'],
             'bisoprololhctz': ['', ''],
             'bisoprolol': ['LD50 information\nOral LD50 of bisoprolol in the mouse was 730 mg/kg.20',
              'LD50 information\nOral LD50 of bisoprolol in the mouse was 730 mg/kg.20'],
             'indomethacin sr': ['Acute oral LD50 is 2.42 mg/kg in rats and 13 mg/kg in mice.MSDS The oral LD50 of indomethacin in mice and rats (based on 14-day mortality response) was 50 and 12 mg/kg, respectively.13 ',
              'Acute oral LD50 is 2.42 mg/kg in rats and 13 mg/kg in mice.MSDS The oral LD50 of indomethacin in mice and rats (based on 14-day mortality response) was 50 and 12 mg/kg, respectively.13 '],
             'bisoprolol fum': ['LD50 information\nOral LD50 of bisoprolol in the mouse was 730 mg/kg.20',
              'LD50 information\nOral LD50 of bisoprolol in the mouse was 730 mg/kg.20'],
             'isoniazid': ['0 mg/kg',
              'oses interfere with the coenzyme function of the latter'],
             'tizanidine': ['LD50 information ', 'LD50 information '],
             'phendimetraz er': ['Acute overdosage of phendimetrazine may manifest itself by the following signs and symptoms: unusual restlessness, confusion, belligerance, hallucinations, and panic states. Fatigue and depression usually follow the central stimulation. Cardiovascular effects include arrhythmias, hypertension, or hypotension and circulatory collapse. Gastrointestinal symptoms include nausea, vomiting, diarrhea, and abdominal cramps. Poisoning may result in convulsions, coma and death.',
              'Acute overdosage of phendimetrazine may manifest itself by the following signs and symptoms: unusual restlessness, confusion, belligerance, hallucinations, and panic states. Fatigue and depression usually follow the central stimulation. Cardiovascular effects include arrhythmias, hypertension, or hypotension and circulatory collapse. Gastrointestinal symptoms include nausea, vomiting, diarrhea, and abdominal cramps. Poisoning may result in convulsions, coma and death.'],
             'atacand': ['No lethality was observed in acute toxicity studies in mice, rats and dogs given single oral doses of up to 2000 mg/kg of candesartan cilexetil or in rats given single oral doses of up to 2000 mg/kg of candesartan cilexetil in combination with 1000 mg/kg of hydrochlorothiazide. In mice given single oral doses of the primary metabolite, candesartan, the minimum lethal dose was greater than 1000 mg/kg but less than 2000 mg/kg.',
              'No lethality was observed in acute toxicity studies in mice, rats and dogs given single oral doses of up to 2000 mg/kg of candesartan cilexetil or in rats given single oral doses of up to 2000 mg/kg of candesartan cilexetil in combination with 1000 mg/kg of hydrochlorothiazide. In mice given single oral doses of the primary metabolite, candesartan, the minimum lethal dose was greater than 1000 mg/kg but less than 2000 mg/kg.'],
             'atacand hct': ['No lethality was observed in acute toxicity studies in mice, rats and dogs given single oral doses of up to 2000 mg/kg of candesartan cilexetil or in rats given single oral doses of up to 2000 mg/kg of candesartan cilexetil in combination with 1000 mg/kg of hydrochlorothiazide. In mice given single oral doses of the primary metabolite, candesartan, the minimum lethal dose was greater than 1000 mg/kg but less than 2000 mg/kg.',
              'No lethality was observed in acute toxicity studies in mice, rats and dogs given single oral doses of up to 2000 mg/kg of candesartan cilexetil or in rats given single oral doses of up to 2000 mg/kg of candesartan cilexetil in combination with 1000 mg/kg of hydrochlorothiazide. In mice given single oral doses of the primary metabolite, candesartan, the minimum lethal dose was greater than 1000 mg/kg but less than 2000 mg/kg.'],
             'vimovo': ['Although the over-the-counter (OTC) availability of naproxen provides convenience to patients, it also increases the likelihood of overdose.3 Thankfully, the extent of overdose is typically mild with adverse effects normally limited to drowsiness, lethargy, epigastric pain, nausea and vomiting.31213 Although there is no antidote for naproxen overdose, symptoms will typically subside with appropriate supportive care.12133',
              'Although the over-the-counter (OTC) availability of naproxen provides convenience to patients, it also increases the likelihood of overdose.3 Thankfully, the extent of overdose is typically mild with adverse effects normally limited to drowsiness, lethargy, epigastric pain, nausea and vomiting.31213 Although there is no antidote for naproxen overdose, symptoms will typically subside with appropriate supportive care.12133'],
             'brilinta ticagrelor': ['Patients experiencing an overdose may present with bleeding, nausea, vomiting, diarrhea, and ventricular pauses.6 Overdose can be managed through symptomatic and supportive treatment, including ECG monitoring.6,7 Dialysis is not expected to remove ticagrelor from the blood due to it being highly protein bound.6',
              'Patients experiencing an overdose may present with bleeding, nausea, vomiting, diarrhea, and ventricular pauses.6 Overdose can be managed through symptomatic and supportive treatment, including ECG monitoring.6,7 Dialysis is not expected to remove ticagrelor from the blood due to it being highly protein bound.6'],
             'toprol xl': ['Oral administration of metoprolol to rats presents an LD50 in the range of 3090 to 4670 mg/kg. Cases of overdose have reported bradycardia, hypotension, bronchospasm, and cardiac failure. In the case of an overdose, gastric lavage is recommended followed by specific treatment according to symptoms.Label',
              'Oral administration of metoprolol to rats presents an LD50 in the range of 3090 to 4670 mg/kg. Cases of overdose have reported bradycardia, hypotension, bronchospasm, and cardiac failure. In the case of an overdose, gastric lavage is recommended followed by specific treatment according to symptoms.Label'],
             'nexium': ['Blurred vision, confusion, drowsiness, dry mouth, flushing headache, nausea, rapid heartbeat, sweating',
              'Blurred vision, confusion, drowsiness, dry mouth, flushing headache, nausea, rapid heartbeat, sweating'],
             'cardizem cd': ['Clinical Toxicity and Overdose',
              'Clinical Toxicity and Overdose'],
             'phrenilin forte s': ['The oral LD50 of caffeine in rats is 192 mg/kg.MSDS An acute fatal overdose of caffeine in humans is about 10–14 grams (equivalent to 150–200 mg/kg of body weight).14',
              'The oral LD50 of caffeine in rats is 192 mg/kg.MSDS An acute fatal overdose of caffeine in humans is about 10–14 grams (equivalent to 150–200 mg/kg of body weight).14'],
             'mephyton': ['High doses of vitamin K1 are not associated with toxicity.6 Intravenous administration has been associated with an increased risk of toxicity.9 These patients should be treated with symptomatic and supportive measures.',
              'High doses of vitamin K1 are not associated with toxicity.6 Intravenous administration has been associated with an increased risk of toxicity.9 These patients should be treated with symptomatic and supportive measures.'],
             'alprazolam': ['Alprazolam overdose can present as sleepiness, confusion, poor coordination, slow reflexes, coma, and death.18,19 Taking alprazolam with alcohol lowers the threshold for overdose.18,19 Patients should have their respiration, pulse, and blood pressure monitored.18,19 Patients can be treated by gastric lavage and intravenous fluids.18,19. If hypotension occurs, patients may be treated with vasopressors.18,19 In known, or suspected overdoses, patients can be given the benzodiazepine receptor antagonist flumazenil in addition to other methods of management.18,19',
              'Alprazolam overdose can present as sleepiness, confusion, poor coordination, slow reflexes, coma, and death.18,19 Taking alprazolam with alcohol lowers the threshold for overdose.18,19 Patients should have their respiration, pulse, and blood pressure monitored.18,19 Patients can be treated by gastric lavage and intravenous fluids.18,19. If hypotension occurs, patients may be treated with vasopressors.18,19 In known, or suspected overdoses, patients can be given the benzodiazepine receptor antagonist flumazenil in addition to other methods of management.18,19'],
             'oxazepam s': ['The oral LD50 in rats and mice is >8000 mg/kg and 1540 mg/kg, respectively.16',
              'The oral LD50 in rats and mice is >8000 mg/kg and 1540 mg/kg, respectively.16'],
             'oxazepam': ['The oral LD50 in rats and mice is >8000 mg/kg and 1540 mg/kg, respectively.16',
              'The oral LD50 in rats and mice is >8000 mg/kg and 1540 mg/kg, respectively.16'],
             'temazepam': ['Manifestations of acute overdosage of temazepam, as with other benzodiazepines, can be expected to reflect the increasing CNS effects of the drug and include somnolence, confusion, and coma, with reduced or absent reflexes Label 12,13,14. With large overdoses, respiratory depression, hypotension, and finally coma can occur Label 12,13,14.',
              'Manifestations of acute overdosage of temazepam, as with other benzodiazepines, can be expected to reflect the increasing CNS effects of the drug and include somnolence, confusion, and coma, with reduced or absent reflexes Label 12,13,14. With large overdoses, respiratory depression, hypotension, and finally coma can occur Label 12,13,14.'],
             'clonidine hydrochloride': ['Oral LD50 is 126 mg/kg in rats.12 The TDLO is 70µg/kg in children, 126µg/kg in women, and 69µg/kg in men.12',
              'Oral LD50 is 126 mg/kg in rats.12 The TDLO is 70µg/kg in children, 126µg/kg in women, and 69µg/kg in men.12'],
             'clonidine': ['Oral LD50 is 126 mg/kg in rats.12 The TDLO is 70µg/kg in children, 126µg/kg in women, and 69µg/kg in men.12',
              'Oral LD50 is 126 mg/kg in rats.12 The TDLO is 70µg/kg in children, 126µg/kg in women, and 69µg/kg in men.12'],
             'propylthiouracil': ['Oral, rat: LD50 = 1250 mg/kg.',
              'Oral, rat: LD50 = 1250 mg/kg.'],
             'carblevo': ['', ''],
             'indapamide': ['ide overdose symptoms may include but are not limited to nausea, vomiting, gastrointestinal disorders, electrolyte disturbances and weakness.23 Other signs of overdose include respiratory depression and severe hypotension.23 In cases of overdose, supportive care interventions may be necessary to manage symptoms.23 Emesis and gastric lavage may be recommended to empty the stomac',
              'r, patients should be monitored closely for any electrolyte or fluid imbalances.2'],
             'diltiazemcd': ['Clinical Toxicity and Overdose',
              'Clinical Toxicity and Overdose'],
             'spironolactone': ['Patients experiencing an overdose may present with drowsiness, mental confusion, maculopapular or erythematous rash, nausea, vomiting, dizziness, or diarrhea. Vomiting is generally induced or a gastric lavage is performed.Label Supportive treatment involves maintining hydration, electrolyte balance, and vital functionsLabel.',
              'Patients experiencing an overdose may present with drowsiness, mental confusion, maculopapular or erythematous rash, nausea, vomiting, dizziness, or diarrhea. Vomiting is generally induced or a gastric lavage is performed.Label Supportive treatment involves maintining hydration, electrolyte balance, and vital functionsLabel.'],
             'glybmetform': ['', ''],
             'pilocarpine hydrochloride': ['Not Available', 'Not Available'],
             'fenofibric acid sule': ['Oral LD50 (rat): 1242 mg/kg, Oral LD50 (mouse): 100 mg/kg, lntraperitoneal LD50 (mouse): 500 mg/kg MSDS',
              'Oral LD50 (rat): 1242 mg/kg, Oral LD50 (mouse): 100 mg/kg, lntraperitoneal LD50 (mouse): 500 mg/kg MSDS'],
             'fenofibric acid': ['Oral LD50 (rat): 1242 mg/kg, Oral LD50 (mouse): 100 mg/kg, lntraperitoneal LD50 (mouse): 500 mg/kg MSDS',
              'Oral LD50 (rat): 1242 mg/kg, Oral LD50 (mouse): 100 mg/kg, lntraperitoneal LD50 (mouse): 500 mg/kg MSDS'],
             'fluvoxamine maleate': ['Fluvoxamine is a member of antidepressants that possess an increased risk compared to placebo of suicidal thinking and behaviour (suicidality) in children, adolescents, and young adults (ages including and and below 24) in short-term studies of major depressive disorder and other psychiatric disorders Label.',
              'Fluvoxamine is a member of antidepressants that possess an increased risk compared to placebo of suicidal thinking and behaviour (suicidality) in children, adolescents, and young adults (ages including and and below 24) in short-term studies of major depressive disorder and other psychiatric disorders Label.'],
             'glipizide er': ['In rats, the oral LD50 is reported to be greater than 4000 mg/kg and the intraperitoneal LD50 is 1200 mg/kg. The lowest published toxic dose (TDLo) via oral route in child was 379 μg/kg.MSDS ',
              'In rats, the oral LD50 is reported to be greater than 4000 mg/kg and the intraperitoneal LD50 is 1200 mg/kg. The lowest published toxic dose (TDLo) via oral route in child was 379 μg/kg.MSDS '],
             'tamsulosin': ['In the event of overdose, patients may experience hypotension and should lie down in a supine position to maintain blood pressure and heart rate.Label If further measures are required intravenous fluids should be considered.Label If further progression is required, vasopressors may be used and renal function should be monitored.Label Dialysis is unlikely to assist in treating overdose because tamsulosin is extensively protein bound.Label',
              'In the event of overdose, patients may experience hypotension and should lie down in a supine position to maintain blood pressure and heart rate.Label If further measures are required intravenous fluids should be considered.Label If further progression is required, vasopressors may be used and renal function should be monitored.Label Dialysis is unlikely to assist in treating overdose because tamsulosin is extensively protein bound.Label'],
             'buprenorphine sublingual': ['Manifestations of acute overdose include pinpoint pupils, sedation, hypotension, respiratory depression and death.',
              'Manifestations of acute overdose include pinpoint pupils, sedation, hypotension, respiratory depression and death.'],
             'buprenorphine and naloxone sublingual': ['Manifestations of acute overdose include pinpoint pupils, sedation, hypotension, respiratory depression and death.',
              'Manifestations of acute overdose include pinpoint pupils, sedation, hypotension, respiratory depression and death.'],
             'ropinirole': ['Overdose', 'Overdose'],
             'alprazolam orally disintegrating': ['Alprazolam overdose can present as sleepiness, confusion, poor coordination, slow reflexes, coma, and death.18,19 Taking alprazolam with alcohol lowers the threshold for overdose.18,19 Patients should have their respiration, pulse, and blood pressure monitored.18,19 Patients can be treated by gastric lavage and intravenous fluids.18,19. If hypotension occurs, patients may be treated with vasopressors.18,19 In known, or suspected overdoses, patients can be given the benzodiazepine receptor antagonist flumazenil in addition to other methods of management.18,19',
              'Alprazolam overdose can present as sleepiness, confusion, poor coordination, slow reflexes, coma, and death.18,19 Taking alprazolam with alcohol lowers the threshold for overdose.18,19 Patients should have their respiration, pulse, and blood pressure monitored.18,19 Patients can be treated by gastric lavage and intravenous fluids.18,19. If hypotension occurs, patients may be treated with vasopressors.18,19 In known, or suspected overdoses, patients can be given the benzodiazepine receptor antagonist flumazenil in addition to other methods of management.18,19'],
             'donepezil hydrochloride': ['LD50', 'LD50'],
             'klorcon meq': ['LD50=537 mg/kg (Orally in rats). Signs of overdose include respiratory depression, muscle weakness, somnolence (general depressed activity).',
              'LD50=537 mg/kg (Orally in rats). Signs of overdose include respiratory depression, muscle weakness, somnolence (general depressed activity).'],
             'klorcon &lt;suffix&gt;&lt;suffix&gt;': ['LD50=537 mg/kg (Orally in rats). Signs of overdose include respiratory depression, muscle weakness, somnolence (general depressed activity).',
              'LD50=537 mg/kg (Orally in rats). Signs of overdose include respiratory depression, muscle weakness, somnolence (general depressed activity).'],
             'klorcon m': ['LD50=537 mg/kg (Orally in rats). Signs of overdose include respiratory depression, muscle weakness, somnolence (general depressed activity).',
              'LD50=537 mg/kg (Orally in rats). Signs of overdose include respiratory depression, muscle weakness, somnolence (general depressed activity).'],
             'pot citrate': ['In a mice study, the LD50 was reported as 3621 mg/kg of following 33 weeks of oral administration 9. A study reports that at doses of 400 mg/d for several weeks had adverse effects such as GI irritation, skin rash, and lymphadenopathy but cited no serious complications 3. In patients treated with 400 to 1,000 mg perchlorate daily, cases of agranulocytosis and fatal aplastic anemia have been reported. Several deaths from aplastic anemia in patients treated with perchlorate at doses of 400 to 1000 mg/d for 8 to 33 weeks were also reported 3. The last report of fatal bone marrow toxicity related to treatment with potassium perchlorate was in the 1960s 3. ',
              'In a mice study, the LD50 was reported as 3621 mg/kg of following 33 weeks of oral administration 9. A study reports that at doses of 400 mg/d for several weeks had adverse effects such as GI irritation, skin rash, and lymphadenopathy but cited no serious complications 3. In patients treated with 400 to 1,000 mg perchlorate daily, cases of agranulocytosis and fatal aplastic anemia have been reported. Several deaths from aplastic anemia in patients treated with perchlorate at doses of 400 to 1000 mg/d for 8 to 33 weeks were also reported 3. The last report of fatal bone marrow toxicity related to treatment with potassium perchlorate was in the 1960s 3. '],
             'oxandrolone': ['The oral LD50 of oxandrolone in mice and dogs is greater than 5,000 mg/kg.',
              'The oral LD50 of oxandrolone in mice and dogs is greater than 5,000 mg/kg.'],
             'oracea': ['Oral LD50 is 2000 mg/kg in rats, 1870 mg/kg in mice, and 500 mg/kg in dog.14',
              'Oral LD50 is 2000 mg/kg in rats, 1870 mg/kg in mice, and 500 mg/kg in dog.14'],
             'daliresp': ['There are no data regarding overdosage with orally administered roflumilast. Phase I studies in which roflumilast was administered at single doses up to 5000 mcg showed an increase in the incidence of headache, gastrointestinal disorders, dizziness, palpitations, lightheadedness, clamminess, and arterial hypotension.8 In the event of an overdose, administer support medical care as soon as possible. Hemodialysis is unlikely to be of benefit given the extensive protein binding of roflumilast.8',
              'There are no data regarding overdosage with orally administered roflumilast. Phase I studies in which roflumilast was administered at single doses up to 5000 mcg showed an increase in the incidence of headache, gastrointestinal disorders, dizziness, palpitations, lightheadedness, clamminess, and arterial hypotension.8 In the event of an overdose, administer support medical care as soon as possible. Hemodialysis is unlikely to be of benefit given the extensive protein binding of roflumilast.8'],
             'zestril': ['The oral and subcutaneous LD50 in rats is >8500mg/kg and in mice is >9100mg/kg.11 The oral LDLO in women is 1200µg/kg/16D and in men is 43mg/kg/43W.11',
              'The oral and subcutaneous LD50 in rats is >8500mg/kg and in mice is >9100mg/kg.11 The oral LDLO in women is 1200µg/kg/16D and in men is 43mg/kg/43W.11'],
             'zestoretic': ['The oral LD50 of hydrochlorothiazide is >10g/kg in mice and rats.10,11',
              'The oral LD50 of hydrochlorothiazide is >10g/kg in mice and rats.10,11'],
             'zomig zmt': ['Toxicity information regarding zolmitriptan is not readily available. Patients experiencing an overdose are at an increased risk of severe adverse effects such as cardiovascular symptoms due to excessive vasoconstriction and activation of serotonergic receptors. Patients receiving a single 50 mg oral dose of zolmitriptan often experienced sedation. Symptomatic and supportive measures are recommended.14',
              'Toxicity information regarding zolmitriptan is not readily available. Patients experiencing an overdose are at an increased risk of severe adverse effects such as cardiovascular symptoms due to excessive vasoconstriction and activation of serotonergic receptors. Patients receiving a single 50 mg oral dose of zolmitriptan often experienced sedation. Symptomatic and supportive measures are recommended.14'],
             'seroquel': ['The oral LD50 if quetiapine in rats is 2000 mg/kg.20',
              'The oral LD50 if quetiapine in rats is 2000 mg/kg.20'],
             'seroquel xr': ['The oral LD50 if quetiapine in rats is 2000 mg/kg.20',
              'The oral LD50 if quetiapine in rats is 2000 mg/kg.20'],
             'accolate': ['Side effects include rash and upset stomach.',
              'Side effects include rash and upset stomach.'],
             'crestor': ['Generally well-tolerated. Side effects may include myalgia, constipation, asthenia, abdominal pain, and nausea. Other possible side effects include myotoxicity (myopathy, myositis, rhabdomyolysis) and hepatotoxicity. To avoid toxicity in Asian patients, lower doses should be considered. Pharmacokinetic studies show an approximately two-fold increase in peak plasma concentration and AUC in Asian patients (Philippino, Chinese, Japanese, Korean, Vietnamese, or Asian-Indian descent) compared to Caucasian patients. ',
              'Generally well-tolerated. Side effects may include myalgia, constipation, asthenia, abdominal pain, and nausea. Other possible side effects include myotoxicity (myopathy, myositis, rhabdomyolysis) and hepatotoxicity. To avoid toxicity in Asian patients, lower doses should be considered. Pharmacokinetic studies show an approximately two-fold increase in peak plasma concentration and AUC in Asian patients (Philippino, Chinese, Japanese, Korean, Vietnamese, or Asian-Indian descent) compared to Caucasian patients. '],
             'onglyza (saxagliptin)': ['Adverse reactions reported in ≥5% of patients treated with saxagliptin and more commonly than in patients treated with placebo are: upper respiratory tract infection, urinary tract infection, and headache.',
              'Adverse reactions reported in ≥5% of patients treated with saxagliptin and more commonly than in patients treated with placebo are: upper respiratory tract infection, urinary tract infection, and headache.'],
             'kombiglyze xr': ['Adverse reactions reported in ≥5% of patients treated with saxagliptin and more commonly than in patients treated with placebo are: upper respiratory tract infection, urinary tract infection, and headache.',
              'Adverse reactions reported in ≥5% of patients treated with saxagliptin and more commonly than in patients treated with placebo are: upper respiratory tract infection, urinary tract infection, and headache.'],
             'tension headache relief': ['Not Available', 'Not Available'],
             'methotrexate': ['The oral LD50 in rats is 135mg/kg and in mice is 146mg/kg.9',
              'The oral LD50 in rats is 135mg/kg and in mice is 146mg/kg.9'],
             'pindolol': ['Patients experiencing an overdose may experience excessive bradycardia, cardiac failure, hypotension, and bronchospasm.9 Initiate treatment with symptomatic and supportive measures.9',
              'Patients experiencing an overdose may experience excessive bradycardia, cardiac failure, hypotension, and bronchospasm.9 Initiate treatment with symptomatic and supportive measures.9'],
             'spironolactone and hydrochlorothiazide': ['Patients experiencing an overdose may present with drowsiness, mental confusion, maculopapular or erythematous rash, nausea, vomiting, dizziness, or diarrhea. Vomiting is generally induced or a gastric lavage is performed.Label Supportive treatment involves maintining hydration, electrolyte balance, and vital functionsLabel.',
              'Patients experiencing an overdose may present with drowsiness, mental confusion, maculopapular or erythematous rash, nausea, vomiting, dizziness, or diarrhea. Vomiting is generally induced or a gastric lavage is performed.Label Supportive treatment involves maintining hydration, electrolyte balance, and vital functionsLabel.'],
             'allopurinol': ['Oral TDLO (rat): 10 mg/kg; Oral LD50 (mouse): 78 mg/kg; Oral TDLO (mouse): 100 mg/kg 14 ',
              'Oral TDLO (rat): 10 mg/kg; Oral LD50 (mouse): 78 mg/kg; Oral TDLO (mouse): 100 mg/kg 14 '],
             'chlorthalidone': ['Not Available', 'Not Available'],
             'haloperidol': ['Acute oral toxicity (LD50): 71 mg/kg in rats MSDS. ',
              'Acute oral toxicity (LD50): 71 mg/kg in rats MSDS. '],
             'chlorthalidone s': ['Not Available', 'Not Available'],
             'cdpamitrip': ['', ''],
             'perphenamit': ['s of overdose include stupor or coma, and children may have convulsive seizures. Signs of arousal may not occur for 48 hours. Oral LD50=318 mg/kg',
              '50=64 mg/kg'],
             'diphenoxylate w atropine': ['Coma, dry skin and mucous membranes, enlarged pupils of the eyes, extremely high body temperature, flushing, involuntary eyeball movement, lower than normal muscle tone, pinpoint pupils, rapid heartbeat, restlessness, sluggishness, suppressed breathing',
              'Coma, dry skin and mucous membranes, enlarged pupils of the eyes, extremely high body temperature, flushing, involuntary eyeball movement, lower than normal muscle tone, pinpoint pupils, rapid heartbeat, restlessness, sluggishness, suppressed breathing'],
             'metoprlhctz': ['', ''],
             'bupropion hydrochloride': ['Symptoms of overdose include seizures, hallucinations, loss of consciousness, tachycardia, and cardiac arrest.',
              'Symptoms of overdose include seizures, hallucinations, loss of consciousness, tachycardia, and cardiac arrest.'],
             'divalproex er': ['LD50 Values', 'LD50 Values'],
             'perphenamitrip': ['', ''],
             'amiloride hcl and hydrochlorothiazide': ['No data are available in regard to overdosage in humans. The oral LD50 of amiloride hydrochloride (calculated as the base) is 56 mg/kg in mice and 36 to 85 mg/kg in rats, depending on the strain. The most likely signs and symptoms to be expected with overdosage are dehydration and electrolyte imbalance.',
              'No data are available in regard to overdosage in humans. The oral LD50 of amiloride hydrochloride (calculated as the base) is 56 mg/kg in mice and 36 to 85 mg/kg in rats, depending on the strain. The most likely signs and symptoms to be expected with overdosage are dehydration and electrolyte imbalance.'],
             'propranhctz': ['', ''],
             'cyclobenzaprine hydrochloride': ['The oral LD50 of cyclobenzaprine in mice and rats is 338 mg/kg and 425 mg/kg, respectively. Signs of overdose may develop rapidly after ingestion and commonly include significant drowsiness and tachycardia, with less common manifestations including tremor, agitation, ataxia, GI upset, and other CNS effects such as confusion and hallucinations. Potentially critical manifestations, though rare, include cardiac arrest or dysrhythmias, severe hypotension, seizures, and neuroleptic malignant syndrome.15,16',
              'The oral LD50 of cyclobenzaprine in mice and rats is 338 mg/kg and 425 mg/kg, respectively. Signs of overdose may develop rapidly after ingestion and commonly include significant drowsiness and tachycardia, with less common manifestations including tremor, agitation, ataxia, GI upset, and other CNS effects such as confusion and hallucinations. Potentially critical manifestations, though rare, include cardiac arrest or dysrhythmias, severe hypotension, seizures, and neuroleptic malignant syndrome.15,16'],
             'fexofenadine hcl': ["No deaths were observed following the oral administration of up to 5000 mg/kg in both mice and rats (equivalent to approximately 100-200x the recommended human dose). Single doses of up to 800 mg and chronic exposure of up to 690 mg twice daily for 1 month in humans did not result in clinically significant adverse events. Symptoms of overdosage are consistent with fexofenadine's adverse effect profile and are likely to include dizziness, drowsiness, and dry mouth.10",
              "No deaths were observed following the oral administration of up to 5000 mg/kg in both mice and rats (equivalent to approximately 100-200x the recommended human dose). Single doses of up to 800 mg and chronic exposure of up to 690 mg twice daily for 1 month in humans did not result in clinically significant adverse events. Symptoms of overdosage are consistent with fexofenadine's adverse effect profile and are likely to include dizziness, drowsiness, and dry mouth.10"],
             'doxepin hydrochloride': ['Oral LD50 values of doxepin in mouse and rat are 180 mg/kg and 147 mg/kg, respectively.MSDS In an overdose state, symptoms of convulsions, dysrhythmias, coma, severe hypotension, central nervous system depression, changes on electrocardiography results and death have been observed.7',
              'Oral LD50 values of doxepin in mouse and rat are 180 mg/kg and 147 mg/kg, respectively.MSDS In an overdose state, symptoms of convulsions, dysrhythmias, coma, severe hypotension, central nervous system depression, changes on electrocardiography results and death have been observed.7'],
             'triamterene and hydrochlorothiazide': ['Acute oral LD50 of triamterene in rats is 400 mg/kg and 285-380 mg/kg in mice.15,MSDS There has been a case of reversible acute renal failure following ingestion of 50 combination pills containing 50 mg triamterene and 25 mg hydrochlorothiazide. Symptoms of overdose, such as nausea, vomiting, gastrointestinal disturbances, weakness, and hypotension, are related to electrolyte imbalances, such as hyperkalemia. As there is no specific antidote, emesis and gastric lavage should be use to induce immediate evacuation of the stomach and careful evaluation of the electrolyte pattern and fluid balance should be made. Dialysis may be somewhat effective in case of an overdosage.15',
              'Acute oral LD50 of triamterene in rats is 400 mg/kg and 285-380 mg/kg in mice.15,MSDS There has been a case of reversible acute renal failure following ingestion of 50 combination pills containing 50 mg triamterene and 25 mg hydrochlorothiazide. Symptoms of overdose, such as nausea, vomiting, gastrointestinal disturbances, weakness, and hypotension, are related to electrolyte imbalances, such as hyperkalemia. As there is no specific antidote, emesis and gastric lavage should be use to induce immediate evacuation of the stomach and careful evaluation of the electrolyte pattern and fluid balance should be made. Dialysis may be somewhat effective in case of an overdosage.15'],
             'acebutolol': ['Symptoms of overdose include extreme bradycardia, advanced atrioventricular block, intraventricular conduction defects, hypotension, severe congestive heart failure, seizures, and in susceptible patients, bronchospasm, and hypoglycemia.',
              'Symptoms of overdose include extreme bradycardia, advanced atrioventricular block, intraventricular conduction defects, hypotension, severe congestive heart failure, seizures, and in susceptible patients, bronchospasm, and hypoglycemia.'],
             'estradiol': ['The NOAEL (no-observed-adverse-effect-level) oral toxicity of estradiol after 90 day in rats was 0.003 mg/kg/day for blood, female reproductive, and male reproductive, endocrine, and liver toxicity.36 Oral TDLO of ethinyl estradiol is 21 mg/kg/21D intermittent, woman) with an oral LD50 of 960 mg/kg in the rat.37 ',
              'The NOAEL (no-observed-adverse-effect-level) oral toxicity of estradiol after 90 day in rats was 0.003 mg/kg/day for blood, female reproductive, and male reproductive, endocrine, and liver toxicity.36 Oral TDLO of ethinyl estradiol is 21 mg/kg/21D intermittent, woman) with an oral LD50 of 960 mg/kg in the rat.37 '],
             'extended phenytoin sodium': ['The experience of phenytoin toxicity is not limited to situations of acute ingestion, but may also occur due to drug interactions or due to physiological circumstances that impact serum albumin (ie. kidney disease) or drug metabolism.3 Other changes that may result in phenytoin toxicity include pregnancy, malnutrition and malignancy.3',
              'The experience of phenytoin toxicity is not limited to situations of acute ingestion, but may also occur due to drug interactions or due to physiological circumstances that impact serum albumin (ie. kidney disease) or drug metabolism.3 Other changes that may result in phenytoin toxicity include pregnancy, malnutrition and malignancy.3'],
             'terazosin': ['In the event of an overdose, patients may experience hypotensionLabel. Blood pressure and heart rate should be controlled by having the patient lie down or by treating with volume expanders or if necessary vasopressorsLabel. Patients should be monitored for renal functionLabel. Because terazosin is highly protein bound, dialysis is unlikely to provide benefit to overdosing patientsLabel.',
              'In the event of an overdose, patients may experience hypotensionLabel. Blood pressure and heart rate should be controlled by having the patient lie down or by treating with volume expanders or if necessary vasopressorsLabel. Patients should be monitored for renal functionLabel. Because terazosin is highly protein bound, dialysis is unlikely to provide benefit to overdosing patientsLabel.'],
             'dicyclomine hydrochloride': ['Patients experiencing an overdose may present with headache, nausea, vomiting, blurred vision, dilated pupils, dizziness, dry mouth, difficulty swallowing, CNS stimulation, as well as hot, dry skin.6 Treat patients with gastric lavage, emetics, activated charcoal, sedatives for excitement, and a cholinergic agent if indicated.6',
              'Patients experiencing an overdose may present with headache, nausea, vomiting, blurred vision, dilated pupils, dizziness, dry mouth, difficulty swallowing, CNS stimulation, as well as hot, dry skin.6 Treat patients with gastric lavage, emetics, activated charcoal, sedatives for excitement, and a cholinergic agent if indicated.6'],
             'levothyroxine sodium': ['LD50=20 mg/kg (orally in rat). Hypermetabolic state indistinguishable from thyrotoxicosis of endogenous origin. Symptoms of thyrotoxicosis include weight loss, increased appetite, palpitations, nervousness, diarrhea, abdominal cramps, sweating, tachycardia, increased pulse and blood pressure, cardiac arrhythmias, tremors, insomnia, heat intolerance, fever, and menstrual irregularities.',
              'LD50=20 mg/kg (orally in rat). Hypermetabolic state indistinguishable from thyrotoxicosis of endogenous origin. Symptoms of thyrotoxicosis include weight loss, increased appetite, palpitations, nervousness, diarrhea, abdominal cramps, sweating, tachycardia, increased pulse and blood pressure, cardiac arrhythmias, tremors, insomnia, heat intolerance, fever, and menstrual irregularities.'],
             'midodrine hydrochloride': ['Symptoms of overdose could include hypertension, piloerection (goosebumps), a sensation of coldness and urinary retention. The single doses that would be associated with symptoms of overdosage or would be potentially life- threatening are unknown. The oral LD50 is approximately 30 to 50 mg/kg in rats, 675 mg/kg in mice, and 125 to 160 mg/kg in dogs. Desglymidodrine is dialyzable.',
              'Symptoms of overdose could include hypertension, piloerection (goosebumps), a sensation of coldness and urinary retention. The single doses that would be associated with symptoms of overdosage or would be potentially life- threatening are unknown. The oral LD50 is approximately 30 to 50 mg/kg in rats, 675 mg/kg in mice, and 125 to 160 mg/kg in dogs. Desglymidodrine is dialyzable.'],
             'paroxetine hcl': ['The acute LD50 in mice and rats is 350 mg/kg.35',
              'The acute LD50 in mice and rats is 350 mg/kg.35'],
             'paroxetine er': ['The acute LD50 in mice and rats is 350 mg/kg.35',
              'The acute LD50 in mice and rats is 350 mg/kg.35'],
             'bromocriptine mesylate': ['Symptoms of overdosage include nausea, vomiting, and severe hypotension. The most common adverse effects include nausea, headache, vertigo, constipation, light-headedness, abdominal cramps, nasal congestion, diarrhea, and hypotension. ',
              'Symptoms of overdosage include nausea, vomiting, and severe hypotension. The most common adverse effects include nausea, headache, vertigo, constipation, light-headedness, abdominal cramps, nasal congestion, diarrhea, and hypotension. '],
             'atenolol and chlorthalidone': ['LD50 Values', 'LD50 Values'],
             'nisoldipine': ['Not Available', 'Not Available'],
             'terazosin hydrochloride anhydrous': ['In the event of an overdose, patients may experience hypotensionLabel. Blood pressure and heart rate should be controlled by having the patient lie down or by treating with volume expanders or if necessary vasopressorsLabel. Patients should be monitored for renal functionLabel. Because terazosin is highly protein bound, dialysis is unlikely to provide benefit to overdosing patientsLabel.',
              'In the event of an overdose, patients may experience hypotensionLabel. Blood pressure and heart rate should be controlled by having the patient lie down or by treating with volume expanders or if necessary vasopressorsLabel. Patients should be monitored for renal functionLabel. Because terazosin is highly protein bound, dialysis is unlikely to provide benefit to overdosing patientsLabel.'],
             'trifluoperazine hydrochloride': ['Symptoms of overdose include agitation, coma, convulsions, difficulty breathing, difficulty swallowing, dry mouth, extreme sleepiness, fever, intestinal blockage, irregular heart rate, low blood pressure, and restlessness.',
              'Symptoms of overdose include agitation, coma, convulsions, difficulty breathing, difficulty swallowing, dry mouth, extreme sleepiness, fever, intestinal blockage, irregular heart rate, low blood pressure, and restlessness.'],
             'amitriptyline hydrochloride': ['Toxicity Data: Oral TDLO (child): 4167 μg/kg; Oral TDLO (man): 714 μg/kg/1D (intermittent); Oral TDLO (woman): 10 mg/kg 12. ',
              'Toxicity Data: Oral TDLO (child): 4167 μg/kg; Oral TDLO (man): 714 μg/kg/1D (intermittent); Oral TDLO (woman): 10 mg/kg 12. '],
             'galantamine': ['The oral LD50 of the active ingredient, galantamine hydrobromide, in rats is 75 mg/kg.12 Symptoms of overdose are expected to be similar to those of cholinomimetics, which involve the central nervous system, the parasympathetic nervous system, and the neuromuscular junction. Effects of a cholinergic crisis include severe nausea, vomiting, gastrointestinal cramping, salivation, lacrimation, urination, defecation, sweating, bradycardia, hypotension, respiratory depression, collapse, and convulsions. Muscle weakness or fasciculations may also occur, with respiratory muscle weakness having the potential to bring fatal results. In one patient who consumed an oral daily dose of 32 mg developed bradycardia, QT prolongation, ventricular tachycardia and torsades de pointes accompanied by a brief loss of consciousness. In one patient with a history of hallucinations who consumed a daily dose of 24 mg galantamine, hallucinations requiring hospitalization occurred. A patient who ingested 160 mg of galantamine from an oral solution developed sweating, vomiting, bradycardia, and near-syncope one hour following consumption.10 ',
              'The oral LD50 of the active ingredient, galantamine hydrobromide, in rats is 75 mg/kg.12 Symptoms of overdose are expected to be similar to those of cholinomimetics, which involve the central nervous system, the parasympathetic nervous system, and the neuromuscular junction. Effects of a cholinergic crisis include severe nausea, vomiting, gastrointestinal cramping, salivation, lacrimation, urination, defecation, sweating, bradycardia, hypotension, respiratory depression, collapse, and convulsions. Muscle weakness or fasciculations may also occur, with respiratory muscle weakness having the potential to bring fatal results. In one patient who consumed an oral daily dose of 32 mg developed bradycardia, QT prolongation, ventricular tachycardia and torsades de pointes accompanied by a brief loss of consciousness. In one patient with a history of hallucinations who consumed a daily dose of 24 mg galantamine, hallucinations requiring hospitalization occurred. A patient who ingested 160 mg of galantamine from an oral solution developed sweating, vomiting, bradycardia, and near-syncope one hour following consumption.10 '],
             'telmisartan': ['nous LD50 in rats is 150-200 mg/kg in males and 200 to 250 mg/kg in females. Acute oral toxicity is low: no deaths and no changes occurred in rats or dogs at 2000 mg/kg, the highest dose tested. Limited data are available with regard to overdosage in humans. The most likely manifestations of overdosage with telmisartan would be hypotension, dizziness and tachycardi',
              'ardia could occur from parasympathetic'],
             'candesartan cilexetil and hydrochlorothiazide': ['Not Available',
              'Not Available'],
             'repaglidine': ['LD50 >1 g/kg (rat) (W. Grell)',
              'LD50 >1 g/kg (rat) (W. Grell)'],
             'doxepin': ['Oral LD50 values of doxepin in mouse and rat are 180 mg/kg and 147 mg/kg, respectively.MSDS In an overdose state, symptoms of convulsions, dysrhythmias, coma, severe hypotension, central nervous system depression, changes on electrocardiography results and death have been observed.7',
              'Oral LD50 values of doxepin in mouse and rat are 180 mg/kg and 147 mg/kg, respectively.MSDS In an overdose state, symptoms of convulsions, dysrhythmias, coma, severe hypotension, central nervous system depression, changes on electrocardiography results and death have been observed.7'],
             'glipizide and metformin hcl': ['In rats, the oral LD50 is reported to be greater than 4000 mg/kg and the intraperitoneal LD50 is 1200 mg/kg. The lowest published toxic dose (TDLo) via oral route in child was 379 μg/kg.MSDS ',
              'In rats, the oral LD50 is reported to be greater than 4000 mg/kg and the intraperitoneal LD50 is 1200 mg/kg. The lowest published toxic dose (TDLo) via oral route in child was 379 μg/kg.MSDS '],
             'nitrofurantoin sule': ['Symptoms of overdose include vomiting.8,9,10 In case of overdose, induce vomiting if it has not already occurred and increase fluid intake to promote urination.8,9,10 In extreme cases, nitrofurantoin can be removed from circulation by dialysis.8,9,10',
              'Symptoms of overdose include vomiting.8,9,10 In case of overdose, induce vomiting if it has not already occurred and increase fluid intake to promote urination.8,9,10 In extreme cases, nitrofurantoin can be removed from circulation by dialysis.8,9,10'],
             'phenytoin chewable': ['The experience of phenytoin toxicity is not limited to situations of acute ingestion, but may also occur due to drug interactions or due to physiological circumstances that impact serum albumin (ie. kidney disease) or drug metabolism.3 Other changes that may result in phenytoin toxicity include pregnancy, malnutrition and malignancy.3',
              'The experience of phenytoin toxicity is not limited to situations of acute ingestion, but may also occur due to drug interactions or due to physiological circumstances that impact serum albumin (ie. kidney disease) or drug metabolism.3 Other changes that may result in phenytoin toxicity include pregnancy, malnutrition and malignancy.3'],
             'desloratadine': ['tion regarding desloratadine overdose is limited, although somnolence has been reported. In case of overdose, symptomatic and supportive treatment, including removing the unabsorbed drug, is recommende',
              'however, that desloratadine and its active metabolite 3-hydroxydesloratadine cannot be eliminated by hemodialysis.'],
             'atovaquone and proguanil hcl': ['The median lethal dose is higher than the maximum oral dose tested in mice and rats (1825 mg/kg per day). Overdoses up to 31,500 mg of atovaquone have been reported. In one such patient who also took an unspecified dose of dapsone, methemoglobinemia occurred. Rash has also been reported after overdose.',
              'The median lethal dose is higher than the maximum oral dose tested in mice and rats (1825 mg/kg per day). Overdoses up to 31,500 mg of atovaquone have been reported. In one such patient who also took an unspecified dose of dapsone, methemoglobinemia occurred. Rash has also been reported after overdose.'],
             'sertraline hydrochloride': ['The LD50 of sertraline is >2000 mg/kg in rats according to the FDA label.25 One other references indicates an oral LD50 of in mice and rats of 419 - 548 mg/kg and 1327 - 1591mg/kg, respectively.MSDS',
              'The LD50 of sertraline is >2000 mg/kg in rats according to the FDA label.25 One other references indicates an oral LD50 of in mice and rats of 419 - 548 mg/kg and 1327 - 1591mg/kg, respectively.MSDS'],
             'valacyclovir': ['LD50 Oral ', 'LD50 Oral '],
             'metoprolol succinate': ['Oral administration of metoprolol to rats presents an LD50 in the range of 3090 to 4670 mg/kg. Cases of overdose have reported bradycardia, hypotension, bronchospasm, and cardiac failure. In the case of an overdose, gastric lavage is recommended followed by specific treatment according to symptoms.Label',
              'Oral administration of metoprolol to rats presents an LD50 in the range of 3090 to 4670 mg/kg. Cases of overdose have reported bradycardia, hypotension, bronchospasm, and cardiac failure. In the case of an overdose, gastric lavage is recommended followed by specific treatment according to symptoms.Label'],
             'benazepril hcl and hydrochlorothiazide': ['The most common adverse effects include headache, dizziness, fatigue, somnolence, postural dizziness, nausea, and coughLabel,4.',
              'The most common adverse effects include headache, dizziness, fatigue, somnolence, postural dizziness, nausea, and coughLabel,4.'],
             'venlafaxine': ['Oral LD50 was 350 mg/kg in female rats and 700 mg/kg in male rats.16',
              'Oral LD50 was 350 mg/kg in female rats and 700 mg/kg in male rats.16'],
             'diltiazem hcl': ['Clinical Toxicity and Overdose',
              'Clinical Toxicity and Overdose'],
             'doxepin hcl': ['Oral LD50 values of doxepin in mouse and rat are 180 mg/kg and 147 mg/kg, respectively.MSDS In an overdose state, symptoms of convulsions, dysrhythmias, coma, severe hypotension, central nervous system depression, changes on electrocardiography results and death have been observed.7',
              'Oral LD50 values of doxepin in mouse and rat are 180 mg/kg and 147 mg/kg, respectively.MSDS In an overdose state, symptoms of convulsions, dysrhythmias, coma, severe hypotension, central nervous system depression, changes on electrocardiography results and death have been observed.7'],
             'risperidone orally disintegrating': ['Symptoms of overdose include lethargy, dystonia/spasm, tachycardia, bradycardia, and seizures. LD50=57.7 mg/kg (rat, oral) and 34 mg/kg (rat, intravenous).5,6,7',
              'Symptoms of overdose include lethargy, dystonia/spasm, tachycardia, bradycardia, and seizures. LD50=57.7 mg/kg (rat, oral) and 34 mg/kg (rat, intravenous).5,6,7'],
             'riperidone orally disintegrating': ['Symptoms of overdose include lethargy, dystonia/spasm, tachycardia, bradycardia, and seizures. LD50=57.7 mg/kg (rat, oral) and 34 mg/kg (rat, intravenous).5,6,7',
              'Symptoms of overdose include lethargy, dystonia/spasm, tachycardia, bradycardia, and seizures. LD50=57.7 mg/kg (rat, oral) and 34 mg/kg (rat, intravenous).5,6,7'],
             'diltiazem sr': ['Clinical Toxicity and Overdose',
              'Clinical Toxicity and Overdose'],
             'fluphenazine hydrochloride': ['Not Available', 'Not Available'],
             'omeprazole': ['Oral acute (LD50): 4000 mg/kg (mouse), 2210 mg/kg (rat) MSDS. ',
              'Oral acute (LD50): 4000 mg/kg (mouse), 2210 mg/kg (rat) MSDS. '],
             'metolazone': ['Symptoms of overdose include difficulty breathing, dizziness, dizziness on standing up, drowsiness, fainting, irritation of the stomach and intestines, and lethargy leading to coma.',
              'Symptoms of overdose include difficulty breathing, dizziness, dizziness on standing up, drowsiness, fainting, irritation of the stomach and intestines, and lethargy leading to coma.'],
             'verapamil er': ["Verapamil's reported oral TDLo is 14.4 mg/kg in women and 3.429 mg/kg in men.21 The oral LD50 is 150 mg/kg in rats and 163 mg/kg in mice.21",
              "Verapamil's reported oral TDLo is 14.4 mg/kg in women and 3.429 mg/kg in men.21 The oral LD50 is 150 mg/kg in rats and 163 mg/kg in mice.21"],
             'verapamil hcl sule (pm)': ["Verapamil's reported oral TDLo is 14.4 mg/kg in women and 3.429 mg/kg in men.21 The oral LD50 is 150 mg/kg in rats and 163 mg/kg in mice.21",
              "Verapamil's reported oral TDLo is 14.4 mg/kg in women and 3.429 mg/kg in men.21 The oral LD50 is 150 mg/kg in rats and 163 mg/kg in mice.21"],
             'zonisamide': ['Information on daily doses over 800 mg/day of zonisamide is limited. During clinical development, three patients ingested unknown amounts of zonisamide as suicide attempts, and all of them were hospitalized with central nervous system symptoms. One patient became comatose and developed bradycardia, hypotension, and respiratory depression; 31 hours after zonisamide ingestion, plasma level was 100.1 µg/mL. Zonisamide plasma levels fell with a half-life of 57 hours, and the patient became alert five days later.7,8 There are no specific antidotes for zonisamide overdosage. In case of a suspected recent overdose, emesis should be induced or gastric lavage performed with the usual precautions to protect the airway. General supportive care is indicated, including frequent monitoring of vital signs and close observation.7,8 Due to its long half-life and low protein binding, renal dialysis may be effective in treating zonisamide overdose; however, the effectiveness of this procedure has not been formally studied.8 ',
              'Information on daily doses over 800 mg/day of zonisamide is limited. During clinical development, three patients ingested unknown amounts of zonisamide as suicide attempts, and all of them were hospitalized with central nervous system symptoms. One patient became comatose and developed bradycardia, hypotension, and respiratory depression; 31 hours after zonisamide ingestion, plasma level was 100.1 µg/mL. Zonisamide plasma levels fell with a half-life of 57 hours, and the patient became alert five days later.7,8 There are no specific antidotes for zonisamide overdosage. In case of a suspected recent overdose, emesis should be induced or gastric lavage performed with the usual precautions to protect the airway. General supportive care is indicated, including frequent monitoring of vital signs and close observation.7,8 Due to its long half-life and low protein binding, renal dialysis may be effective in treating zonisamide overdose; however, the effectiveness of this procedure has not been formally studied.8 '],
             'amlodipine and benazepril hcl': ['Acute oral toxicity (LD50): 37 mg/kg (mouse) MSDS. ',
              'Acute oral toxicity (LD50): 37 mg/kg (mouse) MSDS. '],
             'drospirenone and ethinyl estradiol': ['The oral LD50 of drospirenone in rats is >2000 mg/kg.22',
              'The oral LD50 of drospirenone in rats is >2000 mg/kg.22'],
             'ondansetron odt': ['At present, there is little information concerning overdosage with ondansetron 8,9,10. Nevertheless, there have been certain cases of somewhat idiosyncratic adverse effects associated with particular dosages of ondansetron used 8,9,10.',
              'At present, there is little information concerning overdosage with ondansetron 8,9,10. Nevertheless, there have been certain cases of somewhat idiosyncratic adverse effects associated with particular dosages of ondansetron used 8,9,10.'],
             'carbidopa levodopa and entaone': ['The LD50 of carbidopa is reported to be in the rat of 4810 mg/kg.MSDS In animal studies, carbidopa showed no incidences on neoplasia and showed no effect on the fertility status and development.Label',
              'The LD50 of carbidopa is reported to be in the rat of 4810 mg/kg.MSDS In animal studies, carbidopa showed no incidences on neoplasia and showed no effect on the fertility status and development.Label'],
             'selegiline': ['LD50=63 mg/kg (rats, IV)',
              'LD50=63 mg/kg (rats, IV)'],
             'hydrocodone acetaminophen': ['Overdosage with hydrocodone presents as opioid intoxication including respiratory depression, somnolence, coma, skeletal muscle flaccidity, cold and clammy skin, constricted pupils, pulmonary edema, bradycardia, hypotension, partial or complete airway obstruction, atypical snoring, and death.15',
              'Overdosage with hydrocodone presents as opioid intoxication including respiratory depression, somnolence, coma, skeletal muscle flaccidity, cold and clammy skin, constricted pupils, pulmonary edema, bradycardia, hypotension, partial or complete airway obstruction, atypical snoring, and death.15'],
             'hydrocodone apap': ['Overdosage with hydrocodone presents as opioid intoxication including respiratory depression, somnolence, coma, skeletal muscle flaccidity, cold and clammy skin, constricted pupils, pulmonary edema, bradycardia, hypotension, partial or complete airway obstruction, atypical snoring, and death.15',
              'Overdosage with hydrocodone presents as opioid intoxication including respiratory depression, somnolence, coma, skeletal muscle flaccidity, cold and clammy skin, constricted pupils, pulmonary edema, bradycardia, hypotension, partial or complete airway obstruction, atypical snoring, and death.15'],
             'hydroco apap': ['Side effects include inhibition of bone formation, suppression of calcium absorption and delayed wound healing',
              'Side effects include inhibition of bone formation, suppression of calcium absorption and delayed wound healing'],
             'naltrexone': ['In the mouse, rat and guinea pig, the oral LD50s were 1,100-1,550 mg/kg; 1,450 mg/kg; and 1,490 mg/kg; respectively. High doses of naltrexone (generally >1,000 mg/kg) produce salivation, depression/reduced activity, tremors, and convulsions.',
              'In the mouse, rat and guinea pig, the oral LD50s were 1,100-1,550 mg/kg; 1,450 mg/kg; and 1,490 mg/kg; respectively. High doses of naltrexone (generally >1,000 mg/kg) produce salivation, depression/reduced activity, tremors, and convulsions.'],
             'imipramine pamoate': ['The anticholinergic actvity of imipramine can produce dry mucous membranes, blurred vision, increased intraocular pressure, hyperthermia, constipation, adynamic ileus, urinary retention, delayed micturition, and dilation of the urinary tract 16.',
              'The anticholinergic actvity of imipramine can produce dry mucous membranes, blurred vision, increased intraocular pressure, hyperthermia, constipation, adynamic ileus, urinary retention, delayed micturition, and dilation of the urinary tract 16.'],
             'doryx (doxycycline hyclate )': ['Oral LD50 is 2000 mg/kg in rats, 1870 mg/kg in mice, and 500 mg/kg in dog.14',
              'Oral LD50 is 2000 mg/kg in rats, 1870 mg/kg in mice, and 500 mg/kg in dog.14'],
             'femhrt': ['Female patients experiencing and overdose may present with withdrawal bleeding, nausea, vomiting, breast tenderness, abdominal pain, drowsiness, and fatigue.15,25,16,17,24,18,19,20,21,22,23 Overdose should be treated with symptomatic and supportive care including monitoring for potassium concentrations, sodium concentrations, and signs of metabolic acidosis.15,25,16,17,24,18,19,20,21,22,23',
              'Female patients experiencing and overdose may present with withdrawal bleeding, nausea, vomiting, breast tenderness, abdominal pain, drowsiness, and fatigue.15,25,16,17,24,18,19,20,21,22,23 Overdose should be treated with symptomatic and supportive care including monitoring for potassium concentrations, sodium concentrations, and signs of metabolic acidosis.15,25,16,17,24,18,19,20,21,22,23'],
             'enablex (darifenacin)': ['Overdosage can potentially result in severe central anticholinergic effects.',
              'Overdosage can potentially result in severe central anticholinergic effects.'],
             'lo loestrin fe': ['Adverse effects include anorexia, nausea, vomiting, bleeding, anemia, diarrhea, and constipation.13 Loxoprofen toxicity may lead to gastrointestinal disturbance (including flatulence, dyspepsia and gastritis) and renal failure.14',
              'Adverse effects include anorexia, nausea, vomiting, bleeding, anemia, diarrhea, and constipation.13 Loxoprofen toxicity may lead to gastrointestinal disturbance (including flatulence, dyspepsia and gastritis) and renal failure.14'],
             'actonel': ['In clinical trials, over 10% of patients experienced back pain, arthralgia, abdominal pain, and dyspepsiaLabel. Less commonly, patients experience angioedema, generalized rash, bullous skin reactions, iritis, and uveitisLabel.',
              'In clinical trials, over 10% of patients experienced back pain, arthralgia, abdominal pain, and dyspepsiaLabel. Less commonly, patients experience angioedema, generalized rash, bullous skin reactions, iritis, and uveitisLabel.'],
             'minastrin fe': ['Female patients experiencing and overdose may present with withdrawal bleeding, nausea, vomiting, breast tenderness, abdominal pain, drowsiness, and fatigue.15,25,16,17,24,18,19,20,21,22,23 Overdose should be treated with symptomatic and supportive care including monitoring for potassium concentrations, sodium concentrations, and signs of metabolic acidosis.15,25,16,17,24,18,19,20,21,22,23',
              'Female patients experiencing and overdose may present with withdrawal bleeding, nausea, vomiting, breast tenderness, abdominal pain, drowsiness, and fatigue.15,25,16,17,24,18,19,20,21,22,23 Overdose should be treated with symptomatic and supportive care including monitoring for potassium concentrations, sodium concentrations, and signs of metabolic acidosis.15,25,16,17,24,18,19,20,21,22,23'],
             'delzicol': ['Oral, mouse: LD50 = 3370 mg/kg; Oral, rat: LD50 = 2800 mg/kg; Skin, rabbit: LD50 = >5 gm/kg MSDS.',
              'Oral, mouse: LD50 = 3370 mg/kg; Oral, rat: LD50 = 2800 mg/kg; Skin, rabbit: LD50 = >5 gm/kg MSDS.'],
             'asacol hd': ['Oral, mouse: LD50 = 3370 mg/kg; Oral, rat: LD50 = 2800 mg/kg; Skin, rabbit: LD50 = >5 gm/kg MSDS.',
              'Oral, mouse: LD50 = 3370 mg/kg; Oral, rat: LD50 = 2800 mg/kg; Skin, rabbit: LD50 = >5 gm/kg MSDS.'],
             'atelvia': ['In clinical trials, over 10% of patients experienced back pain, arthralgia, abdominal pain, and dyspepsiaLabel. Less commonly, patients experience angioedema, generalized rash, bullous skin reactions, iritis, and uveitisLabel.',
              'In clinical trials, over 10% of patients experienced back pain, arthralgia, abdominal pain, and dyspepsiaLabel. Less commonly, patients experience angioedema, generalized rash, bullous skin reactions, iritis, and uveitisLabel.'],
             'thyroid armour': ['Patients experiencing and overdose may present with symptoms of a hypermetabolic state.10 Overdose may be treated by symptomatic and supportive treatment, dose reduction or temporarily stopping the medication, induction of vomiting, administering oxygen, cardiac glycosides, as well as methods to control fever, hypoglycemia, and fluid loss.10',
              'Patients experiencing and overdose may present with symptoms of a hypermetabolic state.10 Overdose may be treated by symptomatic and supportive treatment, dose reduction or temporarily stopping the medication, induction of vomiting, administering oxygen, cardiac glycosides, as well as methods to control fever, hypoglycemia, and fluid loss.10'],
             'thyroid \\ gr': ['Patients experiencing and overdose may present with symptoms of a hypermetabolic state.10 Overdose may be treated by symptomatic and supportive treatment, dose reduction or temporarily stopping the medication, induction of vomiting, administering oxygen, cardiac glycosides, as well as methods to control fever, hypoglycemia, and fluid loss.10',
              'Patients experiencing and overdose may present with symptoms of a hypermetabolic state.10 Overdose may be treated by symptomatic and supportive treatment, dose reduction or temporarily stopping the medication, induction of vomiting, administering oxygen, cardiac glycosides, as well as methods to control fever, hypoglycemia, and fluid loss.10'],
             'thyroid gr armour': ['Patients experiencing and overdose may present with symptoms of a hypermetabolic state.10 Overdose may be treated by symptomatic and supportive treatment, dose reduction or temporarily stopping the medication, induction of vomiting, administering oxygen, cardiac glycosides, as well as methods to control fever, hypoglycemia, and fluid loss.10',
              'Patients experiencing and overdose may present with symptoms of a hypermetabolic state.10 Overdose may be treated by symptomatic and supportive treatment, dose reduction or temporarily stopping the medication, induction of vomiting, administering oxygen, cardiac glycosides, as well as methods to control fever, hypoglycemia, and fluid loss.10'],
             'thyroid gr s': ['Patients experiencing and overdose may present with symptoms of a hypermetabolic state.10 Overdose may be treated by symptomatic and supportive treatment, dose reduction or temporarily stopping the medication, induction of vomiting, administering oxygen, cardiac glycosides, as well as methods to control fever, hypoglycemia, and fluid loss.10',
              'Patients experiencing and overdose may present with symptoms of a hypermetabolic state.10 Overdose may be treated by symptomatic and supportive treatment, dose reduction or temporarily stopping the medication, induction of vomiting, administering oxygen, cardiac glycosides, as well as methods to control fever, hypoglycemia, and fluid loss.10'],
             'armour thyroid': ['Patients experiencing and overdose may present with symptoms of a hypermetabolic state.10 Overdose may be treated by symptomatic and supportive treatment, dose reduction or temporarily stopping the medication, induction of vomiting, administering oxygen, cardiac glycosides, as well as methods to control fever, hypoglycemia, and fluid loss.10',
              'Patients experiencing and overdose may present with symptoms of a hypermetabolic state.10 Overdose may be treated by symptomatic and supportive treatment, dose reduction or temporarily stopping the medication, induction of vomiting, administering oxygen, cardiac glycosides, as well as methods to control fever, hypoglycemia, and fluid loss.10'],
             'viibryd': ['There is a lack of clinical studies of vilazodone in pregnancyLabel. Animal studies have shown the effects on offspring to be reduced fetal weight, increased mortality, delayed maturation, and decreased fertility in adulthood at doses well above the maximum recommended human doseLabel. Clinical cases of fetal and neonatal exposure to SSRIs and SNRIs have lead to a number of complications including respiratory distress, seizures, and temperature instabilityLabel. It is not know whether vilazodone is excreted in the breast milk of nursing mothers but animal studies show this is the case for ratsLabel. The risk and benefit of breast feeding while taking vilazodone for mother and child must be considered before a decision is madeLabel. Safety and effectiveness in pediatric patients has not been established in clinical trials though antidepressants are associated with an increased risk of suicidal thought and behaviour in patients under 24 yearsLabel. Clinical studies in geriatric patients showed to significant difference in response to vilazodone compared to younger patientsLabel. Geriatric patients should be started at a lower dose and titrated to an effective dose as they are more likely to have other comorbiditiesLabel. Dosage adjustments are not necessary for patients of different genders or with reduced hepatic and renal functionLabel.',
              'There is a lack of clinical studies of vilazodone in pregnancyLabel. Animal studies have shown the effects on offspring to be reduced fetal weight, increased mortality, delayed maturation, and decreased fertility in adulthood at doses well above the maximum recommended human doseLabel. Clinical cases of fetal and neonatal exposure to SSRIs and SNRIs have lead to a number of complications including respiratory distress, seizures, and temperature instabilityLabel. It is not know whether vilazodone is excreted in the breast milk of nursing mothers but animal studies show this is the case for ratsLabel. The risk and benefit of breast feeding while taking vilazodone for mother and child must be considered before a decision is madeLabel. Safety and effectiveness in pediatric patients has not been established in clinical trials though antidepressants are associated with an increased risk of suicidal thought and behaviour in patients under 24 yearsLabel. Clinical studies in geriatric patients showed to significant difference in response to vilazodone compared to younger patientsLabel. Geriatric patients should be started at a lower dose and titrated to an effective dose as they are more likely to have other comorbiditiesLabel. Dosage adjustments are not necessary for patients of different genders or with reduced hepatic and renal functionLabel.'],
             'linzess': ['Most common adverse reactions (incidence of at least 2%) reported in IBS-C or CIC patients are diarrhea, abdominal pain, flatulence and abdominal distension.',
              'Most common adverse reactions (incidence of at least 2%) reported in IBS-C or CIC patients are diarrhea, abdominal pain, flatulence and abdominal distension.'],
             'bystolic (nebivolol)': ['Patients experiencing an overdose may present with bradycardia, hypotension, cardiac failure, dizziness, hypoglycemia, fatigue, vomiting, bronchospasm and heart block.9 Treat overdose with general supportive measures including intravenous atropine for bradycardia, vasopressors and intravenous fluids for hypotension, isoproterenol infusion for heart block, digitalis glycosides and diuretics for congestive heart failure, bronchodilators for bronchospasm, and intravenous glucose for hypoglycemia.9',
              'Patients experiencing an overdose may present with bradycardia, hypotension, cardiac failure, dizziness, hypoglycemia, fatigue, vomiting, bronchospasm and heart block.9 Treat overdose with general supportive measures including intravenous atropine for bradycardia, vasopressors and intravenous fluids for hypotension, isoproterenol infusion for heart block, digitalis glycosides and diuretics for congestive heart failure, bronchodilators for bronchospasm, and intravenous glucose for hypoglycemia.9'],
             'bystolic': ['Patients experiencing an overdose may present with bradycardia, hypotension, cardiac failure, dizziness, hypoglycemia, fatigue, vomiting, bronchospasm and heart block.9 Treat overdose with general supportive measures including intravenous atropine for bradycardia, vasopressors and intravenous fluids for hypotension, isoproterenol infusion for heart block, digitalis glycosides and diuretics for congestive heart failure, bronchodilators for bronchospasm, and intravenous glucose for hypoglycemia.9',
              'Patients experiencing an overdose may present with bradycardia, hypotension, cardiac failure, dizziness, hypoglycemia, fatigue, vomiting, bronchospasm and heart block.9 Treat overdose with general supportive measures including intravenous atropine for bradycardia, vasopressors and intravenous fluids for hypotension, isoproterenol infusion for heart block, digitalis glycosides and diuretics for congestive heart failure, bronchodilators for bronchospasm, and intravenous glucose for hypoglycemia.9'],
             'savella': ['There is limited clinical experience with milnacipran overdose in humans. In clinical trials, cases of acute ingestions up to 1000 mg daily were reported with none being fatal 26. In postmarketing experience, fatal outcomes have been reported for acute overdoses primarily involving multiple drugs but also with milnacipran only 26. The most common signs and symptoms of overdose included increased blood pressure, cardio-respiratory arrest, changes in the level of consciousness (ranging from somnolence to coma), confusional state, dizziness, and increased hepatic enzymes 24,25,26.',
              'There is limited clinical experience with milnacipran overdose in humans. In clinical trials, cases of acute ingestions up to 1000 mg daily were reported with none being fatal 26. In postmarketing experience, fatal outcomes have been reported for acute overdoses primarily involving multiple drugs but also with milnacipran only 26. The most common signs and symptoms of overdose included increased blood pressure, cardio-respiratory arrest, changes in the level of consciousness (ranging from somnolence to coma), confusional state, dizziness, and increased hepatic enzymes 24,25,26.'],
             'savella (milnacipran hcl)': ['There is limited clinical experience with milnacipran overdose in humans. In clinical trials, cases of acute ingestions up to 1000 mg daily were reported with none being fatal 26. In postmarketing experience, fatal outcomes have been reported for acute overdoses primarily involving multiple drugs but also with milnacipran only 26. The most common signs and symptoms of overdose included increased blood pressure, cardio-respiratory arrest, changes in the level of consciousness (ranging from somnolence to coma), confusional state, dizziness, and increased hepatic enzymes 24,25,26.',
              'There is limited clinical experience with milnacipran overdose in humans. In clinical trials, cases of acute ingestions up to 1000 mg daily were reported with none being fatal 26. In postmarketing experience, fatal outcomes have been reported for acute overdoses primarily involving multiple drugs but also with milnacipran only 26. The most common signs and symptoms of overdose included increased blood pressure, cardio-respiratory arrest, changes in the level of consciousness (ranging from somnolence to coma), confusional state, dizziness, and increased hepatic enzymes 24,25,26.'],
             'lexapro let': ['Symptoms of overdose may include CNS effects (dizziness, convulsions, coma, somnolence), gastrointestinal distress (nausea, vomiting), and/or cardiac abnormalities (hypotension, tachycardia, ECG changes).18,19,20 There is no specific antidote for escitalopram overdose. Management of overdose should focus on monitoring for cardiac abnormalities and changes to vital signs as well as treatment with supportive measures as indicated. As escitalopram is highly distributed into tissue following oral administration, forced diuresis, dialysis, and other methods of extracting drug from plasma are unlikely to be beneficial.',
              'Symptoms of overdose may include CNS effects (dizziness, convulsions, coma, somnolence), gastrointestinal distress (nausea, vomiting), and/or cardiac abnormalities (hypotension, tachycardia, ECG changes).18,19,20 There is no specific antidote for escitalopram overdose. Management of overdose should focus on monitoring for cardiac abnormalities and changes to vital signs as well as treatment with supportive measures as indicated. As escitalopram is highly distributed into tissue following oral administration, forced diuresis, dialysis, and other methods of extracting drug from plasma are unlikely to be beneficial.'],
             'lexapro': ['Symptoms of overdose may include CNS effects (dizziness, convulsions, coma, somnolence), gastrointestinal distress (nausea, vomiting), and/or cardiac abnormalities (hypotension, tachycardia, ECG changes).18,19,20 There is no specific antidote for escitalopram overdose. Management of overdose should focus on monitoring for cardiac abnormalities and changes to vital signs as well as treatment with supportive measures as indicated. As escitalopram is highly distributed into tissue following oral administration, forced diuresis, dialysis, and other methods of extracting drug from plasma are unlikely to be beneficial.',
              'Symptoms of overdose may include CNS effects (dizziness, convulsions, coma, somnolence), gastrointestinal distress (nausea, vomiting), and/or cardiac abnormalities (hypotension, tachycardia, ECG changes).18,19,20 There is no specific antidote for escitalopram overdose. Management of overdose should focus on monitoring for cardiac abnormalities and changes to vital signs as well as treatment with supportive measures as indicated. As escitalopram is highly distributed into tissue following oral administration, forced diuresis, dialysis, and other methods of extracting drug from plasma are unlikely to be beneficial.'],
             'fetzima': ['The most common adverse reactions are nausea, constipation, hyperhidrosis, heart rate increase, erectile dysfunction, tachycardia, vomiting, and palpitations. ',
              'The most common adverse reactions are nausea, constipation, hyperhidrosis, heart rate increase, erectile dysfunction, tachycardia, vomiting, and palpitations. '],
             'fetzima levomilnacipran': ['The most common adverse reactions are nausea, constipation, hyperhidrosis, heart rate increase, erectile dysfunction, tachycardia, vomiting, and palpitations. ',
              'The most common adverse reactions are nausea, constipation, hyperhidrosis, heart rate increase, erectile dysfunction, tachycardia, vomiting, and palpitations. '],
             'namenda s': ['LD50', 'LD50'],
             'namenda': ['LD50', 'LD50'],
             'campral s': ['The intraperitoneal LD50 in male mice is 1.87 g/kg.14 In reported cases of acute overdosage with acamprosate (doses of up to 56 grams of acamprosate calcium) diarrhea was the only reported symptom attributable to acamprosate. In the case of an overdose, supportive and symptomatic treatment is recommended.11',
              'The intraperitoneal LD50 in male mice is 1.87 g/kg.14 In reported cases of acute overdosage with acamprosate (doses of up to 56 grams of acamprosate calcium) diarrhea was the only reported symptom attributable to acamprosate. In the case of an overdose, supportive and symptomatic treatment is recommended.11'],
             'namenda xr extended release': ['LD50', 'LD50'],
             'prograf': ['Side effects can be severe and include blurred vision, liver and kidney problems (it is nephrotoxic), seizures, tremors, hypertension, hypomagnesemia, diabetes mellitus, hyperkalemia, itching, insomnia, confusion. LD50=134-194 mg/kg (rat).',
              'Side effects can be severe and include blurred vision, liver and kidney problems (it is nephrotoxic), seizures, tremors, hypertension, hypomagnesemia, diabetes mellitus, hyperkalemia, itching, insomnia, confusion. LD50=134-194 mg/kg (rat).'],
             'myrbetriq extended release': ['At doses of up to 400mg in healthy volunteers (~8x the recommended maximum), reported symptoms of overdose included palpitations and increased heart rate. Symptoms of chronic overdosage are similar in presentation and may also include a rise in systolic blood pressure. In cases of overdosage, employ standard symptomatic and supportive measures in addition to ECG monitoring.6',
              'At doses of up to 400mg in healthy volunteers (~8x the recommended maximum), reported symptoms of overdose included palpitations and increased heart rate. Symptoms of chronic overdosage are similar in presentation and may also include a rise in systolic blood pressure. In cases of overdosage, employ standard symptomatic and supportive measures in addition to ECG monitoring.6'],
             'kphos original (sodiumfree) urinary acidifier': ['Not Available',
              'Not Available'],
             'uroqid #': ['', ''],
             'kphos neutral phosphorus supplement': ['Not Available',
              'Not Available'],
             'deplin': ['Not Available', 'Not Available'],
             'cerefolin nac': ['', ''],
             'foltx &quot;new formula&quot;': ['Oral Rat LD50 = 4 gm/kg. Toxic effects include convulsions, dyspnea, hypermotility, diarrhea, ataxia and muscle weakness.',
              'Oral Rat LD50 = 4 gm/kg. Toxic effects include convulsions, dyspnea, hypermotility, diarrhea, ataxia and muscle weakness.'],
             'deplin (lmethylfolate)': ['Not Available', 'Not Available'],
             'metanx': ['The severity of local phototoxic reactions such as erythema, pain and burning sensation may increase in case of prolonged application time or very high light intensity.',
              'The severity of local phototoxic reactions such as erythema, pain and burning sensation may increase in case of prolonged application time or very high light intensity.'],
             'dicyclomine s': ['Patients experiencing an overdose may present with headache, nausea, vomiting, blurred vision, dilated pupils, dizziness, dry mouth, difficulty swallowing, CNS stimulation, as well as hot, dry skin.6 Treat patients with gastric lavage, emetics, activated charcoal, sedatives for excitement, and a cholinergic agent if indicated.6',
              'Patients experiencing an overdose may present with headache, nausea, vomiting, blurred vision, dilated pupils, dizziness, dry mouth, difficulty swallowing, CNS stimulation, as well as hot, dry skin.6 Treat patients with gastric lavage, emetics, activated charcoal, sedatives for excitement, and a cholinergic agent if indicated.6'],
             'acetazolamide s': ['Not Available', 'Not Available'],
             'terbutaline sulfate': ['Patients experiencing an overdose may present with abdominal pain, agitation, palpitations, seizures, angina, hypertension, hypotension, tachycardia, arrhythmias, nervousness, headache, tremor, dry mouth, nausea, dizziness, fatigue, malaise, insomnia.10,16,17 Discontinue treatment with terbutaline and initiate symptomatic and supportive therapy.16,17',
              'Patients experiencing an overdose may present with abdominal pain, agitation, palpitations, seizures, angina, hypertension, hypotension, tachycardia, arrhythmias, nervousness, headache, tremor, dry mouth, nausea, dizziness, fatigue, malaise, insomnia.10,16,17 Discontinue treatment with terbutaline and initiate symptomatic and supportive therapy.16,17'],
             'pilocarpine': ['Not Available', 'Not Available'],
             'rifampin': ['LD50=1570 mg/kg (rat), chronic exposure may cause nausea and vomiting and unconsciousness',
              'LD50=1570 mg/kg (rat), chronic exposure may cause nausea and vomiting and unconsciousness'],
             'digoxin s': ['Oral TDLO (human female): 100 ug/kg, Oral TDLO (human male): 75 ug/kg, Oral LD50 (rat): 28270 ug/kgMSDS',
              'Oral TDLO (human female): 100 ug/kg, Oral TDLO (human male): 75 ug/kg, Oral LD50 (rat): 28270 ug/kgMSDS'],
             'levothyroxine s': ['LD50=20 mg/kg (orally in rat). Hypermetabolic state indistinguishable from thyrotoxicosis of endogenous origin. Symptoms of thyrotoxicosis include weight loss, increased appetite, palpitations, nervousness, diarrhea, abdominal cramps, sweating, tachycardia, increased pulse and blood pressure, cardiac arrhythmias, tremors, insomnia, heat intolerance, fever, and menstrual irregularities.',
              'LD50=20 mg/kg (orally in rat). Hypermetabolic state indistinguishable from thyrotoxicosis of endogenous origin. Symptoms of thyrotoxicosis include weight loss, increased appetite, palpitations, nervousness, diarrhea, abdominal cramps, sweating, tachycardia, increased pulse and blood pressure, cardiac arrhythmias, tremors, insomnia, heat intolerance, fever, and menstrual irregularities.'],
             'levothyroxine mcg': ['LD50=20 mg/kg (orally in rat). Hypermetabolic state indistinguishable from thyrotoxicosis of endogenous origin. Symptoms of thyrotoxicosis include weight loss, increased appetite, palpitations, nervousness, diarrhea, abdominal cramps, sweating, tachycardia, increased pulse and blood pressure, cardiac arrhythmias, tremors, insomnia, heat intolerance, fever, and menstrual irregularities.',
              'LD50=20 mg/kg (orally in rat). Hypermetabolic state indistinguishable from thyrotoxicosis of endogenous origin. Symptoms of thyrotoxicosis include weight loss, increased appetite, palpitations, nervousness, diarrhea, abdominal cramps, sweating, tachycardia, increased pulse and blood pressure, cardiac arrhythmias, tremors, insomnia, heat intolerance, fever, and menstrual irregularities.'],
             'levothyroxine': ['LD50=20 mg/kg (orally in rat). Hypermetabolic state indistinguishable from thyrotoxicosis of endogenous origin. Symptoms of thyrotoxicosis include weight loss, increased appetite, palpitations, nervousness, diarrhea, abdominal cramps, sweating, tachycardia, increased pulse and blood pressure, cardiac arrhythmias, tremors, insomnia, heat intolerance, fever, and menstrual irregularities.',
              'LD50=20 mg/kg (orally in rat). Hypermetabolic state indistinguishable from thyrotoxicosis of endogenous origin. Symptoms of thyrotoxicosis include weight loss, increased appetite, palpitations, nervousness, diarrhea, abdominal cramps, sweating, tachycardia, increased pulse and blood pressure, cardiac arrhythmias, tremors, insomnia, heat intolerance, fever, and menstrual irregularities.'],
             'levothyroxin mcg': ['LD50=20 mg/kg (orally in rat). Hypermetabolic state indistinguishable from thyrotoxicosis of endogenous origin. Symptoms of thyrotoxicosis include weight loss, increased appetite, palpitations, nervousness, diarrhea, abdominal cramps, sweating, tachycardia, increased pulse and blood pressure, cardiac arrhythmias, tremors, insomnia, heat intolerance, fever, and menstrual irregularities.',
              'LD50=20 mg/kg (orally in rat). Hypermetabolic state indistinguishable from thyrotoxicosis of endogenous origin. Symptoms of thyrotoxicosis include weight loss, increased appetite, palpitations, nervousness, diarrhea, abdominal cramps, sweating, tachycardia, increased pulse and blood pressure, cardiac arrhythmias, tremors, insomnia, heat intolerance, fever, and menstrual irregularities.'],
             'clindamycin hcl': ['The oral LD50 in mice and rats is 2540 mg/kg and 2190 mg/kg, respectively.19',
              'The oral LD50 in mice and rats is 2540 mg/kg and 2190 mg/kg, respectively.19'],
             'mtestest estrogen': ['', ''],
             'doxycycline mono': ['Oral LD50 is 2000 mg/kg in rats, 1870 mg/kg in mice, and 500 mg/kg in dog.14',
              'Oral LD50 is 2000 mg/kg in rats, 1870 mg/kg in mice, and 500 mg/kg in dog.14'],
             'docusate sodium &amp; senna': ['Docusate is not known to be carcinogenic or cause reproductive toxicity. It was not mutagenic in the Ames assay MSDS. Acute oral LD50 in rat is 1900 mg/kg MSDS. ',
              'Docusate is not known to be carcinogenic or cause reproductive toxicity. It was not mutagenic in the Ames assay MSDS. Acute oral LD50 in rat is 1900 mg/kg MSDS. '],
             'enteric coated aspirin (nsaid)': ['Not Available',
              'Not Available'],
             'meclizine hcl': ['The oral and intraperitoneal LD50 in mouse are 1600 mg/kg and 625 mg/kg, respectively. The lowest published toxic dose (TDLo) in rats via the oral route is 800 mg/kg.MSDS ',
              'The oral and intraperitoneal LD50 in mouse are 1600 mg/kg and 625 mg/kg, respectively. The lowest published toxic dose (TDLo) in rats via the oral route is 800 mg/kg.MSDS '],
             'travel sickness': ['Infants and children experiencing an overdose may lead to hallucinations, convulsions, or death.10 Adults experiencing an overdose may present with drowsiness, convulsions, coma, or respiratory depression.10 Treat overdoses with symptomatic and supportive measures including mechanically assisted ventilation.10',
              'Infants and children experiencing an overdose may lead to hallucinations, convulsions, or death.10 Adults experiencing an overdose may present with drowsiness, convulsions, coma, or respiratory depression.10 Treat overdoses with symptomatic and supportive measures including mechanically assisted ventilation.10'],
             'aspirin (nsaid)': ['Lethal doses', 'Lethal doses'],
             'acetaminophen': ['338 mg/kg', '1944 mg/kg'],
             'aspirin (nsaid) microcoated': ['Lethal doses', 'Lethal doses'],
             'meclizine s': ['The oral and intraperitoneal LD50 in mouse are 1600 mg/kg and 625 mg/kg, respectively. The lowest published toxic dose (TDLo) in rats via the oral route is 800 mg/kg.MSDS ',
              'The oral and intraperitoneal LD50 in mouse are 1600 mg/kg and 625 mg/kg, respectively. The lowest published toxic dose (TDLo) in rats via the oral route is 800 mg/kg.MSDS '],
             'meclizine hci': ['The oral and intraperitoneal LD50 in mouse are 1600 mg/kg and 625 mg/kg, respectively. The lowest published toxic dose (TDLo) in rats via the oral route is 800 mg/kg.MSDS ',
              'The oral and intraperitoneal LD50 in mouse are 1600 mg/kg and 625 mg/kg, respectively. The lowest published toxic dose (TDLo) in rats via the oral route is 800 mg/kg.MSDS '],
             'sodium bicar': ['The acute oral LD50 in rats is 4500-5000 mg/kg and the intradermal LD50 in rabbits is 10,000 mg/kg.6 Individuals are likely to be exposed to boric acid from industrial manufacturing or processing. Local tissue injury from boric acid exposure is likely due to caustic effects. Systemic effects from boric acid poisoning usually occur from multiple exposures over a period of days and involve gastrointestinal, dermal, CNS, and renal manifestations. Gastrointestinal toxicity include persistent nausea, vomiting, diarrhea, epigastric pain, hematemesis, and blue-green discoloration of the feces and vomit 4. Following the onset of GI symptoms, a characteristic intense generalized erythroderma follows 4. Management of mild to moderate toxicity should be supportive. In case of severe toxicity, dialysis may be required in addition to supportive treatment. ',
              'The acute oral LD50 in rats is 4500-5000 mg/kg and the intradermal LD50 in rabbits is 10,000 mg/kg.6 Individuals are likely to be exposed to boric acid from industrial manufacturing or processing. Local tissue injury from boric acid exposure is likely due to caustic effects. Systemic effects from boric acid poisoning usually occur from multiple exposures over a period of days and involve gastrointestinal, dermal, CNS, and renal manifestations. Gastrointestinal toxicity include persistent nausea, vomiting, diarrhea, epigastric pain, hematemesis, and blue-green discoloration of the feces and vomit 4. Following the onset of GI symptoms, a characteristic intense generalized erythroderma follows 4. Management of mild to moderate toxicity should be supportive. In case of severe toxicity, dialysis may be required in addition to supportive treatment. '],
             'nephrovite': ['', ''],
             'chlordiazep': ['LD50=537 mg/kg (Orally in rats). Signs of overdose include respiratory depression, muscle weakness, somnolence (general depressed activity).',
              'LD50=537 mg/kg (Orally in rats). Signs of overdose include respiratory depression, muscle weakness, somnolence (general depressed activity).'],
             'diphenhyramine hydrochloride': ['e is expected to result in effects similar to the adverse effects that are ordinarily associated with the use of diphenhydramine, including drowsiness, hyperpyrexia, and anticholinergic effects, among others 10,11,12,13,18. Additional symptoms during overdose may include mydriasis, fever, flushing, agitation, tremor, dystonic reactions, hallucinations and ECG changes 13. Large overdose may cause rhabdomyolysis, convulsions, delirium, toxic psychosis, arrhythmias, coma and cardiovascular collapse 13. Moreover, with higher doses, and particularly in children, symptoms of CNS excitation including hallucinations and convulsions may appea',
              'assive doses, coma or cardiovascular collapse may follow 18'],
             'clonazepam orally disintegrating': ['Benzodiazepines like clonazepam commonly cause drowsiness, ataxia, dysarthria, and nystagmus. Overdose with clonazepam is generally not life-threatening if the drug is taken alone, but may lead to areflexia, apnea, hypotension, cardiorespiratory depression, and coma. Coma, if it does occur, usually lasts a few hours but it can become more protracted and cyclical, especially in elderly patients. Increased frequency of seizures may occur in patients at supratherapeutic plasma concentrations. Benzodiazepine respiratory depressant effects are more serious when compounded in patients with respiratory disease.',
              'Benzodiazepines like clonazepam commonly cause drowsiness, ataxia, dysarthria, and nystagmus. Overdose with clonazepam is generally not life-threatening if the drug is taken alone, but may lead to areflexia, apnea, hypotension, cardiorespiratory depression, and coma. Coma, if it does occur, usually lasts a few hours but it can become more protracted and cyclical, especially in elderly patients. Increased frequency of seizures may occur in patients at supratherapeutic plasma concentrations. Benzodiazepine respiratory depressant effects are more serious when compounded in patients with respiratory disease.'],
             'chlordiazepoxide s': ['LD50=537 mg/kg (Orally in rats). Signs of overdose include respiratory depression, muscle weakness, somnolence (general depressed activity).',
              'LD50=537 mg/kg (Orally in rats). Signs of overdose include respiratory depression, muscle weakness, somnolence (general depressed activity).'],
             'mefloquine hcl': ['The oral TDLO of mefloquine in humans is 11 mg/kg/2W (intermittent) and 880 mg/kg in the rat. Intraperitoneal LD50 in the rat is 130 mg/kg.17\nSymptoms of an overdose with mefloquine may manifest as a worsening of adverse effects. In the case of an overdose, symptomatic and supportive care should be provided. There is no known antidote for an overdose with mefloquine.13 Monitor cardiac function by ECG, follow neuropsychiatric status for at least 24 hours, and provide treatment as required.13 ',
              'The oral TDLO of mefloquine in humans is 11 mg/kg/2W (intermittent) and 880 mg/kg in the rat. Intraperitoneal LD50 in the rat is 130 mg/kg.17\nSymptoms of an overdose with mefloquine may manifest as a worsening of adverse effects. In the case of an overdose, symptomatic and supportive care should be provided. There is no known antidote for an overdose with mefloquine.13 Monitor cardiac function by ECG, follow neuropsychiatric status for at least 24 hours, and provide treatment as required.13 '],
             'norethindrone': ['The oral LD50 in mice 6 g/kg and the TDLo in human women is 42 mg/kg.20 There have been no reports of serious ill effects following overdose of oral contraceptives, including following ingestion by children.19,18 Symptoms of overdosage are likely to be consistent with the adverse effect profile of the contraceptive and may, therefore, include significant nausea and/or vomiting.',
              'The oral LD50 in mice 6 g/kg and the TDLo in human women is 42 mg/kg.20 There have been no reports of serious ill effects following overdose of oral contraceptives, including following ingestion by children.19,18 Symptoms of overdosage are likely to be consistent with the adverse effect profile of the contraceptive and may, therefore, include significant nausea and/or vomiting.'],
             'norethindrone acetate': ['The oral LD50 in mice 6 g/kg and the TDLo in human women is 42 mg/kg.20 There have been no reports of serious ill effects following overdose of oral contraceptives, including following ingestion by children.19,18 Symptoms of overdosage are likely to be consistent with the adverse effect profile of the contraceptive and may, therefore, include significant nausea and/or vomiting.',
              'The oral LD50 in mice 6 g/kg and the TDLo in human women is 42 mg/kg.20 There have been no reports of serious ill effects following overdose of oral contraceptives, including following ingestion by children.19,18 Symptoms of overdosage are likely to be consistent with the adverse effect profile of the contraceptive and may, therefore, include significant nausea and/or vomiting.'],
             'hydroxyzine pam s': ['The oral LD50 is 840 mg/kg in rats and 400 mg/kg in mice.16 ',
              'The oral LD50 is 840 mg/kg in rats and 400 mg/kg in mice.16 '],
             'errin': ['The oral LD50 in mice 6 g/kg and the TDLo in human women is 42 mg/kg.20 There have been no reports of serious ill effects following overdose of oral contraceptives, including following ingestion by children.19,18 Symptoms of overdosage are likely to be consistent with the adverse effect profile of the contraceptive and may, therefore, include significant nausea and/or vomiting.',
              'The oral LD50 in mice 6 g/kg and the TDLo in human women is 42 mg/kg.20 There have been no reports of serious ill effects following overdose of oral contraceptives, including following ingestion by children.19,18 Symptoms of overdosage are likely to be consistent with the adverse effect profile of the contraceptive and may, therefore, include significant nausea and/or vomiting.'],
             'meperidine hcl': ['Not Available', 'Not Available'],
             'amiloridehctz': ['', ''],
             'amiloride hydrochloride and hydrochlorothiazide': ['No data are available in regard to overdosage in humans. The oral LD50 of amiloride hydrochloride (calculated as the base) is 56 mg/kg in mice and 36 to 85 mg/kg in rats, depending on the strain. The most likely signs and symptoms to be expected with overdosage are dehydration and electrolyte imbalance.',
              'No data are available in regard to overdosage in humans. The oral LD50 of amiloride hydrochloride (calculated as the base) is 56 mg/kg in mice and 36 to 85 mg/kg in rats, depending on the strain. The most likely signs and symptoms to be expected with overdosage are dehydration and electrolyte imbalance.'],
             'acetazolamide': ['Not Available', 'Not Available'],
             'didanosine': ['Side effects include pancreatitis, peripheral neuropathy, diarrhea, hyperuricemia and hepatic dysfunction',
              'Side effects include pancreatitis, peripheral neuropathy, diarrhea, hyperuricemia and hepatic dysfunction'],
             'megestrol acetate :ets': ['No serious unexpected side effects have resulted from studies involving megestrol acetate oral suspension administered in dosages as high as 1200 mg/day. Treatment with megestrol acetate, an orexigenic agent, has also resulted in iatrogenic adrenal suppression. The mechanism is presumably related to the glucocorticoid properties of megestrol acetate [PMID: 12872362].',
              'No serious unexpected side effects have resulted from studies involving megestrol acetate oral suspension administered in dosages as high as 1200 mg/day. Treatment with megestrol acetate, an orexigenic agent, has also resulted in iatrogenic adrenal suppression. The mechanism is presumably related to the glucocorticoid properties of megestrol acetate [PMID: 12872362].'],
             'megestrol acetate': ['No serious unexpected side effects have resulted from studies involving megestrol acetate oral suspension administered in dosages as high as 1200 mg/day. Treatment with megestrol acetate, an orexigenic agent, has also resulted in iatrogenic adrenal suppression. The mechanism is presumably related to the glucocorticoid properties of megestrol acetate [PMID: 12872362].',
              'No serious unexpected side effects have resulted from studies involving megestrol acetate oral suspension administered in dosages as high as 1200 mg/day. Treatment with megestrol acetate, an orexigenic agent, has also resulted in iatrogenic adrenal suppression. The mechanism is presumably related to the glucocorticoid properties of megestrol acetate [PMID: 12872362].'],
             'pramipexole': ['LD50', 'LD50'],
             'danazol': ['Not Available', 'Not Available'],
             'oxycodone and acetaminophen': ['Patients experiencing an overdose may present with respiratory depression, sleepiness, stupor, coma, skeletal muscle flaccidity, cold sweat, constricted pupils, bradycardia, hypotension, partial or complete airway obstruction, atypical snoring, and death.Label Overdose should be treated by maintaining airway, ventilation, and oxygenation.Label Oxygen and vasopressor treatment may be necessary to treat circulatory shock and pulmonary edema and defibrillation may be required for cardiac arrest of arrhythmia.Label Naloxone, nalmefene, or naltrexone may be used to counteract the effects of opioids but patients should be monitored in case further doses are required.Label',
              'Patients experiencing an overdose may present with respiratory depression, sleepiness, stupor, coma, skeletal muscle flaccidity, cold sweat, constricted pupils, bradycardia, hypotension, partial or complete airway obstruction, atypical snoring, and death.Label Overdose should be treated by maintaining airway, ventilation, and oxygenation.Label Oxygen and vasopressor treatment may be necessary to treat circulatory shock and pulmonary edema and defibrillation may be required for cardiac arrest of arrhythmia.Label Naloxone, nalmefene, or naltrexone may be used to counteract the effects of opioids but patients should be monitored in case further doses are required.Label'],
             'demeclocycline hydrochloride': ['Oral, rat: LD50 = 2372 mg/kg',
              'Oral, rat: LD50 = 2372 mg/kg'],
             'camila': ['The oral LD50 in mice 6 g/kg and the TDLo in human women is 42 mg/kg.20 There have been no reports of serious ill effects following overdose of oral contraceptives, including following ingestion by children.19,18 Symptoms of overdosage are likely to be consistent with the adverse effect profile of the contraceptive and may, therefore, include significant nausea and/or vomiting.',
              'The oral LD50 in mice 6 g/kg and the TDLo in human women is 42 mg/kg.20 There have been no reports of serious ill effects following overdose of oral contraceptives, including following ingestion by children.19,18 Symptoms of overdosage are likely to be consistent with the adverse effect profile of the contraceptive and may, therefore, include significant nausea and/or vomiting.'],
             'estropipate ()': ['Not Available', 'Not Available'],
             'trazodone hydrochloride': ['The oral LD50 of trazodone is 690 mg/kg in rats.22',
              'The oral LD50 of trazodone is 690 mg/kg in rats.22'],
             'adderall': ['The mean lethal serum concentration is reported to be of 6.4 mg/l. Acute amphetamine overdose can lead to hyperthermia, respiratory depression, seizures, metabolic acidosis, renal failure, hepatic injury, and coma. Some of the neurologic effects have been shown to be agitation, aggressive behavior, irritability, headache, and hallucinations. In the cardiovascular site, there have been reports of arrhythmia, cardiomyopathy, myocardial infarction or ischemic stroke. Lastly, in the GI tract, there are reports if abdominal pain, vomiting, diarrhea, cramps, anorexia and GI hemorrhage. A dose of 1-2 g of amphetamine is known to cause severe intoxication but some chronic abusers can report usage of even 5-15 g per day.16',
              'The mean lethal serum concentration is reported to be of 6.4 mg/l. Acute amphetamine overdose can lead to hyperthermia, respiratory depression, seizures, metabolic acidosis, renal failure, hepatic injury, and coma. Some of the neurologic effects have been shown to be agitation, aggressive behavior, irritability, headache, and hallucinations. In the cardiovascular site, there have been reports of arrhythmia, cardiomyopathy, myocardial infarction or ischemic stroke. Lastly, in the GI tract, there are reports if abdominal pain, vomiting, diarrhea, cramps, anorexia and GI hemorrhage. A dose of 1-2 g of amphetamine is known to cause severe intoxication but some chronic abusers can report usage of even 5-15 g per day.16'],
             'dextroamphetamine saccharate amphetamine aspartate dextroamphetamine sulfate and amphetamine sulfate': ['Dextroamphetamine has been shown to be teratogenic and embryotoxic in mice at 41 times the maximum human dose.Label These effects were not seen in rat or rabbit studies, and the effects on human pregnancy have not been studied.Label The risk and benefit of use during pregnancy should be weighed as bone deformities, tracheoesophageal fistula, anal atresia, low birthweight, and withdrawl have been reported in the children of mothers who were taking dextroamphetamine during pregnancy.Label Mothers should not take amphetamines while nursing as the drug is excreted in breast milk.Label Long term effects of dextroamphetamine have not bee determined in pediatric patients and dextroamphetamine should be avoided in children under 3 years.Label',
              'Dextroamphetamine has been shown to be teratogenic and embryotoxic in mice at 41 times the maximum human dose.Label These effects were not seen in rat or rabbit studies, and the effects on human pregnancy have not been studied.Label The risk and benefit of use during pregnancy should be weighed as bone deformities, tracheoesophageal fistula, anal atresia, low birthweight, and withdrawl have been reported in the children of mothers who were taking dextroamphetamine during pregnancy.Label Mothers should not take amphetamines while nursing as the drug is excreted in breast milk.Label Long term effects of dextroamphetamine have not bee determined in pediatric patients and dextroamphetamine should be avoided in children under 3 years.Label'],
             'medroxyprogesterone acetate': ['The oral LD50 in rats is >6400mg/kg and in mice is >16g/kg.16 The intraperitoneal LD50 in rats is >900mg/kg and in mice is >1500mg/kg.16 The subcutaneous LD50 in rats is >900mg/kg and in mice is>1500mg/kg.16',
              'The oral LD50 in rats is >6400mg/kg and in mice is >16g/kg.16 The intraperitoneal LD50 in rats is >900mg/kg and in mice is >1500mg/kg.16 The subcutaneous LD50 in rats is >900mg/kg and in mice is>1500mg/kg.16'],
             'medroxypr ac': ['The oral LD50 in rats is >6400mg/kg and in mice is >16g/kg.16 The intraperitoneal LD50 in rats is >900mg/kg and in mice is >1500mg/kg.16 The subcutaneous LD50 in rats is >900mg/kg and in mice is>1500mg/kg.16',
              'The oral LD50 in rats is >6400mg/kg and in mice is >16g/kg.16 The intraperitoneal LD50 in rats is >900mg/kg and in mice is >1500mg/kg.16 The subcutaneous LD50 in rats is >900mg/kg and in mice is>1500mg/kg.16'],
             'tretinoin': ['Not Available', 'Not Available'],
             'warfarin': ['LD50 Values', 'LD50 Values'],
             'flecainide acetate': ['The oral LD50 in rats is 1346mg/kg and in mice is 170mg/kg.14 The subcutaneous LD50 in rats is 215mg/kg and in mice is 188mg/kg.14 The oral TDLO in women is 20mg/kg and in men is 40mg/kg/2W.14',
              'The oral LD50 in rats is 1346mg/kg and in mice is 170mg/kg.14 The subcutaneous LD50 in rats is 215mg/kg and in mice is 188mg/kg.14 The oral TDLO in women is 20mg/kg and in men is 40mg/kg/2W.14'],
             'medroxyprogest acet': ['The oral LD50 in rats is >6400mg/kg and in mice is >16g/kg.16 The intraperitoneal LD50 in rats is >900mg/kg and in mice is >1500mg/kg.16 The subcutaneous LD50 in rats is >900mg/kg and in mice is>1500mg/kg.16',
              'The oral LD50 in rats is >6400mg/kg and in mice is >16g/kg.16 The intraperitoneal LD50 in rats is >900mg/kg and in mice is >1500mg/kg.16 The subcutaneous LD50 in rats is >900mg/kg and in mice is>1500mg/kg.16'],
             'hydroxyurea': ['ouse: LD50 = 7330 mg/k',
              'rat: LD50 = 5760 mg/kg\nTeratogenicity: Teratogenic effects have occurred in experimental animals.Hydroxyurea use during a small number of human pregnancies has been reported. Adverse effects have not been observed in any of the exposed newborns.\nReproductive Effects: Adverse reproductive effects have occurred in experimental animals.\nMutagenicity: Mutagenic effects have occurred in experimental animals.Mutagenic effects have occurred in humans'],
             'ethambutol hydrochloride': ['Patients experiencing a chronic overdose of ethambutol may present with disturbances in colour vision and reduced visual acuity as symptoms of optic neuropathy.8 In these cases, ethambutol should be stopped.8 Data regarding acute overdose of ethambutol are not readily available. Patients experiencing an acute overdose of ethambutol may be experience an increased risk and severity of adverse effects such as pruritus, joint pain, gastrointestinal upset, abdominal pain, malaise, headache, dizziness, mental confusion, disorientation, and possible hallucinations.12,14 Patients should be treated with symptomatic and supportive measures.',
              'Patients experiencing a chronic overdose of ethambutol may present with disturbances in colour vision and reduced visual acuity as symptoms of optic neuropathy.8 In these cases, ethambutol should be stopped.8 Data regarding acute overdose of ethambutol are not readily available. Patients experiencing an acute overdose of ethambutol may be experience an increased risk and severity of adverse effects such as pruritus, joint pain, gastrointestinal upset, abdominal pain, malaise, headache, dizziness, mental confusion, disorientation, and possible hallucinations.12,14 Patients should be treated with symptomatic and supportive measures.'],
             'dextroamphetamine sulfate': ['Dextroamphetamine has been shown to be teratogenic and embryotoxic in mice at 41 times the maximum human dose.Label These effects were not seen in rat or rabbit studies, and the effects on human pregnancy have not been studied.Label The risk and benefit of use during pregnancy should be weighed as bone deformities, tracheoesophageal fistula, anal atresia, low birthweight, and withdrawl have been reported in the children of mothers who were taking dextroamphetamine during pregnancy.Label Mothers should not take amphetamines while nursing as the drug is excreted in breast milk.Label Long term effects of dextroamphetamine have not bee determined in pediatric patients and dextroamphetamine should be avoided in children under 3 years.Label',
              'Dextroamphetamine has been shown to be teratogenic and embryotoxic in mice at 41 times the maximum human dose.Label These effects were not seen in rat or rabbit studies, and the effects on human pregnancy have not been studied.Label The risk and benefit of use during pregnancy should be weighed as bone deformities, tracheoesophageal fistula, anal atresia, low birthweight, and withdrawl have been reported in the children of mothers who were taking dextroamphetamine during pregnancy.Label Mothers should not take amphetamines while nursing as the drug is excreted in breast milk.Label Long term effects of dextroamphetamine have not bee determined in pediatric patients and dextroamphetamine should be avoided in children under 3 years.Label'],
             'nimodipine': ['Symptoms of overdosage would be expected to be related to cardiovascular effects such as excessive peripheral vasodilation with marked systemic hypotension.',
              'Symptoms of overdosage would be expected to be related to cardiovascular effects such as excessive peripheral vasodilation with marked systemic hypotension.'],
             'fludcortisone': ['The oral LD50 of fludrocortisone in rats is >1g/kg.17 Acute overdosage of fludrocortisone is likely to result in symptoms consistent with its adverse effect profile. Patients receiving a single large dose should be treated with plenty of water by mouth and should undergo monitoring of serum electrolytes, particularly potassium and sodium, and be treated appropriately for any developing imbalances.15,16 ',
              'The oral LD50 of fludrocortisone in rats is >1g/kg.17 Acute overdosage of fludrocortisone is likely to result in symptoms consistent with its adverse effect profile. Patients receiving a single large dose should be treated with plenty of water by mouth and should undergo monitoring of serum electrolytes, particularly potassium and sodium, and be treated appropriately for any developing imbalances.15,16 '],
             'galantamine hydrobromide': ['The oral LD50 of the active ingredient, galantamine hydrobromide, in rats is 75 mg/kg.12 Symptoms of overdose are expected to be similar to those of cholinomimetics, which involve the central nervous system, the parasympathetic nervous system, and the neuromuscular junction. Effects of a cholinergic crisis include severe nausea, vomiting, gastrointestinal cramping, salivation, lacrimation, urination, defecation, sweating, bradycardia, hypotension, respiratory depression, collapse, and convulsions. Muscle weakness or fasciculations may also occur, with respiratory muscle weakness having the potential to bring fatal results. In one patient who consumed an oral daily dose of 32 mg developed bradycardia, QT prolongation, ventricular tachycardia and torsades de pointes accompanied by a brief loss of consciousness. In one patient with a history of hallucinations who consumed a daily dose of 24 mg galantamine, hallucinations requiring hospitalization occurred. A patient who ingested 160 mg of galantamine from an oral solution developed sweating, vomiting, bradycardia, and near-syncope one hour following consumption.10 ',
              'The oral LD50 of the active ingredient, galantamine hydrobromide, in rats is 75 mg/kg.12 Symptoms of overdose are expected to be similar to those of cholinomimetics, which involve the central nervous system, the parasympathetic nervous system, and the neuromuscular junction. Effects of a cholinergic crisis include severe nausea, vomiting, gastrointestinal cramping, salivation, lacrimation, urination, defecation, sweating, bradycardia, hypotension, respiratory depression, collapse, and convulsions. Muscle weakness or fasciculations may also occur, with respiratory muscle weakness having the potential to bring fatal results. In one patient who consumed an oral daily dose of 32 mg developed bradycardia, QT prolongation, ventricular tachycardia and torsades de pointes accompanied by a brief loss of consciousness. In one patient with a history of hallucinations who consumed a daily dose of 24 mg galantamine, hallucinations requiring hospitalization occurred. A patient who ingested 160 mg of galantamine from an oral solution developed sweating, vomiting, bradycardia, and near-syncope one hour following consumption.10 '],
             'claravis': ['Patients experiencing an overdose may present with vomiting, facial flushing, cheilosis, abdominal pain, headache, dizziness, and ataxia.Label These symptoms may rapidly resolve.Label Generally no treatment is required for these overdoses.7',
              'Patients experiencing an overdose may present with vomiting, facial flushing, cheilosis, abdominal pain, headache, dizziness, and ataxia.Label These symptoms may rapidly resolve.Label Generally no treatment is required for these overdoses.7'],
             'benzonatate': ['The lowest published toxic dose (TDLo) from oral administration in man is 51 mg/kg. The oral LD50 is 400 mg/kg and the subcutaneous LD50 is 100 mg/kg in mice.8',
              'The lowest published toxic dose (TDLo) from oral administration in man is 51 mg/kg. The oral LD50 is 400 mg/kg and the subcutaneous LD50 is 100 mg/kg in mice.8'],
             'nortrel': ['Female patients experiencing and overdose may present with withdrawal bleeding, nausea, vomiting, breast tenderness, abdominal pain, drowsiness, and fatigue.15,25,16,17,24,18,19,20,21,22,23 Overdose should be treated with symptomatic and supportive care including monitoring for potassium concentrations, sodium concentrations, and signs of metabolic acidosis.15,25,16,17,24,18,19,20,21,22,23',
              'Female patients experiencing and overdose may present with withdrawal bleeding, nausea, vomiting, breast tenderness, abdominal pain, drowsiness, and fatigue.15,25,16,17,24,18,19,20,21,22,23 Overdose should be treated with symptomatic and supportive care including monitoring for potassium concentrations, sodium concentrations, and signs of metabolic acidosis.15,25,16,17,24,18,19,20,21,22,23'],
             'nortel': ['Not Available', 'Not Available'],
             'lessina': ['Female patients experiencing and overdose may present with withdrawal bleeding, nausea, vomiting, breast tenderness, abdominal pain, drowsiness, and fatigue.15,25,16,17,24,18,19,20,21,22,23 Overdose should be treated with symptomatic and supportive care including monitoring for potassium concentrations, sodium concentrations, and signs of metabolic acidosis.15,25,16,17,24,18,19,20,21,22,23',
              'Female patients experiencing and overdose may present with withdrawal bleeding, nausea, vomiting, breast tenderness, abdominal pain, drowsiness, and fatigue.15,25,16,17,24,18,19,20,21,22,23 Overdose should be treated with symptomatic and supportive care including monitoring for potassium concentrations, sodium concentrations, and signs of metabolic acidosis.15,25,16,17,24,18,19,20,21,22,23'],
             'sprintec': ['Data regarding overdoses of norgestimate are rare.10,11 However, the majority of patients overdosing on oral contraceptives do not become seriously ill.6 Treat overdoses with symptomatic and supportive care.6,10,11',
              'Data regarding overdoses of norgestimate are rare.10,11 However, the majority of patients overdosing on oral contraceptives do not become seriously ill.6 Treat overdoses with symptomatic and supportive care.6,10,11'],
             'trisprintec': ['', ''],
             'portia': ['Female patients experiencing and overdose may present with withdrawal bleeding, nausea, vomiting, breast tenderness, abdominal pain, drowsiness, and fatigue.15,25,16,17,24,18,19,20,21,22,23 Overdose should be treated with symptomatic and supportive care including monitoring for potassium concentrations, sodium concentrations, and signs of metabolic acidosis.15,25,16,17,24,18,19,20,21,22,23',
              'Female patients experiencing and overdose may present with withdrawal bleeding, nausea, vomiting, breast tenderness, abdominal pain, drowsiness, and fatigue.15,25,16,17,24,18,19,20,21,22,23 Overdose should be treated with symptomatic and supportive care including monitoring for potassium concentrations, sodium concentrations, and signs of metabolic acidosis.15,25,16,17,24,18,19,20,21,22,23'],
             'junel': ['Female patients experiencing and overdose may present with withdrawal bleeding, nausea, vomiting, breast tenderness, abdominal pain, drowsiness, and fatigue.15,25,16,17,24,18,19,20,21,22,23 Overdose should be treated with symptomatic and supportive care including monitoring for potassium concentrations, sodium concentrations, and signs of metabolic acidosis.15,25,16,17,24,18,19,20,21,22,23',
              'Female patients experiencing and overdose may present with withdrawal bleeding, nausea, vomiting, breast tenderness, abdominal pain, drowsiness, and fatigue.15,25,16,17,24,18,19,20,21,22,23 Overdose should be treated with symptomatic and supportive care including monitoring for potassium concentrations, sodium concentrations, and signs of metabolic acidosis.15,25,16,17,24,18,19,20,21,22,23'],
             'trilegest fe': ['', ''],
             'balziva': ['Female patients experiencing and overdose may present with withdrawal bleeding, nausea, vomiting, breast tenderness, abdominal pain, drowsiness, and fatigue.15,25,16,17,24,18,19,20,21,22,23 Overdose should be treated with symptomatic and supportive care including monitoring for potassium concentrations, sodium concentrations, and signs of metabolic acidosis.15,25,16,17,24,18,19,20,21,22,23',
              'Female patients experiencing and overdose may present with withdrawal bleeding, nausea, vomiting, breast tenderness, abdominal pain, drowsiness, and fatigue.15,25,16,17,24,18,19,20,21,22,23 Overdose should be treated with symptomatic and supportive care including monitoring for potassium concentrations, sodium concentrations, and signs of metabolic acidosis.15,25,16,17,24,18,19,20,21,22,23'],
             'apri days': ['Not Available', 'Not Available'],
             'aviane': ['Female patients experiencing and overdose may present with withdrawal bleeding, nausea, vomiting, breast tenderness, abdominal pain, drowsiness, and fatigue.15,25,16,17,24,18,19,20,21,22,23 Overdose should be treated with symptomatic and supportive care including monitoring for potassium concentrations, sodium concentrations, and signs of metabolic acidosis.15,25,16,17,24,18,19,20,21,22,23',
              'Female patients experiencing and overdose may present with withdrawal bleeding, nausea, vomiting, breast tenderness, abdominal pain, drowsiness, and fatigue.15,25,16,17,24,18,19,20,21,22,23 Overdose should be treated with symptomatic and supportive care including monitoring for potassium concentrations, sodium concentrations, and signs of metabolic acidosis.15,25,16,17,24,18,19,20,21,22,23'],
             'enpresse': ['Female patients experiencing and overdose may present with withdrawal bleeding, nausea, vomiting, breast tenderness, abdominal pain, drowsiness, and fatigue.15,25,16,17,24,18,19,20,21,22,23 Overdose should be treated with symptomatic and supportive care including monitoring for potassium concentrations, sodium concentrations, and signs of metabolic acidosis.15,25,16,17,24,18,19,20,21,22,23',
              'Female patients experiencing and overdose may present with withdrawal bleeding, nausea, vomiting, breast tenderness, abdominal pain, drowsiness, and fatigue.15,25,16,17,24,18,19,20,21,22,23 Overdose should be treated with symptomatic and supportive care including monitoring for potassium concentrations, sodium concentrations, and signs of metabolic acidosis.15,25,16,17,24,18,19,20,21,22,23'],
             'cryselle': ['Not Available', 'Not Available'],
             'kariva': ['Female patients experiencing and overdose may present with withdrawal bleeding, nausea, vomiting, breast tenderness, abdominal pain, drowsiness, and fatigue.15,25,16,17,24,18,19,20,21,22,23 Overdose should be treated with symptomatic and supportive care including monitoring for potassium concentrations, sodium concentrations, and signs of metabolic acidosis.15,25,16,17,24,18,19,20,21,22,23',
              'Female patients experiencing and overdose may present with withdrawal bleeding, nausea, vomiting, breast tenderness, abdominal pain, drowsiness, and fatigue.15,25,16,17,24,18,19,20,21,22,23 Overdose should be treated with symptomatic and supportive care including monitoring for potassium concentrations, sodium concentrations, and signs of metabolic acidosis.15,25,16,17,24,18,19,20,21,22,23'],
             'velivet': ['Female patients experiencing and overdose may present with withdrawal bleeding, nausea, vomiting, breast tenderness, abdominal pain, drowsiness, and fatigue.15,25,16,17,24,18,19,20,21,22,23 Overdose should be treated with symptomatic and supportive care including monitoring for potassium concentrations, sodium concentrations, and signs of metabolic acidosis.15,25,16,17,24,18,19,20,21,22,23',
              'Female patients experiencing and overdose may present with withdrawal bleeding, nausea, vomiting, breast tenderness, abdominal pain, drowsiness, and fatigue.15,25,16,17,24,18,19,20,21,22,23 Overdose should be treated with symptomatic and supportive care including monitoring for potassium concentrations, sodium concentrations, and signs of metabolic acidosis.15,25,16,17,24,18,19,20,21,22,23'],
             'kelnor': ['Not Available', 'Not Available'],
             'aranelle': ['Female patients experiencing and overdose may present with withdrawal bleeding, nausea, vomiting, breast tenderness, abdominal pain, drowsiness, and fatigue.15,25,16,17,24,18,19,20,21,22,23 Overdose should be treated with symptomatic and supportive care including monitoring for potassium concentrations, sodium concentrations, and signs of metabolic acidosis.15,25,16,17,24,18,19,20,21,22,23',
              'Female patients experiencing and overdose may present with withdrawal bleeding, nausea, vomiting, breast tenderness, abdominal pain, drowsiness, and fatigue.15,25,16,17,24,18,19,20,21,22,23 Overdose should be treated with symptomatic and supportive care including monitoring for potassium concentrations, sodium concentrations, and signs of metabolic acidosis.15,25,16,17,24,18,19,20,21,22,23'],
             'jolessa': ['Female patients experiencing and overdose may present with withdrawal bleeding, nausea, vomiting, breast tenderness, abdominal pain, drowsiness, and fatigue.15,25,16,17,24,18,19,20,21,22,23 Overdose should be treated with symptomatic and supportive care including monitoring for potassium concentrations, sodium concentrations, and signs of metabolic acidosis.15,25,16,17,24,18,19,20,21,22,23',
              'Female patients experiencing and overdose may present with withdrawal bleeding, nausea, vomiting, breast tenderness, abdominal pain, drowsiness, and fatigue.15,25,16,17,24,18,19,20,21,22,23 Overdose should be treated with symptomatic and supportive care including monitoring for potassium concentrations, sodium concentrations, and signs of metabolic acidosis.15,25,16,17,24,18,19,20,21,22,23'],
             'ocella': ['The oral LD50 of drospirenone in rats is >2000 mg/kg.22',
              'The oral LD50 of drospirenone in rats is >2000 mg/kg.22'],
             'alavert d hour': ['generation antihistamines such as loratadine have very few adverse effect',
              'r, insomnia, headache, fatigue, drowsiness and rash have been reported.7 Symptoms of loratadine overdose include gastrointestinal side effects, agitation, drowsiness, tachycardia, and headache.7 It is advised to obtain an ECG in the event of loratadine overdose.'],
             'clotrimazole troches': ['Symptoms of overdose include erythema, stinging, blistering, peeling, edema, pruritus, urticaria, burning, and general irritation of the skin, and cramps. As with all topical agents, skin sensitization may result 12.',
              'Symptoms of overdose include erythema, stinging, blistering, peeling, edema, pruritus, urticaria, burning, and general irritation of the skin, and cramps. As with all topical agents, skin sensitization may result 12.'],
             'moexipril': ['Human overdoses of moexipril have not been reported. In case reports of overdoses with other ACE inhibitors, hypotension has been the principal adverse effect noted. Single oral doses of 2 g/kg moexipril were associated with significant lethality in mice. Rats, however, tolerated single oral doses of up to 3 g/kg. Common adverse effects include cough, dizziness, diarrhea, flu syndrome, fatigue, pharyngitis, flushing, rash, and myalgia',
              'Human overdoses of moexipril have not been reported. In case reports of overdoses with other ACE inhibitors, hypotension has been the principal adverse effect noted. Single oral doses of 2 g/kg moexipril were associated with significant lethality in mice. Rats, however, tolerated single oral doses of up to 3 g/kg. Common adverse effects include cough, dizziness, diarrhea, flu syndrome, fatigue, pharyngitis, flushing, rash, and myalgia'],
             'trospium chloride': ['Not Available', 'Not Available'],
             'liothyronine mcg': ['The reported oral LD50 of liothyronine in the rat is higher than 4540 mg/kg. When overdosage is registered, symptoms of hyperthyroidism are reported as well as confusion, disorientation, cerebral embolism, seizure, shock, coma, and death. The symptoms of overdose can be presented immediately or several days after overdose ingestion. In an overdose state, reduce the dose of liothyronine and do supportive treatment.Label',
              'The reported oral LD50 of liothyronine in the rat is higher than 4540 mg/kg. When overdosage is registered, symptoms of hyperthyroidism are reported as well as confusion, disorientation, cerebral embolism, seizure, shock, coma, and death. The symptoms of overdose can be presented immediately or several days after overdose ingestion. In an overdose state, reduce the dose of liothyronine and do supportive treatment.Label'],
             'repaglinide': ['LD50 >1 g/kg (rat) (W. Grell)',
              'LD50 >1 g/kg (rat) (W. Grell)'],
             'hyoscyamine': ['Patients experiencing an overdose may present with headache, nausea, vomiting, dizziness, dry mouth, difficulty in swallowing, dilated pupils, blurred vision, urinary retention, hot dry and flushed skin, tachycardia, hypertension, hypotension, respiratory depression, CNS stimulation, fever, ataxia, excitation, lethargy, stupor, coma, and paralysis.7,8 Patients should be treated with symptomatic and supportive therapy which may include emesis, gastric lavage, activated charcoal, artificial respiration, or intravenous physostigmine.2,7,8 Dialysis is expected to remove hyoscyamine sulfate from circulation.7,8',
              'Patients experiencing an overdose may present with headache, nausea, vomiting, dizziness, dry mouth, difficulty in swallowing, dilated pupils, blurred vision, urinary retention, hot dry and flushed skin, tachycardia, hypertension, hypotension, respiratory depression, CNS stimulation, fever, ataxia, excitation, lethargy, stupor, coma, and paralysis.7,8 Patients should be treated with symptomatic and supportive therapy which may include emesis, gastric lavage, activated charcoal, artificial respiration, or intravenous physostigmine.2,7,8 Dialysis is expected to remove hyoscyamine sulfate from circulation.7,8'],
             'hyoscyamine sub': ['Patients experiencing an overdose may present with headache, nausea, vomiting, dizziness, dry mouth, difficulty in swallowing, dilated pupils, blurred vision, urinary retention, hot dry and flushed skin, tachycardia, hypertension, hypotension, respiratory depression, CNS stimulation, fever, ataxia, excitation, lethargy, stupor, coma, and paralysis.7,8 Patients should be treated with symptomatic and supportive therapy which may include emesis, gastric lavage, activated charcoal, artificial respiration, or intravenous physostigmine.2,7,8 Dialysis is expected to remove hyoscyamine sulfate from circulation.7,8',
              'Patients experiencing an overdose may present with headache, nausea, vomiting, dizziness, dry mouth, difficulty in swallowing, dilated pupils, blurred vision, urinary retention, hot dry and flushed skin, tachycardia, hypertension, hypotension, respiratory depression, CNS stimulation, fever, ataxia, excitation, lethargy, stupor, coma, and paralysis.7,8 Patients should be treated with symptomatic and supportive therapy which may include emesis, gastric lavage, activated charcoal, artificial respiration, or intravenous physostigmine.2,7,8 Dialysis is expected to remove hyoscyamine sulfate from circulation.7,8'],
             'hyoscyamine er': ['Patients experiencing an overdose may present with headache, nausea, vomiting, dizziness, dry mouth, difficulty in swallowing, dilated pupils, blurred vision, urinary retention, hot dry and flushed skin, tachycardia, hypertension, hypotension, respiratory depression, CNS stimulation, fever, ataxia, excitation, lethargy, stupor, coma, and paralysis.7,8 Patients should be treated with symptomatic and supportive therapy which may include emesis, gastric lavage, activated charcoal, artificial respiration, or intravenous physostigmine.2,7,8 Dialysis is expected to remove hyoscyamine sulfate from circulation.7,8',
              'Patients experiencing an overdose may present with headache, nausea, vomiting, dizziness, dry mouth, difficulty in swallowing, dilated pupils, blurred vision, urinary retention, hot dry and flushed skin, tachycardia, hypertension, hypotension, respiratory depression, CNS stimulation, fever, ataxia, excitation, lethargy, stupor, coma, and paralysis.7,8 Patients should be treated with symptomatic and supportive therapy which may include emesis, gastric lavage, activated charcoal, artificial respiration, or intravenous physostigmine.2,7,8 Dialysis is expected to remove hyoscyamine sulfate from circulation.7,8'],
             'amiloride hcl': ['No data are available in regard to overdosage in humans. The oral LD50 of amiloride hydrochloride (calculated as the base) is 56 mg/kg in mice and 36 to 85 mg/kg in rats, depending on the strain. The most likely signs and symptoms to be expected with overdosage are dehydration and electrolyte imbalance.',
              'No data are available in regard to overdosage in humans. The oral LD50 of amiloride hydrochloride (calculated as the base) is 56 mg/kg in mice and 36 to 85 mg/kg in rats, depending on the strain. The most likely signs and symptoms to be expected with overdosage are dehydration and electrolyte imbalance.'],
             'lorazepam': ['The LD50 observed by oral administration in a mouse is of 1850 mg/kg.16 When an overdose administration is registered, signs of CNS and respiratory depression are rapidly observed. An overdose stage can result in profound sedation, deep respiratory depression, coma, and death.12 When overdose is observed, it is recommended to administer emergency symptomatic medical support with attention to produce an increase in lorazepam elimination.13',
              'The LD50 observed by oral administration in a mouse is of 1850 mg/kg.16 When an overdose administration is registered, signs of CNS and respiratory depression are rapidly observed. An overdose stage can result in profound sedation, deep respiratory depression, coma, and death.12 When overdose is observed, it is recommended to administer emergency symptomatic medical support with attention to produce an increase in lorazepam elimination.13'],
             'verapamil': ["Verapamil's reported oral TDLo is 14.4 mg/kg in women and 3.429 mg/kg in men.21 The oral LD50 is 150 mg/kg in rats and 163 mg/kg in mice.21",
              "Verapamil's reported oral TDLo is 14.4 mg/kg in women and 3.429 mg/kg in men.21 The oral LD50 is 150 mg/kg in rats and 163 mg/kg in mice.21"],
             'hctz s': ['The oral LD50 of hydrochlorothiazide is >10g/kg in mice and rats.10,11',
              'The oral LD50 of hydrochlorothiazide is >10g/kg in mice and rats.10,11'],
             'triamterenhctz s': ['', ''],
             'loxapine': ['LD50=65 mg/kg (Orally in mice)',
              'LD50=65 mg/kg (Orally in mice)'],
             'pentazocine and naloxone': ['Not Available', 'Not Available'],
             'diclofenac sodium and misoprostol': ['Symptoms of overdose include lethargy, drowsiness, nausea, vomiting, and epigastric pain, and gastrointestinal bleeding.Label Hypertension, acute renal failure, respiratory depression and coma occur rarely. In case of overdose, provide supportive care and consider inducing emesis and administering activated charcoal if overdose occurred less than 4 hours prior.',
              'Symptoms of overdose include lethargy, drowsiness, nausea, vomiting, and epigastric pain, and gastrointestinal bleeding.Label Hypertension, acute renal failure, respiratory depression and coma occur rarely. In case of overdose, provide supportive care and consider inducing emesis and administering activated charcoal if overdose occurred less than 4 hours prior.'],
             'doxycycline monohydrate': ['Oral LD50 is 2000 mg/kg in rats, 1870 mg/kg in mice, and 500 mg/kg in dog.14',
              'Oral LD50 is 2000 mg/kg in rats, 1870 mg/kg in mice, and 500 mg/kg in dog.14'],
             'estropipate': ['Not Available', 'Not Available'],
             'metoprol tar': ['Oral administration of metoprolol to rats presents an LD50 in the range of 3090 to 4670 mg/kg. Cases of overdose have reported bradycardia, hypotension, bronchospasm, and cardiac failure. In the case of an overdose, gastric lavage is recommended followed by specific treatment according to symptoms.Label',
              'Oral administration of metoprolol to rats presents an LD50 in the range of 3090 to 4670 mg/kg. Cases of overdose have reported bradycardia, hypotension, bronchospasm, and cardiac failure. In the case of an overdose, gastric lavage is recommended followed by specific treatment according to symptoms.Label'],
             'propafenone hydrochloride': ['Symptoms of propafenone overdose (usually most severe within the first 3 hours) may include convulsions (rarely), heartbeat irregularities, low blood pressure, and sleepiness.',
              'Symptoms of propafenone overdose (usually most severe within the first 3 hours) may include convulsions (rarely), heartbeat irregularities, low blood pressure, and sleepiness.'],
             'labetalol': ['The oral LD50 in mice is 600mg/kg and in rats is >2g/kg.7,8 The intravenous LD50 in mice and rats is 50-60mg/kg.7,8',
              'The oral LD50 in mice is 600mg/kg and in rats is >2g/kg.7,8 The intravenous LD50 in mice and rats is 50-60mg/kg.7,8'],
             'diclofenac er': ['Symptoms of overdose include lethargy, drowsiness, nausea, vomiting, and epigastric pain, and gastrointestinal bleeding.Label Hypertension, acute renal failure, respiratory depression and coma occur rarely. In case of overdose, provide supportive care and consider inducing emesis and administering activated charcoal if overdose occurred less than 4 hours prior.',
              'Symptoms of overdose include lethargy, drowsiness, nausea, vomiting, and epigastric pain, and gastrointestinal bleeding.Label Hypertension, acute renal failure, respiratory depression and coma occur rarely. In case of overdose, provide supportive care and consider inducing emesis and administering activated charcoal if overdose occurred less than 4 hours prior.'],
             'hydroxychlor': ['Patients experiencing an overdose may present with headache, drowsiness, visual disturbances, cardiovascular collapse, convulsions, hypokalemia, rhythm and conduction disorders including QT prolongation, torsades de pointes, ventricular tachycardia, and ventricular fibrillation.13 This may progress to sudden respiratory and cardiac arrest.13 Overdose should be treated with immediate gastric lavage and activated charcoal at a dose of at least 5 times the hydroxychloroquine dose within 30 minutes.2,13 Parenteral diazepam may be given to treat cardiotoxicity, transfusion may reduce serum concentrations of drug, patients should be monitored for at least 6 hours, fluids should be given, and ammonium chloride should be given to acidify urine and promote urinary excretion.13 Patients may also be given epinephrine.11',
              'Patients experiencing an overdose may present with headache, drowsiness, visual disturbances, cardiovascular collapse, convulsions, hypokalemia, rhythm and conduction disorders including QT prolongation, torsades de pointes, ventricular tachycardia, and ventricular fibrillation.13 This may progress to sudden respiratory and cardiac arrest.13 Overdose should be treated with immediate gastric lavage and activated charcoal at a dose of at least 5 times the hydroxychloroquine dose within 30 minutes.2,13 Parenteral diazepam may be given to treat cardiotoxicity, transfusion may reduce serum concentrations of drug, patients should be monitored for at least 6 hours, fluids should be given, and ammonium chloride should be given to acidify urine and promote urinary excretion.13 Patients may also be given epinephrine.11'],
             'sucralfate gram': ['Overdose', 'Overdose'],
             'diethylpropion hydrochloride controlledrelease': ['The reported oral LD50 for mice is 600 mg/kg, for rats is 250 mg/kg and for dogs is 225 mg/kg. Manifestation of acute overdosage include restlessness, tremor, hyperreflexia, rapid respiration, confusion, assaultiveness, hallucinations, and panic states.',
              'The reported oral LD50 for mice is 600 mg/kg, for rats is 250 mg/kg and for dogs is 225 mg/kg. Manifestation of acute overdosage include restlessness, tremor, hyperreflexia, rapid respiration, confusion, assaultiveness, hallucinations, and panic states.'],
             'diethylprop': ['The reported oral LD50 for mice is 600 mg/kg, for rats is 250 mg/kg and for dogs is 225 mg/kg. Manifestation of acute overdosage include restlessness, tremor, hyperreflexia, rapid respiration, confusion, assaultiveness, hallucinations, and panic states.',
              'The reported oral LD50 for mice is 600 mg/kg, for rats is 250 mg/kg and for dogs is 225 mg/kg. Manifestation of acute overdosage include restlessness, tremor, hyperreflexia, rapid respiration, confusion, assaultiveness, hallucinations, and panic states.'],
             'methylprednisolone': ['The oral LD50 in rats is >4g/kg.14 The intraperitoneal LD50 in mice is 2292mg/kg and in rats is 100mg/kg.14',
              'The oral LD50 in rats is >4g/kg.14 The intraperitoneal LD50 in mice is 2292mg/kg and in rats is 100mg/kg.14'],
             'dicyclomine hcl': ['Patients experiencing an overdose may present with headache, nausea, vomiting, blurred vision, dilated pupils, dizziness, dry mouth, difficulty swallowing, CNS stimulation, as well as hot, dry skin.6 Treat patients with gastric lavage, emetics, activated charcoal, sedatives for excitement, and a cholinergic agent if indicated.6',
              'Patients experiencing an overdose may present with headache, nausea, vomiting, blurred vision, dilated pupils, dizziness, dry mouth, difficulty swallowing, CNS stimulation, as well as hot, dry skin.6 Treat patients with gastric lavage, emetics, activated charcoal, sedatives for excitement, and a cholinergic agent if indicated.6'],
             'sulfasalazine s': ['Not Available', 'Not Available'],
             'hydroxyzine pam': ['The oral LD50 is 840 mg/kg in rats and 400 mg/kg in mice.16 ',
              'The oral LD50 is 840 mg/kg in rats and 400 mg/kg in mice.16 '],
             'oncedaily lisinopril': ['', ''],
             'desmopressin': ['Intravenous TDLo in humans is reported to be 0.3 µg/kg/10M MSDS. Desmopressin is associated with hyponatremia in case of overdose, which may require temporary or permanent discontinuation of the therapy depending on severity. The effects of hyponatremia include seizure, altered mental status (confusion, drowsiness or continuing headache), cardiac arrhythmias and worsening edema. Other signs of overdose may include oliguria and rapid weight gain due to fluid retention Label. In case of overdose, reduce the dose or frequency of drug administration, or discontinue use if appropriate. Assessment of serum sodium and initiation of appropriate medical treatment is recommended. ',
              'Intravenous TDLo in humans is reported to be 0.3 µg/kg/10M MSDS. Desmopressin is associated with hyponatremia in case of overdose, which may require temporary or permanent discontinuation of the therapy depending on severity. The effects of hyponatremia include seizure, altered mental status (confusion, drowsiness or continuing headache), cardiac arrhythmias and worsening edema. Other signs of overdose may include oliguria and rapid weight gain due to fluid retention Label. In case of overdose, reduce the dose or frequency of drug administration, or discontinue use if appropriate. Assessment of serum sodium and initiation of appropriate medical treatment is recommended. '],
             'metoclopram': ['The rat oral LD50 of metoclopramide is 750 mg/kg.20',
              'The rat oral LD50 of metoclopramide is 750 mg/kg.20'],
             'mirtazapine solu': ['LD50', 'LD50'],
             'tetracycline hydrochloride': ['LD50=808mg/kg (orally in mice)',
              'LD50=808mg/kg (orally in mice)'],
             'valsartan and hydrochlorothiazide': ['Approximate LD50 >2000 mg/kg (Gavage, rat) 24',
              'Approximate LD50 >2000 mg/kg (Gavage, rat) 24'],
             'lansoprazole acid reducer': ["The most commonly reported adverse events occurring more frequently in lansoprazole treated patients compared to placebo include abdominal pain, constipation, diarrhea, and nausea.Label There is a case report of toxic epidermal necrolysis (TEN), which is a rare but very serious cutaneous reaction, caused by lansoprazole.8 The previously healthy patient presented with symptoms of TEN 15 days after starting lansoprazole to manage peptic disease.8 Although the use of PPI's is rarely associated with TEN, causation should be considered if a patient presents with TEN shortly after newly commencing a PPI.8 ",
              "The most commonly reported adverse events occurring more frequently in lansoprazole treated patients compared to placebo include abdominal pain, constipation, diarrhea, and nausea.Label There is a case report of toxic epidermal necrolysis (TEN), which is a rare but very serious cutaneous reaction, caused by lansoprazole.8 The previously healthy patient presented with symptoms of TEN 15 days after starting lansoprazole to manage peptic disease.8 Although the use of PPI's is rarely associated with TEN, causation should be considered if a patient presents with TEN shortly after newly commencing a PPI.8 "],
             'chlorzoxazone': ['Oral, mouse: LD50 = 440 mg/kg; Oral, rat: LD50 = 763 mg/kg; Symptoms of overdose include diarrhea, dizziness, drowsiness, headache, light-headedness, nausea, and vomiting.',
              'Oral, mouse: LD50 = 440 mg/kg; Oral, rat: LD50 = 763 mg/kg; Symptoms of overdose include diarrhea, dizziness, drowsiness, headache, light-headedness, nausea, and vomiting.'],
             'butalbital acetaminophen and caffeine': ['Reported oral TDLO (woman) is 400 mg/kg and subcutaneous LD50 in rat is 160 mg/kg.MSDS The lowest acute dose of butalbital alone associated with death in adults is 2.0 g.1 Symptoms of acute barbiturate poisoning include drowsiness, confusion, coma, respiratory depression, hypotension, and shock. Due to the CNS depressant effects, an overdose of barbiturates may lead to death.10 Barbiturates are also associated with withdrawal reactions, which may lead to death if severe.1',
              'Reported oral TDLO (woman) is 400 mg/kg and subcutaneous LD50 in rat is 160 mg/kg.MSDS The lowest acute dose of butalbital alone associated with death in adults is 2.0 g.1 Symptoms of acute barbiturate poisoning include drowsiness, confusion, coma, respiratory depression, hypotension, and shock. Due to the CNS depressant effects, an overdose of barbiturates may lead to death.10 Barbiturates are also associated with withdrawal reactions, which may lead to death if severe.1'],
             'butalbital acetaminophen caffeine and codeine phospahte': ['Reported oral TDLO (woman) is 400 mg/kg and subcutaneous LD50 in rat is 160 mg/kg.MSDS The lowest acute dose of butalbital alone associated with death in adults is 2.0 g.1 Symptoms of acute barbiturate poisoning include drowsiness, confusion, coma, respiratory depression, hypotension, and shock. Due to the CNS depressant effects, an overdose of barbiturates may lead to death.10 Barbiturates are also associated with withdrawal reactions, which may lead to death if severe.1',
              'Reported oral TDLO (woman) is 400 mg/kg and subcutaneous LD50 in rat is 160 mg/kg.MSDS The lowest acute dose of butalbital alone associated with death in adults is 2.0 g.1 Symptoms of acute barbiturate poisoning include drowsiness, confusion, coma, respiratory depression, hypotension, and shock. Due to the CNS depressant effects, an overdose of barbiturates may lead to death.10 Barbiturates are also associated with withdrawal reactions, which may lead to death if severe.1'],
             'tizanidine hydrochloride': ['LD50 information ',
              'LD50 information '],
             'verapamil hcl sustainedrelease': ["Verapamil's reported oral TDLo is 14.4 mg/kg in women and 3.429 mg/kg in men.21 The oral LD50 is 150 mg/kg in rats and 163 mg/kg in mice.21",
              "Verapamil's reported oral TDLo is 14.4 mg/kg in women and 3.429 mg/kg in men.21 The oral LD50 is 150 mg/kg in rats and 163 mg/kg in mice.21"],
             'verapamil s sr': ["Verapamil's reported oral TDLo is 14.4 mg/kg in women and 3.429 mg/kg in men.21 The oral LD50 is 150 mg/kg in rats and 163 mg/kg in mice.21",
              "Verapamil's reported oral TDLo is 14.4 mg/kg in women and 3.429 mg/kg in men.21 The oral LD50 is 150 mg/kg in rats and 163 mg/kg in mice.21"],
             'clindamycin': ['The oral LD50 in mice and rats is 2540 mg/kg and 2190 mg/kg, respectively.19',
              'The oral LD50 in mice and rats is 2540 mg/kg and 2190 mg/kg, respectively.19'],
             'nizatidine': ['Oral, rat LD50: 301 mg/kg. Symptoms of overdose include cholinergic-type effects including lacrimation, salivation, emesis, miosis, and diarrhea.',
              'Oral, rat LD50: 301 mg/kg. Symptoms of overdose include cholinergic-type effects including lacrimation, salivation, emesis, miosis, and diarrhea.'],
             'minocycline': ['The oral LD50 in rats is 2380mg/kg and in mice is 3600mg/kg.17 The intraperitoneal LD50 in rats is 331mg/kg and in mice is 299mg/kg.17 The subcutaneous LD50 in rats is 1700mg/kg and in mice is 2290mg/kg.17',
              'The oral LD50 in rats is 2380mg/kg and in mice is 3600mg/kg.17 The intraperitoneal LD50 in rats is 331mg/kg and in mice is 299mg/kg.17 The subcutaneous LD50 in rats is 1700mg/kg and in mice is 2290mg/kg.17'],
             'ursodiol s': ['Neither accidental nor intentional overdosing with ursodeoxycholic acid has been reported. Doses of ursodeoxycholic acid in the range of 16-20 mg/kg/day have been tolerated for 6-37 months without symptoms by 7 patients. The LD50 for ursodeoxycholic acid in rats is over 5000 mg/kg given over 7-10 days and over 7500 mg/kg for mice. The most likely manifestation of severe overdose with ursodeoxycholic acid would probably be diarrhea, which should be treated symptomatically.',
              'Neither accidental nor intentional overdosing with ursodeoxycholic acid has been reported. Doses of ursodeoxycholic acid in the range of 16-20 mg/kg/day have been tolerated for 6-37 months without symptoms by 7 patients. The LD50 for ursodeoxycholic acid in rats is over 5000 mg/kg given over 7-10 days and over 7500 mg/kg for mice. The most likely manifestation of severe overdose with ursodeoxycholic acid would probably be diarrhea, which should be treated symptomatically.'],
             'afedi cr': ['The oral LD50 in rats is 1022mg/kg and in mice is 202mg/kg.16',
              'The oral LD50 in rats is 1022mg/kg and in mice is 202mg/kg.16'],
             'afedi cr (nifedipine )': ['The oral LD50 in rats is 1022mg/kg and in mice is 202mg/kg.16',
              'The oral LD50 in rats is 1022mg/kg and in mice is 202mg/kg.16'],
             'rivastigmine tartrate': ['Not Available', 'Not Available'],
             'rivastigmine': ['Not Available', 'Not Available'],
             'butalbitalasacaff s': ['', ''],
             'butacetaminophencaffcodeine': ['', ''],
             'valacyclovir gm': ['LD50 Oral ', 'LD50 Oral '],
             'bupropion xl': ['Symptoms of overdose include seizures, hallucinations, loss of consciousness, tachycardia, and cardiac arrest.',
              'Symptoms of overdose include seizures, hallucinations, loss of consciousness, tachycardia, and cardiac arrest.'],
             'bupropion hydrochloride (xl)': ['Symptoms of overdose include seizures, hallucinations, loss of consciousness, tachycardia, and cardiac arrest.',
              'Symptoms of overdose include seizures, hallucinations, loss of consciousness, tachycardia, and cardiac arrest.'],
             'nateglinide': ['An overdose may result in an exaggerated glucose-lowering effect with the development of hypoglycemic symptoms.',
              'An overdose may result in an exaggerated glucose-lowering effect with the development of hypoglycemic symptoms.'],
             'hydroxyzine': ['The oral LD50 is 840 mg/kg in rats and 400 mg/kg in mice.16 ',
              'The oral LD50 is 840 mg/kg in rats and 400 mg/kg in mice.16 '],
             'modafinil': ['Not Available', 'Not Available'],
             'bupropion hcl (sr)': ['Symptoms of overdose include seizures, hallucinations, loss of consciousness, tachycardia, and cardiac arrest.',
              'Symptoms of overdose include seizures, hallucinations, loss of consciousness, tachycardia, and cardiac arrest.'],
             'butalbital aspirin caffeine and codeine phosphate': ['Reported oral TDLO (woman) is 400 mg/kg and subcutaneous LD50 in rat is 160 mg/kg.MSDS The lowest acute dose of butalbital alone associated with death in adults is 2.0 g.1 Symptoms of acute barbiturate poisoning include drowsiness, confusion, coma, respiratory depression, hypotension, and shock. Due to the CNS depressant effects, an overdose of barbiturates may lead to death.10 Barbiturates are also associated with withdrawal reactions, which may lead to death if severe.1',
              'Reported oral TDLO (woman) is 400 mg/kg and subcutaneous LD50 in rat is 160 mg/kg.MSDS The lowest acute dose of butalbital alone associated with death in adults is 2.0 g.1 Symptoms of acute barbiturate poisoning include drowsiness, confusion, coma, respiratory depression, hypotension, and shock. Due to the CNS depressant effects, an overdose of barbiturates may lead to death.10 Barbiturates are also associated with withdrawal reactions, which may lead to death if severe.1'],
             'vancomycin hydrochloride': ['The oral LD50 in mice is 5000 mg/kg. The median lethal intravenous dose is 319 mg/kg in rats and 400 mg/kg in mice. Label',
              'The oral LD50 in mice is 5000 mg/kg. The median lethal intravenous dose is 319 mg/kg in rats and 400 mg/kg in mice. Label'],
             'dronabinol': ['SYNDROS, a synthetic cannabinoid containing alcohol, may cause fetal harm. Avoid the use of SYNDROS in pregnant women. Although there is little published data on the use of synthetic cannabinoids during pregnancy, the use of cannabis (e.g., marijuana) and the use of alcohol during pregnancy have been associated with adverse fetal/neonatal outcomes (see Clinical Considerations). Cannabinoids have been found in the umbilical cord blood of pregnant women who smoke cannabis. In animal reproduction studies, no teratogenicity was reported in mice administered dronabinol (delta-9-THC) at up to 30 times the MRHD (maximum recommended human doses) and up to 5 times the MRHD for patients with AIDS and cancer, respectively. Similar findings were reported in pregnant rats administered dronabinol at up to 5 to 20 times the MRHD and 3 times the MRHD for patients with AIDS and cancer, respectively.\nDecreased maternal weight gain and the number of viable pups and increased fetal mortality and early resorptions were observed in both species at doses that induced maternal toxicity. In rats, maternal administration of dronabinol from pregnancy (implantation) through weaning was associated with maternal toxicity, including mortality of pups, and adverse developmental and 10 neurodevelopmental effects on the pups at 2 to 20 times the MRHD for patients with AIDS and less than and up to 3.3 times the MRHD for patients with cancer.8',
              'SYNDROS, a synthetic cannabinoid containing alcohol, may cause fetal harm. Avoid the use of SYNDROS in pregnant women. Although there is little published data on the use of synthetic cannabinoids during pregnancy, the use of cannabis (e.g., marijuana) and the use of alcohol during pregnancy have been associated with adverse fetal/neonatal outcomes (see Clinical Considerations). Cannabinoids have been found in the umbilical cord blood of pregnant women who smoke cannabis. In animal reproduction studies, no teratogenicity was reported in mice administered dronabinol (delta-9-THC) at up to 30 times the MRHD (maximum recommended human doses) and up to 5 times the MRHD for patients with AIDS and cancer, respectively. Similar findings were reported in pregnant rats administered dronabinol at up to 5 to 20 times the MRHD and 3 times the MRHD for patients with AIDS and cancer, respectively.\nDecreased maternal weight gain and the number of viable pups and increased fetal mortality and early resorptions were observed in both species at doses that induced maternal toxicity. In rats, maternal administration of dronabinol from pregnancy (implantation) through weaning was associated with maternal toxicity, including mortality of pups, and adverse developmental and 10 neurodevelopmental effects on the pups at 2 to 20 times the MRHD for patients with AIDS and less than and up to 3.3 times the MRHD for patients with cancer.8'],
             'oncedaily trospium chloride': ['', ''],
             'nitrofurantoin (macrocrystals)': ['Symptoms of overdose include vomiting.8,9,10 In case of overdose, induce vomiting if it has not already occurred and increase fluid intake to promote urination.8,9,10 In extreme cases, nitrofurantoin can be removed from circulation by dialysis.8,9,10',
              'Symptoms of overdose include vomiting.8,9,10 In case of overdose, induce vomiting if it has not already occurred and increase fluid intake to promote urination.8,9,10 In extreme cases, nitrofurantoin can be removed from circulation by dialysis.8,9,10'],
             'paliperidone': ['The possibility of obtundation, seizures, or dystonic reaction of the head and neck following overdose may create a risk of aspiration with induced emesis. ',
              'The possibility of obtundation, seizures, or dystonic reaction of the head and neck following overdose may create a risk of aspiration with induced emesis. '],
             'oncedaily dosing levetiracetam': ['', ''],
             'ibandronate sodium let': ['Patients experiencing an overdose may present with hypocalcemia, hypophosphatemia, upset stomach, dyspepsia, esophagitis, and uclers.11,12 Oral overdose can be managed by giving patients milk or antacids to bind excess unabsorbed ibandronate.11 Overdoses can be managed by providing intravenous electrolytes and dialysis is not expected to remove excess drug from serum.11,12',
              'Patients experiencing an overdose may present with hypocalcemia, hypophosphatemia, upset stomach, dyspepsia, esophagitis, and uclers.11,12 Oral overdose can be managed by giving patients milk or antacids to bind excess unabsorbed ibandronate.11 Overdoses can be managed by providing intravenous electrolytes and dialysis is not expected to remove excess drug from serum.11,12'],
             'atorvastatin': ['The reported LD50 of oral atorvastatin in mice is higher than 5000 mg/kg.MSDS In cases of overdose with atorvastatin, there is reported symptoms of complicated breathing, jaundice, liver damage, dark urine, muscle pain, and seizures.38 In case of overdose, symptomatic treatment is recommended and due to the high plasma protein binding, hemodialysis is not expected to generate significant improvement.Label',
              'The reported LD50 of oral atorvastatin in mice is higher than 5000 mg/kg.MSDS In cases of overdose with atorvastatin, there is reported symptoms of complicated breathing, jaundice, liver damage, dark urine, muscle pain, and seizures.38 In case of overdose, symptomatic treatment is recommended and due to the high plasma protein binding, hemodialysis is not expected to generate significant improvement.Label'],
             'memantine hcl': ['LD50', 'LD50'],
             'memantine hcl titration pak': ['LD50', 'LD50'],
             'chlorzoxazone s': ['Oral, mouse: LD50 = 440 mg/kg; Oral, rat: LD50 = 763 mg/kg; Symptoms of overdose include diarrhea, dizziness, drowsiness, headache, light-headedness, nausea, and vomiting.',
              'Oral, mouse: LD50 = 440 mg/kg; Oral, rat: LD50 = 763 mg/kg; Symptoms of overdose include diarrhea, dizziness, drowsiness, headache, light-headedness, nausea, and vomiting.'],
             'glipmetform': ['', ''],
             'valproic acd': ['LD50 Values', 'LD50 Values'],
             'perphenazine': ['s of overdose include stupor or coma, and children may have convulsive seizures. Signs of arousal may not occur for 48 hours. Oral LD50=318 mg/kg',
              '50=64 mg/kg'],
             'darifenacin': ['Overdosage can potentially result in severe central anticholinergic effects.',
              'Overdosage can potentially result in severe central anticholinergic effects.'],
             'amantadine hydrochloride': ['Deaths have been reported from overdose with amantadine. The lowest reported acute lethal dose was 2 grams. Drug overdose has resulted in cardiac, respiratory, renal or central nervous system toxicity. Cardiac dysfunction includes arrhythmia, tachycardia and hypertension. Pulmonary edema and respiratory distress (including ARDS) have been reported. Renal dysfunction including increased BUN, decreased creatinine clearance and renal insufficiency can occur. Central nervous system effects that have been reported include insomnia, anxiety, aggressive behavior, hypertonia, hyperkinesia, tremor, confusion, disorientation, depersonalization, fear, delirium, hallucination, psychotic reactions, lethargy, somnolence and coma. Seizures may be exacerbated in patients with prior history of seizure disorders. Hyperthermia has also been observed in cases where a drug overdose has occurred.',
              'Deaths have been reported from overdose with amantadine. The lowest reported acute lethal dose was 2 grams. Drug overdose has resulted in cardiac, respiratory, renal or central nervous system toxicity. Cardiac dysfunction includes arrhythmia, tachycardia and hypertension. Pulmonary edema and respiratory distress (including ARDS) have been reported. Renal dysfunction including increased BUN, decreased creatinine clearance and renal insufficiency can occur. Central nervous system effects that have been reported include insomnia, anxiety, aggressive behavior, hypertonia, hyperkinesia, tremor, confusion, disorientation, depersonalization, fear, delirium, hallucination, psychotic reactions, lethargy, somnolence and coma. Seizures may be exacerbated in patients with prior history of seizure disorders. Hyperthermia has also been observed in cases where a drug overdose has occurred.'],
             'promethazine': ['The intraperitoneal LD50 in rats is 170mg/kg and in mice is 160mg/kg.14 The subcutaneous LD50 in rats is 400mg/kg and in mice is 240mg/kg.14 The oral LD50 in mice is 255mg/kg.14',
              'The intraperitoneal LD50 in rats is 170mg/kg and in mice is 160mg/kg.14 The subcutaneous LD50 in rats is 400mg/kg and in mice is 240mg/kg.14 The oral LD50 in mice is 255mg/kg.14'],
             'primidone': ['The oral LD50 in rats is 1500mg/kg and in mice is 280mg/kg.16 The intraperitoneal LD50 in rats was 240mg/kg and in mice was 332mg/kg.16',
              'The oral LD50 in rats is 1500mg/kg and in mice is 280mg/kg.16 The intraperitoneal LD50 in rats was 240mg/kg and in mice was 332mg/kg.16'],
             'colprobenecid s': ['', ''],
             'trihexyphen': ['Symptoms of overdose include mydriasis, dryness of mucous membranes, red face, atonic states of bowels and bladder, and hyperthermia in high doses. Trihexyphenidyl causes agitation, confusion, and hallucinations due to its effects on the central nervous system. Untreated overdose may result in death, especially in children. Respiratory depression and cardiac arrest may be seen as premortal signs.',
              'Symptoms of overdose include mydriasis, dryness of mucous membranes, red face, atonic states of bowels and bladder, and hyperthermia in high doses. Trihexyphenidyl causes agitation, confusion, and hallucinations due to its effects on the central nervous system. Untreated overdose may result in death, especially in children. Respiratory depression and cardiac arrest may be seen as premortal signs.'],
             'trihexyphenidyl hydrochloride': ['Symptoms of overdose include mydriasis, dryness of mucous membranes, red face, atonic states of bowels and bladder, and hyperthermia in high doses. Trihexyphenidyl causes agitation, confusion, and hallucinations due to its effects on the central nervous system. Untreated overdose may result in death, especially in children. Respiratory depression and cardiac arrest may be seen as premortal signs.',
              'Symptoms of overdose include mydriasis, dryness of mucous membranes, red face, atonic states of bowels and bladder, and hyperthermia in high doses. Trihexyphenidyl causes agitation, confusion, and hallucinations due to its effects on the central nervous system. Untreated overdose may result in death, especially in children. Respiratory depression and cardiac arrest may be seen as premortal signs.'],
             'probenecid s': ['Not Available', 'Not Available'],
             'methocarbam': ['Overdose of methocarbamol may be associated with alcohol and other central nervous system depressants.Label Patients may experience nausea, drowsiness, blurred vision, hypotension, seizures, and coma.Label Treatment of overdose is generally through airway maintenance, monitoring urinary output, vital signs, and giving fluid intravenously if necessary.Label',
              'Overdose of methocarbamol may be associated with alcohol and other central nervous system depressants.Label Patients may experience nausea, drowsiness, blurred vision, hypotension, seizures, and coma.Label Treatment of overdose is generally through airway maintenance, monitoring urinary output, vital signs, and giving fluid intravenously if necessary.Label'],
             'methocarbamol s': ['Overdose of methocarbamol may be associated with alcohol and other central nervous system depressants.Label Patients may experience nausea, drowsiness, blurred vision, hypotension, seizures, and coma.Label Treatment of overdose is generally through airway maintenance, monitoring urinary output, vital signs, and giving fluid intravenously if necessary.Label',
              'Overdose of methocarbamol may be associated with alcohol and other central nervous system depressants.Label Patients may experience nausea, drowsiness, blurred vision, hypotension, seizures, and coma.Label Treatment of overdose is generally through airway maintenance, monitoring urinary output, vital signs, and giving fluid intravenously if necessary.Label'],
             'carisoprodol': ['LD50 values', 'LD50 values'],
             'doxycycl hyc': ['Oral LD50 is 2000 mg/kg in rats, 1870 mg/kg in mice, and 500 mg/kg in dog.14',
              'Oral LD50 is 2000 mg/kg in rats, 1870 mg/kg in mice, and 500 mg/kg in dog.14'],
             'allopurinol s': ['Oral TDLO (rat): 10 mg/kg; Oral LD50 (mouse): 78 mg/kg; Oral TDLO (mouse): 100 mg/kg 14 ',
              'Oral TDLO (rat): 10 mg/kg; Oral LD50 (mouse): 78 mg/kg; Oral TDLO (mouse): 100 mg/kg 14 '],
             'propranolol': ['Symptoms of overdose include hypotension, hypoglycemic seizure, restlessness, euphoria, insomnia.8 Patients with asthma may develop bronchospasm.8 In case of overdose, monitor vital signs, mental status, and blood glucose.8 Treat hypotension with intravenous fluids, bradycardia with atropine, and isoproterenol and aminophylline for bronchospasm.8 If patients do not respond to intravenous fluids, follow up with glucagon 50-150µg/kg intravenously, then 1-5mg/hour, followed by catecholamines.8,9,10 Dialysis will not be useful as propranolol is highly protein bound.8,9,10',
              'Symptoms of overdose include hypotension, hypoglycemic seizure, restlessness, euphoria, insomnia.8 Patients with asthma may develop bronchospasm.8 In case of overdose, monitor vital signs, mental status, and blood glucose.8 Treat hypotension with intravenous fluids, bradycardia with atropine, and isoproterenol and aminophylline for bronchospasm.8 If patients do not respond to intravenous fluids, follow up with glucagon 50-150µg/kg intravenously, then 1-5mg/hour, followed by catecholamines.8,9,10 Dialysis will not be useful as propranolol is highly protein bound.8,9,10'],
             'trimethoprim s': ['The oral LD50 in mice and rats is 2764 mg/kg and >5300 mg/kg, respectively.18',
              'The oral LD50 in mice and rats is 2764 mg/kg and >5300 mg/kg, respectively.18'],
             'tranylcypromine sulfate': ['In overdosage, some patients exhibit insomnia, restlessness and anxiety, progressing in severe cases to agitation, mental confusion and incoherence. Hypotension, dizziness, weakness and drowsiness may occur, progressing in severe cases to extreme dizziness and shock. A few patients have displayed hypertension with severe headache and other symptoms. Rare instances have been reported in which hypertension was accompanied by twitching or myoclonic fibrillation of skeletal muscles with hyperpyrexia, sometimes progressing to generalized rigidity and coma.',
              'In overdosage, some patients exhibit insomnia, restlessness and anxiety, progressing in severe cases to agitation, mental confusion and incoherence. Hypotension, dizziness, weakness and drowsiness may occur, progressing in severe cases to extreme dizziness and shock. A few patients have displayed hypertension with severe headache and other symptoms. Rare instances have been reported in which hypertension was accompanied by twitching or myoclonic fibrillation of skeletal muscles with hyperpyrexia, sometimes progressing to generalized rigidity and coma.'],
             'minoxidil': ['50 in rats has ranged from 1321-3492 mg/k',
              'e, 2456-2648 mg/kg. Side effects include cardiovascular effects associated with hypotension such as sudden weight gain, rapid heart beat, faintness or dizziness'],
             'cyclobenzapr': ['The oral LD50 of cyclobenzaprine in mice and rats is 338 mg/kg and 425 mg/kg, respectively. Signs of overdose may develop rapidly after ingestion and commonly include significant drowsiness and tachycardia, with less common manifestations including tremor, agitation, ataxia, GI upset, and other CNS effects such as confusion and hallucinations. Potentially critical manifestations, though rare, include cardiac arrest or dysrhythmias, severe hypotension, seizures, and neuroleptic malignant syndrome.15,16',
              'The oral LD50 of cyclobenzaprine in mice and rats is 338 mg/kg and 425 mg/kg, respectively. Signs of overdose may develop rapidly after ingestion and commonly include significant drowsiness and tachycardia, with less common manifestations including tremor, agitation, ataxia, GI upset, and other CNS effects such as confusion and hallucinations. Potentially critical manifestations, though rare, include cardiac arrest or dysrhythmias, severe hypotension, seizures, and neuroleptic malignant syndrome.15,16'],
             'sulindac': ['Acute oral toxicity (LD50) in rats is 264 mg/kg. Cases of overdose have been reported and rarely, deaths have occurred. The following signs and symptoms may be observed following overdose: stupor, coma, diminished urine output and hypotension.',
              'Acute oral toxicity (LD50) in rats is 264 mg/kg. Cases of overdose have been reported and rarely, deaths have occurred. The following signs and symptoms may be observed following overdose: stupor, coma, diminished urine output and hypotension.'],
             'atenololchlortahal': ['', ''],
             'atenolol w chlorth': ['LD50 Values', 'LD50 Values'],
             'nortriptyline s': ['The oral LD50 of nortriptyline in the rat is 405 mg/kg.22Symptoms of overdose with nortriptyline include cardiac arrhythmias, severe hypotension, shock, congestive heart failure, pulmonary edema, convulsions, coma, and CNS depression.18,20 Changes in the electrocardiogram, particularly in QRS segment, may be indicative of tricyclic antidepressant toxicity.13 ',
              'The oral LD50 of nortriptyline in the rat is 405 mg/kg.22Symptoms of overdose with nortriptyline include cardiac arrhythmias, severe hypotension, shock, congestive heart failure, pulmonary edema, convulsions, coma, and CNS depression.18,20 Changes in the electrocardiogram, particularly in QRS segment, may be indicative of tricyclic antidepressant toxicity.13 '],
             'aggrenox': ['Hypotension, if it occurs, is likely to be of short duration, but a vasopressor drug may be used if necessary. The oral LD50 in rats is greater than 6,000 mg/kg while in the dogs, the oral LD50 is approximately 400 mg/kg. LD50=8.4g/kg (orally in rat)',
              'Hypotension, if it occurs, is likely to be of short duration, but a vasopressor drug may be used if necessary. The oral LD50 in rats is greater than 6,000 mg/kg while in the dogs, the oral LD50 is approximately 400 mg/kg. LD50=8.4g/kg (orally in rat)'],
             'micardis': ['nous LD50 in rats is 150-200 mg/kg in males and 200 to 250 mg/kg in females. Acute oral toxicity is low: no deaths and no changes occurred in rats or dogs at 2000 mg/kg, the highest dose tested. Limited data are available with regard to overdosage in humans. The most likely manifestations of overdosage with telmisartan would be hypotension, dizziness and tachycardi',
              'ardia could occur from parasympathetic'],
             'micardis hct': ['nous LD50 in rats is 150-200 mg/kg in males and 200 to 250 mg/kg in females. Acute oral toxicity is low: no deaths and no changes occurred in rats or dogs at 2000 mg/kg, the highest dose tested. Limited data are available with regard to overdosage in humans. The most likely manifestations of overdosage with telmisartan would be hypotension, dizziness and tachycardi',
              'ardia could occur from parasympathetic'],
             'viramune': ['Symptoms of overdose include edema, erythema nodosum, fatigue, fever, headache, insomnia, nausea, pulmonaryinfiltrates, rash, vertigo, vomiting, and weight decrease. The most common adverse reaction is rash. ',
              'Symptoms of overdose include edema, erythema nodosum, fatigue, fever, headache, insomnia, nausea, pulmonaryinfiltrates, rash, vertigo, vomiting, and weight decrease. The most common adverse reaction is rash. '],
             'flomax': ['In the event of overdose, patients may experience hypotension and should lie down in a supine position to maintain blood pressure and heart rate.Label If further measures are required intravenous fluids should be considered.Label If further progression is required, vasopressors may be used and renal function should be monitored.Label Dialysis is unlikely to assist in treating overdose because tamsulosin is extensively protein bound.Label',
              'In the event of overdose, patients may experience hypotension and should lie down in a supine position to maintain blood pressure and heart rate.Label If further measures are required intravenous fluids should be considered.Label If further progression is required, vasopressors may be used and renal function should be monitored.Label Dialysis is unlikely to assist in treating overdose because tamsulosin is extensively protein bound.Label'],
             'tradjenta (linagliptin)': ['No dosage adjustment is necessary based on race, age, weight, sex, renal impairment, or hepatic impairment3.',
              'No dosage adjustment is necessary based on race, age, weight, sex, renal impairment, or hepatic impairment3.'],
             'jardiance': ["Experience with empagliflozin overdose is limited - employ standard symptomatic and supportive measures, as well as gastric decontamination when appropriate. The use of hemodialysis in empagliflozin overdose has not been studied but is unlikely to be of benefit given the drug's relatively high protein-binding.18",
              "Experience with empagliflozin overdose is limited - employ standard symptomatic and supportive measures, as well as gastric decontamination when appropriate. The use of hemodialysis in empagliflozin overdose has not been studied but is unlikely to be of benefit given the drug's relatively high protein-binding.18"],
             'glyxambi': ['No dosage adjustment is necessary based on race, age, weight, sex, renal impairment, or hepatic impairment3.',
              'No dosage adjustment is necessary based on race, age, weight, sex, renal impairment, or hepatic impairment3.'],
             'aspirin chewable': ['Lethal doses', 'Lethal doses'],
             'low dose aspirin (nsaid)': ['Not Available', 'Not Available'],
             'aspirin enteric coated': ['Lethal doses', 'Lethal doses'],
             'ferrous sulfate': ['Not Available', 'Not Available'],
             'senna lax': ['Senna causes increased amounts of apoptosis in the large intestine shortly after use due to upregulated p53 activity3. This is normally reversed after 18 hours however chronic use has been shown to be associated with p53 resistance and potential carcinogenicity leading to colon cancer3. The LD50 value in rats was 5000mg/kg. Subacute studies in rats receiving 20mg/kg and dogs receiving 500mg/kg did not produce signs of toxicity5. Tests for mutagenicity and reproductive toxicity do not indicate toxic effects5.',
              'Senna causes increased amounts of apoptosis in the large intestine shortly after use due to upregulated p53 activity3. This is normally reversed after 18 hours however chronic use has been shown to be associated with p53 resistance and potential carcinogenicity leading to colon cancer3. The LD50 value in rats was 5000mg/kg. Subacute studies in rats receiving 20mg/kg and dogs receiving 500mg/kg did not produce signs of toxicity5. Tests for mutagenicity and reproductive toxicity do not indicate toxic effects5.'],
             'amitriptyline s': ['Toxicity Data: Oral TDLO (child): 4167 μg/kg; Oral TDLO (man): 714 μg/kg/1D (intermittent); Oral TDLO (woman): 10 mg/kg 12. ',
              'Toxicity Data: Oral TDLO (child): 4167 μg/kg; Oral TDLO (man): 714 μg/kg/1D (intermittent); Oral TDLO (woman): 10 mg/kg 12. '],
             'amitriptylin': ['Toxicity Data: Oral TDLO (child): 4167 μg/kg; Oral TDLO (man): 714 μg/kg/1D (intermittent); Oral TDLO (woman): 10 mg/kg 12. ',
              'Toxicity Data: Oral TDLO (child): 4167 μg/kg; Oral TDLO (man): 714 μg/kg/1D (intermittent); Oral TDLO (woman): 10 mg/kg 12. '],
             'amitriptyline hcl': ['Toxicity Data: Oral TDLO (child): 4167 μg/kg; Oral TDLO (man): 714 μg/kg/1D (intermittent); Oral TDLO (woman): 10 mg/kg 12. ',
              'Toxicity Data: Oral TDLO (child): 4167 μg/kg; Oral TDLO (man): 714 μg/kg/1D (intermittent); Oral TDLO (woman): 10 mg/kg 12. '],
             'acetaminophen codeine': ['338 mg/kg', '1944 mg/kg'],
             'benztropine': ['The oral LD50 of benztropine is reported to be of 940 mg/kg in rats.MSDS In the presence of overdose with benztropine, it has been observed symptoms of circulatory collapse, cardiac arrest, respiratory depression, respiratory arrest, psychosis, shock, coma, seizure, ataxia, combativeness, anhidrosis, hyperthermia, fever, dysphagia, decreased bowel sounds and sluggish pupils.13',
              'The oral LD50 of benztropine is reported to be of 940 mg/kg in rats.MSDS In the presence of overdose with benztropine, it has been observed symptoms of circulatory collapse, cardiac arrest, respiratory depression, respiratory arrest, psychosis, shock, coma, seizure, ataxia, combativeness, anhidrosis, hyperthermia, fever, dysphagia, decreased bowel sounds and sluggish pupils.13'],
             'butalapap': ['', ''],
             'butalbital acetaminophen caffeine and codeine': ['Reported oral TDLO (woman) is 400 mg/kg and subcutaneous LD50 in rat is 160 mg/kg.MSDS The lowest acute dose of butalbital alone associated with death in adults is 2.0 g.1 Symptoms of acute barbiturate poisoning include drowsiness, confusion, coma, respiratory depression, hypotension, and shock. Due to the CNS depressant effects, an overdose of barbiturates may lead to death.10 Barbiturates are also associated with withdrawal reactions, which may lead to death if severe.1',
              'Reported oral TDLO (woman) is 400 mg/kg and subcutaneous LD50 in rat is 160 mg/kg.MSDS The lowest acute dose of butalbital alone associated with death in adults is 2.0 g.1 Symptoms of acute barbiturate poisoning include drowsiness, confusion, coma, respiratory depression, hypotension, and shock. Due to the CNS depressant effects, an overdose of barbiturates may lead to death.10 Barbiturates are also associated with withdrawal reactions, which may lead to death if severe.1'],
             'carisoprodol s': ['LD50 values', 'LD50 values'],
             'cyclobenzaprine': ['The oral LD50 of cyclobenzaprine in mice and rats is 338 mg/kg and 425 mg/kg, respectively. Signs of overdose may develop rapidly after ingestion and commonly include significant drowsiness and tachycardia, with less common manifestations including tremor, agitation, ataxia, GI upset, and other CNS effects such as confusion and hallucinations. Potentially critical manifestations, though rare, include cardiac arrest or dysrhythmias, severe hypotension, seizures, and neuroleptic malignant syndrome.15,16',
              'The oral LD50 of cyclobenzaprine in mice and rats is 338 mg/kg and 425 mg/kg, respectively. Signs of overdose may develop rapidly after ingestion and commonly include significant drowsiness and tachycardia, with less common manifestations including tremor, agitation, ataxia, GI upset, and other CNS effects such as confusion and hallucinations. Potentially critical manifestations, though rare, include cardiac arrest or dysrhythmias, severe hypotension, seizures, and neuroleptic malignant syndrome.15,16'],
             'glycopyrrolate': ['Patients presenting with an overdose typically present with flushing, hyperthermia, tachycardia, ileus, urinary retention, loss of ocular accommodation, light sensitivity, mydriasis, nausea, vomiting, dizziness, light headedness, and obstipation.12,13 Patients should be treated with symptomatic and supportive therapy, which may include the use of catheters for urinary retention, cardiovascular support, airway maintenance, ventilation, or neostigmine.12',
              'Patients presenting with an overdose typically present with flushing, hyperthermia, tachycardia, ileus, urinary retention, loss of ocular accommodation, light sensitivity, mydriasis, nausea, vomiting, dizziness, light headedness, and obstipation.12,13 Patients should be treated with symptomatic and supportive therapy, which may include the use of catheters for urinary retention, cardiovascular support, airway maintenance, ventilation, or neostigmine.12'],
             'diazepam s': ['The symptoms of diazepam overdose are mainly an intensification of the therapeutic effects (ataxia, drowsiness, dysarthria, sedation, muscle weakness, profound sleep, hypotension, bradycardia, nystagmus) or paradoxical excitation 9,10,6. In most cases only observation of vital functions is required 9,10,6.',
              'The symptoms of diazepam overdose are mainly an intensification of the therapeutic effects (ataxia, drowsiness, dysarthria, sedation, muscle weakness, profound sleep, hypotension, bradycardia, nystagmus) or paradoxical excitation 9,10,6. In most cases only observation of vital functions is required 9,10,6.'],
             'diphenhydram': ['e is expected to result in effects similar to the adverse effects that are ordinarily associated with the use of diphenhydramine, including drowsiness, hyperpyrexia, and anticholinergic effects, among others 10,11,12,13,18. Additional symptoms during overdose may include mydriasis, fever, flushing, agitation, tremor, dystonic reactions, hallucinations and ECG changes 13. Large overdose may cause rhabdomyolysis, convulsions, delirium, toxic psychosis, arrhythmias, coma and cardiovascular collapse 13. Moreover, with higher doses, and particularly in children, symptoms of CNS excitation including hallucinations and convulsions may appea',
              'assive doses, coma or cardiovascular collapse may follow 18'],
             'diphenhydramine hydrochloride': ['e is expected to result in effects similar to the adverse effects that are ordinarily associated with the use of diphenhydramine, including drowsiness, hyperpyrexia, and anticholinergic effects, among others 10,11,12,13,18. Additional symptoms during overdose may include mydriasis, fever, flushing, agitation, tremor, dystonic reactions, hallucinations and ECG changes 13. Large overdose may cause rhabdomyolysis, convulsions, delirium, toxic psychosis, arrhythmias, coma and cardiovascular collapse 13. Moreover, with higher doses, and particularly in children, symptoms of CNS excitation including hallucinations and convulsions may appea',
              'assive doses, coma or cardiovascular collapse may follow 18'],
             'divalproex dr': ['LD50 Values', 'LD50 Values'],
             'once daily lisinopril': ['Mild (e.g., headache, nausea, confusion), but adverse cardiac events including cardiac arrest, ventricular arrhythmias including torsades de pointes and QT prolongation have been reported. LD50=mg/kg (orally in mice)',
              'Mild (e.g., headache, nausea, confusion), but adverse cardiac events including cardiac arrest, ventricular arrhythmias including torsades de pointes and QT prolongation have been reported. LD50=mg/kg (orally in mice)'],
             'methocarbamol': ['Overdose of methocarbamol may be associated with alcohol and other central nervous system depressants.Label Patients may experience nausea, drowsiness, blurred vision, hypotension, seizures, and coma.Label Treatment of overdose is generally through airway maintenance, monitoring urinary output, vital signs, and giving fluid intravenously if necessary.Label',
              'Overdose of methocarbamol may be associated with alcohol and other central nervous system depressants.Label Patients may experience nausea, drowsiness, blurred vision, hypotension, seizures, and coma.Label Treatment of overdose is generally through airway maintenance, monitoring urinary output, vital signs, and giving fluid intravenously if necessary.Label'],
             'methylpred': ['Not Available', 'Not Available'],
             'oxybutynin': ['The acute oral LD50 in rats is 460 mg/kg.13',
              'The acute oral LD50 in rats is 460 mg/kg.13'],
             'phenobarbital s': ['CNS and respiratory depression which may progress to Cheyne-Stokes respiration, areflexia, constriction of the pupils to a slight degree (though in severe poisoning they may wshow paralytic dilation), oliguria, tachycardia, hypotension, lowered body temperature, and coma. Typical shock syndrome (apnea, circulatory collapse, respiratory arrest, and death) may occur.',
              'CNS and respiratory depression which may progress to Cheyne-Stokes respiration, areflexia, constriction of the pupils to a slight degree (though in severe poisoning they may wshow paralytic dilation), oliguria, tachycardia, hypotension, lowered body temperature, and coma. Typical shock syndrome (apnea, circulatory collapse, respiratory arrest, and death) may occur.'],
             'prednisone s': ['Data regarding acute overdoses of prednisone are rare but prolonged high doses of prednisone can lead to a higher incidence and severity of adverse effects such as mental symptoms, moon face, abnormal fat deposits, fluid retention, excessive appetite, weight gain, hypertrichosis, acne, striae, ecchymosis, increased sweating, pigmentation, dry scaly skin, thinning scalp hair, increased blood pressure, tachycardia, thrombophlebitis, decreased resistance to infection, negative nitrogen balance with delayed bone and wound healing, headache, weakness, menstrual disorders, accentuated menopausal symptoms, neuropathy, fractures, osteoporosis, peptic ulcer, decreased glucose tolerance, hypokalemia, and adrenal insufficiency.9',
              'Data regarding acute overdoses of prednisone are rare but prolonged high doses of prednisone can lead to a higher incidence and severity of adverse effects such as mental symptoms, moon face, abnormal fat deposits, fluid retention, excessive appetite, weight gain, hypertrichosis, acne, striae, ecchymosis, increased sweating, pigmentation, dry scaly skin, thinning scalp hair, increased blood pressure, tachycardia, thrombophlebitis, decreased resistance to infection, negative nitrogen balance with delayed bone and wound healing, headache, weakness, menstrual disorders, accentuated menopausal symptoms, neuropathy, fractures, osteoporosis, peptic ulcer, decreased glucose tolerance, hypokalemia, and adrenal insufficiency.9'],
             'spironolactone s': ['Patients experiencing an overdose may present with drowsiness, mental confusion, maculopapular or erythematous rash, nausea, vomiting, dizziness, or diarrhea. Vomiting is generally induced or a gastric lavage is performed.Label Supportive treatment involves maintining hydration, electrolyte balance, and vital functionsLabel.',
              'Patients experiencing an overdose may present with drowsiness, mental confusion, maculopapular or erythematous rash, nausea, vomiting, dizziness, or diarrhea. Vomiting is generally induced or a gastric lavage is performed.Label Supportive treatment involves maintining hydration, electrolyte balance, and vital functionsLabel.'],
             'sotalol': ['Patients experiencing an overdose may present with bradycardia, congestive heart failure, hypotension, bronchospasm, and hypoglycemia.Label Larger intentional overdoses may present as hypotension, bradycardia, cardiac asystole, prolonged QT interval, torsade de pointes, ventricular tachycardia, and premature ventricular complexes.Label Stop administering sotalol and observe the patient until the QT interval returns to normal and the heart rate rises above 50 beats per minute.Label Hemodialysis may help lower plasma concentrations of sotalol as it is not bound to plasma proteins.Label Bradycardia and cardiac asystole may be treated with atropine, other anticholinergic drugs, beta adrenergic agonists, or transvenous cardiac pacing.Label. Second or third degree heart block may be treated with a transvenous cardiac pacemaker.Label Hypotension may be treated with epinephrine or norepinephrine.Label Bronchospasm may be treated with aminophylline or a beta-2 agonist, possibly at higher than normal doses.Label Torsade de pointes may be treated with DC cardioversion, transvenous cardiac pacing, epinephrine, or magnesium sulfate.Label',
              'Patients experiencing an overdose may present with bradycardia, congestive heart failure, hypotension, bronchospasm, and hypoglycemia.Label Larger intentional overdoses may present as hypotension, bradycardia, cardiac asystole, prolonged QT interval, torsade de pointes, ventricular tachycardia, and premature ventricular complexes.Label Stop administering sotalol and observe the patient until the QT interval returns to normal and the heart rate rises above 50 beats per minute.Label Hemodialysis may help lower plasma concentrations of sotalol as it is not bound to plasma proteins.Label Bradycardia and cardiac asystole may be treated with atropine, other anticholinergic drugs, beta adrenergic agonists, or transvenous cardiac pacing.Label. Second or third degree heart block may be treated with a transvenous cardiac pacemaker.Label Hypotension may be treated with epinephrine or norepinephrine.Label Bronchospasm may be treated with aminophylline or a beta-2 agonist, possibly at higher than normal doses.Label Torsade de pointes may be treated with DC cardioversion, transvenous cardiac pacing, epinephrine, or magnesium sulfate.Label'],
             'sulfazine': ['Oral LD50 in mouse is 1500 mg/kg.',
              'Oral LD50 in mouse is 1500 mg/kg.'],
             'sulfazine ec': ['Oral LD50 in mouse is 1500 mg/kg.',
              'Oral LD50 in mouse is 1500 mg/kg.'],
             'trazodone': ['The oral LD50 of trazodone is 690 mg/kg in rats.22',
              'The oral LD50 of trazodone is 690 mg/kg in rats.22'],
             'trihexyphenidyl s': ['Symptoms of overdose include mydriasis, dryness of mucous membranes, red face, atonic states of bowels and bladder, and hyperthermia in high doses. Trihexyphenidyl causes agitation, confusion, and hallucinations due to its effects on the central nervous system. Untreated overdose may result in death, especially in children. Respiratory depression and cardiac arrest may be seen as premortal signs.',
              'Symptoms of overdose include mydriasis, dryness of mucous membranes, red face, atonic states of bowels and bladder, and hyperthermia in high doses. Trihexyphenidyl causes agitation, confusion, and hallucinations due to its effects on the central nervous system. Untreated overdose may result in death, especially in children. Respiratory depression and cardiac arrest may be seen as premortal signs.'],
             'valganciclovir': ['It is expected that an overdose of valganciclovir could also possibly result in increased renal toxicity.',
              'It is expected that an overdose of valganciclovir could also possibly result in increased renal toxicity.'],
             'gildess fe': ['Female patients experiencing and overdose may present with withdrawal bleeding, nausea, vomiting, breast tenderness, abdominal pain, drowsiness, and fatigue.15,25,16,17,24,18,19,20,21,22,23 Overdose should be treated with symptomatic and supportive care including monitoring for potassium concentrations, sodium concentrations, and signs of metabolic acidosis.15,25,16,17,24,18,19,20,21,22,23',
              'Female patients experiencing and overdose may present with withdrawal bleeding, nausea, vomiting, breast tenderness, abdominal pain, drowsiness, and fatigue.15,25,16,17,24,18,19,20,21,22,23 Overdose should be treated with symptomatic and supportive care including monitoring for potassium concentrations, sodium concentrations, and signs of metabolic acidosis.15,25,16,17,24,18,19,20,21,22,23'],})



In [ ]:
toxicity_mapping['acarbose']

['The symptoms of acarbose overdose are likely to be consistent with its adverse effect profile and may therefore include significant gastrointestinal (GI) symptoms (flatulence, distension, etc), although an overdose on an empty stomach (i.e. when not co-administered with food) is less likely to result in these GI symptoms.6 In the event of an overdose, patients should be instructed to avoid carbohydrate-containing foods for 4-6 hours following administration as these can precipitate the aforementioned GI symptoms.6',
 'The symptoms of acarbose overdose are likely to be consistent with its adverse effect profile and may therefore include significant gastrointestinal (GI) symptoms (flatulence, distension, etc), although an overdose on an empty stomach (i.e. when not co-administered with food) is less likely to result in these GI symptoms.6 In the event of an overdose, patients should be instructed to avoid carbohydrate-containing foods for 4-6 hours following administration as these can

In [ ]:
 current_drugs = ['acarbose', 'acyclovir', 'alendronate sodium', 'allopurinol s',
       'alprazolam', 'amaryl', 'amerge', 'amitriptyline hydrochloride',
       'amlodipine besylate and benazepril hydrochloride', 'amoxicillin',
       'amoxicillin and clavulanate potassium', 'amrix', 'atenolol',
       'atorvastatin', 'azithromycin', 'baclofen',
       'benazepril hydrochloride', 'benztropine mesylate', 'betapace',
       'birone', 'birone hydrochloride', 'bumetanide', 'carbamazepine',
       'carvedilol', 'cefuroxime', 'cephalexin', 'ciprofloxacin',
       'citalopram', 'clarithromycin', 'clonazepam',
       'clonidine hydrochloride', 'clozapine', 'coreg cr',
       'cyclobenzaprine hydrochloride', 'dexamethasone',
       'diclofenac sodium', 'diltiazem hydrochloride', 'doxazosin',
       'doxycycline hyclate', 'enalapril maleate', 'escitalopram',
       'estradiol', 'etodolac', 'famotidine', 'fenofibrate',
       'fexofenadine', 'finasteride', 'flecainide acetate', 'fluconazole',
       'fluoxetine', 'fluvoxamine', 'furosemide', 'gabapentin',
       'gabitril', 'glimepiride', 'glipizide', 'glipizide er',
       'glyburide', 'haloperidol', 'hctz s', 'hydralazine',
       'hydralazine hydrochloride', 'hydrochlorothiazide',
       'hydrocodone acetaminophen', 'hydroxyzine hydrochloride', 'hyzaar',
       'indomethacin', 'isosorbide mononitrate', 'lamictal',
       'lamictal xr', 'lamotrigine', 'lanoxin', 'lansoprazole',
       'levetiraceta', 'levetiracetam', 'levofloxacin', 'levothyroxine',
       'levothyroxine s', 'levothyroxine sodium', 'levoxyl', 'lisinopril',
       'lovastatin', 'lyrica', 'meloxicam', 'metformin',
       'metformin hydrochloride', 'metoclopramide',
       'metoprolol succinate', 'metoprolol tartrate', 'metronidazole',
       'mirtazapine', 'nabumetone', 'nadolol', 'naproxen', 'nifedipine',
       'nortel', 'olanzapine', 'omeprazole', 'pantoprazole', 'paroxetine',
       'pioglitazone hydrochloride', 'pravastatin sodium', 'prednisone',
       'propranolol hydrochloride', 'quinapril', 'ramipril', 'ranitidine',
       'requip', 'risperidone', 'ropinirole', 'sertraline hydrochloride',
       'simvastatin', 'sotalol hydrochloride', 'spironolactone',
       'strattera', 'synthroid mcg', 'terazosin', 'tizanidine',
       'topiramate', 'trandolapril', 'trazodone hydrochloride',
       'tribenzor', 'urecholine', 'venlafaxine', 'venlafaxine er',
       'venlafaxine hydrochloride', 'verapamil er', 'warfarin',
       'warfarin sodium', 'ziprasidone']

In [ ]:
def get_new_toxicity():
  result = []
  for drug in current_drugs:
    result.append([drug, toxicity_mapping[drug]])
  return result

In [ ]:
drug_to_toxicity = get_new_toxicity()

In [ ]:
import pandas as pd

In [ ]:
new_df = pd.DataFrame(drug_to_toxicity, columns=['Name', 'toxicity'])

In [ ]:
new_df.to_csv()

',Name,toxicity\n0,acarbose,"[\'The symptoms of acarbose overdose are likely to be consistent with its adverse effect profile and may therefore include significant gastrointestinal (GI) symptoms (flatulence, distension, etc), although an overdose on an empty stomach (i.e. when not co-administered with food) is less likely to result in these GI symptoms.6 In the event of an overdose, patients should be instructed to avoid carbohydrate-containing foods for 4-6 hours following administration as these can precipitate the aforementioned GI symptoms.6\', \'The symptoms of acarbose overdose are likely to be consistent with its adverse effect profile and may therefore include significant gastrointestinal (GI) symptoms (flatulence, distension, etc), although an overdose on an empty stomach (i.e. when not co-administered with food) is less likely to result in these GI symptoms.6 In the event of an overdose, patients should be instructed to avoid carbohydrate-containing foods for 4-6 hours follow

In [ ]:
new_df.to_csv("gdrive/MyDrive/Project/toxicity2.csv", index=False)

In [ ]:
import json
json = json.dumps(toxicity_mapping)
f = open("gdrive/MyDrive/Project/toxicity_mapping.json","w")
f.write(json)
f.close()